In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import sys
import os
import scipy
import math


from sklearn.metrics import r2_score
sys.path.append('../')
sys.path.append('../../')


In [2]:
from Bio.SeqIO import read, parse
import orthoani

In [3]:
from multiprocess import Pool

max_pool = 16
lim_file = 200

In [4]:
from db_sketching.kmer_set import KMerSet, FracMinHash, CMashKmerSet

In [5]:
def cond(kmer_hash):
    hash = (976369 * kmer_hash + 1982627) % 10000
    if hash < 50:
        return True
    else:
        return False
    
random.seed(10)

mod = 1000000007
r0 = random.randint(0,1000000006)
r1 = random.randint(0,1000000006)
r2 = random.randint(0,1000000006)
r3 = random.randint(0,1000000006)
c = 1.1

def poly_cond(kmer_hash):
    hash_mod = kmer_hash % mod
    hash = (r0 + hash_mod * (r1 + hash_mod * (r2 + hash_mod * (r3)))) % mod
    return hash < (mod / c)

def all_cond(kmer_hash):
    return True

In [6]:
def compare_ANI(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition):
    """
    Function to compute the estimated ANI using FracMinHash 
    and compare it to the value from OrthoANI
    """
    return compute_ortho_ani(genome_file_1, genome_file_2), compute_kmer_ani(genome_file_1, genome_file_2, kmer_length, kmer_class, sketching_condition)

def compute_kmer_sketches(genome_file, kmer_class, sketching_condition, kmer_length):
    genome_kmer = kmer_class(sketching_condition, kmer_length)
    genome_kmer.insert_file(genome_file)
    return genome_kmer

def compute_kmer_sketches_parallel(genome_files, kmer_length, kmer_class, sketching_cond):
    args = [(g,kmer_class,sketching_cond,kmer_length) for g in genome_files]
    with Pool(max_pool) as p:
        return p.starmap(compute_kmer_sketches,args)


def compute_kmer_ani(genome_1_kmer, genome_2_kmer):    
    kmer_estimated_ani = genome_1_kmer.ANI_estimation(genome_2_kmer)
    return kmer_estimated_ani


def compute_kmer_ani_parallel(genome_files, kmer_length, kmer_class, sketching_cond):
    kmer_sketches = compute_kmer_sketches_parallel(genome_files, kmer_length, kmer_class, sketching_cond)
    args = [(s1,s2) for s1,s2 in zip(kmer_sketches,kmer_sketches[1:])]
    with Pool(max_pool) as p:
        return p.starmap(compute_kmer_ani,args)

def compute_ortho_ani(genome_file_1, genome_file_2):
    try:
        genome_1_read = parse(genome_file_1,"fasta")
        genome_2_read = parse(genome_file_2,"fasta")
        ortho_ani_value = orthoani.orthoani(genome_1_read,genome_2_read)

        return ortho_ani_value
    except:
        return 0




In [7]:
data_dir = "../../data_temp/"
genome_files = []


for collection in os.listdir(data_dir):
    collection_dir = os.path.join(data_dir,collection)
    for filename in os.listdir(collection_dir):
        full_filename = os.path.join(collection_dir,filename)
        genome_files.append(full_filename)

random.seed(10)

print(genome_files)
print(len(genome_files))

['../../data_temp/388449/7.fna', '../../data_temp/388449/2.fna', '../../data_temp/388449/9.fna', '../../data_temp/388449/5.fna', '../../data_temp/388449/4.fna', '../../data_temp/388449/1.fna', '../../data_temp/388449/8.fna', '../../data_temp/388449/3.fna', '../../data_temp/388449/0.fna', '../../data_temp/388449/6.fna', '../../data_temp/1902584/1.fna', '../../data_temp/1902584/0.fna', '../../data_temp/1331736/7.fna', '../../data_temp/1331736/2.fna', '../../data_temp/1331736/9.fna', '../../data_temp/1331736/5.fna', '../../data_temp/1331736/4.fna', '../../data_temp/1331736/1.fna', '../../data_temp/1331736/8.fna', '../../data_temp/1331736/3.fna', '../../data_temp/1331736/0.fna', '../../data_temp/1331736/6.fna', '../../data_temp/1892252/7.fna', '../../data_temp/1892252/2.fna', '../../data_temp/1892252/9.fna', '../../data_temp/1892252/5.fna', '../../data_temp/1892252/4.fna', '../../data_temp/1892252/1.fna', '../../data_temp/1892252/8.fna', '../../data_temp/1892252/3.fna', '../../data_temp/18

In [8]:
checked_genome_files = []
for file in genome_files:
    try:
        parse(file,"fasta")
        checked_genome_files.append(file)
    except:
        print(f"File {file} is damaged / invalid")


In [9]:
checked_genome_files = ['../../data_temp/388449/7.fna', '../../data_temp/388449/2.fna', '../../data_temp/388449/9.fna', '../../data_temp/388449/5.fna', '../../data_temp/388449/4.fna', '../../data_temp/388449/1.fna', '../../data_temp/388449/8.fna', '../../data_temp/388449/3.fna', '../../data_temp/388449/0.fna', '../../data_temp/388449/6.fna', '../../data_temp/1902584/1.fna', '../../data_temp/1902584/0.fna', '../../data_temp/1331736/7.fna', '../../data_temp/1331736/2.fna', '../../data_temp/1331736/9.fna', '../../data_temp/1331736/5.fna', '../../data_temp/1331736/4.fna', '../../data_temp/1331736/1.fna', '../../data_temp/1331736/8.fna', '../../data_temp/1331736/3.fna', '../../data_temp/1331736/0.fna', '../../data_temp/1331736/6.fna', '../../data_temp/1892252/7.fna', '../../data_temp/1892252/2.fna', '../../data_temp/1892252/9.fna', '../../data_temp/1892252/5.fna', '../../data_temp/1892252/4.fna', '../../data_temp/1892252/1.fna', '../../data_temp/1892252/8.fna', '../../data_temp/1892252/3.fna', '../../data_temp/1892252/0.fna', '../../data_temp/1892252/6.fna', '../../data_temp/3056372/2.fna', '../../data_temp/3056372/1.fna', '../../data_temp/3056372/0.fna', '../../data_temp/2873895/2.fna', '../../data_temp/2873895/1.fna', '../../data_temp/2873895/3.fna', '../../data_temp/2873895/0.fna', '../../data_temp/2848916/7.fna', '../../data_temp/2848916/2.fna', '../../data_temp/2848916/9.fna', '../../data_temp/2848916/5.fna', '../../data_temp/2848916/4.fna', '../../data_temp/2848916/1.fna', '../../data_temp/2848916/8.fna', '../../data_temp/2848916/3.fna', '../../data_temp/2848916/0.fna', '../../data_temp/2848916/6.fna', '../../data_temp/2008794/7.fna', '../../data_temp/2008794/2.fna', '../../data_temp/2008794/9.fna', '../../data_temp/2008794/5.fna', '../../data_temp/2008794/4.fna', '../../data_temp/2008794/1.fna', '../../data_temp/2008794/8.fna', '../../data_temp/2008794/3.fna', '../../data_temp/2008794/0.fna', '../../data_temp/2008794/6.fna', '../../data_temp/2956789/2.fna', '../../data_temp/2956789/1.fna', '../../data_temp/2956789/3.fna', '../../data_temp/2956789/0.fna', '../../data_temp/3104747/0.fna', '../../data_temp/2904714/0.fna', '../../data_temp/1495648/7.fna', '../../data_temp/1495648/2.fna', '../../data_temp/1495648/9.fna', '../../data_temp/1495648/5.fna', '../../data_temp/1495648/4.fna', '../../data_temp/1495648/1.fna', '../../data_temp/1495648/8.fna', '../../data_temp/1495648/3.fna', '../../data_temp/1495648/0.fna', '../../data_temp/1495648/6.fna', '../../data_temp/3039986/7.fna', '../../data_temp/3039986/2.fna', '../../data_temp/3039986/9.fna', '../../data_temp/3039986/5.fna', '../../data_temp/3039986/4.fna', '../../data_temp/3039986/1.fna', '../../data_temp/3039986/8.fna', '../../data_temp/3039986/3.fna', '../../data_temp/3039986/0.fna', '../../data_temp/3039986/6.fna', '../../data_temp/1524216/7.fna', '../../data_temp/1524216/2.fna', '../../data_temp/1524216/5.fna', '../../data_temp/1524216/4.fna', '../../data_temp/1524216/1.fna', '../../data_temp/1524216/3.fna', '../../data_temp/1524216/0.fna', '../../data_temp/1524216/6.fna', '../../data_temp/2005473/7.fna', '../../data_temp/2005473/2.fna', '../../data_temp/2005473/9.fna', '../../data_temp/2005473/5.fna', '../../data_temp/2005473/4.fna', '../../data_temp/2005473/1.fna', '../../data_temp/2005473/8.fna', '../../data_temp/2005473/3.fna', '../../data_temp/2005473/0.fna', '../../data_temp/2005473/6.fna', '../../data_temp/2770093/2.fna', '../../data_temp/2770093/5.fna', '../../data_temp/2770093/4.fna', '../../data_temp/2770093/1.fna', '../../data_temp/2770093/3.fna', '../../data_temp/2770093/0.fna', '../../data_temp/2770093/6.fna', '../../data_temp/3040680/2.fna', '../../data_temp/3040680/5.fna', '../../data_temp/3040680/4.fna', '../../data_temp/3040680/1.fna', '../../data_temp/3040680/3.fna', '../../data_temp/3040680/0.fna', '../../data_temp/3040680/6.fna', '../../data_temp/1903414/7.fna', '../../data_temp/1903414/2.fna', '../../data_temp/1903414/9.fna', '../../data_temp/1903414/5.fna', '../../data_temp/1903414/4.fna', '../../data_temp/1903414/1.fna', '../../data_temp/1903414/8.fna', '../../data_temp/1903414/3.fna', '../../data_temp/1903414/0.fna', '../../data_temp/1903414/6.fna', '../../data_temp/128827/7.fna', '../../data_temp/128827/2.fna', '../../data_temp/128827/9.fna', '../../data_temp/128827/5.fna', '../../data_temp/128827/4.fna', '../../data_temp/128827/1.fna', '../../data_temp/128827/8.fna', '../../data_temp/128827/3.fna', '../../data_temp/128827/0.fna', '../../data_temp/128827/6.fna', '../../data_temp/2844403/2.fna', '../../data_temp/2844403/5.fna', '../../data_temp/2844403/4.fna', '../../data_temp/2844403/1.fna', '../../data_temp/2844403/3.fna', '../../data_temp/2844403/0.fna', '../../data_temp/2844403/6.fna', '../../data_temp/92712/7.fna', '../../data_temp/92712/2.fna', '../../data_temp/92712/9.fna', '../../data_temp/92712/5.fna', '../../data_temp/92712/4.fna', '../../data_temp/92712/1.fna', '../../data_temp/92712/8.fna', '../../data_temp/92712/3.fna', '../../data_temp/92712/0.fna', '../../data_temp/92712/6.fna', '../../data_temp/407032/7.fna', '../../data_temp/407032/2.fna', '../../data_temp/407032/9.fna', '../../data_temp/407032/5.fna', '../../data_temp/407032/4.fna', '../../data_temp/407032/1.fna', '../../data_temp/407032/8.fna', '../../data_temp/407032/3.fna', '../../data_temp/407032/0.fna', '../../data_temp/407032/6.fna', '../../data_temp/2975285/1.fna', '../../data_temp/2975285/0.fna', '../../data_temp/2854170/7.fna', '../../data_temp/2854170/2.fna', '../../data_temp/2854170/9.fna', '../../data_temp/2854170/5.fna', '../../data_temp/2854170/4.fna', '../../data_temp/2854170/1.fna', '../../data_temp/2854170/8.fna', '../../data_temp/2854170/3.fna', '../../data_temp/2854170/0.fna', '../../data_temp/2854170/6.fna', '../../data_temp/2171987/0.fna', '../../data_temp/563843/7.fna', '../../data_temp/563843/2.fna', '../../data_temp/563843/9.fna', '../../data_temp/563843/5.fna', '../../data_temp/563843/4.fna', '../../data_temp/563843/1.fna', '../../data_temp/563843/8.fna', '../../data_temp/563843/3.fna', '../../data_temp/563843/0.fna', '../../data_temp/563843/6.fna', '../../data_temp/2066474/32.fna', '../../data_temp/2066474/7.fna', '../../data_temp/2066474/14.fna', '../../data_temp/2066474/17.fna', '../../data_temp/2066474/2.fna', '../../data_temp/2066474/12.fna', '../../data_temp/2066474/25.fna', '../../data_temp/2066474/22.fna', '../../data_temp/2066474/24.fna', '../../data_temp/2066474/9.fna', '../../data_temp/2066474/5.fna', '../../data_temp/2066474/23.fna', '../../data_temp/2066474/18.fna', '../../data_temp/2066474/4.fna', '../../data_temp/2066474/20.fna', '../../data_temp/2066474/26.fna', '../../data_temp/2066474/1.fna', '../../data_temp/2066474/27.fna', '../../data_temp/2066474/19.fna', '../../data_temp/2066474/13.fna', '../../data_temp/2066474/16.fna', '../../data_temp/2066474/29.fna', '../../data_temp/2066474/8.fna', '../../data_temp/2066474/21.fna', '../../data_temp/2066474/3.fna', '../../data_temp/2066474/10.fna', '../../data_temp/2066474/30.fna', '../../data_temp/2066474/31.fna', '../../data_temp/2066474/28.fna', '../../data_temp/2066474/0.fna', '../../data_temp/2066474/11.fna', '../../data_temp/2066474/6.fna', '../../data_temp/2066474/15.fna', '../../data_temp/1890429/7.fna', '../../data_temp/1890429/2.fna', '../../data_temp/1890429/9.fna', '../../data_temp/1890429/5.fna', '../../data_temp/1890429/4.fna', '../../data_temp/1890429/1.fna', '../../data_temp/1890429/8.fna', '../../data_temp/1890429/3.fna', '../../data_temp/1890429/0.fna', '../../data_temp/1890429/6.fna', '../../data_temp/186806/7.fna', '../../data_temp/186806/2.fna', '../../data_temp/186806/9.fna', '../../data_temp/186806/5.fna', '../../data_temp/186806/4.fna', '../../data_temp/186806/1.fna', '../../data_temp/186806/8.fna', '../../data_temp/186806/3.fna', '../../data_temp/186806/0.fna', '../../data_temp/186806/6.fna', '../../data_temp/2897346/7.fna', '../../data_temp/2897346/2.fna', '../../data_temp/2897346/9.fna', '../../data_temp/2897346/5.fna', '../../data_temp/2897346/4.fna', '../../data_temp/2897346/1.fna', '../../data_temp/2897346/8.fna', '../../data_temp/2897346/3.fna', '../../data_temp/2897346/0.fna', '../../data_temp/2897346/6.fna', '../../data_temp/1903412/7.fna', '../../data_temp/1903412/2.fna', '../../data_temp/1903412/9.fna', '../../data_temp/1903412/5.fna', '../../data_temp/1903412/4.fna', '../../data_temp/1903412/1.fna', '../../data_temp/1903412/8.fna', '../../data_temp/1903412/3.fna', '../../data_temp/1903412/0.fna', '../../data_temp/1903412/6.fna', '../../data_temp/2042895/2.fna', '../../data_temp/2042895/1.fna', '../../data_temp/2042895/0.fna', '../../data_temp/85032/2.fna', '../../data_temp/85032/1.fna', '../../data_temp/85032/3.fna', '../../data_temp/85032/0.fna', '../../data_temp/1643824/7.fna', '../../data_temp/1643824/2.fna', '../../data_temp/1643824/9.fna', '../../data_temp/1643824/5.fna', '../../data_temp/1643824/4.fna', '../../data_temp/1643824/1.fna', '../../data_temp/1643824/8.fna', '../../data_temp/1643824/3.fna', '../../data_temp/1643824/0.fna', '../../data_temp/1643824/6.fna', '../../data_temp/2803844/1.fna', '../../data_temp/2803844/0.fna', '../../data_temp/433/64.fna', '../../data_temp/433/32.fna', '../../data_temp/433/99.fna', '../../data_temp/433/43.fna', '../../data_temp/433/7.fna', '../../data_temp/433/98.fna', '../../data_temp/433/58.fna', '../../data_temp/433/90.fna', '../../data_temp/433/14.fna', '../../data_temp/433/41.fna', '../../data_temp/433/67.fna', '../../data_temp/433/73.fna', '../../data_temp/433/17.fna', '../../data_temp/433/56.fna', '../../data_temp/433/50.fna', '../../data_temp/433/2.fna', '../../data_temp/433/46.fna', '../../data_temp/433/52.fna', '../../data_temp/433/65.fna', '../../data_temp/433/95.fna', '../../data_temp/433/45.fna', '../../data_temp/433/74.fna', '../../data_temp/433/44.fna', '../../data_temp/433/12.fna', '../../data_temp/433/94.fna', '../../data_temp/433/25.fna', '../../data_temp/433/60.fna', '../../data_temp/433/39.fna', '../../data_temp/433/22.fna', '../../data_temp/433/48.fna', '../../data_temp/433/88.fna', '../../data_temp/433/24.fna', '../../data_temp/433/9.fna', '../../data_temp/433/86.fna', '../../data_temp/433/5.fna', '../../data_temp/433/89.fna', '../../data_temp/433/71.fna', '../../data_temp/433/54.fna', '../../data_temp/433/23.fna', '../../data_temp/433/49.fna', '../../data_temp/433/40.fna', '../../data_temp/433/18.fna', '../../data_temp/433/4.fna', '../../data_temp/433/20.fna', '../../data_temp/433/63.fna', '../../data_temp/433/33.fna', '../../data_temp/433/87.fna', '../../data_temp/433/80.fna', '../../data_temp/433/26.fna', '../../data_temp/433/1.fna', '../../data_temp/433/47.fna', '../../data_temp/433/92.fna', '../../data_temp/433/51.fna', '../../data_temp/433/37.fna', '../../data_temp/433/84.fna', '../../data_temp/433/53.fna', '../../data_temp/433/70.fna', '../../data_temp/433/42.fna', '../../data_temp/433/57.fna', '../../data_temp/433/72.fna', '../../data_temp/433/91.fna', '../../data_temp/433/27.fna', '../../data_temp/433/19.fna', '../../data_temp/433/66.fna', '../../data_temp/433/13.fna', '../../data_temp/433/16.fna', '../../data_temp/433/29.fna', '../../data_temp/433/8.fna', '../../data_temp/433/21.fna', '../../data_temp/433/3.fna', '../../data_temp/433/10.fna', '../../data_temp/433/30.fna', '../../data_temp/433/38.fna', '../../data_temp/433/93.fna', '../../data_temp/433/78.fna', '../../data_temp/433/77.fna', '../../data_temp/433/75.fna', '../../data_temp/433/35.fna', '../../data_temp/433/31.fna', '../../data_temp/433/28.fna', '../../data_temp/433/82.fna', '../../data_temp/433/59.fna', '../../data_temp/433/69.fna', '../../data_temp/433/62.fna', '../../data_temp/433/85.fna', '../../data_temp/433/0.fna', '../../data_temp/433/79.fna', '../../data_temp/433/11.fna', '../../data_temp/433/34.fna', '../../data_temp/433/6.fna', '../../data_temp/433/68.fna', '../../data_temp/433/97.fna', '../../data_temp/433/15.fna', '../../data_temp/433/55.fna', '../../data_temp/433/61.fna', '../../data_temp/433/36.fna', '../../data_temp/433/81.fna', '../../data_temp/433/96.fna', '../../data_temp/433/76.fna', '../../data_temp/433/83.fna', '../../data_temp/3031146/2.fna', '../../data_temp/3031146/5.fna', '../../data_temp/3031146/4.fna', '../../data_temp/3031146/1.fna', '../../data_temp/3031146/3.fna', '../../data_temp/3031146/0.fna', '../../data_temp/3031146/6.fna', '../../data_temp/171550/7.fna', '../../data_temp/171550/2.fna', '../../data_temp/171550/9.fna', '../../data_temp/171550/5.fna', '../../data_temp/171550/4.fna', '../../data_temp/171550/1.fna', '../../data_temp/171550/8.fna', '../../data_temp/171550/3.fna', '../../data_temp/171550/0.fna', '../../data_temp/171550/6.fna', '../../data_temp/170/64.fna', '../../data_temp/170/32.fna', '../../data_temp/170/43.fna', '../../data_temp/170/7.fna', '../../data_temp/170/58.fna', '../../data_temp/170/14.fna', '../../data_temp/170/41.fna', '../../data_temp/170/67.fna', '../../data_temp/170/73.fna', '../../data_temp/170/17.fna', '../../data_temp/170/56.fna', '../../data_temp/170/50.fna', '../../data_temp/170/2.fna', '../../data_temp/170/46.fna', '../../data_temp/170/52.fna', '../../data_temp/170/65.fna', '../../data_temp/170/45.fna', '../../data_temp/170/74.fna', '../../data_temp/170/44.fna', '../../data_temp/170/12.fna', '../../data_temp/170/25.fna', '../../data_temp/170/60.fna', '../../data_temp/170/39.fna', '../../data_temp/170/22.fna', '../../data_temp/170/48.fna', '../../data_temp/170/24.fna', '../../data_temp/170/9.fna', '../../data_temp/170/86.fna', '../../data_temp/170/5.fna', '../../data_temp/170/71.fna', '../../data_temp/170/54.fna', '../../data_temp/170/23.fna', '../../data_temp/170/49.fna', '../../data_temp/170/40.fna', '../../data_temp/170/18.fna', '../../data_temp/170/4.fna', '../../data_temp/170/20.fna', '../../data_temp/170/63.fna', '../../data_temp/170/33.fna', '../../data_temp/170/87.fna', '../../data_temp/170/80.fna', '../../data_temp/170/26.fna', '../../data_temp/170/1.fna', '../../data_temp/170/47.fna', '../../data_temp/170/51.fna', '../../data_temp/170/37.fna', '../../data_temp/170/84.fna', '../../data_temp/170/53.fna', '../../data_temp/170/70.fna', '../../data_temp/170/42.fna', '../../data_temp/170/57.fna', '../../data_temp/170/72.fna', '../../data_temp/170/27.fna', '../../data_temp/170/19.fna', '../../data_temp/170/66.fna', '../../data_temp/170/13.fna', '../../data_temp/170/16.fna', '../../data_temp/170/29.fna', '../../data_temp/170/8.fna', '../../data_temp/170/21.fna', '../../data_temp/170/3.fna', '../../data_temp/170/10.fna', '../../data_temp/170/30.fna', '../../data_temp/170/38.fna', '../../data_temp/170/78.fna', '../../data_temp/170/77.fna', '../../data_temp/170/75.fna', '../../data_temp/170/35.fna', '../../data_temp/170/31.fna', '../../data_temp/170/28.fna', '../../data_temp/170/82.fna', '../../data_temp/170/59.fna', '../../data_temp/170/69.fna', '../../data_temp/170/62.fna', '../../data_temp/170/85.fna', '../../data_temp/170/0.fna', '../../data_temp/170/79.fna', '../../data_temp/170/11.fna', '../../data_temp/170/34.fna', '../../data_temp/170/6.fna', '../../data_temp/170/68.fna', '../../data_temp/170/15.fna', '../../data_temp/170/55.fna', '../../data_temp/170/61.fna', '../../data_temp/170/36.fna', '../../data_temp/170/81.fna', '../../data_temp/170/76.fna', '../../data_temp/170/83.fna', '../../data_temp/558415/7.fna', '../../data_temp/558415/2.fna', '../../data_temp/558415/9.fna', '../../data_temp/558415/5.fna', '../../data_temp/558415/4.fna', '../../data_temp/558415/1.fna', '../../data_temp/558415/8.fna', '../../data_temp/558415/3.fna', '../../data_temp/558415/0.fna', '../../data_temp/558415/6.fna', '../../data_temp/200667/7.fna', '../../data_temp/200667/2.fna', '../../data_temp/200667/9.fna', '../../data_temp/200667/5.fna', '../../data_temp/200667/4.fna', '../../data_temp/200667/1.fna', '../../data_temp/200667/8.fna', '../../data_temp/200667/3.fna', '../../data_temp/200667/0.fna', '../../data_temp/200667/6.fna', '../../data_temp/717964/7.fna', '../../data_temp/717964/2.fna', '../../data_temp/717964/9.fna', '../../data_temp/717964/5.fna', '../../data_temp/717964/4.fna', '../../data_temp/717964/1.fna', '../../data_temp/717964/8.fna', '../../data_temp/717964/3.fna', '../../data_temp/717964/0.fna', '../../data_temp/717964/6.fna', '../../data_temp/213483/7.fna', '../../data_temp/213483/2.fna', '../../data_temp/213483/9.fna', '../../data_temp/213483/5.fna', '../../data_temp/213483/4.fna', '../../data_temp/213483/1.fna', '../../data_temp/213483/8.fna', '../../data_temp/213483/3.fna', '../../data_temp/213483/0.fna', '../../data_temp/213483/6.fna', '../../data_temp/3031624/7.fna', '../../data_temp/3031624/2.fna', '../../data_temp/3031624/9.fna', '../../data_temp/3031624/5.fna', '../../data_temp/3031624/4.fna', '../../data_temp/3031624/1.fna', '../../data_temp/3031624/8.fna', '../../data_temp/3031624/3.fna', '../../data_temp/3031624/0.fna', '../../data_temp/3031624/6.fna', '../../data_temp/3090884/1.fna', '../../data_temp/3090884/0.fna', '../../data_temp/2762302/7.fna', '../../data_temp/2762302/2.fna', '../../data_temp/2762302/9.fna', '../../data_temp/2762302/5.fna', '../../data_temp/2762302/4.fna', '../../data_temp/2762302/1.fna', '../../data_temp/2762302/8.fna', '../../data_temp/2762302/3.fna', '../../data_temp/2762302/0.fna', '../../data_temp/2762302/6.fna', '../../data_temp/3031623/0.fna', '../../data_temp/3103149/2.fna', '../../data_temp/3103149/4.fna', '../../data_temp/3103149/1.fna', '../../data_temp/3103149/3.fna', '../../data_temp/3103149/0.fna', '../../data_temp/1663426/7.fna', '../../data_temp/1663426/2.fna', '../../data_temp/1663426/9.fna', '../../data_temp/1663426/5.fna', '../../data_temp/1663426/4.fna', '../../data_temp/1663426/1.fna', '../../data_temp/1663426/8.fna', '../../data_temp/1663426/3.fna', '../../data_temp/1663426/0.fna', '../../data_temp/1663426/6.fna', '../../data_temp/2494208/0.fna', '../../data_temp/84642/7.fna', '../../data_temp/84642/2.fna', '../../data_temp/84642/9.fna', '../../data_temp/84642/5.fna', '../../data_temp/84642/4.fna', '../../data_temp/84642/1.fna', '../../data_temp/84642/8.fna', '../../data_temp/84642/3.fna', '../../data_temp/84642/0.fna', '../../data_temp/84642/6.fna', '../../data_temp/135621/7.fna', '../../data_temp/135621/2.fna', '../../data_temp/135621/9.fna', '../../data_temp/135621/5.fna', '../../data_temp/135621/4.fna', '../../data_temp/135621/1.fna', '../../data_temp/135621/8.fna', '../../data_temp/135621/3.fna', '../../data_temp/135621/0.fna', '../../data_temp/135621/6.fna', '../../data_temp/72293/7.fna', '../../data_temp/72293/2.fna', '../../data_temp/72293/9.fna', '../../data_temp/72293/5.fna', '../../data_temp/72293/4.fna', '../../data_temp/72293/1.fna', '../../data_temp/72293/8.fna', '../../data_temp/72293/3.fna', '../../data_temp/72293/0.fna', '../../data_temp/72293/6.fna', '../../data_temp/1544800/2.fna', '../../data_temp/1544800/4.fna', '../../data_temp/1544800/1.fna', '../../data_temp/1544800/3.fna', '../../data_temp/1544800/0.fna', '../../data_temp/186820/7.fna', '../../data_temp/186820/2.fna', '../../data_temp/186820/9.fna', '../../data_temp/186820/5.fna', '../../data_temp/186820/4.fna', '../../data_temp/186820/1.fna', '../../data_temp/186820/8.fna', '../../data_temp/186820/3.fna', '../../data_temp/186820/0.fna', '../../data_temp/186820/6.fna', '../../data_temp/2844375/7.fna', '../../data_temp/2844375/2.fna', '../../data_temp/2844375/9.fna', '../../data_temp/2844375/5.fna', '../../data_temp/2844375/4.fna', '../../data_temp/2844375/1.fna', '../../data_temp/2844375/8.fna', '../../data_temp/2844375/3.fna', '../../data_temp/2844375/0.fna', '../../data_temp/2844375/6.fna', '../../data_temp/81852/7.fna', '../../data_temp/81852/2.fna', '../../data_temp/81852/9.fna', '../../data_temp/81852/5.fna', '../../data_temp/81852/4.fna', '../../data_temp/81852/1.fna', '../../data_temp/81852/8.fna', '../../data_temp/81852/3.fna', '../../data_temp/81852/0.fna', '../../data_temp/81852/6.fna', '../../data_temp/3056374/2.fna', '../../data_temp/3056374/1.fna', '../../data_temp/3056374/3.fna', '../../data_temp/3056374/0.fna', '../../data_temp/76892/7.fna', '../../data_temp/76892/2.fna', '../../data_temp/76892/9.fna', '../../data_temp/76892/5.fna', '../../data_temp/76892/4.fna', '../../data_temp/76892/1.fna', '../../data_temp/76892/8.fna', '../../data_temp/76892/3.fna', '../../data_temp/76892/0.fna', '../../data_temp/76892/6.fna', '../../data_temp/622451/7.fna', '../../data_temp/622451/2.fna', '../../data_temp/622451/9.fna', '../../data_temp/622451/5.fna', '../../data_temp/622451/4.fna', '../../data_temp/622451/1.fna', '../../data_temp/622451/8.fna', '../../data_temp/622451/3.fna', '../../data_temp/622451/0.fna', '../../data_temp/622451/6.fna', '../../data_temp/2881426/7.fna', '../../data_temp/2881426/2.fna', '../../data_temp/2881426/9.fna', '../../data_temp/2881426/5.fna', '../../data_temp/2881426/4.fna', '../../data_temp/2881426/1.fna', '../../data_temp/2881426/8.fna', '../../data_temp/2881426/3.fna', '../../data_temp/2881426/0.fna', '../../data_temp/2881426/6.fna', '../../data_temp/3031658/7.fna', '../../data_temp/3031658/2.fna', '../../data_temp/3031658/9.fna', '../../data_temp/3031658/5.fna', '../../data_temp/3031658/4.fna', '../../data_temp/3031658/1.fna', '../../data_temp/3031658/8.fna', '../../data_temp/3031658/3.fna', '../../data_temp/3031658/0.fna', '../../data_temp/3031658/6.fna', '../../data_temp/2802693/0.fna', '../../data_temp/543349/7.fna', '../../data_temp/543349/2.fna', '../../data_temp/543349/9.fna', '../../data_temp/543349/5.fna', '../../data_temp/543349/4.fna', '../../data_temp/543349/1.fna', '../../data_temp/543349/8.fna', '../../data_temp/543349/3.fna', '../../data_temp/543349/0.fna', '../../data_temp/543349/6.fna', '../../data_temp/1471398/7.fna', '../../data_temp/1471398/2.fna', '../../data_temp/1471398/9.fna', '../../data_temp/1471398/5.fna', '../../data_temp/1471398/4.fna', '../../data_temp/1471398/1.fna', '../../data_temp/1471398/8.fna', '../../data_temp/1471398/3.fna', '../../data_temp/1471398/0.fna', '../../data_temp/1471398/6.fna', '../../data_temp/2968741/7.fna', '../../data_temp/2968741/2.fna', '../../data_temp/2968741/5.fna', '../../data_temp/2968741/4.fna', '../../data_temp/2968741/1.fna', '../../data_temp/2968741/8.fna', '../../data_temp/2968741/3.fna', '../../data_temp/2968741/0.fna', '../../data_temp/2968741/6.fna', '../../data_temp/1706372/7.fna', '../../data_temp/1706372/2.fna', '../../data_temp/1706372/9.fna', '../../data_temp/1706372/5.fna', '../../data_temp/1706372/4.fna', '../../data_temp/1706372/1.fna', '../../data_temp/1706372/8.fna', '../../data_temp/1706372/3.fna', '../../data_temp/1706372/0.fna', '../../data_temp/1706372/6.fna', '../../data_temp/2762286/7.fna', '../../data_temp/2762286/2.fna', '../../data_temp/2762286/9.fna', '../../data_temp/2762286/5.fna', '../../data_temp/2762286/4.fna', '../../data_temp/2762286/1.fna', '../../data_temp/2762286/8.fna', '../../data_temp/2762286/3.fna', '../../data_temp/2762286/0.fna', '../../data_temp/2762286/6.fna', '../../data_temp/2062/7.fna', '../../data_temp/2062/2.fna', '../../data_temp/2062/9.fna', '../../data_temp/2062/5.fna', '../../data_temp/2062/4.fna', '../../data_temp/2062/1.fna', '../../data_temp/2062/8.fna', '../../data_temp/2062/3.fna', '../../data_temp/2062/0.fna', '../../data_temp/2062/6.fna', '../../data_temp/1643950/7.fna', '../../data_temp/1643950/2.fna', '../../data_temp/1643950/9.fna', '../../data_temp/1643950/5.fna', '../../data_temp/1643950/4.fna', '../../data_temp/1643950/1.fna', '../../data_temp/1643950/8.fna', '../../data_temp/1643950/3.fna', '../../data_temp/1643950/0.fna', '../../data_temp/1643950/6.fna', '../../data_temp/2172021/2.fna', '../../data_temp/2172021/1.fna', '../../data_temp/2172021/3.fna', '../../data_temp/2172021/0.fna', '../../data_temp/2008795/7.fna', '../../data_temp/2008795/2.fna', '../../data_temp/2008795/9.fna', '../../data_temp/2008795/5.fna', '../../data_temp/2008795/4.fna', '../../data_temp/2008795/1.fna', '../../data_temp/2008795/8.fna', '../../data_temp/2008795/3.fna', '../../data_temp/2008795/0.fna', '../../data_temp/2008795/6.fna', '../../data_temp/2886002/2.fna', '../../data_temp/2886002/1.fna', '../../data_temp/2886002/0.fna', '../../data_temp/2945014/1.fna', '../../data_temp/2945014/0.fna', '../../data_temp/28056/7.fna', '../../data_temp/28056/2.fna', '../../data_temp/28056/9.fna', '../../data_temp/28056/5.fna', '../../data_temp/28056/4.fna', '../../data_temp/28056/1.fna', '../../data_temp/28056/8.fna', '../../data_temp/28056/3.fna', '../../data_temp/28056/0.fna', '../../data_temp/28056/6.fna', '../../data_temp/1755824/1.fna', '../../data_temp/1755824/0.fna', '../../data_temp/3121609/2.fna', '../../data_temp/3121609/4.fna', '../../data_temp/3121609/1.fna', '../../data_temp/3121609/3.fna', '../../data_temp/3121609/0.fna', '../../data_temp/543371/2.fna', '../../data_temp/543371/1.fna', '../../data_temp/543371/0.fna', '../../data_temp/3079741/2.fna', '../../data_temp/3079741/1.fna', '../../data_temp/3079741/0.fna', '../../data_temp/3050006/0.fna', '../../data_temp/31979/7.fna', '../../data_temp/31979/2.fna', '../../data_temp/31979/9.fna', '../../data_temp/31979/5.fna', '../../data_temp/31979/4.fna', '../../data_temp/31979/1.fna', '../../data_temp/31979/8.fna', '../../data_temp/31979/3.fna', '../../data_temp/31979/0.fna', '../../data_temp/31979/6.fna', '../../data_temp/2703813/1.fna', '../../data_temp/2703813/0.fna', '../../data_temp/186823/7.fna', '../../data_temp/186823/2.fna', '../../data_temp/186823/9.fna', '../../data_temp/186823/5.fna', '../../data_temp/186823/4.fna', '../../data_temp/186823/1.fna', '../../data_temp/186823/8.fna', '../../data_temp/186823/3.fna', '../../data_temp/186823/0.fna', '../../data_temp/186823/6.fna', '../../data_temp/3029086/7.fna', '../../data_temp/3029086/2.fna', '../../data_temp/3029086/12.fna', '../../data_temp/3029086/9.fna', '../../data_temp/3029086/5.fna', '../../data_temp/3029086/4.fna', '../../data_temp/3029086/1.fna', '../../data_temp/3029086/13.fna', '../../data_temp/3029086/8.fna', '../../data_temp/3029086/3.fna', '../../data_temp/3029086/10.fna', '../../data_temp/3029086/0.fna', '../../data_temp/3029086/11.fna', '../../data_temp/3029086/6.fna', '../../data_temp/1499392/64.fna', '../../data_temp/1499392/32.fna', '../../data_temp/1499392/43.fna', '../../data_temp/1499392/7.fna', '../../data_temp/1499392/58.fna', '../../data_temp/1499392/14.fna', '../../data_temp/1499392/41.fna', '../../data_temp/1499392/67.fna', '../../data_temp/1499392/17.fna', '../../data_temp/1499392/56.fna', '../../data_temp/1499392/50.fna', '../../data_temp/1499392/2.fna', '../../data_temp/1499392/46.fna', '../../data_temp/1499392/52.fna', '../../data_temp/1499392/65.fna', '../../data_temp/1499392/45.fna', '../../data_temp/1499392/44.fna', '../../data_temp/1499392/12.fna', '../../data_temp/1499392/25.fna', '../../data_temp/1499392/60.fna', '../../data_temp/1499392/39.fna', '../../data_temp/1499392/22.fna', '../../data_temp/1499392/48.fna', '../../data_temp/1499392/24.fna', '../../data_temp/1499392/9.fna', '../../data_temp/1499392/5.fna', '../../data_temp/1499392/54.fna', '../../data_temp/1499392/23.fna', '../../data_temp/1499392/49.fna', '../../data_temp/1499392/40.fna', '../../data_temp/1499392/18.fna', '../../data_temp/1499392/4.fna', '../../data_temp/1499392/20.fna', '../../data_temp/1499392/63.fna', '../../data_temp/1499392/33.fna', '../../data_temp/1499392/26.fna', '../../data_temp/1499392/1.fna', '../../data_temp/1499392/47.fna', '../../data_temp/1499392/51.fna', '../../data_temp/1499392/37.fna', '../../data_temp/1499392/53.fna', '../../data_temp/1499392/42.fna', '../../data_temp/1499392/57.fna', '../../data_temp/1499392/27.fna', '../../data_temp/1499392/19.fna', '../../data_temp/1499392/66.fna', '../../data_temp/1499392/13.fna', '../../data_temp/1499392/16.fna', '../../data_temp/1499392/29.fna', '../../data_temp/1499392/8.fna', '../../data_temp/1499392/21.fna', '../../data_temp/1499392/3.fna', '../../data_temp/1499392/10.fna', '../../data_temp/1499392/30.fna', '../../data_temp/1499392/38.fna', '../../data_temp/1499392/35.fna', '../../data_temp/1499392/31.fna', '../../data_temp/1499392/28.fna', '../../data_temp/1499392/59.fna', '../../data_temp/1499392/62.fna', '../../data_temp/1499392/0.fna', '../../data_temp/1499392/11.fna', '../../data_temp/1499392/34.fna', '../../data_temp/1499392/6.fna', '../../data_temp/1499392/15.fna', '../../data_temp/1499392/55.fna', '../../data_temp/1499392/61.fna', '../../data_temp/1499392/36.fna', '../../data_temp/1499392/68.zip', '../../data_temp/45404/7.fna', '../../data_temp/45404/2.fna', '../../data_temp/45404/9.fna', '../../data_temp/45404/5.fna', '../../data_temp/45404/4.fna', '../../data_temp/45404/1.fna', '../../data_temp/45404/8.fna', '../../data_temp/45404/3.fna', '../../data_temp/45404/0.fna', '../../data_temp/45404/6.fna', '../../data_temp/2066491/7.fna', '../../data_temp/2066491/2.fna', '../../data_temp/2066491/9.fna', '../../data_temp/2066491/5.fna', '../../data_temp/2066491/4.fna', '../../data_temp/2066491/1.fna', '../../data_temp/2066491/8.fna', '../../data_temp/2066491/3.fna', '../../data_temp/2066491/0.fna', '../../data_temp/2066491/6.fna', '../../data_temp/2932362/7.fna', '../../data_temp/2932362/2.fna', '../../data_temp/2932362/5.fna', '../../data_temp/2932362/4.fna', '../../data_temp/2932362/1.fna', '../../data_temp/2932362/3.fna', '../../data_temp/2932362/0.fna', '../../data_temp/2932362/6.fna', '../../data_temp/1890498/1.fna', '../../data_temp/1890498/0.fna', '../../data_temp/186817/64.fna', '../../data_temp/186817/32.fna', '../../data_temp/186817/43.fna', '../../data_temp/186817/7.fna', '../../data_temp/186817/58.fna', '../../data_temp/186817/14.fna', '../../data_temp/186817/41.fna', '../../data_temp/186817/67.fna', '../../data_temp/186817/73.fna', '../../data_temp/186817/17.fna', '../../data_temp/186817/56.fna', '../../data_temp/186817/50.fna', '../../data_temp/186817/2.fna', '../../data_temp/186817/46.fna', '../../data_temp/186817/52.fna', '../../data_temp/186817/65.fna', '../../data_temp/186817/45.fna', '../../data_temp/186817/74.fna', '../../data_temp/186817/44.fna', '../../data_temp/186817/12.fna', '../../data_temp/186817/25.fna', '../../data_temp/186817/60.fna', '../../data_temp/186817/39.fna', '../../data_temp/186817/22.fna', '../../data_temp/186817/48.fna', '../../data_temp/186817/24.fna', '../../data_temp/186817/9.fna', '../../data_temp/186817/5.fna', '../../data_temp/186817/71.fna', '../../data_temp/186817/54.fna', '../../data_temp/186817/23.fna', '../../data_temp/186817/49.fna', '../../data_temp/186817/40.fna', '../../data_temp/186817/18.fna', '../../data_temp/186817/4.fna', '../../data_temp/186817/20.fna', '../../data_temp/186817/63.fna', '../../data_temp/186817/33.fna', '../../data_temp/186817/80.fna', '../../data_temp/186817/26.fna', '../../data_temp/186817/1.fna', '../../data_temp/186817/47.fna', '../../data_temp/186817/51.fna', '../../data_temp/186817/37.fna', '../../data_temp/186817/53.fna', '../../data_temp/186817/70.fna', '../../data_temp/186817/42.fna', '../../data_temp/186817/57.fna', '../../data_temp/186817/72.fna', '../../data_temp/186817/27.fna', '../../data_temp/186817/19.fna', '../../data_temp/186817/66.fna', '../../data_temp/186817/13.fna', '../../data_temp/186817/16.fna', '../../data_temp/186817/29.fna', '../../data_temp/186817/8.fna', '../../data_temp/186817/21.fna', '../../data_temp/186817/3.fna', '../../data_temp/186817/10.fna', '../../data_temp/186817/30.fna', '../../data_temp/186817/83.zip', '../../data_temp/186817/38.fna', '../../data_temp/186817/78.fna', '../../data_temp/186817/77.fna', '../../data_temp/186817/75.fna', '../../data_temp/186817/35.fna', '../../data_temp/186817/31.fna', '../../data_temp/186817/28.fna', '../../data_temp/186817/82.fna', '../../data_temp/186817/59.fna', '../../data_temp/186817/69.fna', '../../data_temp/186817/62.fna', '../../data_temp/186817/0.fna', '../../data_temp/186817/79.fna', '../../data_temp/186817/11.fna', '../../data_temp/186817/34.fna', '../../data_temp/186817/6.fna', '../../data_temp/186817/68.fna', '../../data_temp/186817/15.fna', '../../data_temp/186817/55.fna', '../../data_temp/186817/61.fna', '../../data_temp/186817/36.fna', '../../data_temp/186817/81.fna', '../../data_temp/186817/76.fna', '../../data_temp/320599/7.fna', '../../data_temp/320599/2.fna', '../../data_temp/320599/9.fna', '../../data_temp/320599/5.fna', '../../data_temp/320599/4.fna', '../../data_temp/320599/1.fna', '../../data_temp/320599/8.fna', '../../data_temp/320599/3.fna', '../../data_temp/320599/0.fna', '../../data_temp/320599/6.fna', '../../data_temp/1775411/7.fna', '../../data_temp/1775411/2.fna', '../../data_temp/1775411/9.fna', '../../data_temp/1775411/5.fna', '../../data_temp/1775411/4.fna', '../../data_temp/1775411/1.fna', '../../data_temp/1775411/8.fna', '../../data_temp/1775411/3.fna', '../../data_temp/1775411/0.fna', '../../data_temp/1775411/6.fna', '../../data_temp/145360/7.fna', '../../data_temp/145360/2.fna', '../../data_temp/145360/9.fna', '../../data_temp/145360/5.fna', '../../data_temp/145360/4.fna', '../../data_temp/145360/1.fna', '../../data_temp/145360/8.fna', '../../data_temp/145360/3.fna', '../../data_temp/145360/0.fna', '../../data_temp/145360/6.fna', '../../data_temp/1501348/7.fna', '../../data_temp/1501348/2.fna', '../../data_temp/1501348/9.fna', '../../data_temp/1501348/5.fna', '../../data_temp/1501348/4.fna', '../../data_temp/1501348/1.fna', '../../data_temp/1501348/8.fna', '../../data_temp/1501348/3.fna', '../../data_temp/1501348/0.fna', '../../data_temp/1501348/6.fna', '../../data_temp/3079750/0.fna', '../../data_temp/2800061/7.fna', '../../data_temp/2800061/2.fna', '../../data_temp/2800061/9.fna', '../../data_temp/2800061/5.fna', '../../data_temp/2800061/4.fna', '../../data_temp/2800061/1.fna', '../../data_temp/2800061/8.fna', '../../data_temp/2800061/3.fna', '../../data_temp/2800061/0.fna', '../../data_temp/2800061/6.fna', '../../data_temp/1890422/2.fna', '../../data_temp/1890422/1.fna', '../../data_temp/1890422/0.fna', '../../data_temp/2945019/7.fna', '../../data_temp/2945019/2.fna', '../../data_temp/2945019/9.fna', '../../data_temp/2945019/5.fna', '../../data_temp/2945019/4.fna', '../../data_temp/2945019/1.fna', '../../data_temp/2945019/8.fna', '../../data_temp/2945019/3.fna', '../../data_temp/2945019/0.fna', '../../data_temp/2945019/6.fna', '../../data_temp/633392/2.fna', '../../data_temp/633392/5.fna', '../../data_temp/633392/4.fna', '../../data_temp/633392/1.fna', '../../data_temp/633392/3.fna', '../../data_temp/633392/0.fna', '../../data_temp/1890432/0.fna', '../../data_temp/3079746/1.fna', '../../data_temp/3079746/0.fna', '../../data_temp/84997/7.fna', '../../data_temp/84997/2.fna', '../../data_temp/84997/9.fna', '../../data_temp/84997/5.fna', '../../data_temp/84997/4.fna', '../../data_temp/84997/1.fna', '../../data_temp/84997/8.fna', '../../data_temp/84997/3.fna', '../../data_temp/84997/0.fna', '../../data_temp/84997/6.fna', '../../data_temp/2887365/7.fna', '../../data_temp/2887365/2.fna', '../../data_temp/2887365/9.fna', '../../data_temp/2887365/5.fna', '../../data_temp/2887365/4.fna', '../../data_temp/2887365/1.fna', '../../data_temp/2887365/8.fna', '../../data_temp/2887365/3.fna', '../../data_temp/2887365/0.fna', '../../data_temp/2887365/6.fna', '../../data_temp/3031471/0.fna', '../../data_temp/2109261/1.fna', '../../data_temp/2109261/0.fna', '../../data_temp/32011/7.fna', '../../data_temp/32011/2.fna', '../../data_temp/32011/9.fna', '../../data_temp/32011/5.fna', '../../data_temp/32011/4.fna', '../../data_temp/32011/1.fna', '../../data_temp/32011/8.fna', '../../data_temp/32011/3.fna', '../../data_temp/32011/0.fna', '../../data_temp/32011/6.fna', '../../data_temp/204431/7.fna', '../../data_temp/204431/2.fna', '../../data_temp/204431/9.fna', '../../data_temp/204431/5.fna', '../../data_temp/204431/4.fna', '../../data_temp/204431/1.fna', '../../data_temp/204431/8.fna', '../../data_temp/204431/3.fna', '../../data_temp/204431/0.fna', '../../data_temp/204431/6.fna', '../../data_temp/2954146/2.fna', '../../data_temp/2954146/1.fna', '../../data_temp/2954146/0.fna', '../../data_temp/3141700/2.fna', '../../data_temp/3141700/4.fna', '../../data_temp/3141700/1.fna', '../../data_temp/3141700/3.fna', '../../data_temp/3141700/0.fna', '../../data_temp/2844885/1.fna', '../../data_temp/2844885/0.fna', '../../data_temp/1937962/2.fna', '../../data_temp/1937962/5.fna', '../../data_temp/1937962/4.fna', '../../data_temp/1937962/1.fna', '../../data_temp/1937962/3.fna', '../../data_temp/1937962/0.fna', '../../data_temp/1937962/6.fna', '../../data_temp/1843490/7.fna', '../../data_temp/1843490/2.fna', '../../data_temp/1843490/9.fna', '../../data_temp/1843490/5.fna', '../../data_temp/1843490/1.zip', '../../data_temp/1843490/4.fna', '../../data_temp/1843490/1.fna', '../../data_temp/1843490/8.fna', '../../data_temp/1843490/3.fna', '../../data_temp/1843490/0.fna', '../../data_temp/1843490/6.fna', '../../data_temp/2900662/0.fna', '../../data_temp/2964538/7.fna', '../../data_temp/2964538/2.fna', '../../data_temp/2964538/9.fna', '../../data_temp/2964538/5.fna', '../../data_temp/2964538/4.fna', '../../data_temp/2964538/1.fna', '../../data_temp/2964538/8.fna', '../../data_temp/2964538/3.fna', '../../data_temp/2964538/0.fna', '../../data_temp/2964538/6.fna', '../../data_temp/1892263/7.fna', '../../data_temp/1892263/2.fna', '../../data_temp/1892263/9.fna', '../../data_temp/1892263/5.fna', '../../data_temp/1892263/4.fna', '../../data_temp/1892263/1.fna', '../../data_temp/1892263/8.fna', '../../data_temp/1892263/3.fna', '../../data_temp/1892263/0.fna', '../../data_temp/1892263/6.fna', '../../data_temp/2507544/2.fna', '../../data_temp/2507544/1.fna', '../../data_temp/2507544/0.fna', '../../data_temp/186807/7.fna', '../../data_temp/186807/2.fna', '../../data_temp/186807/9.fna', '../../data_temp/186807/5.fna', '../../data_temp/186807/4.fna', '../../data_temp/186807/1.fna', '../../data_temp/186807/8.fna', '../../data_temp/186807/3.fna', '../../data_temp/186807/0.fna', '../../data_temp/186807/6.fna', '../../data_temp/85018/7.fna', '../../data_temp/85018/2.fna', '../../data_temp/85018/9.fna', '../../data_temp/85018/5.fna', '../../data_temp/85018/4.fna', '../../data_temp/85018/1.fna', '../../data_temp/85018/8.fna', '../../data_temp/85018/3.fna', '../../data_temp/85018/0.fna', '../../data_temp/85018/6.fna', '../../data_temp/263369/7.fna', '../../data_temp/263369/2.fna', '../../data_temp/263369/9.fna', '../../data_temp/263369/5.fna', '../../data_temp/263369/4.fna', '../../data_temp/263369/1.fna', '../../data_temp/263369/8.fna', '../../data_temp/263369/3.fna', '../../data_temp/263369/0.fna', '../../data_temp/263369/6.fna', '../../data_temp/2690195/2.fna', '../../data_temp/2690195/5.fna', '../../data_temp/2690195/4.fna', '../../data_temp/2690195/1.fna', '../../data_temp/2690195/3.fna', '../../data_temp/2690195/0.fna', '../../data_temp/3064153/0.fna', '../../data_temp/224372/7.fna', '../../data_temp/224372/2.fna', '../../data_temp/224372/9.fna', '../../data_temp/224372/5.fna', '../../data_temp/224372/4.fna', '../../data_temp/224372/1.fna', '../../data_temp/224372/8.fna', '../../data_temp/224372/3.fna', '../../data_temp/224372/0.fna', '../../data_temp/224372/6.fna', '../../data_temp/85002/7.fna', '../../data_temp/85002/2.fna', '../../data_temp/85002/5.fna', '../../data_temp/85002/4.fna', '../../data_temp/85002/1.fna', '../../data_temp/85002/8.fna', '../../data_temp/85002/3.fna', '../../data_temp/85002/0.fna', '../../data_temp/85002/6.fna', '../../data_temp/2946505/0.fna', '../../data_temp/3118656/7.fna', '../../data_temp/3118656/2.fna', '../../data_temp/3118656/9.fna', '../../data_temp/3118656/5.fna', '../../data_temp/3118656/4.fna', '../../data_temp/3118656/1.fna', '../../data_temp/3118656/8.fna', '../../data_temp/3118656/3.fna', '../../data_temp/3118656/0.fna', '../../data_temp/3118656/6.fna', '../../data_temp/1982354/0.fna', '../../data_temp/183710/7.fna', '../../data_temp/183710/2.fna', '../../data_temp/183710/9.fna', '../../data_temp/183710/5.fna', '../../data_temp/183710/4.fna', '../../data_temp/183710/1.fna', '../../data_temp/183710/8.fna', '../../data_temp/183710/3.fna', '../../data_temp/183710/0.fna', '../../data_temp/183710/6.fna', '../../data_temp/135616/7.fna', '../../data_temp/135616/2.fna', '../../data_temp/135616/9.fna', '../../data_temp/135616/5.fna', '../../data_temp/135616/4.fna', '../../data_temp/135616/1.fna', '../../data_temp/135616/8.fna', '../../data_temp/135616/3.fna', '../../data_temp/135616/0.fna', '../../data_temp/135616/6.fna', '../../data_temp/80864/7.fna', '../../data_temp/80864/2.fna', '../../data_temp/80864/9.fna', '../../data_temp/80864/5.fna', '../../data_temp/80864/4.fna', '../../data_temp/80864/1.fna', '../../data_temp/80864/8.fna', '../../data_temp/80864/3.fna', '../../data_temp/80864/0.fna', '../../data_temp/80864/6.fna', '../../data_temp/206379/7.fna', '../../data_temp/206379/2.fna', '../../data_temp/206379/9.fna', '../../data_temp/206379/5.fna', '../../data_temp/206379/4.fna', '../../data_temp/206379/1.fna', '../../data_temp/206379/8.fna', '../../data_temp/206379/3.fna', '../../data_temp/206379/0.fna', '../../data_temp/206379/6.fna', '../../data_temp/449719/7.fna', '../../data_temp/449719/2.fna', '../../data_temp/449719/9.fna', '../../data_temp/449719/5.fna', '../../data_temp/449719/4.fna', '../../data_temp/449719/1.fna', '../../data_temp/449719/8.fna', '../../data_temp/449719/3.fna', '../../data_temp/449719/0.fna', '../../data_temp/449719/6.fna', '../../data_temp/3031670/0.fna', '../../data_temp/2844881/7.fna', '../../data_temp/2844881/2.fna', '../../data_temp/2844881/9.fna', '../../data_temp/2844881/5.fna', '../../data_temp/2844881/4.fna', '../../data_temp/2844881/1.fna', '../../data_temp/2844881/8.fna', '../../data_temp/2844881/3.fna', '../../data_temp/2844881/0.fna', '../../data_temp/2844881/6.fna', '../../data_temp/2717338/0.fna', '../../data_temp/104174/7.fna', '../../data_temp/104174/2.fna', '../../data_temp/104174/9.fna', '../../data_temp/104174/5.fna', '../../data_temp/104174/4.fna', '../../data_temp/104174/1.fna', '../../data_temp/104174/8.fna', '../../data_temp/104174/3.fna', '../../data_temp/104174/0.fna', '../../data_temp/104174/6.fna', '../../data_temp/2571193/2.fna', '../../data_temp/2571193/5.fna', '../../data_temp/2571193/4.fna', '../../data_temp/2571193/1.fna', '../../data_temp/2571193/3.fna', '../../data_temp/2571193/0.fna', '../../data_temp/118882/7.fna', '../../data_temp/118882/2.fna', '../../data_temp/118882/9.fna', '../../data_temp/118882/5.fna', '../../data_temp/118882/4.fna', '../../data_temp/118882/1.fna', '../../data_temp/118882/8.fna', '../../data_temp/118882/3.fna', '../../data_temp/118882/0.fna', '../../data_temp/118882/6.fna', '../../data_temp/1653/7.fna', '../../data_temp/1653/2.fna', '../../data_temp/1653/9.fna', '../../data_temp/1653/5.fna', '../../data_temp/1653/4.fna', '../../data_temp/1653/1.fna', '../../data_temp/1653/8.fna', '../../data_temp/1653/3.fna', '../../data_temp/1653/0.fna', '../../data_temp/1653/6.fna', '../../data_temp/1491775/2.fna', '../../data_temp/1491775/1.fna', '../../data_temp/1491775/0.fna', '../../data_temp/2885993/1.fna', '../../data_temp/2885993/0.fna', '../../data_temp/3120162/2.fna', '../../data_temp/3120162/1.fna', '../../data_temp/3120162/0.fna', '../../data_temp/2829814/7.fna', '../../data_temp/2829814/2.fna', '../../data_temp/2829814/5.fna', '../../data_temp/2829814/4.fna', '../../data_temp/2829814/1.fna', '../../data_temp/2829814/3.fna', '../../data_temp/2829814/0.fna', '../../data_temp/2829814/6.fna', '../../data_temp/3085657/2.fna', '../../data_temp/3085657/1.fna', '../../data_temp/3085657/0.fna', '../../data_temp/3031458/0.fna', '../../data_temp/83763/7.fna', '../../data_temp/83763/2.fna', '../../data_temp/83763/9.fna', '../../data_temp/83763/5.fna', '../../data_temp/83763/4.fna', '../../data_temp/83763/1.fna', '../../data_temp/83763/8.fna', '../../data_temp/83763/3.fna', '../../data_temp/83763/0.fna', '../../data_temp/83763/6.fna', '../../data_temp/475964/7.fna', '../../data_temp/475964/2.fna', '../../data_temp/475964/9.fna', '../../data_temp/475964/5.fna', '../../data_temp/475964/4.fna', '../../data_temp/475964/1.fna', '../../data_temp/475964/8.fna', '../../data_temp/475964/3.fna', '../../data_temp/475964/0.fna', '../../data_temp/475964/6.fna', '../../data_temp/2171990/2.fna', '../../data_temp/2171990/4.fna', '../../data_temp/2171990/1.fna', '../../data_temp/2171990/3.fna', '../../data_temp/2171990/0.fna', '../../data_temp/1647988/7.fna', '../../data_temp/1647988/2.fna', '../../data_temp/1647988/9.fna', '../../data_temp/1647988/5.fna', '../../data_temp/1647988/4.fna', '../../data_temp/1647988/1.fna', '../../data_temp/1647988/8.fna', '../../data_temp/1647988/3.fna', '../../data_temp/1647988/0.fna', '../../data_temp/1647988/6.fna', '../../data_temp/2813859/1.fna', '../../data_temp/2813859/0.fna', '../../data_temp/2005523/7.fna', '../../data_temp/2005523/2.fna', '../../data_temp/2005523/9.fna', '../../data_temp/2005523/5.fna', '../../data_temp/2005523/4.fna', '../../data_temp/2005523/1.fna', '../../data_temp/2005523/8.fna', '../../data_temp/2005523/3.fna', '../../data_temp/2005523/0.fna', '../../data_temp/2005523/6.fna', '../../data_temp/3118652/7.fna', '../../data_temp/3118652/2.fna', '../../data_temp/3118652/9.fna', '../../data_temp/3118652/5.fna', '../../data_temp/3118652/4.fna', '../../data_temp/3118652/1.fna', '../../data_temp/3118652/8.fna', '../../data_temp/3118652/3.fna', '../../data_temp/3118652/0.fna', '../../data_temp/3118652/6.fna', '../../data_temp/33958/7.fna', '../../data_temp/33958/2.fna', '../../data_temp/33958/9.fna', '../../data_temp/33958/5.fna', '../../data_temp/33958/4.fna', '../../data_temp/33958/1.fna', '../../data_temp/33958/8.fna', '../../data_temp/33958/3.fna', '../../data_temp/33958/0.fna', '../../data_temp/33958/6.fna', '../../data_temp/224467/7.fna', '../../data_temp/224467/2.fna', '../../data_temp/224467/9.fna', '../../data_temp/224467/5.fna', '../../data_temp/224467/4.fna', '../../data_temp/224467/1.fna', '../../data_temp/224467/8.fna', '../../data_temp/224467/3.fna', '../../data_temp/224467/0.fna', '../../data_temp/224467/6.fna', '../../data_temp/2811505/0.fna', '../../data_temp/3031665/2.fna', '../../data_temp/3031665/1.fna', '../../data_temp/3031665/0.fna', '../../data_temp/1382930/7.fna', '../../data_temp/1382930/2.fna', '../../data_temp/1382930/9.fna', '../../data_temp/1382930/5.fna', '../../data_temp/1382930/4.fna', '../../data_temp/1382930/1.fna', '../../data_temp/1382930/8.fna', '../../data_temp/1382930/3.fna', '../../data_temp/1382930/0.fna', '../../data_temp/1382930/6.fna', '../../data_temp/2714749/2.fna', '../../data_temp/2714749/1.fna', '../../data_temp/2714749/0.fna', '../../data_temp/2975520/2.fna', '../../data_temp/2975520/1.fna', '../../data_temp/2975520/0.fna', '../../data_temp/188711/7.fna', '../../data_temp/188711/2.fna', '../../data_temp/188711/9.fna', '../../data_temp/188711/5.fna', '../../data_temp/188711/4.fna', '../../data_temp/188711/1.fna', '../../data_temp/188711/8.fna', '../../data_temp/188711/3.fna', '../../data_temp/188711/0.fna', '../../data_temp/188711/6.fna', '../../data_temp/2562244/0.fna', '../../data_temp/772/7.fna', '../../data_temp/772/2.fna', '../../data_temp/772/9.fna', '../../data_temp/772/5.fna', '../../data_temp/772/4.fna', '../../data_temp/772/1.fna', '../../data_temp/772/8.fna', '../../data_temp/772/3.fna', '../../data_temp/772/0.fna', '../../data_temp/772/6.fna', '../../data_temp/444/7.fna', '../../data_temp/444/2.fna', '../../data_temp/444/9.fna', '../../data_temp/444/5.fna', '../../data_temp/444/4.fna', '../../data_temp/444/1.fna', '../../data_temp/444/8.fna', '../../data_temp/444/3.fna', '../../data_temp/444/0.fna', '../../data_temp/444/6.fna', '../../data_temp/3067276/7.fna', '../../data_temp/3067276/2.fna', '../../data_temp/3067276/9.fna', '../../data_temp/3067276/5.fna', '../../data_temp/3067276/4.fna', '../../data_temp/3067276/1.fna', '../../data_temp/3067276/8.fna', '../../data_temp/3067276/3.fna', '../../data_temp/3067276/0.fna', '../../data_temp/3067276/6.fna', '../../data_temp/2975259/0.fna', '../../data_temp/768649/0.fna', '../../data_temp/3039170/1.fna', '../../data_temp/3039170/0.fna', '../../data_temp/1868322/0.fna', '../../data_temp/1266054/2.fna', '../../data_temp/1266054/1.fna', '../../data_temp/1266054/3.fna', '../../data_temp/1266054/0.fna', '../../data_temp/3120708/1.fna', '../../data_temp/3120708/0.fna', '../../data_temp/145357/7.fna', '../../data_temp/145357/2.fna', '../../data_temp/145357/9.fna', '../../data_temp/145357/5.fna', '../../data_temp/145357/4.fna', '../../data_temp/145357/1.fna', '../../data_temp/145357/8.fna', '../../data_temp/145357/3.fna', '../../data_temp/145357/0.fna', '../../data_temp/145357/6.fna', '../../data_temp/39782/7.fna', '../../data_temp/39782/2.fna', '../../data_temp/39782/9.fna', '../../data_temp/39782/5.fna', '../../data_temp/39782/4.fna', '../../data_temp/39782/1.fna', '../../data_temp/39782/8.fna', '../../data_temp/39782/3.fna', '../../data_temp/39782/0.fna', '../../data_temp/39782/6.fna', '../../data_temp/213422/7.fna', '../../data_temp/213422/2.fna', '../../data_temp/213422/9.fna', '../../data_temp/213422/5.fna', '../../data_temp/213422/4.fna', '../../data_temp/213422/1.fna', '../../data_temp/213422/8.fna', '../../data_temp/213422/3.fna', '../../data_temp/213422/0.fna', '../../data_temp/213422/6.fna', '../../data_temp/119859/7.fna', '../../data_temp/119859/2.fna', '../../data_temp/119859/9.fna', '../../data_temp/119859/5.fna', '../../data_temp/119859/4.fna', '../../data_temp/119859/1.fna', '../../data_temp/119859/8.fna', '../../data_temp/119859/3.fna', '../../data_temp/119859/0.fna', '../../data_temp/119859/6.fna', '../../data_temp/3085096/2.fna', '../../data_temp/3085096/5.fna', '../../data_temp/3085096/4.fna', '../../data_temp/3085096/1.fna', '../../data_temp/3085096/3.fna', '../../data_temp/3085096/0.fna', '../../data_temp/3085096/6.fna', '../../data_temp/85029/7.fna', '../../data_temp/85029/2.fna', '../../data_temp/85029/9.fna', '../../data_temp/85029/5.fna', '../../data_temp/85029/4.fna', '../../data_temp/85029/1.fna', '../../data_temp/85029/8.fna', '../../data_temp/85029/3.fna', '../../data_temp/85029/0.fna', '../../data_temp/85029/6.fna', '../../data_temp/3039169/0.fna', '../../data_temp/3085114/0.fna', '../../data_temp/2806305/0.fna', '../../data_temp/213122/0.fna', '../../data_temp/3018741/7.fna', '../../data_temp/3018741/2.fna', '../../data_temp/3018741/5.fna', '../../data_temp/3018741/4.fna', '../../data_temp/3018741/1.fna', '../../data_temp/3018741/8.fna', '../../data_temp/3018741/3.fna', '../../data_temp/3018741/0.fna', '../../data_temp/3018741/6.fna', '../../data_temp/186828/7.fna', '../../data_temp/186828/2.fna', '../../data_temp/186828/9.fna', '../../data_temp/186828/5.fna', '../../data_temp/186828/4.fna', '../../data_temp/186828/1.fna', '../../data_temp/186828/8.fna', '../../data_temp/186828/3.fna', '../../data_temp/186828/0.fna', '../../data_temp/186828/6.fna', '../../data_temp/186818/7.fna', '../../data_temp/186818/2.fna', '../../data_temp/186818/9.fna', '../../data_temp/186818/5.fna', '../../data_temp/186818/4.fna', '../../data_temp/186818/1.fna', '../../data_temp/186818/8.fna', '../../data_temp/186818/3.fna', '../../data_temp/186818/0.fna', '../../data_temp/186818/6.fna', '../../data_temp/2845253/7.fna', '../../data_temp/2845253/2.fna', '../../data_temp/2845253/9.fna', '../../data_temp/2845253/5.fna', '../../data_temp/2845253/4.fna', '../../data_temp/2845253/1.fna', '../../data_temp/2845253/8.fna', '../../data_temp/2845253/3.fna', '../../data_temp/2845253/0.fna', '../../data_temp/2845253/6.fna', '../../data_temp/2131/64.fna', '../../data_temp/2131/32.fna', '../../data_temp/2131/43.fna', '../../data_temp/2131/7.fna', '../../data_temp/2131/58.fna', '../../data_temp/2131/14.fna', '../../data_temp/2131/41.fna', '../../data_temp/2131/67.fna', '../../data_temp/2131/17.fna', '../../data_temp/2131/56.fna', '../../data_temp/2131/50.fna', '../../data_temp/2131/2.fna', '../../data_temp/2131/46.fna', '../../data_temp/2131/52.fna', '../../data_temp/2131/65.fna', '../../data_temp/2131/45.fna', '../../data_temp/2131/44.fna', '../../data_temp/2131/12.fna', '../../data_temp/2131/25.fna', '../../data_temp/2131/60.fna', '../../data_temp/2131/39.fna', '../../data_temp/2131/22.fna', '../../data_temp/2131/48.fna', '../../data_temp/2131/24.fna', '../../data_temp/2131/9.fna', '../../data_temp/2131/5.fna', '../../data_temp/2131/54.fna', '../../data_temp/2131/23.fna', '../../data_temp/2131/49.fna', '../../data_temp/2131/40.fna', '../../data_temp/2131/18.fna', '../../data_temp/2131/4.fna', '../../data_temp/2131/20.fna', '../../data_temp/2131/63.fna', '../../data_temp/2131/33.fna', '../../data_temp/2131/26.fna', '../../data_temp/2131/1.fna', '../../data_temp/2131/47.fna', '../../data_temp/2131/51.fna', '../../data_temp/2131/37.fna', '../../data_temp/2131/53.fna', '../../data_temp/2131/42.fna', '../../data_temp/2131/57.fna', '../../data_temp/2131/27.fna', '../../data_temp/2131/19.fna', '../../data_temp/2131/66.fna', '../../data_temp/2131/13.fna', '../../data_temp/2131/16.fna', '../../data_temp/2131/29.fna', '../../data_temp/2131/8.fna', '../../data_temp/2131/21.fna', '../../data_temp/2131/3.fna', '../../data_temp/2131/10.fna', '../../data_temp/2131/30.fna', '../../data_temp/2131/38.fna', '../../data_temp/2131/35.fna', '../../data_temp/2131/31.fna', '../../data_temp/2131/28.fna', '../../data_temp/2131/59.fna', '../../data_temp/2131/69.fna', '../../data_temp/2131/62.fna', '../../data_temp/2131/0.fna', '../../data_temp/2131/11.fna', '../../data_temp/2131/34.fna', '../../data_temp/2131/6.fna', '../../data_temp/2131/68.fna', '../../data_temp/2131/15.fna', '../../data_temp/2131/55.fna', '../../data_temp/2131/61.fna', '../../data_temp/2131/36.fna', '../../data_temp/2614438/0.fna', '../../data_temp/1890528/7.fna', '../../data_temp/1890528/2.fna', '../../data_temp/1890528/9.fna', '../../data_temp/1890528/5.fna', '../../data_temp/1890528/4.fna', '../../data_temp/1890528/1.fna', '../../data_temp/1890528/8.fna', '../../data_temp/1890528/3.fna', '../../data_temp/1890528/0.fna', '../../data_temp/1890528/6.fna', '../../data_temp/3142785/0.fna', '../../data_temp/224379/7.fna', '../../data_temp/224379/2.fna', '../../data_temp/224379/9.fna', '../../data_temp/224379/5.fna', '../../data_temp/224379/4.fna', '../../data_temp/224379/1.fna', '../../data_temp/224379/8.fna', '../../data_temp/224379/3.fna', '../../data_temp/224379/0.fna', '../../data_temp/224379/6.fna', '../../data_temp/85017/7.fna', '../../data_temp/85017/2.fna', '../../data_temp/85017/9.fna', '../../data_temp/85017/5.fna', '../../data_temp/85017/4.fna', '../../data_temp/85017/1.fna', '../../data_temp/85017/8.fna', '../../data_temp/85017/3.fna', '../../data_temp/85017/0.fna', '../../data_temp/85017/6.fna', '../../data_temp/3119833/7.fna', '../../data_temp/3119833/2.fna', '../../data_temp/3119833/9.fna', '../../data_temp/3119833/5.fna', '../../data_temp/3119833/4.fna', '../../data_temp/3119833/1.fna', '../../data_temp/3119833/8.fna', '../../data_temp/3119833/3.fna', '../../data_temp/3119833/0.fna', '../../data_temp/3119833/6.fna', '../../data_temp/89374/7.fna', '../../data_temp/89374/2.fna', '../../data_temp/89374/9.fna', '../../data_temp/89374/5.fna', '../../data_temp/89374/4.fna', '../../data_temp/89374/1.fna', '../../data_temp/89374/8.fna', '../../data_temp/89374/3.fna', '../../data_temp/89374/0.fna', '../../data_temp/89374/6.fna', '../../data_temp/2813035/7.fna', '../../data_temp/2813035/2.fna', '../../data_temp/2813035/9.fna', '../../data_temp/2813035/5.fna', '../../data_temp/2813035/4.fna', '../../data_temp/2813035/1.fna', '../../data_temp/2813035/8.fna', '../../data_temp/2813035/3.fna', '../../data_temp/2813035/0.fna', '../../data_temp/2813035/6.fna', '../../data_temp/2759950/1.fna', '../../data_temp/2759950/0.fna', '../../data_temp/85028/7.fna', '../../data_temp/85028/2.fna', '../../data_temp/85028/9.fna', '../../data_temp/85028/5.fna', '../../data_temp/85028/4.fna', '../../data_temp/85028/1.fna', '../../data_temp/85028/8.fna', '../../data_temp/85028/3.fna', '../../data_temp/85028/0.fna', '../../data_temp/85028/6.fna', '../../data_temp/2897177/7.fna', '../../data_temp/2897177/2.fna', '../../data_temp/2897177/9.fna', '../../data_temp/2897177/5.fna', '../../data_temp/2897177/4.fna', '../../data_temp/2897177/1.fna', '../../data_temp/2897177/8.fna', '../../data_temp/2897177/3.fna', '../../data_temp/2897177/0.fna', '../../data_temp/2897177/6.fna', '../../data_temp/2918833/0.fna', '../../data_temp/868/7.fna', '../../data_temp/868/2.fna', '../../data_temp/868/9.fna', '../../data_temp/868/5.fna', '../../data_temp/868/4.fna', '../../data_temp/868/1.fna', '../../data_temp/868/8.fna', '../../data_temp/868/3.fna', '../../data_temp/868/0.fna', '../../data_temp/868/6.fna', '../../data_temp/1762/7.fna', '../../data_temp/1762/2.fna', '../../data_temp/1762/9.fna', '../../data_temp/1762/5.fna', '../../data_temp/1762/4.fna', '../../data_temp/1762/1.fna', '../../data_temp/1762/8.fna', '../../data_temp/1762/3.fna', '../../data_temp/1762/0.fna', '../../data_temp/1762/6.fna', '../../data_temp/3031656/2.fna', '../../data_temp/3031656/5.fna', '../../data_temp/3031656/4.fna', '../../data_temp/3031656/1.fna', '../../data_temp/3031656/3.fna', '../../data_temp/3031656/0.fna', '../../data_temp/2992717/0.fna', '../../data_temp/39/7.fna', '../../data_temp/39/2.fna', '../../data_temp/39/9.fna', '../../data_temp/39/5.fna', '../../data_temp/39/4.fna', '../../data_temp/39/1.fna', '../../data_temp/39/8.fna', '../../data_temp/39/3.fna', '../../data_temp/39/0.fna', '../../data_temp/39/6.fna', '../../data_temp/1524213/0.fna', '../../data_temp/3085097/7.fna', '../../data_temp/3085097/2.fna', '../../data_temp/3085097/5.fna', '../../data_temp/3085097/4.fna', '../../data_temp/3085097/1.fna', '../../data_temp/3085097/8.fna', '../../data_temp/3085097/3.fna', '../../data_temp/3085097/0.fna', '../../data_temp/3085097/6.fna', '../../data_temp/2992719/2.fna', '../../data_temp/2992719/5.fna', '../../data_temp/2992719/4.fna', '../../data_temp/2992719/1.fna', '../../data_temp/2992719/3.fna', '../../data_temp/2992719/0.fna', '../../data_temp/2992719/6.fna', '../../data_temp/2787155/0.fna', '../../data_temp/204434/7.fna', '../../data_temp/204434/2.fna', '../../data_temp/204434/9.fna', '../../data_temp/204434/5.fna', '../../data_temp/204434/4.fna', '../../data_temp/204434/1.fna', '../../data_temp/204434/8.fna', '../../data_temp/204434/3.fna', '../../data_temp/204434/0.fna', '../../data_temp/204434/6.fna', '../../data_temp/2762318/7.fna', '../../data_temp/2762318/2.fna', '../../data_temp/2762318/9.fna', '../../data_temp/2762318/5.fna', '../../data_temp/2762318/4.fna', '../../data_temp/2762318/1.fna', '../../data_temp/2762318/8.fna', '../../data_temp/2762318/3.fna', '../../data_temp/2762318/0.fna', '../../data_temp/2762318/6.fna', '../../data_temp/3073607/2.fna', '../../data_temp/3073607/1.fna', '../../data_temp/3073607/0.fna', '../../data_temp/3031626/0.fna', '../../data_temp/2843305/7.fna', '../../data_temp/2843305/2.fna', '../../data_temp/2843305/5.fna', '../../data_temp/2843305/4.fna', '../../data_temp/2843305/1.fna', '../../data_temp/2843305/3.fna', '../../data_temp/2843305/0.fna', '../../data_temp/2843305/6.fna', '../../data_temp/3023783/2.fna', '../../data_temp/3023783/1.fna', '../../data_temp/3023783/0.fna', '../../data_temp/815/7.fna', '../../data_temp/815/2.fna', '../../data_temp/815/9.fna', '../../data_temp/815/5.fna', '../../data_temp/815/4.fna', '../../data_temp/815/1.fna', '../../data_temp/815/8.fna', '../../data_temp/815/3.fna', '../../data_temp/815/0.fna', '../../data_temp/815/6.fna', '../../data_temp/2805590/7.fna', '../../data_temp/2805590/2.fna', '../../data_temp/2805590/9.fna', '../../data_temp/2805590/5.fna', '../../data_temp/2805590/4.fna', '../../data_temp/2805590/1.fna', '../../data_temp/2805590/8.fna', '../../data_temp/2805590/3.fna', '../../data_temp/2805590/0.fna', '../../data_temp/2805590/6.fna', '../../data_temp/2881384/2.fna', '../../data_temp/2881384/1.fna', '../../data_temp/2881384/0.fna', '../../data_temp/574976/7.fna', '../../data_temp/574976/2.fna', '../../data_temp/574976/9.fna', '../../data_temp/574976/5.fna', '../../data_temp/574976/4.fna', '../../data_temp/574976/1.fna', '../../data_temp/574976/8.fna', '../../data_temp/574976/3.fna', '../../data_temp/574976/0.fna', '../../data_temp/574976/6.fna', '../../data_temp/1189/2.fna', '../../data_temp/1189/1.fna', '../../data_temp/1189/0.fna', '../../data_temp/3029090/1.fna', '../../data_temp/3029090/0.fna', '../../data_temp/2829815/7.fna', '../../data_temp/2829815/2.fna', '../../data_temp/2829815/9.fna', '../../data_temp/2829815/5.fna', '../../data_temp/2829815/4.fna', '../../data_temp/2829815/1.fna', '../../data_temp/2829815/8.fna', '../../data_temp/2829815/3.fna', '../../data_temp/2829815/0.fna', '../../data_temp/2829815/6.fna', '../../data_temp/1890438/7.fna', '../../data_temp/1890438/2.fna', '../../data_temp/1890438/9.fna', '../../data_temp/1890438/5.fna', '../../data_temp/1890438/4.fna', '../../data_temp/1890438/1.fna', '../../data_temp/1890438/8.fna', '../../data_temp/1890438/3.fna', '../../data_temp/1890438/0.fna', '../../data_temp/1890438/6.fna', '../../data_temp/3018589/7.fna', '../../data_temp/3018589/2.fna', '../../data_temp/3018589/9.fna', '../../data_temp/3018589/5.fna', '../../data_temp/3018589/4.fna', '../../data_temp/3018589/1.fna', '../../data_temp/3018589/8.fna', '../../data_temp/3018589/3.fna', '../../data_temp/3018589/0.fna', '../../data_temp/3018589/6.fna', '../../data_temp/2864377/2.fna', '../../data_temp/2864377/1.fna', '../../data_temp/2864377/0.fna', '../../data_temp/2661849/7.fna', '../../data_temp/2661849/2.fna', '../../data_temp/2661849/9.fna', '../../data_temp/2661849/5.fna', '../../data_temp/2661849/4.fna', '../../data_temp/2661849/1.fna', '../../data_temp/2661849/8.fna', '../../data_temp/2661849/3.fna', '../../data_temp/2661849/0.fna', '../../data_temp/2661849/6.fna', '../../data_temp/2100208/2.fna', '../../data_temp/2100208/5.fna', '../../data_temp/2100208/4.fna', '../../data_temp/2100208/1.fna', '../../data_temp/2100208/3.fna', '../../data_temp/2100208/0.fna', '../../data_temp/1706373/7.fna', '../../data_temp/1706373/2.fna', '../../data_temp/1706373/9.fna', '../../data_temp/1706373/5.fna', '../../data_temp/1706373/4.fna', '../../data_temp/1706373/1.fna', '../../data_temp/1706373/8.fna', '../../data_temp/1706373/3.fna', '../../data_temp/1706373/0.fna', '../../data_temp/1706373/6.fna', '../../data_temp/194924/7.fna', '../../data_temp/194924/2.fna', '../../data_temp/194924/9.fna', '../../data_temp/194924/5.fna', '../../data_temp/194924/4.fna', '../../data_temp/194924/1.fna', '../../data_temp/194924/8.fna', '../../data_temp/194924/3.fna', '../../data_temp/194924/0.fna', '../../data_temp/194924/6.fna', '../../data_temp/3071002/7.fna', '../../data_temp/3071002/2.fna', '../../data_temp/3071002/9.fna', '../../data_temp/3071002/5.fna', '../../data_temp/3071002/4.fna', '../../data_temp/3071002/1.fna', '../../data_temp/3071002/8.fna', '../../data_temp/3071002/3.fna', '../../data_temp/3071002/0.fna', '../../data_temp/3071002/6.fna', '../../data_temp/558314/7.fna', '../../data_temp/558314/2.fna', '../../data_temp/558314/9.fna', '../../data_temp/558314/5.fna', '../../data_temp/558314/4.fna', '../../data_temp/558314/1.fna', '../../data_temp/558314/8.fna', '../../data_temp/558314/3.fna', '../../data_temp/558314/0.fna', '../../data_temp/558314/6.fna', '../../data_temp/1334117/7.fna', '../../data_temp/1334117/2.fna', '../../data_temp/1334117/9.fna', '../../data_temp/1334117/5.fna', '../../data_temp/1334117/4.fna', '../../data_temp/1334117/1.fna', '../../data_temp/1334117/8.fna', '../../data_temp/1334117/3.fna', '../../data_temp/1334117/0.fna', '../../data_temp/1334117/6.fna', '../../data_temp/597359/7.fna', '../../data_temp/597359/2.fna', '../../data_temp/597359/9.fna', '../../data_temp/597359/5.fna', '../../data_temp/597359/4.fna', '../../data_temp/597359/1.fna', '../../data_temp/597359/8.fna', '../../data_temp/597359/3.fna', '../../data_temp/597359/0.fna', '../../data_temp/597359/6.fna', '../../data_temp/85034/7.fna', '../../data_temp/85034/2.fna', '../../data_temp/85034/9.fna', '../../data_temp/85034/5.fna', '../../data_temp/85034/4.fna', '../../data_temp/85034/1.fna', '../../data_temp/85034/8.fna', '../../data_temp/85034/3.fna', '../../data_temp/85034/0.fna', '../../data_temp/85034/6.fna', '../../data_temp/191394/7.fna', '../../data_temp/191394/2.fna', '../../data_temp/191394/5.fna', '../../data_temp/191394/4.fna', '../../data_temp/191394/1.fna', '../../data_temp/191394/8.fna', '../../data_temp/191394/3.fna', '../../data_temp/191394/0.fna', '../../data_temp/191394/6.fna', '../../data_temp/3071913/0.fna', '../../data_temp/2771472/7.fna', '../../data_temp/2771472/2.fna', '../../data_temp/2771472/9.fna', '../../data_temp/2771472/5.fna', '../../data_temp/2771472/4.fna', '../../data_temp/2771472/1.fna', '../../data_temp/2771472/8.fna', '../../data_temp/2771472/3.fna', '../../data_temp/2771472/0.fna', '../../data_temp/2771472/6.fna', '../../data_temp/2847776/0.fna', '../../data_temp/332161/7.fna', '../../data_temp/332161/14.fna', '../../data_temp/332161/17.fna', '../../data_temp/332161/2.fna', '../../data_temp/332161/12.fna', '../../data_temp/332161/22.fna', '../../data_temp/332161/9.fna', '../../data_temp/332161/5.fna', '../../data_temp/332161/23.fna', '../../data_temp/332161/18.fna', '../../data_temp/332161/4.fna', '../../data_temp/332161/20.fna', '../../data_temp/332161/1.fna', '../../data_temp/332161/19.fna', '../../data_temp/332161/13.fna', '../../data_temp/332161/16.fna', '../../data_temp/332161/8.fna', '../../data_temp/332161/21.fna', '../../data_temp/332161/3.fna', '../../data_temp/332161/10.fna', '../../data_temp/332161/0.fna', '../../data_temp/332161/11.fna', '../../data_temp/332161/6.fna', '../../data_temp/332161/15.fna', '../../data_temp/908622/2.fna', '../../data_temp/908622/5.fna', '../../data_temp/908622/4.fna', '../../data_temp/908622/1.fna', '../../data_temp/908622/3.fna', '../../data_temp/908622/0.fna', '../../data_temp/316606/1.fna', '../../data_temp/316606/0.fna', '../../data_temp/1921783/2.fna', '../../data_temp/1921783/4.fna', '../../data_temp/1921783/1.fna', '../../data_temp/1921783/3.fna', '../../data_temp/1921783/0.fna', '../../data_temp/31993/7.fna', '../../data_temp/31993/2.fna', '../../data_temp/31993/9.fna', '../../data_temp/31993/5.fna', '../../data_temp/31993/4.fna', '../../data_temp/31993/1.fna', '../../data_temp/31993/8.fna', '../../data_temp/31993/3.fna', '../../data_temp/31993/0.fna', '../../data_temp/31993/6.fna', '../../data_temp/1892255/2.fna', '../../data_temp/1892255/5.fna', '../../data_temp/1892255/4.fna', '../../data_temp/1892255/1.fna', '../../data_temp/1892255/3.fna', '../../data_temp/1892255/0.fna', '../../data_temp/2937909/7.fna', '../../data_temp/2937909/2.fna', '../../data_temp/2937909/9.fna', '../../data_temp/2937909/5.fna', '../../data_temp/2937909/4.fna', '../../data_temp/2937909/1.fna', '../../data_temp/2937909/8.fna', '../../data_temp/2937909/3.fna', '../../data_temp/2937909/0.fna', '../../data_temp/2937909/6.fna', '../../data_temp/3132029/1.fna', '../../data_temp/3132029/0.fna', '../../data_temp/2900569/0.fna', '../../data_temp/1676141/7.fna', '../../data_temp/1676141/2.fna', '../../data_temp/1676141/9.fna', '../../data_temp/1676141/5.fna', '../../data_temp/1676141/4.fna', '../../data_temp/1676141/1.fna', '../../data_temp/1676141/8.fna', '../../data_temp/1676141/3.fna', '../../data_temp/1676141/0.fna', '../../data_temp/1676141/6.fna', '../../data_temp/2056687/7.fna', '../../data_temp/2056687/2.fna', '../../data_temp/2056687/9.fna', '../../data_temp/2056687/5.fna', '../../data_temp/2056687/4.fna', '../../data_temp/2056687/1.fna', '../../data_temp/2056687/8.fna', '../../data_temp/2056687/3.fna', '../../data_temp/2056687/0.fna', '../../data_temp/2056687/6.fna', '../../data_temp/1400857/7.fna', '../../data_temp/1400857/2.fna', '../../data_temp/1400857/9.fna', '../../data_temp/1400857/5.fna', '../../data_temp/1400857/4.fna', '../../data_temp/1400857/1.fna', '../../data_temp/1400857/8.fna', '../../data_temp/1400857/3.fna', '../../data_temp/1400857/0.fna', '../../data_temp/1400857/6.fna', '../../data_temp/3120161/2.fna', '../../data_temp/3120161/5.fna', '../../data_temp/3120161/4.fna', '../../data_temp/3120161/1.fna', '../../data_temp/3120161/3.fna', '../../data_temp/3120161/0.fna', '../../data_temp/3120161/6.fna', '../../data_temp/3031627/2.fna', '../../data_temp/3031627/4.fna', '../../data_temp/3031627/1.fna', '../../data_temp/3031627/3.fna', '../../data_temp/3031627/0.fna', '../../data_temp/2808963/7.fna', '../../data_temp/2808963/2.fna', '../../data_temp/2808963/9.fna', '../../data_temp/2808963/5.fna', '../../data_temp/2808963/4.fna', '../../data_temp/2808963/1.fna', '../../data_temp/2808963/8.fna', '../../data_temp/2808963/3.fna', '../../data_temp/2808963/0.fna', '../../data_temp/2808963/6.fna', '../../data_temp/471823/1.fna', '../../data_temp/471823/0.fna', '../../data_temp/2975441/7.fna', '../../data_temp/2975441/2.fna', '../../data_temp/2975441/9.fna', '../../data_temp/2975441/5.fna', '../../data_temp/2975441/4.fna', '../../data_temp/2975441/1.fna', '../../data_temp/2975441/8.fna', '../../data_temp/2975441/3.fna', '../../data_temp/2975441/0.fna', '../../data_temp/2975441/6.fna', '../../data_temp/189777/7.fna', '../../data_temp/189777/2.fna', '../../data_temp/189777/9.fna', '../../data_temp/189777/5.fna', '../../data_temp/189777/4.fna', '../../data_temp/189777/1.fna', '../../data_temp/189777/8.fna', '../../data_temp/189777/3.fna', '../../data_temp/189777/0.fna', '../../data_temp/189777/6.fna', '../../data_temp/2944142/0.fna', '../../data_temp/89373/7.fna', '../../data_temp/89373/2.fna', '../../data_temp/89373/9.fna', '../../data_temp/89373/5.fna', '../../data_temp/89373/4.fna', '../../data_temp/89373/1.fna', '../../data_temp/89373/8.fna', '../../data_temp/89373/3.fna', '../../data_temp/89373/0.fna', '../../data_temp/89373/6.fna', '../../data_temp/186822/7.fna', '../../data_temp/186822/2.fna', '../../data_temp/186822/9.fna', '../../data_temp/186822/5.fna', '../../data_temp/186822/4.fna', '../../data_temp/186822/1.fna', '../../data_temp/186822/8.fna', '../../data_temp/186822/3.fna', '../../data_temp/186822/0.fna', '../../data_temp/186822/6.fna', '../../data_temp/3046412/2.fna', '../../data_temp/3046412/1.fna', '../../data_temp/3046412/3.fna', '../../data_temp/3046412/0.fna', '../../data_temp/415002/7.fna', '../../data_temp/415002/2.fna', '../../data_temp/415002/9.fna', '../../data_temp/415002/5.fna', '../../data_temp/415002/4.fna', '../../data_temp/415002/1.fna', '../../data_temp/415002/8.fna', '../../data_temp/415002/3.fna', '../../data_temp/415002/0.fna', '../../data_temp/415002/6.fna', '../../data_temp/85031/7.fna', '../../data_temp/85031/2.fna', '../../data_temp/85031/9.fna', '../../data_temp/85031/5.fna', '../../data_temp/85031/4.fna', '../../data_temp/85031/1.fna', '../../data_temp/85031/8.fna', '../../data_temp/85031/3.fna', '../../data_temp/85031/0.fna', '../../data_temp/85031/6.fna', '../../data_temp/219687/7.fna', '../../data_temp/219687/2.fna', '../../data_temp/219687/9.fna', '../../data_temp/219687/5.fna', '../../data_temp/219687/4.fna', '../../data_temp/219687/1.fna', '../../data_temp/219687/8.fna', '../../data_temp/219687/3.fna', '../../data_temp/219687/0.fna', '../../data_temp/219687/6.fna', '../../data_temp/2896860/7.fna', '../../data_temp/2896860/2.fna', '../../data_temp/2896860/9.fna', '../../data_temp/2896860/5.fna', '../../data_temp/2896860/4.fna', '../../data_temp/2896860/1.fna', '../../data_temp/2896860/8.fna', '../../data_temp/2896860/3.fna', '../../data_temp/2896860/0.fna', '../../data_temp/2896860/6.fna', '../../data_temp/2146/7.fna', '../../data_temp/2146/2.fna', '../../data_temp/2146/9.fna', '../../data_temp/2146/5.fna', '../../data_temp/2146/4.fna', '../../data_temp/2146/1.fna', '../../data_temp/2146/8.fna', '../../data_temp/2146/3.fna', '../../data_temp/2146/0.fna', '../../data_temp/2146/6.fna', '../../data_temp/1853232/7.fna', '../../data_temp/1853232/2.fna', '../../data_temp/1853232/9.fna', '../../data_temp/1853232/5.fna', '../../data_temp/1853232/4.fna', '../../data_temp/1853232/1.fna', '../../data_temp/1853232/8.fna', '../../data_temp/1853232/3.fna', '../../data_temp/1853232/0.fna', '../../data_temp/1853232/6.fna', '../../data_temp/2807496/0.fna', '../../data_temp/32033/7.fna', '../../data_temp/32033/2.fna', '../../data_temp/32033/9.fna', '../../data_temp/32033/5.fna', '../../data_temp/32033/4.fna', '../../data_temp/32033/1.fna', '../../data_temp/32033/8.fna', '../../data_temp/32033/3.fna', '../../data_temp/32033/0.fna', '../../data_temp/32033/6.fna', '../../data_temp/2856854/0.fna', '../../data_temp/2811504/7.fna', '../../data_temp/2811504/2.fna', '../../data_temp/2811504/9.fna', '../../data_temp/2811504/5.fna', '../../data_temp/2811504/4.fna', '../../data_temp/2811504/1.fna', '../../data_temp/2811504/8.fna', '../../data_temp/2811504/3.fna', '../../data_temp/2811504/0.fna', '../../data_temp/2811504/6.fna', '../../data_temp/1798913/7.fna', '../../data_temp/1798913/2.fna', '../../data_temp/1798913/9.fna', '../../data_temp/1798913/5.fna', '../../data_temp/1798913/4.fna', '../../data_temp/1798913/1.fna', '../../data_temp/1798913/8.fna', '../../data_temp/1798913/3.fna', '../../data_temp/1798913/0.fna', '../../data_temp/1798913/6.fna', '../../data_temp/2810281/7.fna', '../../data_temp/2810281/2.fna', '../../data_temp/2810281/9.fna', '../../data_temp/2810281/5.fna', '../../data_temp/2810281/4.fna', '../../data_temp/2810281/1.fna', '../../data_temp/2810281/8.fna', '../../data_temp/2810281/3.fna', '../../data_temp/2810281/0.fna', '../../data_temp/2810281/6.fna', '../../data_temp/90964/7.fna', '../../data_temp/90964/2.fna', '../../data_temp/90964/9.fna', '../../data_temp/90964/5.fna', '../../data_temp/90964/4.fna', '../../data_temp/90964/1.fna', '../../data_temp/90964/8.fna', '../../data_temp/90964/3.fna', '../../data_temp/90964/0.fna', '../../data_temp/90964/6.fna', '../../data_temp/1892249/2.fna', '../../data_temp/1892249/1.fna', '../../data_temp/1892249/0.fna', '../../data_temp/1860138/7.fna', '../../data_temp/1860138/2.fna', '../../data_temp/1860138/5.fna', '../../data_temp/1860138/4.fna', '../../data_temp/1860138/1.fna', '../../data_temp/1860138/3.fna', '../../data_temp/1860138/0.fna', '../../data_temp/1860138/6.fna', '../../data_temp/510685/7.fna', '../../data_temp/510685/2.fna', '../../data_temp/510685/9.fna', '../../data_temp/510685/5.fna', '../../data_temp/510685/4.fna', '../../data_temp/510685/1.fna', '../../data_temp/510685/8.fna', '../../data_temp/510685/3.fna', '../../data_temp/510685/0.fna', '../../data_temp/510685/6.fna', '../../data_temp/3135529/2.fna', '../../data_temp/3135529/4.fna', '../../data_temp/3135529/1.fna', '../../data_temp/3135529/3.fna', '../../data_temp/3135529/0.fna', '../../data_temp/1154585/2.fna', '../../data_temp/1154585/1.fna', '../../data_temp/1154585/0.fna', '../../data_temp/224380/7.fna', '../../data_temp/224380/2.fna', '../../data_temp/224380/5.fna', '../../data_temp/224380/4.fna', '../../data_temp/224380/1.fna', '../../data_temp/224380/3.fna', '../../data_temp/224380/0.fna', '../../data_temp/224380/6.fna', '../../data_temp/1853230/7.fna', '../../data_temp/1853230/2.fna', '../../data_temp/1853230/9.fna', '../../data_temp/1853230/5.fna', '../../data_temp/1853230/4.fna', '../../data_temp/1853230/1.fna', '../../data_temp/1853230/8.fna', '../../data_temp/1853230/3.fna', '../../data_temp/1853230/0.fna', '../../data_temp/1853230/6.fna', '../../data_temp/3031145/7.fna', '../../data_temp/3031145/14.fna', '../../data_temp/3031145/17.fna', '../../data_temp/3031145/2.fna', '../../data_temp/3031145/12.fna', '../../data_temp/3031145/9.fna', '../../data_temp/3031145/5.fna', '../../data_temp/3031145/18.fna', '../../data_temp/3031145/4.fna', '../../data_temp/3031145/20.fna', '../../data_temp/3031145/1.fna', '../../data_temp/3031145/19.fna', '../../data_temp/3031145/13.fna', '../../data_temp/3031145/16.fna', '../../data_temp/3031145/8.fna', '../../data_temp/3031145/21.fna', '../../data_temp/3031145/3.fna', '../../data_temp/3031145/10.fna', '../../data_temp/3031145/0.fna', '../../data_temp/3031145/11.fna', '../../data_temp/3031145/6.fna', '../../data_temp/3031145/15.fna', '../../data_temp/2858885/0.fna', '../../data_temp/3079643/1.fna', '../../data_temp/3079643/0.fna', '../../data_temp/543313/7.fna', '../../data_temp/543313/2.fna', '../../data_temp/543313/5.fna', '../../data_temp/543313/4.fna', '../../data_temp/543313/1.fna', '../../data_temp/543313/3.fna', '../../data_temp/543313/0.fna', '../../data_temp/543313/6.fna', '../../data_temp/89377/7.fna', '../../data_temp/89377/2.fna', '../../data_temp/89377/9.fna', '../../data_temp/89377/5.fna', '../../data_temp/89377/4.fna', '../../data_temp/89377/1.fna', '../../data_temp/89377/8.fna', '../../data_temp/89377/3.fna', '../../data_temp/89377/0.fna', '../../data_temp/89377/6.fna', '../../data_temp/3120698/2.fna', '../../data_temp/3120698/1.fna', '../../data_temp/3120698/3.fna', '../../data_temp/3120698/0.fna', '../../data_temp/1892265/0.fna', '../../data_temp/213116/7.fna', '../../data_temp/213116/2.fna', '../../data_temp/213116/9.fna', '../../data_temp/213116/5.fna', '../../data_temp/213116/4.fna', '../../data_temp/213116/1.fna', '../../data_temp/213116/8.fna', '../../data_temp/213116/3.fna', '../../data_temp/213116/0.fna', '../../data_temp/213116/6.fna', '../../data_temp/2712270/0.fna', '../../data_temp/1934946/2.fna', '../../data_temp/1934946/1.fna', '../../data_temp/1934946/3.fna', '../../data_temp/1934946/0.fna', '../../data_temp/2931039/7.fna', '../../data_temp/2931039/2.fna', '../../data_temp/2931039/9.fna', '../../data_temp/2931039/5.fna', '../../data_temp/2931039/4.fna', '../../data_temp/2931039/1.fna', '../../data_temp/2931039/8.fna', '../../data_temp/2931039/3.fna', '../../data_temp/2931039/0.fna', '../../data_temp/2931039/6.fna', '../../data_temp/84566/7.fna', '../../data_temp/84566/2.fna', '../../data_temp/84566/9.fna', '../../data_temp/84566/5.fna', '../../data_temp/84566/4.fna', '../../data_temp/84566/1.fna', '../../data_temp/84566/8.fna', '../../data_temp/84566/3.fna', '../../data_temp/84566/0.fna', '../../data_temp/84566/6.fna', '../../data_temp/543314/7.fna', '../../data_temp/543314/2.fna', '../../data_temp/543314/9.fna', '../../data_temp/543314/5.fna', '../../data_temp/543314/4.fna', '../../data_temp/543314/1.fna', '../../data_temp/543314/8.fna', '../../data_temp/543314/3.fna', '../../data_temp/543314/0.fna', '../../data_temp/543314/6.fna', '../../data_temp/72294/7.fna', '../../data_temp/72294/2.fna', '../../data_temp/72294/9.fna', '../../data_temp/72294/5.fna', '../../data_temp/72294/4.fna', '../../data_temp/72294/1.fna', '../../data_temp/72294/8.fna', '../../data_temp/72294/3.fna', '../../data_temp/72294/0.fna', '../../data_temp/72294/6.fna', '../../data_temp/49/7.fna', '../../data_temp/49/2.fna', '../../data_temp/49/9.fna', '../../data_temp/49/5.fna', '../../data_temp/49/4.fna', '../../data_temp/49/1.fna', '../../data_temp/49/8.fna', '../../data_temp/49/3.fna', '../../data_temp/49/0.fna', '../../data_temp/49/6.fna', '../../data_temp/2945021/2.fna', '../../data_temp/2945021/4.fna', '../../data_temp/2945021/1.fna', '../../data_temp/2945021/3.fna', '../../data_temp/2945021/0.fna', '../../data_temp/2483367/2.fna', '../../data_temp/2483367/1.fna', '../../data_temp/2483367/3.fna', '../../data_temp/2483367/0.fna', '../../data_temp/543372/1.fna', '../../data_temp/543372/0.fna', '../../data_temp/225059/0.fna', '../../data_temp/255527/7.fna', '../../data_temp/255527/2.fna', '../../data_temp/255527/9.fna', '../../data_temp/255527/5.fna', '../../data_temp/255527/4.fna', '../../data_temp/255527/1.fna', '../../data_temp/255527/8.fna', '../../data_temp/255527/3.fna', '../../data_temp/255527/0.fna', '../../data_temp/255527/6.fna', '../../data_temp/191412/7.fna', '../../data_temp/191412/2.fna', '../../data_temp/191412/9.fna', '../../data_temp/191412/5.fna', '../../data_temp/191412/4.fna', '../../data_temp/191412/1.fna', '../../data_temp/191412/8.fna', '../../data_temp/191412/3.fna', '../../data_temp/191412/0.fna', '../../data_temp/191412/6.fna', '../../data_temp/1903335/0.fna', '../../data_temp/539000/7.fna', '../../data_temp/539000/14.fna', '../../data_temp/539000/17.fna', '../../data_temp/539000/2.fna', '../../data_temp/539000/12.fna', '../../data_temp/539000/25.fna', '../../data_temp/539000/22.fna', '../../data_temp/539000/24.fna', '../../data_temp/539000/9.fna', '../../data_temp/539000/5.fna', '../../data_temp/539000/23.fna', '../../data_temp/539000/18.fna', '../../data_temp/539000/4.fna', '../../data_temp/539000/20.fna', '../../data_temp/539000/26.fna', '../../data_temp/539000/1.fna', '../../data_temp/539000/27.fna', '../../data_temp/539000/19.fna', '../../data_temp/539000/13.fna', '../../data_temp/539000/16.fna', '../../data_temp/539000/8.fna', '../../data_temp/539000/21.fna', '../../data_temp/539000/3.fna', '../../data_temp/539000/10.fna', '../../data_temp/539000/28.fna', '../../data_temp/539000/0.fna', '../../data_temp/539000/11.fna', '../../data_temp/539000/6.fna', '../../data_temp/539000/15.fna', '../../data_temp/2762306/7.fna', '../../data_temp/2762306/2.fna', '../../data_temp/2762306/9.fna', '../../data_temp/2762306/5.fna', '../../data_temp/2762306/4.fna', '../../data_temp/2762306/1.fna', '../../data_temp/2762306/8.fna', '../../data_temp/2762306/3.fna', '../../data_temp/2762306/0.fna', '../../data_temp/2762306/6.fna', '../../data_temp/2603433/0.fna', '../../data_temp/2898533/2.fna', '../../data_temp/2898533/5.fna', '../../data_temp/2898533/4.fna', '../../data_temp/2898533/1.fna', '../../data_temp/2898533/3.fna', '../../data_temp/2898533/0.fna', '../../data_temp/2898533/6.fna', '../../data_temp/2005519/7.fna', '../../data_temp/2005519/2.fna', '../../data_temp/2005519/9.fna', '../../data_temp/2005519/5.fna', '../../data_temp/2005519/4.fna', '../../data_temp/2005519/1.fna', '../../data_temp/2005519/8.fna', '../../data_temp/2005519/3.fna', '../../data_temp/2005519/0.fna', '../../data_temp/2005519/6.fna', '../../data_temp/972/7.fna', '../../data_temp/972/2.fna', '../../data_temp/972/9.fna', '../../data_temp/972/5.fna', '../../data_temp/972/4.fna', '../../data_temp/972/1.fna', '../../data_temp/972/8.fna', '../../data_temp/972/3.fna', '../../data_temp/972/0.fna', '../../data_temp/972/6.fna', '../../data_temp/2831106/7.fna', '../../data_temp/2831106/2.fna', '../../data_temp/2831106/9.fna', '../../data_temp/2831106/5.fna', '../../data_temp/2831106/4.fna', '../../data_temp/2831106/1.fna', '../../data_temp/2831106/8.fna', '../../data_temp/2831106/3.fna', '../../data_temp/2831106/0.fna', '../../data_temp/2831106/6.fna', '../../data_temp/2212439/0.fna', '../../data_temp/135617/7.fna', '../../data_temp/135617/2.fna', '../../data_temp/135617/9.fna', '../../data_temp/135617/5.fna', '../../data_temp/135617/4.fna', '../../data_temp/135617/1.fna', '../../data_temp/135617/8.fna', '../../data_temp/135617/3.fna', '../../data_temp/135617/0.fna', '../../data_temp/135617/6.fna', '../../data_temp/69657/7.fna', '../../data_temp/69657/2.fna', '../../data_temp/69657/9.fna', '../../data_temp/69657/5.fna', '../../data_temp/69657/4.fna', '../../data_temp/69657/1.fna', '../../data_temp/69657/8.fna', '../../data_temp/69657/3.fna', '../../data_temp/69657/0.fna', '../../data_temp/69657/6.fna', '../../data_temp/255475/7.fna', '../../data_temp/255475/2.fna', '../../data_temp/255475/9.fna', '../../data_temp/255475/5.fna', '../../data_temp/255475/4.fna', '../../data_temp/255475/1.fna', '../../data_temp/255475/8.fna', '../../data_temp/255475/3.fna', '../../data_temp/255475/0.fna', '../../data_temp/255475/6.fna', '../../data_temp/641/7.fna', '../../data_temp/641/2.fna', '../../data_temp/641/9.fna', '../../data_temp/641/5.fna', '../../data_temp/641/4.fna', '../../data_temp/641/1.fna', '../../data_temp/641/8.fna', '../../data_temp/641/3.fna', '../../data_temp/641/0.fna', '../../data_temp/641/6.fna', '../../data_temp/795749/7.fna', '../../data_temp/795749/2.fna', '../../data_temp/795749/9.fna', '../../data_temp/795749/5.fna', '../../data_temp/795749/4.fna', '../../data_temp/795749/1.fna', '../../data_temp/795749/8.fna', '../../data_temp/795749/3.fna', '../../data_temp/795749/0.fna', '../../data_temp/795749/6.fna', '../../data_temp/2932623/7.fna', '../../data_temp/2932623/2.fna', '../../data_temp/2932623/9.fna', '../../data_temp/2932623/5.fna', '../../data_temp/2932623/4.fna', '../../data_temp/2932623/1.fna', '../../data_temp/2932623/8.fna', '../../data_temp/2932623/3.fna', '../../data_temp/2932623/0.fna', '../../data_temp/2932623/6.fna', '../../data_temp/1937960/7.fna', '../../data_temp/1937960/2.fna', '../../data_temp/1937960/9.fna', '../../data_temp/1937960/5.fna', '../../data_temp/1937960/4.fna', '../../data_temp/1937960/1.fna', '../../data_temp/1937960/8.fna', '../../data_temp/1937960/3.fna', '../../data_temp/1937960/0.fna', '../../data_temp/1937960/6.fna', '../../data_temp/1890436/7.fna', '../../data_temp/1890436/2.fna', '../../data_temp/1890436/9.fna', '../../data_temp/1890436/5.fna', '../../data_temp/1890436/4.fna', '../../data_temp/1890436/1.fna', '../../data_temp/1890436/8.fna', '../../data_temp/1890436/3.fna', '../../data_temp/1890436/0.fna', '../../data_temp/1890436/6.fna', '../../data_temp/3084969/2.fna', '../../data_temp/3084969/5.fna', '../../data_temp/3084969/4.fna', '../../data_temp/3084969/1.fna', '../../data_temp/3084969/3.fna', '../../data_temp/3084969/0.fna', '../../data_temp/3031464/2.fna', '../../data_temp/3031464/5.fna', '../../data_temp/3031464/4.fna', '../../data_temp/3031464/1.fna', '../../data_temp/3031464/3.fna', '../../data_temp/3031464/0.fna', '../../data_temp/2867375/7.fna', '../../data_temp/2867375/2.fna', '../../data_temp/2867375/9.fna', '../../data_temp/2867375/5.fna', '../../data_temp/2867375/4.fna', '../../data_temp/2867375/1.fna', '../../data_temp/2867375/8.fna', '../../data_temp/2867375/3.fna', '../../data_temp/2867375/0.fna', '../../data_temp/2867375/6.fna', '../../data_temp/1392996/2.fna', '../../data_temp/1392996/1.fna', '../../data_temp/1392996/0.fna', '../../data_temp/1737406/7.fna', '../../data_temp/1737406/2.fna', '../../data_temp/1737406/9.fna', '../../data_temp/1737406/5.fna', '../../data_temp/1737406/4.fna', '../../data_temp/1737406/1.fna', '../../data_temp/1737406/8.fna', '../../data_temp/1737406/3.fna', '../../data_temp/1737406/0.fna', '../../data_temp/1737406/6.fna', '../../data_temp/1812917/0.fna', '../../data_temp/2831111/7.fna', '../../data_temp/2831111/2.fna', '../../data_temp/2831111/9.fna', '../../data_temp/2831111/5.fna', '../../data_temp/2831111/4.fna', '../../data_temp/2831111/1.fna', '../../data_temp/2831111/8.fna', '../../data_temp/2831111/3.fna', '../../data_temp/2831111/0.fna', '../../data_temp/2831111/6.fna', '../../data_temp/3023805/0.fna', '../../data_temp/2498708/2.fna', '../../data_temp/2498708/5.fna', '../../data_temp/2498708/4.fna', '../../data_temp/2498708/1.fna', '../../data_temp/2498708/3.fna', '../../data_temp/2498708/0.fna', '../../data_temp/2498708/6.fna', '../../data_temp/3031147/2.fna', '../../data_temp/3031147/4.fna', '../../data_temp/3031147/1.fna', '../../data_temp/3031147/3.fna', '../../data_temp/3031147/0.fna', '../../data_temp/2831104/1.fna', '../../data_temp/2831104/0.fna', '../../data_temp/2762301/7.fna', '../../data_temp/2762301/2.fna', '../../data_temp/2762301/5.fna', '../../data_temp/2762301/4.fna', '../../data_temp/2762301/1.fna', '../../data_temp/2762301/3.fna', '../../data_temp/2762301/0.fna', '../../data_temp/2762301/6.fna', '../../data_temp/3084970/1.fna', '../../data_temp/3084970/0.fna', '../../data_temp/2772535/2.fna', '../../data_temp/2772535/1.fna', '../../data_temp/2772535/0.fna', '../../data_temp/1300/7.fna', '../../data_temp/1300/2.fna', '../../data_temp/1300/9.fna', '../../data_temp/1300/5.fna', '../../data_temp/1300/4.fna', '../../data_temp/1300/1.fna', '../../data_temp/1300/8.fna', '../../data_temp/1300/3.fna', '../../data_temp/1300/0.fna', '../../data_temp/1300/6.fna', '../../data_temp/1570339/7.fna', '../../data_temp/1570339/2.fna', '../../data_temp/1570339/9.fna', '../../data_temp/1570339/5.fna', '../../data_temp/1570339/4.fna', '../../data_temp/1570339/1.fna', '../../data_temp/1570339/8.fna', '../../data_temp/1570339/3.fna', '../../data_temp/1570339/0.fna', '../../data_temp/1570339/6.fna', '../../data_temp/3024158/2.fna', '../../data_temp/3024158/1.fna', '../../data_temp/3024158/0.fna', '../../data_temp/2650922/2.fna', '../../data_temp/2650922/5.fna', '../../data_temp/2650922/4.fna', '../../data_temp/2650922/1.fna', '../../data_temp/2650922/3.fna', '../../data_temp/2650922/0.fna', '../../data_temp/292628/7.fna', '../../data_temp/292628/2.fna', '../../data_temp/292628/9.fna', '../../data_temp/292628/5.fna', '../../data_temp/292628/4.fna', '../../data_temp/292628/1.fna', '../../data_temp/292628/8.fna', '../../data_temp/292628/3.fna', '../../data_temp/292628/0.fna', '../../data_temp/292628/6.fna', '../../data_temp/224463/7.fna', '../../data_temp/224463/2.fna', '../../data_temp/224463/9.fna', '../../data_temp/224463/5.fna', '../../data_temp/224463/4.fna', '../../data_temp/224463/1.fna', '../../data_temp/224463/8.fna', '../../data_temp/224463/3.fna', '../../data_temp/224463/0.fna', '../../data_temp/224463/6.fna', '../../data_temp/1328881/7.fna', '../../data_temp/1328881/2.fna', '../../data_temp/1328881/9.fna', '../../data_temp/1328881/5.fna', '../../data_temp/1328881/4.fna', '../../data_temp/1328881/1.fna', '../../data_temp/1328881/8.fna', '../../data_temp/1328881/3.fna', '../../data_temp/1328881/0.fna', '../../data_temp/1328881/6.fna', '../../data_temp/3076164/1.fna', '../../data_temp/3076164/0.fna', '../../data_temp/2881378/7.fna', '../../data_temp/2881378/2.fna', '../../data_temp/2881378/9.fna', '../../data_temp/2881378/5.fna', '../../data_temp/2881378/4.fna', '../../data_temp/2881378/1.fna', '../../data_temp/2881378/8.fna', '../../data_temp/2881378/3.fna', '../../data_temp/2881378/0.fna', '../../data_temp/2881378/6.fna', '../../data_temp/189779/7.fna', '../../data_temp/189779/2.fna', '../../data_temp/189779/9.fna', '../../data_temp/189779/5.fna', '../../data_temp/189779/4.fna', '../../data_temp/189779/1.fna', '../../data_temp/189779/8.fna', '../../data_temp/189779/3.fna', '../../data_temp/189779/0.fna', '../../data_temp/189779/6.fna', '../../data_temp/1920240/7.fna', '../../data_temp/1920240/2.fna', '../../data_temp/1920240/9.fna', '../../data_temp/1920240/5.fna', '../../data_temp/1920240/4.fna', '../../data_temp/1920240/1.fna', '../../data_temp/1920240/8.fna', '../../data_temp/1920240/3.fna', '../../data_temp/1920240/0.fna', '../../data_temp/1920240/6.fna', '../../data_temp/2900672/1.fna', '../../data_temp/2900672/0.fna', '../../data_temp/255474/7.fna', '../../data_temp/255474/2.fna', '../../data_temp/255474/9.fna', '../../data_temp/255474/5.fna', '../../data_temp/255474/4.fna', '../../data_temp/255474/1.fna', '../../data_temp/255474/8.fna', '../../data_temp/255474/3.fna', '../../data_temp/255474/0.fna', '../../data_temp/255474/6.fna', '../../data_temp/31984/7.fna', '../../data_temp/31984/2.fna', '../../data_temp/31984/9.fna', '../../data_temp/31984/5.fna', '../../data_temp/31984/4.fna', '../../data_temp/31984/1.fna', '../../data_temp/31984/8.fna', '../../data_temp/31984/3.fna', '../../data_temp/31984/0.fna', '../../data_temp/31984/6.fna', '../../data_temp/3033994/0.fna', '../../data_temp/1962854/7.fna', '../../data_temp/1962854/2.fna', '../../data_temp/1962854/9.fna', '../../data_temp/1962854/5.fna', '../../data_temp/1962854/4.fna', '../../data_temp/1962854/1.fna', '../../data_temp/1962854/8.fna', '../../data_temp/1962854/3.fna', '../../data_temp/1962854/0.fna', '../../data_temp/1962854/6.fna', '../../data_temp/2029878/0.fna', '../../data_temp/1769717/7.fna', '../../data_temp/1769717/2.fna', '../../data_temp/1769717/9.fna', '../../data_temp/1769717/5.fna', '../../data_temp/1769717/4.fna', '../../data_temp/1769717/1.fna', '../../data_temp/1769717/8.fna', '../../data_temp/1769717/3.fna', '../../data_temp/1769717/10.fna', '../../data_temp/1769717/0.fna', '../../data_temp/1769717/6.fna', '../../data_temp/1890431/2.fna', '../../data_temp/1890431/1.fna', '../../data_temp/1890431/0.fna', '../../data_temp/538998/0.fna', '../../data_temp/85020/7.fna', '../../data_temp/85020/2.fna', '../../data_temp/85020/9.fna', '../../data_temp/85020/5.fna', '../../data_temp/85020/4.fna', '../../data_temp/85020/1.fna', '../../data_temp/85020/8.fna', '../../data_temp/85020/3.fna', '../../data_temp/85020/0.fna', '../../data_temp/85020/6.fna', '../../data_temp/1042314/0.fna', '../../data_temp/1643826/7.fna', '../../data_temp/1643826/2.fna', '../../data_temp/1643826/9.fna', '../../data_temp/1643826/5.fna', '../../data_temp/1643826/4.fna', '../../data_temp/1643826/1.fna', '../../data_temp/1643826/8.fna', '../../data_temp/1643826/3.fna', '../../data_temp/1643826/0.fna', '../../data_temp/1643826/6.fna', '../../data_temp/1202464/32.fna', '../../data_temp/1202464/43.fna', '../../data_temp/1202464/7.fna', '../../data_temp/1202464/58.fna', '../../data_temp/1202464/14.fna', '../../data_temp/1202464/41.fna', '../../data_temp/1202464/17.fna', '../../data_temp/1202464/56.fna', '../../data_temp/1202464/50.fna', '../../data_temp/1202464/2.fna', '../../data_temp/1202464/46.fna', '../../data_temp/1202464/52.fna', '../../data_temp/1202464/45.fna', '../../data_temp/1202464/44.fna', '../../data_temp/1202464/12.fna', '../../data_temp/1202464/25.fna', '../../data_temp/1202464/39.fna', '../../data_temp/1202464/22.fna', '../../data_temp/1202464/48.fna', '../../data_temp/1202464/24.fna', '../../data_temp/1202464/9.fna', '../../data_temp/1202464/5.fna', '../../data_temp/1202464/54.fna', '../../data_temp/1202464/23.fna', '../../data_temp/1202464/49.fna', '../../data_temp/1202464/40.fna', '../../data_temp/1202464/18.fna', '../../data_temp/1202464/4.fna', '../../data_temp/1202464/20.fna', '../../data_temp/1202464/33.fna', '../../data_temp/1202464/26.fna', '../../data_temp/1202464/1.fna', '../../data_temp/1202464/47.fna', '../../data_temp/1202464/51.fna', '../../data_temp/1202464/37.fna', '../../data_temp/1202464/53.fna', '../../data_temp/1202464/42.fna', '../../data_temp/1202464/57.fna', '../../data_temp/1202464/27.fna', '../../data_temp/1202464/19.fna', '../../data_temp/1202464/13.fna', '../../data_temp/1202464/16.fna', '../../data_temp/1202464/29.fna', '../../data_temp/1202464/8.fna', '../../data_temp/1202464/21.fna', '../../data_temp/1202464/3.fna', '../../data_temp/1202464/10.fna', '../../data_temp/1202464/30.fna', '../../data_temp/1202464/38.fna', '../../data_temp/1202464/35.fna', '../../data_temp/1202464/31.fna', '../../data_temp/1202464/28.fna', '../../data_temp/1202464/0.fna', '../../data_temp/1202464/11.fna', '../../data_temp/1202464/34.fna', '../../data_temp/1202464/6.fna', '../../data_temp/1202464/15.fna', '../../data_temp/1202464/55.fna', '../../data_temp/1202464/36.fna', '../../data_temp/1240483/7.fna', '../../data_temp/1240483/2.fna', '../../data_temp/1240483/9.fna', '../../data_temp/1240483/5.fna', '../../data_temp/1240483/4.fna', '../../data_temp/1240483/1.fna', '../../data_temp/1240483/8.fna', '../../data_temp/1240483/3.fna', '../../data_temp/1240483/0.fna', '../../data_temp/1240483/6.fna', '../../data_temp/1892251/7.fna', '../../data_temp/1892251/2.fna', '../../data_temp/1892251/9.fna', '../../data_temp/1892251/5.fna', '../../data_temp/1892251/4.fna', '../../data_temp/1892251/1.fna', '../../data_temp/1892251/8.fna', '../../data_temp/1892251/3.fna', '../../data_temp/1892251/0.fna', '../../data_temp/1892251/6.fna', '../../data_temp/2137278/2.fna', '../../data_temp/2137278/5.fna', '../../data_temp/2137278/4.fna', '../../data_temp/2137278/1.fna', '../../data_temp/2137278/3.fna', '../../data_temp/2137278/0.fna', '../../data_temp/2137278/6.fna', '../../data_temp/255528/7.fna', '../../data_temp/255528/2.fna', '../../data_temp/255528/9.fna', '../../data_temp/255528/5.fna', '../../data_temp/255528/4.fna', '../../data_temp/255528/1.fna', '../../data_temp/255528/8.fna', '../../data_temp/255528/3.fna', '../../data_temp/255528/0.fna', '../../data_temp/255528/6.fna', '../../data_temp/203492/7.fna', '../../data_temp/203492/2.fna', '../../data_temp/203492/9.fna', '../../data_temp/203492/5.fna', '../../data_temp/203492/4.fna', '../../data_temp/203492/1.fna', '../../data_temp/203492/8.fna', '../../data_temp/203492/3.fna', '../../data_temp/203492/0.fna', '../../data_temp/203492/6.fna', '../../data_temp/44746/7.fna', '../../data_temp/44746/2.fna', '../../data_temp/44746/9.fna', '../../data_temp/44746/5.fna', '../../data_temp/44746/4.fna', '../../data_temp/44746/1.fna', '../../data_temp/44746/8.fna', '../../data_temp/44746/3.fna', '../../data_temp/44746/0.fna', '../../data_temp/44746/6.fna', '../../data_temp/641876/7.fna', '../../data_temp/641876/2.fna', '../../data_temp/641876/9.fna', '../../data_temp/641876/5.fna', '../../data_temp/641876/4.fna', '../../data_temp/641876/1.fna', '../../data_temp/641876/8.fna', '../../data_temp/641876/3.fna', '../../data_temp/641876/0.fna', '../../data_temp/641876/6.fna', '../../data_temp/1843491/7.fna', '../../data_temp/1843491/2.fna', '../../data_temp/1843491/9.fna', '../../data_temp/1843491/5.fna', '../../data_temp/1843491/4.fna', '../../data_temp/1843491/1.fna', '../../data_temp/1843491/8.fna', '../../data_temp/1843491/3.fna', '../../data_temp/1843491/0.fna', '../../data_temp/1843491/6.fna', '../../data_temp/2171994/1.fna', '../../data_temp/2171994/0.fna', '../../data_temp/1914233/7.fna', '../../data_temp/1914233/2.fna', '../../data_temp/1914233/9.fna', '../../data_temp/1914233/5.fna', '../../data_temp/1914233/4.fna', '../../data_temp/1914233/1.fna', '../../data_temp/1914233/8.fna', '../../data_temp/1914233/3.fna', '../../data_temp/1914233/0.fna', '../../data_temp/1914233/6.fna', '../../data_temp/655351/7.fna', '../../data_temp/655351/2.fna', '../../data_temp/655351/5.fna', '../../data_temp/655351/4.fna', '../../data_temp/655351/1.fna', '../../data_temp/655351/3.fna', '../../data_temp/655351/0.fna', '../../data_temp/655351/6.fna', '../../data_temp/1962910/7.fna', '../../data_temp/1962910/2.fna', '../../data_temp/1962910/9.fna', '../../data_temp/1962910/5.fna', '../../data_temp/1962910/4.fna', '../../data_temp/1962910/1.fna', '../../data_temp/1962910/8.fna', '../../data_temp/1962910/3.fna', '../../data_temp/1962910/0.fna', '../../data_temp/1962910/6.fna', '../../data_temp/1389452/7.fna', '../../data_temp/1389452/2.fna', '../../data_temp/1389452/9.fna', '../../data_temp/1389452/5.fna', '../../data_temp/1389452/4.fna', '../../data_temp/1389452/1.fna', '../../data_temp/1389452/8.fna', '../../data_temp/1389452/3.fna', '../../data_temp/1389452/0.fna', '../../data_temp/1389452/6.fna', '../../data_temp/335929/7.fna', '../../data_temp/335929/2.fna', '../../data_temp/335929/9.fna', '../../data_temp/335929/5.fna', '../../data_temp/335929/4.fna', '../../data_temp/335929/1.fna', '../../data_temp/335929/8.fna', '../../data_temp/335929/3.fna', '../../data_temp/335929/0.fna', '../../data_temp/335929/6.fna', '../../data_temp/2844866/64.fna', '../../data_temp/2844866/32.fna', '../../data_temp/2844866/43.fna', '../../data_temp/2844866/7.fna', '../../data_temp/2844866/58.fna', '../../data_temp/2844866/14.fna', '../../data_temp/2844866/41.fna', '../../data_temp/2844866/67.fna', '../../data_temp/2844866/17.fna', '../../data_temp/2844866/56.fna', '../../data_temp/2844866/50.fna', '../../data_temp/2844866/2.fna', '../../data_temp/2844866/46.fna', '../../data_temp/2844866/52.fna', '../../data_temp/2844866/65.fna', '../../data_temp/2844866/45.fna', '../../data_temp/2844866/44.fna', '../../data_temp/2844866/12.fna', '../../data_temp/2844866/25.fna', '../../data_temp/2844866/60.fna', '../../data_temp/2844866/39.fna', '../../data_temp/2844866/22.fna', '../../data_temp/2844866/48.fna', '../../data_temp/2844866/24.fna', '../../data_temp/2844866/9.fna', '../../data_temp/2844866/5.fna', '../../data_temp/2844866/54.fna', '../../data_temp/2844866/23.fna', '../../data_temp/2844866/49.fna', '../../data_temp/2844866/40.fna', '../../data_temp/2844866/18.fna', '../../data_temp/2844866/4.fna', '../../data_temp/2844866/20.fna', '../../data_temp/2844866/63.fna', '../../data_temp/2844866/33.fna', '../../data_temp/2844866/26.fna', '../../data_temp/2844866/1.fna', '../../data_temp/2844866/47.fna', '../../data_temp/2844866/51.fna', '../../data_temp/2844866/37.fna', '../../data_temp/2844866/53.fna', '../../data_temp/2844866/42.fna', '../../data_temp/2844866/57.fna', '../../data_temp/2844866/27.fna', '../../data_temp/2844866/19.fna', '../../data_temp/2844866/66.fna', '../../data_temp/2844866/13.fna', '../../data_temp/2844866/16.fna', '../../data_temp/2844866/29.fna', '../../data_temp/2844866/8.fna', '../../data_temp/2844866/21.fna', '../../data_temp/2844866/3.fna', '../../data_temp/2844866/10.fna', '../../data_temp/2844866/30.fna', '../../data_temp/2844866/38.fna', '../../data_temp/2844866/35.fna', '../../data_temp/2844866/31.fna', '../../data_temp/2844866/28.fna', '../../data_temp/2844866/59.fna', '../../data_temp/2844866/62.fna', '../../data_temp/2844866/0.fna', '../../data_temp/2844866/11.fna', '../../data_temp/2844866/34.fna', '../../data_temp/2844866/6.fna', '../../data_temp/2844866/68.fna', '../../data_temp/2844866/15.fna', '../../data_temp/2844866/55.fna', '../../data_temp/2844866/61.fna', '../../data_temp/2844866/36.fna', '../../data_temp/2814901/0.fna', '../../data_temp/1738654/7.fna', '../../data_temp/1738654/2.fna', '../../data_temp/1738654/9.fna', '../../data_temp/1738654/5.fna', '../../data_temp/1738654/4.fna', '../../data_temp/1738654/1.fna', '../../data_temp/1738654/8.fna', '../../data_temp/1738654/3.fna', '../../data_temp/1738654/0.fna', '../../data_temp/1738654/6.fna', '../../data_temp/2844374/1.fna', '../../data_temp/2844374/0.fna', '../../data_temp/2812024/2.fna', '../../data_temp/2812024/1.fna', '../../data_temp/2812024/3.fna', '../../data_temp/2812024/0.fna', '../../data_temp/3079755/0.fna', '../../data_temp/267891/7.fna', '../../data_temp/267891/2.fna', '../../data_temp/267891/9.fna', '../../data_temp/267891/5.fna', '../../data_temp/267891/4.fna', '../../data_temp/267891/1.fna', '../../data_temp/267891/8.fna', '../../data_temp/267891/3.fna', '../../data_temp/267891/0.fna', '../../data_temp/267891/6.fna', '../../data_temp/2974936/0.fna', '../../data_temp/1077264/7.fna', '../../data_temp/1077264/2.fna', '../../data_temp/1077264/9.fna', '../../data_temp/1077264/5.fna', '../../data_temp/1077264/4.fna', '../../data_temp/1077264/1.fna', '../../data_temp/1077264/8.fna', '../../data_temp/1077264/3.fna', '../../data_temp/1077264/0.fna', '../../data_temp/1077264/6.fna', '../../data_temp/85022/7.fna', '../../data_temp/85022/2.fna', '../../data_temp/85022/9.fna', '../../data_temp/85022/5.fna', '../../data_temp/85022/4.fna', '../../data_temp/85022/1.fna', '../../data_temp/85022/8.fna', '../../data_temp/85022/3.fna', '../../data_temp/85022/0.fna', '../../data_temp/85022/6.fna', '../../data_temp/84107/7.fna', '../../data_temp/84107/2.fna', '../../data_temp/84107/9.fna', '../../data_temp/84107/5.fna', '../../data_temp/84107/4.fna', '../../data_temp/84107/1.fna', '../../data_temp/84107/8.fna', '../../data_temp/84107/3.fna', '../../data_temp/84107/0.fna', '../../data_temp/84107/6.fna', '../../data_temp/3064025/0.fna', '../../data_temp/1771359/0.fna', '../../data_temp/31953/7.fna', '../../data_temp/31953/2.fna', '../../data_temp/31953/9.fna', '../../data_temp/31953/5.fna', '../../data_temp/31953/4.fna', '../../data_temp/31953/1.fna', '../../data_temp/31953/8.fna', '../../data_temp/31953/3.fna', '../../data_temp/31953/0.fna', '../../data_temp/31953/6.fna', '../../data_temp/1903409/7.fna', '../../data_temp/1903409/2.fna', '../../data_temp/1903409/9.fna', '../../data_temp/1903409/5.fna', '../../data_temp/1903409/4.fna', '../../data_temp/1903409/1.fna', '../../data_temp/1903409/8.fna', '../../data_temp/1903409/3.fna', '../../data_temp/1903409/0.fna', '../../data_temp/1903409/6.fna', '../../data_temp/3046411/1.fna', '../../data_temp/3046411/0.fna', '../../data_temp/2012/7.fna', '../../data_temp/2012/2.fna', '../../data_temp/2012/9.fna', '../../data_temp/2012/5.fna', '../../data_temp/2012/4.fna', '../../data_temp/2012/1.fna', '../../data_temp/2012/8.fna', '../../data_temp/2012/3.fna', '../../data_temp/2012/0.fna', '../../data_temp/2012/6.fna', '../../data_temp/1890464/7.fna', '../../data_temp/1890464/2.fna', '../../data_temp/1890464/9.fna', '../../data_temp/1890464/5.fna', '../../data_temp/1890464/4.fna', '../../data_temp/1890464/1.fna', '../../data_temp/1890464/8.fna', '../../data_temp/1890464/3.fna', '../../data_temp/1890464/0.fna', '../../data_temp/1890464/6.fna', '../../data_temp/2171986/7.fna', '../../data_temp/2171986/2.fna', '../../data_temp/2171986/9.fna', '../../data_temp/2171986/5.fna', '../../data_temp/2171986/4.fna', '../../data_temp/2171986/1.fna', '../../data_temp/2171986/8.fna', '../../data_temp/2171986/3.fna', '../../data_temp/2171986/0.fna', '../../data_temp/2171986/6.fna', '../../data_temp/3046600/2.fna', '../../data_temp/3046600/4.fna', '../../data_temp/3046600/1.fna', '../../data_temp/3046600/3.fna', '../../data_temp/3046600/0.fna', '../../data_temp/2937911/7.fna', '../../data_temp/2937911/2.fna', '../../data_temp/2937911/5.fna', '../../data_temp/2937911/4.fna', '../../data_temp/2937911/1.fna', '../../data_temp/2937911/3.fna', '../../data_temp/2937911/0.fna', '../../data_temp/2937911/6.fna', '../../data_temp/2698813/0.fna', '../../data_temp/292630/7.fna', '../../data_temp/292630/2.fna', '../../data_temp/292630/5.fna', '../../data_temp/292630/4.fna', '../../data_temp/292630/1.fna', '../../data_temp/292630/8.fna', '../../data_temp/292630/3.fna', '../../data_temp/292630/0.fna', '../../data_temp/292630/6.fna', '../../data_temp/83676/7.fna', '../../data_temp/83676/2.fna', '../../data_temp/83676/9.fna', '../../data_temp/83676/5.fna', '../../data_temp/83676/4.fna', '../../data_temp/83676/1.fna', '../../data_temp/83676/8.fna', '../../data_temp/83676/3.fna', '../../data_temp/83676/0.fna', '../../data_temp/83676/6.fna', '../../data_temp/2691359/7.fna', '../../data_temp/2691359/14.fna', '../../data_temp/2691359/2.fna', '../../data_temp/2691359/12.fna', '../../data_temp/2691359/9.fna', '../../data_temp/2691359/5.fna', '../../data_temp/2691359/4.fna', '../../data_temp/2691359/1.fna', '../../data_temp/2691359/13.fna', '../../data_temp/2691359/8.fna', '../../data_temp/2691359/3.fna', '../../data_temp/2691359/10.fna', '../../data_temp/2691359/0.fna', '../../data_temp/2691359/11.fna', '../../data_temp/2691359/6.fna', '../../data_temp/49546/7.fna', '../../data_temp/49546/2.fna', '../../data_temp/49546/9.fna', '../../data_temp/49546/5.fna', '../../data_temp/49546/4.fna', '../../data_temp/49546/1.fna', '../../data_temp/49546/8.fna', '../../data_temp/49546/3.fna', '../../data_temp/49546/0.fna', '../../data_temp/49546/6.fna', '../../data_temp/1706371/32.fna', '../../data_temp/1706371/7.fna', '../../data_temp/1706371/14.fna', '../../data_temp/1706371/17.fna', '../../data_temp/1706371/2.fna', '../../data_temp/1706371/12.fna', '../../data_temp/1706371/25.fna', '../../data_temp/1706371/22.fna', '../../data_temp/1706371/24.fna', '../../data_temp/1706371/9.fna', '../../data_temp/1706371/5.fna', '../../data_temp/1706371/23.fna', '../../data_temp/1706371/18.fna', '../../data_temp/1706371/4.fna', '../../data_temp/1706371/20.fna', '../../data_temp/1706371/26.fna', '../../data_temp/1706371/1.fna', '../../data_temp/1706371/27.fna', '../../data_temp/1706371/19.fna', '../../data_temp/1706371/13.fna', '../../data_temp/1706371/16.fna', '../../data_temp/1706371/29.fna', '../../data_temp/1706371/8.fna', '../../data_temp/1706371/21.fna', '../../data_temp/1706371/3.fna', '../../data_temp/1706371/10.fna', '../../data_temp/1706371/30.fna', '../../data_temp/1706371/31.fna', '../../data_temp/1706371/28.fna', '../../data_temp/1706371/0.fna', '../../data_temp/1706371/11.fna', '../../data_temp/1706371/6.fna', '../../data_temp/1706371/15.fna', '../../data_temp/85026/7.fna', '../../data_temp/85026/2.fna', '../../data_temp/85026/9.fna', '../../data_temp/85026/5.fna', '../../data_temp/85026/4.fna', '../../data_temp/85026/1.fna', '../../data_temp/85026/8.fna', '../../data_temp/85026/3.fna', '../../data_temp/85026/0.fna', '../../data_temp/85026/6.fna', '../../data_temp/1892254/7.fna', '../../data_temp/1892254/2.fna', '../../data_temp/1892254/9.fna', '../../data_temp/1892254/5.fna', '../../data_temp/1892254/4.fna', '../../data_temp/1892254/1.fna', '../../data_temp/1892254/8.fna', '../../data_temp/1892254/3.fna', '../../data_temp/1892254/0.fna', '../../data_temp/1892254/6.fna', '../../data_temp/125316/2.fna', '../../data_temp/125316/5.fna', '../../data_temp/125316/4.fna', '../../data_temp/125316/1.fna', '../../data_temp/125316/3.fna', '../../data_temp/125316/0.fna', '../../data_temp/125316/6.fna', '../../data_temp/2812006/7.fna', '../../data_temp/2812006/2.fna', '../../data_temp/2812006/9.fna', '../../data_temp/2812006/5.fna', '../../data_temp/2812006/4.fna', '../../data_temp/2812006/1.fna', '../../data_temp/2812006/8.fna', '../../data_temp/2812006/3.fna', '../../data_temp/2812006/0.fna', '../../data_temp/2812006/6.fna', '../../data_temp/1890497/2.fna', '../../data_temp/1890497/1.fna', '../../data_temp/1890497/3.fna', '../../data_temp/1890497/0.fna', '../../data_temp/403/64.fna', '../../data_temp/403/32.fna', '../../data_temp/403/99.fna', '../../data_temp/403/43.fna', '../../data_temp/403/7.fna', '../../data_temp/403/98.fna', '../../data_temp/403/58.fna', '../../data_temp/403/90.fna', '../../data_temp/403/14.fna', '../../data_temp/403/41.fna', '../../data_temp/403/67.fna', '../../data_temp/403/73.fna', '../../data_temp/403/17.fna', '../../data_temp/403/56.fna', '../../data_temp/403/50.fna', '../../data_temp/403/2.fna', '../../data_temp/403/46.fna', '../../data_temp/403/52.fna', '../../data_temp/403/65.fna', '../../data_temp/403/95.fna', '../../data_temp/403/45.fna', '../../data_temp/403/74.fna', '../../data_temp/403/44.fna', '../../data_temp/403/12.fna', '../../data_temp/403/94.fna', '../../data_temp/403/25.fna', '../../data_temp/403/60.fna', '../../data_temp/403/39.fna', '../../data_temp/403/22.fna', '../../data_temp/403/48.fna', '../../data_temp/403/88.fna', '../../data_temp/403/24.fna', '../../data_temp/403/9.fna', '../../data_temp/403/86.fna', '../../data_temp/403/5.fna', '../../data_temp/403/89.fna', '../../data_temp/403/71.fna', '../../data_temp/403/54.fna', '../../data_temp/403/23.fna', '../../data_temp/403/49.fna', '../../data_temp/403/40.fna', '../../data_temp/403/18.fna', '../../data_temp/403/4.fna', '../../data_temp/403/20.fna', '../../data_temp/403/63.fna', '../../data_temp/403/33.fna', '../../data_temp/403/87.fna', '../../data_temp/403/80.fna', '../../data_temp/403/26.fna', '../../data_temp/403/1.fna', '../../data_temp/403/47.fna', '../../data_temp/403/92.fna', '../../data_temp/403/51.fna', '../../data_temp/403/37.fna', '../../data_temp/403/84.fna', '../../data_temp/403/53.fna', '../../data_temp/403/70.fna', '../../data_temp/403/42.fna', '../../data_temp/403/57.fna', '../../data_temp/403/72.fna', '../../data_temp/403/91.fna', '../../data_temp/403/27.fna', '../../data_temp/403/19.fna', '../../data_temp/403/66.fna', '../../data_temp/403/13.fna', '../../data_temp/403/16.fna', '../../data_temp/403/29.fna', '../../data_temp/403/8.fna', '../../data_temp/403/21.fna', '../../data_temp/403/3.fna', '../../data_temp/403/10.fna', '../../data_temp/403/30.fna', '../../data_temp/403/38.fna', '../../data_temp/403/93.fna', '../../data_temp/403/78.fna', '../../data_temp/403/77.fna', '../../data_temp/403/75.fna', '../../data_temp/403/35.fna', '../../data_temp/403/31.fna', '../../data_temp/403/28.fna', '../../data_temp/403/82.fna', '../../data_temp/403/59.fna', '../../data_temp/403/69.fna', '../../data_temp/403/62.fna', '../../data_temp/403/85.fna', '../../data_temp/403/0.fna', '../../data_temp/403/79.fna', '../../data_temp/403/11.fna', '../../data_temp/403/34.fna', '../../data_temp/403/6.fna', '../../data_temp/403/68.fna', '../../data_temp/403/97.fna', '../../data_temp/403/15.fna', '../../data_temp/403/55.fna', '../../data_temp/403/61.fna', '../../data_temp/403/36.fna', '../../data_temp/403/81.fna', '../../data_temp/403/96.fna', '../../data_temp/403/76.fna', '../../data_temp/403/83.fna', '../../data_temp/145358/7.fna', '../../data_temp/145358/2.fna', '../../data_temp/145358/9.fna', '../../data_temp/145358/5.fna', '../../data_temp/145358/4.fna', '../../data_temp/145358/1.fna', '../../data_temp/145358/8.fna', '../../data_temp/145358/3.fna', '../../data_temp/145358/0.fna', '../../data_temp/145358/6.fna', '../../data_temp/1643957/0.fna', '../../data_temp/1937961/7.fna', '../../data_temp/1937961/2.fna', '../../data_temp/1937961/9.fna', '../../data_temp/1937961/5.fna', '../../data_temp/1937961/4.fna', '../../data_temp/1937961/1.fna', '../../data_temp/1937961/8.fna', '../../data_temp/1937961/3.fna', '../../data_temp/1937961/0.fna', '../../data_temp/1937961/6.fna', '../../data_temp/2945020/7.fna', '../../data_temp/2945020/2.fna', '../../data_temp/2945020/9.fna', '../../data_temp/2945020/5.fna', '../../data_temp/2945020/4.fna', '../../data_temp/2945020/1.fna', '../../data_temp/2945020/8.fna', '../../data_temp/2945020/3.fna', '../../data_temp/2945020/0.fna', '../../data_temp/2945020/6.fna', '../../data_temp/568386/7.fna', '../../data_temp/568386/2.fna', '../../data_temp/568386/9.fna', '../../data_temp/568386/5.fna', '../../data_temp/568386/4.fna', '../../data_temp/568386/1.fna', '../../data_temp/568386/8.fna', '../../data_temp/568386/3.fna', '../../data_temp/568386/0.fna', '../../data_temp/568386/6.fna', '../../data_temp/34064/7.fna', '../../data_temp/34064/2.fna', '../../data_temp/34064/9.fna', '../../data_temp/34064/5.fna', '../../data_temp/34064/4.fna', '../../data_temp/34064/1.fna', '../../data_temp/34064/8.fna', '../../data_temp/34064/3.fna', '../../data_temp/34064/0.fna', '../../data_temp/34064/6.fna', '../../data_temp/1969992/7.fna', '../../data_temp/1969992/2.fna', '../../data_temp/1969992/9.fna', '../../data_temp/1969992/5.fna', '../../data_temp/1969992/4.fna', '../../data_temp/1969992/1.fna', '../../data_temp/1969992/8.fna', '../../data_temp/1969992/3.fna', '../../data_temp/1969992/0.fna', '../../data_temp/1969992/6.fna', '../../data_temp/2963099/1.fna', '../../data_temp/2963099/0.fna', '../../data_temp/137/7.fna', '../../data_temp/137/2.fna', '../../data_temp/137/9.fna', '../../data_temp/137/5.fna', '../../data_temp/137/4.fna', '../../data_temp/137/1.fna', '../../data_temp/137/8.fna', '../../data_temp/137/3.fna', '../../data_temp/137/0.fna', '../../data_temp/137/6.fna', '../../data_temp/1162/7.fna', '../../data_temp/1162/2.fna', '../../data_temp/1162/9.fna', '../../data_temp/1162/5.fna', '../../data_temp/1162/4.fna', '../../data_temp/1162/1.fna', '../../data_temp/1162/8.fna', '../../data_temp/1162/3.fna', '../../data_temp/1162/0.fna', '../../data_temp/1162/6.fna', '../../data_temp/85023/7.fna', '../../data_temp/85023/2.fna', '../../data_temp/85023/9.fna', '../../data_temp/85023/5.fna', '../../data_temp/85023/4.fna', '../../data_temp/85023/1.fna', '../../data_temp/85023/8.fna', '../../data_temp/85023/3.fna', '../../data_temp/85023/0.fna', '../../data_temp/85023/6.fna', '../../data_temp/3118654/7.fna', '../../data_temp/3118654/2.fna', '../../data_temp/3118654/5.fna', '../../data_temp/3118654/4.fna', '../../data_temp/3118654/1.fna', '../../data_temp/3118654/3.fna', '../../data_temp/3118654/0.fna', '../../data_temp/3118654/6.fna', '../../data_temp/2813951/2.fna', '../../data_temp/2813951/5.fna', '../../data_temp/2813951/4.fna', '../../data_temp/2813951/1.fna', '../../data_temp/2813951/3.fna', '../../data_temp/2813951/0.fna', '../../data_temp/1804993/2.fna', '../../data_temp/1804993/1.fna', '../../data_temp/1804993/0.fna', '../../data_temp/1853231/7.fna', '../../data_temp/1853231/2.fna', '../../data_temp/1853231/9.fna', '../../data_temp/1853231/5.fna', '../../data_temp/1853231/4.fna', '../../data_temp/1853231/1.fna', '../../data_temp/1853231/8.fna', '../../data_temp/1853231/3.fna', '../../data_temp/1853231/0.fna', '../../data_temp/1853231/6.fna', '../../data_temp/3056371/7.fna', '../../data_temp/3056371/2.fna', '../../data_temp/3056371/5.fna', '../../data_temp/3056371/4.fna', '../../data_temp/3056371/1.fna', '../../data_temp/3056371/3.fna', '../../data_temp/3056371/0.fna', '../../data_temp/3056371/6.fna', '../../data_temp/1748772/0.fna', '../../data_temp/649777/7.fna', '../../data_temp/649777/2.fna', '../../data_temp/649777/9.fna', '../../data_temp/649777/5.fna', '../../data_temp/649777/4.fna', '../../data_temp/649777/1.fna', '../../data_temp/649777/8.fna', '../../data_temp/649777/3.fna', '../../data_temp/649777/0.fna', '../../data_temp/649777/6.fna', '../../data_temp/2152899/7.fna', '../../data_temp/2152899/2.fna', '../../data_temp/2152899/9.fna', '../../data_temp/2152899/5.fna', '../../data_temp/2152899/4.fna', '../../data_temp/2152899/1.fna', '../../data_temp/2152899/8.fna', '../../data_temp/2152899/3.fna', '../../data_temp/2152899/0.fna', '../../data_temp/2152899/6.fna', '../../data_temp/1853227/2.fna', '../../data_temp/1853227/5.fna', '../../data_temp/1853227/4.fna', '../../data_temp/1853227/1.fna', '../../data_temp/1853227/3.fna', '../../data_temp/1853227/0.fna', '../../data_temp/3079273/2.fna', '../../data_temp/3079273/1.fna', '../../data_temp/3079273/0.fna', '../../data_temp/92714/7.fna', '../../data_temp/92714/2.fna', '../../data_temp/92714/9.fna', '../../data_temp/92714/5.fna', '../../data_temp/92714/4.fna', '../../data_temp/92714/1.fna', '../../data_temp/92714/8.fna', '../../data_temp/92714/3.fna', '../../data_temp/92714/0.fna', '../../data_temp/92714/6.fna', '../../data_temp/990719/7.fna', '../../data_temp/990719/2.fna', '../../data_temp/990719/9.fna', '../../data_temp/990719/5.fna', '../../data_temp/990719/4.fna', '../../data_temp/990719/1.fna', '../../data_temp/990719/8.fna', '../../data_temp/990719/3.fna', '../../data_temp/990719/0.fna', '../../data_temp/990719/6.fna', '../../data_temp/1100104/2.fna', '../../data_temp/1100104/1.fna', '../../data_temp/1100104/0.fna', '../../data_temp/227387/2.fna', '../../data_temp/227387/1.fna', '../../data_temp/227387/0.fna', '../../data_temp/2805401/0.fna', '../../data_temp/145359/1.fna', '../../data_temp/145359/0.fna', '../../data_temp/3047432/0.fna', '../../data_temp/3031459/7.fna', '../../data_temp/3031459/2.fna', '../../data_temp/3031459/9.fna', '../../data_temp/3031459/5.fna', '../../data_temp/3031459/4.fna', '../../data_temp/3031459/1.fna', '../../data_temp/3031459/8.fna', '../../data_temp/3031459/3.fna', '../../data_temp/3031459/0.fna', '../../data_temp/3031459/6.fna', '../../data_temp/1890455/0.fna', '../../data_temp/82115/7.fna', '../../data_temp/82115/2.fna', '../../data_temp/82115/9.fna', '../../data_temp/82115/5.fna', '../../data_temp/82115/4.fna', '../../data_temp/82115/1.fna', '../../data_temp/82115/8.fna', '../../data_temp/82115/3.fna', '../../data_temp/82115/0.fna', '../../data_temp/82115/6.fna', '../../data_temp/85033/2.fna', '../../data_temp/85033/5.fna', '../../data_temp/85033/4.fna', '../../data_temp/85033/1.fna', '../../data_temp/85033/3.fna', '../../data_temp/85033/0.fna', '../../data_temp/3031654/1.fna', '../../data_temp/3031654/0.fna', '../../data_temp/64898/7.fna', '../../data_temp/64898/2.fna', '../../data_temp/64898/9.fna', '../../data_temp/64898/5.fna', '../../data_temp/64898/4.fna', '../../data_temp/64898/1.fna', '../../data_temp/64898/8.fna', '../../data_temp/64898/3.fna', '../../data_temp/64898/0.fna', '../../data_temp/64898/6.fna', '../../data_temp/2772534/2.fna', '../../data_temp/2772534/1.fna', '../../data_temp/2772534/3.fna', '../../data_temp/2772534/0.fna', '../../data_temp/1442156/7.fna', '../../data_temp/1442156/2.fna', '../../data_temp/1442156/9.fna', '../../data_temp/1442156/5.fna', '../../data_temp/1442156/4.fna', '../../data_temp/1442156/1.fna', '../../data_temp/1442156/8.fna', '../../data_temp/1442156/3.fna', '../../data_temp/1442156/0.fna', '../../data_temp/1442156/6.fna', '../../data_temp/3031641/0.fna', '../../data_temp/2946506/2.fna', '../../data_temp/2946506/5.fna', '../../data_temp/2946506/4.fna', '../../data_temp/2946506/1.fna', '../../data_temp/2946506/3.fna', '../../data_temp/2946506/0.fna', '../../data_temp/2946506/6.fna', '../../data_temp/3085642/7.fna', '../../data_temp/3085642/2.fna', '../../data_temp/3085642/9.fna', '../../data_temp/3085642/5.fna', '../../data_temp/3085642/4.fna', '../../data_temp/3085642/1.fna', '../../data_temp/3085642/8.fna', '../../data_temp/3085642/3.fna', '../../data_temp/3085642/0.fna', '../../data_temp/3085642/6.fna', '../../data_temp/1903410/7.fna', '../../data_temp/1903410/2.fna', '../../data_temp/1903410/9.fna', '../../data_temp/1903410/5.fna', '../../data_temp/1903410/4.fna', '../../data_temp/1903410/1.fna', '../../data_temp/1903410/8.fna', '../../data_temp/1903410/3.fna', '../../data_temp/1903410/0.fna', '../../data_temp/1903410/6.fna', '../../data_temp/543/7.fna', '../../data_temp/543/2.fna', '../../data_temp/543/9.fna', '../../data_temp/543/5.fna', '../../data_temp/543/4.fna', '../../data_temp/543/1.fna', '../../data_temp/543/8.fna', '../../data_temp/543/3.fna', '../../data_temp/543/0.fna', '../../data_temp/543/6.fna', '../../data_temp/83778/7.fna', '../../data_temp/83778/2.fna', '../../data_temp/83778/9.fna', '../../data_temp/83778/5.fna', '../../data_temp/83778/4.fna', '../../data_temp/83778/1.fna', '../../data_temp/83778/8.fna', '../../data_temp/83778/3.fna', '../../data_temp/83778/0.fna', '../../data_temp/83778/6.fna', '../../data_temp/246874/7.fna', '../../data_temp/246874/2.fna', '../../data_temp/246874/9.fna', '../../data_temp/246874/5.fna', '../../data_temp/246874/4.fna', '../../data_temp/246874/1.fna', '../../data_temp/246874/8.fna', '../../data_temp/246874/3.fna', '../../data_temp/246874/0.fna', '../../data_temp/246874/6.fna', '../../data_temp/2685818/7.fna', '../../data_temp/2685818/2.fna', '../../data_temp/2685818/5.fna', '../../data_temp/2685818/4.fna', '../../data_temp/2685818/1.fna', '../../data_temp/2685818/3.fna', '../../data_temp/2685818/0.fna', '../../data_temp/2685818/6.fna', '../../data_temp/1185407/7.fna', '../../data_temp/1185407/2.fna', '../../data_temp/1185407/9.fna', '../../data_temp/1185407/5.fna', '../../data_temp/1185407/4.fna', '../../data_temp/1185407/1.fna', '../../data_temp/1185407/8.fna', '../../data_temp/1185407/3.fna', '../../data_temp/1185407/0.fna', '../../data_temp/1185407/6.fna', '../../data_temp/2805426/7.fna', '../../data_temp/2805426/2.fna', '../../data_temp/2805426/9.fna', '../../data_temp/2805426/5.fna', '../../data_temp/2805426/4.fna', '../../data_temp/2805426/1.fna', '../../data_temp/2805426/8.fna', '../../data_temp/2805426/3.fna', '../../data_temp/2805426/0.fna', '../../data_temp/2805426/6.fna', '../../data_temp/2829802/7.fna', '../../data_temp/2829802/2.fna', '../../data_temp/2829802/9.fna', '../../data_temp/2829802/5.fna', '../../data_temp/2829802/4.fna', '../../data_temp/2829802/1.fna', '../../data_temp/2829802/8.fna', '../../data_temp/2829802/3.fna', '../../data_temp/2829802/0.fna', '../../data_temp/2829802/6.fna', '../../data_temp/3141702/7.fna', '../../data_temp/3141702/2.fna', '../../data_temp/3141702/9.fna', '../../data_temp/3141702/5.fna', '../../data_temp/3141702/4.fna', '../../data_temp/3141702/1.fna', '../../data_temp/3141702/8.fna', '../../data_temp/3141702/3.fna', '../../data_temp/3141702/0.fna', '../../data_temp/3141702/6.fna', '../../data_temp/143786/7.fna', '../../data_temp/143786/2.fna', '../../data_temp/143786/9.fna', '../../data_temp/143786/5.fna', '../../data_temp/143786/4.fna', '../../data_temp/143786/1.fna', '../../data_temp/143786/8.fna', '../../data_temp/143786/3.fna', '../../data_temp/143786/0.fna', '../../data_temp/143786/6.fna', '../../data_temp/2992718/7.fna', '../../data_temp/2992718/2.fna', '../../data_temp/2992718/9.fna', '../../data_temp/2992718/5.fna', '../../data_temp/2992718/4.fna', '../../data_temp/2992718/1.fna', '../../data_temp/2992718/8.fna', '../../data_temp/2992718/3.fna', '../../data_temp/2992718/0.fna', '../../data_temp/2992718/6.fna', '../../data_temp/1706374/7.fna', '../../data_temp/1706374/2.fna', '../../data_temp/1706374/9.fna', '../../data_temp/1706374/5.fna', '../../data_temp/1706374/4.fna', '../../data_temp/1706374/1.fna', '../../data_temp/1706374/8.fna', '../../data_temp/1706374/3.fna', '../../data_temp/1706374/0.fna', '../../data_temp/1706374/6.fna', '../../data_temp/1890448/7.fna', '../../data_temp/1890448/2.fna', '../../data_temp/1890448/5.fna', '../../data_temp/1890448/4.fna', '../../data_temp/1890448/1.fna', '../../data_temp/1890448/3.fna', '../../data_temp/1890448/0.fna', '../../data_temp/1890448/6.fna', '../../data_temp/1652132/7.fna', '../../data_temp/1652132/2.fna', '../../data_temp/1652132/9.fna', '../../data_temp/1652132/5.fna', '../../data_temp/1652132/4.fna', '../../data_temp/1652132/1.fna', '../../data_temp/1652132/8.fna', '../../data_temp/1652132/3.fna', '../../data_temp/1652132/0.fna', '../../data_temp/1652132/6.fna', '../../data_temp/2843308/7.fna', '../../data_temp/2843308/2.fna', '../../data_temp/2843308/9.fna', '../../data_temp/2843308/5.fna', '../../data_temp/2843308/4.fna', '../../data_temp/2843308/1.fna', '../../data_temp/2843308/8.fna', '../../data_temp/2843308/3.fna', '../../data_temp/2843308/0.fna', '../../data_temp/2843308/6.fna', '../../data_temp/2772533/0.fna', '../../data_temp/2983183/0.fna', '../../data_temp/942/7.fna', '../../data_temp/942/2.fna', '../../data_temp/942/9.fna', '../../data_temp/942/5.fna', '../../data_temp/942/4.fna', '../../data_temp/942/1.fna', '../../data_temp/942/8.fna', '../../data_temp/942/3.fna', '../../data_temp/942/0.fna', '../../data_temp/942/6.fna', '../../data_temp/2004/7.fna', '../../data_temp/2004/2.fna', '../../data_temp/2004/9.fna', '../../data_temp/2004/5.fna', '../../data_temp/2004/4.fna', '../../data_temp/2004/1.fna', '../../data_temp/2004/8.fna', '../../data_temp/2004/3.fna', '../../data_temp/2004/0.fna', '../../data_temp/2004/6.fna', '../../data_temp/563798/7.fna', '../../data_temp/563798/2.fna', '../../data_temp/563798/9.fna', '../../data_temp/563798/5.fna', '../../data_temp/563798/4.fna', '../../data_temp/563798/1.fna', '../../data_temp/563798/8.fna', '../../data_temp/563798/3.fna', '../../data_temp/563798/0.fna', '../../data_temp/563798/6.fna', '../../data_temp/3023558/0.fna', '../../data_temp/85016/7.fna', '../../data_temp/85016/2.fna', '../../data_temp/85016/9.fna', '../../data_temp/85016/5.fna', '../../data_temp/85016/4.fna', '../../data_temp/85016/1.fna', '../../data_temp/85016/8.fna', '../../data_temp/85016/3.fna', '../../data_temp/85016/0.fna', '../../data_temp/85016/6.fna', '../../data_temp/213119/64.fna', '../../data_temp/213119/32.fna', '../../data_temp/213119/43.fna', '../../data_temp/213119/7.fna', '../../data_temp/213119/58.fna', '../../data_temp/213119/90.fna', '../../data_temp/213119/14.fna', '../../data_temp/213119/41.fna', '../../data_temp/213119/67.fna', '../../data_temp/213119/73.fna', '../../data_temp/213119/17.fna', '../../data_temp/213119/56.fna', '../../data_temp/213119/50.fna', '../../data_temp/213119/2.fna', '../../data_temp/213119/46.fna', '../../data_temp/213119/52.fna', '../../data_temp/213119/65.fna', '../../data_temp/213119/95.fna', '../../data_temp/213119/45.fna', '../../data_temp/213119/74.fna', '../../data_temp/213119/44.fna', '../../data_temp/213119/12.fna', '../../data_temp/213119/94.fna', '../../data_temp/213119/25.fna', '../../data_temp/213119/60.fna', '../../data_temp/213119/39.fna', '../../data_temp/213119/22.fna', '../../data_temp/213119/48.fna', '../../data_temp/213119/88.fna', '../../data_temp/213119/24.fna', '../../data_temp/213119/9.fna', '../../data_temp/213119/86.fna', '../../data_temp/213119/5.fna', '../../data_temp/213119/89.fna', '../../data_temp/213119/71.fna', '../../data_temp/213119/54.fna', '../../data_temp/213119/23.fna', '../../data_temp/213119/49.fna', '../../data_temp/213119/40.fna', '../../data_temp/213119/18.fna', '../../data_temp/213119/4.fna', '../../data_temp/213119/20.fna', '../../data_temp/213119/63.fna', '../../data_temp/213119/33.fna', '../../data_temp/213119/87.fna', '../../data_temp/213119/80.fna', '../../data_temp/213119/26.fna', '../../data_temp/213119/1.fna', '../../data_temp/213119/47.fna', '../../data_temp/213119/92.fna', '../../data_temp/213119/51.fna', '../../data_temp/213119/37.fna', '../../data_temp/213119/84.fna', '../../data_temp/213119/53.fna', '../../data_temp/213119/70.fna', '../../data_temp/213119/42.fna', '../../data_temp/213119/57.fna', '../../data_temp/213119/72.fna', '../../data_temp/213119/91.fna', '../../data_temp/213119/27.fna', '../../data_temp/213119/19.fna', '../../data_temp/213119/66.fna', '../../data_temp/213119/13.fna', '../../data_temp/213119/16.fna', '../../data_temp/213119/29.fna', '../../data_temp/213119/8.fna', '../../data_temp/213119/21.fna', '../../data_temp/213119/3.fna', '../../data_temp/213119/10.fna', '../../data_temp/213119/30.fna', '../../data_temp/213119/38.fna', '../../data_temp/213119/93.fna', '../../data_temp/213119/78.fna', '../../data_temp/213119/77.fna', '../../data_temp/213119/75.fna', '../../data_temp/213119/35.fna', '../../data_temp/213119/31.fna', '../../data_temp/213119/28.fna', '../../data_temp/213119/82.fna', '../../data_temp/213119/59.fna', '../../data_temp/213119/69.fna', '../../data_temp/213119/62.fna', '../../data_temp/213119/85.fna', '../../data_temp/213119/0.fna', '../../data_temp/213119/79.fna', '../../data_temp/213119/11.fna', '../../data_temp/213119/34.fna', '../../data_temp/213119/6.fna', '../../data_temp/213119/68.fna', '../../data_temp/213119/97.fna', '../../data_temp/213119/15.fna', '../../data_temp/213119/55.fna', '../../data_temp/213119/61.fna', '../../data_temp/213119/36.fna', '../../data_temp/213119/81.fna', '../../data_temp/213119/96.fna', '../../data_temp/213119/76.fna', '../../data_temp/213119/83.fna', '../../data_temp/267889/7.fna', '../../data_temp/267889/2.fna', '../../data_temp/267889/9.fna', '../../data_temp/267889/5.fna', '../../data_temp/267889/4.fna', '../../data_temp/267889/1.fna', '../../data_temp/267889/8.fna', '../../data_temp/267889/3.fna', '../../data_temp/267889/0.fna', '../../data_temp/267889/6.fna', '../../data_temp/2831090/2.fna', '../../data_temp/2831090/5.fna', '../../data_temp/2831090/4.fna', '../../data_temp/2831090/1.fna', '../../data_temp/2831090/3.fna', '../../data_temp/2831090/0.fna', '../../data_temp/2831090/6.fna', '../../data_temp/3030910/7.fna', '../../data_temp/3030910/2.fna', '../../data_temp/3030910/9.fna', '../../data_temp/3030910/5.fna', '../../data_temp/3030910/4.fna', '../../data_temp/3030910/1.fna', '../../data_temp/3030910/8.fna', '../../data_temp/3030910/3.fna', '../../data_temp/3030910/0.fna', '../../data_temp/3030910/6.fna', '../../data_temp/2953756/0.fna', '../../data_temp/2093822/7.fna', '../../data_temp/2093822/2.fna', '../../data_temp/2093822/9.fna', '../../data_temp/2093822/5.fna', '../../data_temp/2093822/4.fna', '../../data_temp/2093822/1.fna', '../../data_temp/2093822/8.fna', '../../data_temp/2093822/3.fna', '../../data_temp/2093822/0.fna', '../../data_temp/2093822/6.fna', '../../data_temp/186804/7.fna', '../../data_temp/186804/2.fna', '../../data_temp/186804/9.fna', '../../data_temp/186804/5.fna', '../../data_temp/186804/4.fna', '../../data_temp/186804/1.fna', '../../data_temp/186804/8.fna', '../../data_temp/186804/3.fna', '../../data_temp/186804/0.fna', '../../data_temp/186804/6.fna', '../../data_temp/1617891/7.fna', '../../data_temp/1617891/2.fna', '../../data_temp/1617891/5.fna', '../../data_temp/1617891/4.fna', '../../data_temp/1617891/1.fna', '../../data_temp/1617891/3.fna', '../../data_temp/1617891/0.fna', '../../data_temp/1617891/6.fna', '../../data_temp/1692504/0.fna', '../../data_temp/1783343/7.fna', '../../data_temp/1783343/2.fna', '../../data_temp/1783343/5.fna', '../../data_temp/1783343/4.fna', '../../data_temp/1783343/1.fna', '../../data_temp/1783343/8.fna', '../../data_temp/1783343/3.fna', '../../data_temp/1783343/0.fna', '../../data_temp/1783343/6.fna', '../../data_temp/3079756/2.fna', '../../data_temp/3079756/1.fna', '../../data_temp/3079756/0.fna', '../../data_temp/3031664/2.fna', '../../data_temp/3031664/1.fna', '../../data_temp/3031664/0.fna', '../../data_temp/1892264/2.fna', '../../data_temp/1892264/5.fna', '../../data_temp/1892264/4.fna', '../../data_temp/1892264/1.fna', '../../data_temp/1892264/3.fna', '../../data_temp/1892264/0.fna', '../../data_temp/1890450/7.fna', '../../data_temp/1890450/2.fna', '../../data_temp/1890450/9.fna', '../../data_temp/1890450/5.fna', '../../data_temp/1890450/4.fna', '../../data_temp/1890450/1.fna', '../../data_temp/1890450/8.fna', '../../data_temp/1890450/3.fna', '../../data_temp/1890450/0.fna', '../../data_temp/1890450/6.fna', '../../data_temp/1892261/2.fna', '../../data_temp/1892261/1.fna', '../../data_temp/1892261/3.fna', '../../data_temp/1892261/0.fna', '../../data_temp/267893/7.fna', '../../data_temp/267893/2.fna', '../../data_temp/267893/9.fna', '../../data_temp/267893/5.fna', '../../data_temp/267893/4.fna', '../../data_temp/267893/1.fna', '../../data_temp/267893/8.fna', '../../data_temp/267893/3.fna', '../../data_temp/267893/0.fna', '../../data_temp/267893/6.fna', '../../data_temp/186824/7.fna', '../../data_temp/186824/2.fna', '../../data_temp/186824/9.fna', '../../data_temp/186824/5.fna', '../../data_temp/186824/4.fna', '../../data_temp/186824/1.fna', '../../data_temp/186824/8.fna', '../../data_temp/186824/3.fna', '../../data_temp/186824/0.fna', '../../data_temp/186824/6.fna', '../../data_temp/2771471/7.fna', '../../data_temp/2771471/2.fna', '../../data_temp/2771471/9.fna', '../../data_temp/2771471/5.fna', '../../data_temp/2771471/4.fna', '../../data_temp/2771471/1.fna', '../../data_temp/2771471/8.fna', '../../data_temp/2771471/3.fna', '../../data_temp/2771471/0.fna', '../../data_temp/2771471/6.fna', '../../data_temp/69277/7.fna', '../../data_temp/69277/2.fna', '../../data_temp/69277/9.fna', '../../data_temp/69277/5.fna', '../../data_temp/69277/4.fna', '../../data_temp/69277/1.fna', '../../data_temp/69277/8.fna', '../../data_temp/69277/3.fna', '../../data_temp/69277/0.fna', '../../data_temp/69277/6.fna', '../../data_temp/267888/7.fna', '../../data_temp/267888/2.fna', '../../data_temp/267888/9.fna', '../../data_temp/267888/5.fna', '../../data_temp/267888/4.fna', '../../data_temp/267888/1.fna', '../../data_temp/267888/8.fna', '../../data_temp/267888/3.fna', '../../data_temp/267888/0.fna', '../../data_temp/267888/6.fna', '../../data_temp/3031622/2.fna', '../../data_temp/3031622/1.fna', '../../data_temp/3031622/3.fna', '../../data_temp/3031622/0.fna', '../../data_temp/3076165/2.fna', '../../data_temp/3076165/5.fna', '../../data_temp/3076165/4.fna', '../../data_temp/3076165/1.fna', '../../data_temp/3076165/3.fna', '../../data_temp/3076165/0.fna', '../../data_temp/2497645/2.fna', '../../data_temp/2497645/1.fna', '../../data_temp/2497645/3.fna', '../../data_temp/2497645/0.fna', '../../data_temp/2844877/2.fna', '../../data_temp/2844877/1.fna', '../../data_temp/2844877/3.fna', '../../data_temp/2844877/0.fna', '../../data_temp/1955043/2.fna', '../../data_temp/1955043/5.fna', '../../data_temp/1955043/4.fna', '../../data_temp/1955043/1.fna', '../../data_temp/1955043/3.fna', '../../data_temp/1955043/0.fna', '../../data_temp/1573805/7.fna', '../../data_temp/1573805/2.fna', '../../data_temp/1573805/9.fna', '../../data_temp/1573805/5.fna', '../../data_temp/1573805/4.fna', '../../data_temp/1573805/1.fna', '../../data_temp/1573805/8.fna', '../../data_temp/1573805/3.fna', '../../data_temp/1573805/0.fna', '../../data_temp/1573805/6.fna', '../../data_temp/3086088/7.fna', '../../data_temp/3086088/2.fna', '../../data_temp/3086088/9.fna', '../../data_temp/3086088/5.fna', '../../data_temp/3086088/4.fna', '../../data_temp/3086088/1.fna', '../../data_temp/3086088/8.fna', '../../data_temp/3086088/3.fna', '../../data_temp/3086088/0.fna', '../../data_temp/3086088/6.fna', '../../data_temp/1055686/7.fna', '../../data_temp/1055686/2.fna', '../../data_temp/1055686/9.fna', '../../data_temp/1055686/5.fna', '../../data_temp/1055686/4.fna', '../../data_temp/1055686/1.fna', '../../data_temp/1055686/8.fna', '../../data_temp/1055686/3.fna', '../../data_temp/1055686/0.fna', '../../data_temp/1055686/6.fna', '../../data_temp/414877/7.fna', '../../data_temp/414877/2.fna', '../../data_temp/414877/5.fna', '../../data_temp/414877/4.fna', '../../data_temp/414877/1.fna', '../../data_temp/414877/3.fna', '../../data_temp/414877/0.fna', '../../data_temp/414877/6.fna', '../../data_temp/1759334/0.fna', '../../data_temp/1692041/7.fna', '../../data_temp/1692041/2.fna', '../../data_temp/1692041/9.fna', '../../data_temp/1692041/5.fna', '../../data_temp/1692041/4.fna', '../../data_temp/1692041/1.fna', '../../data_temp/1692041/8.fna', '../../data_temp/1692041/3.fna', '../../data_temp/1692041/0.fna', '../../data_temp/1692041/6.fna', '../../data_temp/216572/7.fna', '../../data_temp/216572/2.fna', '../../data_temp/216572/9.fna', '../../data_temp/216572/5.fna', '../../data_temp/216572/4.fna', '../../data_temp/216572/1.fna', '../../data_temp/216572/8.fna', '../../data_temp/216572/3.fna', '../../data_temp/216572/0.fna', '../../data_temp/216572/6.fna', '../../data_temp/3024550/2.fna', '../../data_temp/3024550/1.fna', '../../data_temp/3024550/0.fna', '../../data_temp/1771355/1.fna', '../../data_temp/1771355/0.fna', '../../data_temp/117942/7.fna', '../../data_temp/117942/2.fna', '../../data_temp/117942/9.fna', '../../data_temp/117942/5.fna', '../../data_temp/117942/4.fna', '../../data_temp/117942/1.fna', '../../data_temp/117942/8.fna', '../../data_temp/117942/3.fna', '../../data_temp/117942/0.fna', '../../data_temp/117942/6.fna', '../../data_temp/1890426/32.fna', '../../data_temp/1890426/7.fna', '../../data_temp/1890426/14.fna', '../../data_temp/1890426/17.fna', '../../data_temp/1890426/2.fna', '../../data_temp/1890426/12.fna', '../../data_temp/1890426/25.fna', '../../data_temp/1890426/22.fna', '../../data_temp/1890426/24.fna', '../../data_temp/1890426/9.fna', '../../data_temp/1890426/39.zip', '../../data_temp/1890426/5.fna', '../../data_temp/1890426/23.fna', '../../data_temp/1890426/18.fna', '../../data_temp/1890426/4.fna', '../../data_temp/1890426/20.fna', '../../data_temp/1890426/33.fna', '../../data_temp/1890426/26.fna', '../../data_temp/1890426/1.fna', '../../data_temp/1890426/37.fna', '../../data_temp/1890426/27.fna', '../../data_temp/1890426/19.fna', '../../data_temp/1890426/13.fna', '../../data_temp/1890426/16.fna', '../../data_temp/1890426/29.fna', '../../data_temp/1890426/8.fna', '../../data_temp/1890426/21.fna', '../../data_temp/1890426/3.fna', '../../data_temp/1890426/10.fna', '../../data_temp/1890426/30.fna', '../../data_temp/1890426/38.fna', '../../data_temp/1890426/35.fna', '../../data_temp/1890426/31.fna', '../../data_temp/1890426/28.fna', '../../data_temp/1890426/0.fna', '../../data_temp/1890426/11.fna', '../../data_temp/1890426/34.fna', '../../data_temp/1890426/6.fna', '../../data_temp/1890426/15.fna', '../../data_temp/1890426/36.fna', '../../data_temp/3071915/0.fna', '../../data_temp/320796/7.fna', '../../data_temp/320796/2.fna', '../../data_temp/320796/9.fna', '../../data_temp/320796/5.fna', '../../data_temp/320796/4.fna', '../../data_temp/320796/1.fna', '../../data_temp/320796/8.fna', '../../data_temp/320796/3.fna', '../../data_temp/320796/0.fna', '../../data_temp/320796/6.fna', '../../data_temp/3082771/7.fna', '../../data_temp/3082771/2.fna', '../../data_temp/3082771/9.fna', '../../data_temp/3082771/5.fna', '../../data_temp/3082771/4.fna', '../../data_temp/3082771/1.fna', '../../data_temp/3082771/8.fna', '../../data_temp/3082771/3.fna', '../../data_temp/3082771/0.fna', '../../data_temp/3082771/6.fna', '../../data_temp/1755828/2.fna', '../../data_temp/1755828/1.fna', '../../data_temp/1755828/0.fna', '../../data_temp/85019/7.fna', '../../data_temp/85019/2.fna', '../../data_temp/85019/9.fna', '../../data_temp/85019/5.fna', '../../data_temp/85019/4.fna', '../../data_temp/85019/1.fna', '../../data_temp/85019/8.fna', '../../data_temp/85019/3.fna', '../../data_temp/85019/0.fna', '../../data_temp/85019/6.fna', '../../data_temp/41295/7.fna', '../../data_temp/41295/2.fna', '../../data_temp/41295/9.fna', '../../data_temp/41295/5.fna', '../../data_temp/41295/4.fna', '../../data_temp/41295/1.fna', '../../data_temp/41295/8.fna', '../../data_temp/41295/3.fna', '../../data_temp/41295/0.fna', '../../data_temp/41295/6.fna', '../../data_temp/2005525/7.fna', '../../data_temp/2005525/2.fna', '../../data_temp/2005525/9.fna', '../../data_temp/2005525/5.fna', '../../data_temp/2005525/4.fna', '../../data_temp/2005525/1.fna', '../../data_temp/2005525/8.fna', '../../data_temp/2005525/3.fna', '../../data_temp/2005525/0.fna', '../../data_temp/2005525/6.fna', '../../data_temp/2715733/1.fna', '../../data_temp/2715733/0.fna', '../../data_temp/2448023/7.fna', '../../data_temp/2448023/2.fna', '../../data_temp/2448023/9.fna', '../../data_temp/2448023/5.fna', '../../data_temp/2448023/4.fna', '../../data_temp/2448023/1.fna', '../../data_temp/2448023/8.fna', '../../data_temp/2448023/3.fna', '../../data_temp/2448023/0.fna', '../../data_temp/2448023/6.fna', '../../data_temp/693073/7.fna', '../../data_temp/693073/2.fna', '../../data_temp/693073/9.fna', '../../data_temp/693073/5.fna', '../../data_temp/693073/4.fna', '../../data_temp/693073/1.fna', '../../data_temp/693073/8.fna', '../../data_temp/693073/3.fna', '../../data_temp/693073/0.fna', '../../data_temp/693073/6.fna', '../../data_temp/2315421/1.fna', '../../data_temp/2315421/0.fna', '../../data_temp/2829290/0.fna', '../../data_temp/2303730/1.fna', '../../data_temp/2303730/0.fna', '../../data_temp/2944140/0.fna', '../../data_temp/539738/7.fna', '../../data_temp/539738/2.fna', '../../data_temp/539738/9.fna', '../../data_temp/539738/5.fna', '../../data_temp/539738/4.fna', '../../data_temp/539738/1.fna', '../../data_temp/539738/8.fna', '../../data_temp/539738/3.fna', '../../data_temp/539738/0.fna', '../../data_temp/539738/6.fna', '../../data_temp/2723775/2.fna', '../../data_temp/2723775/1.fna', '../../data_temp/2723775/0.fna', '../../data_temp/712/7.fna', '../../data_temp/712/2.fna', '../../data_temp/712/9.fna', '../../data_temp/712/5.fna', '../../data_temp/712/4.fna', '../../data_temp/712/1.fna', '../../data_temp/712/8.fna', '../../data_temp/712/3.fna', '../../data_temp/712/0.fna', '../../data_temp/712/6.fna', '../../data_temp/666507/7.fna', '../../data_temp/666507/2.fna', '../../data_temp/666507/9.fna', '../../data_temp/666507/5.fna', '../../data_temp/666507/4.fna', '../../data_temp/666507/1.fna', '../../data_temp/666507/8.fna', '../../data_temp/666507/3.fna', '../../data_temp/666507/0.fna', '../../data_temp/666507/6.fna', '../../data_temp/1524215/7.fna', '../../data_temp/1524215/2.fna', '../../data_temp/1524215/9.fna', '../../data_temp/1524215/5.fna', '../../data_temp/1524215/4.fna', '../../data_temp/1524215/1.fna', '../../data_temp/1524215/8.fna', '../../data_temp/1524215/3.fna', '../../data_temp/1524215/0.fna', '../../data_temp/1524215/6.fna', '../../data_temp/995019/7.fna', '../../data_temp/995019/2.fna', '../../data_temp/995019/9.fna', '../../data_temp/995019/5.fna', '../../data_temp/995019/4.fna', '../../data_temp/995019/1.fna', '../../data_temp/995019/8.fna', '../../data_temp/995019/3.fna', '../../data_temp/995019/0.fna', '../../data_temp/995019/6.fna', '../../data_temp/3036766/2.fna', '../../data_temp/3036766/4.fna', '../../data_temp/3036766/1.fna', '../../data_temp/3036766/3.fna', '../../data_temp/3036766/0.fna', '../../data_temp/1433378/2.fna', '../../data_temp/1433378/1.fna', '../../data_temp/1433378/3.fna', '../../data_temp/1433378/0.fna', '../../data_temp/41294/7.fna', '../../data_temp/41294/2.fna', '../../data_temp/41294/9.fna', '../../data_temp/41294/5.fna', '../../data_temp/41294/4.fna', '../../data_temp/41294/1.fna', '../../data_temp/41294/8.fna', '../../data_temp/41294/3.fna', '../../data_temp/41294/0.fna', '../../data_temp/41294/6.fna', '../../data_temp/660062/0.fna', '../../data_temp/1042322/7.fna', '../../data_temp/1042322/2.fna', '../../data_temp/1042322/9.fna', '../../data_temp/1042322/5.fna', '../../data_temp/1042322/4.fna', '../../data_temp/1042322/1.fna', '../../data_temp/1042322/8.fna', '../../data_temp/1042322/3.fna', '../../data_temp/1042322/0.fna', '../../data_temp/1042322/6.fna', '../../data_temp/2895623/7.fna', '../../data_temp/2895623/2.fna', '../../data_temp/2895623/9.fna', '../../data_temp/2895623/5.fna', '../../data_temp/2895623/4.fna', '../../data_temp/2895623/1.fna', '../../data_temp/2895623/8.fna', '../../data_temp/2895623/3.fna', '../../data_temp/2895623/0.fna', '../../data_temp/2895623/6.fna', '../../data_temp/3039168/7.fna', '../../data_temp/3039168/2.fna', '../../data_temp/3039168/9.fna', '../../data_temp/3039168/5.fna', '../../data_temp/3039168/4.fna', '../../data_temp/3039168/1.fna', '../../data_temp/3039168/8.fna', '../../data_temp/3039168/3.fna', '../../data_temp/3039168/0.fna', '../../data_temp/3039168/6.fna', '../../data_temp/3016347/0.fna', '../../data_temp/134627/7.fna', '../../data_temp/134627/2.fna', '../../data_temp/134627/9.fna', '../../data_temp/134627/5.fna', '../../data_temp/134627/4.fna', '../../data_temp/134627/1.fna', '../../data_temp/134627/8.fna', '../../data_temp/134627/3.fna', '../../data_temp/134627/0.fna', '../../data_temp/134627/6.fna', '../../data_temp/267894/7.fna', '../../data_temp/267894/2.fna', '../../data_temp/267894/9.fna', '../../data_temp/267894/5.fna', '../../data_temp/267894/4.fna', '../../data_temp/267894/1.fna', '../../data_temp/267894/8.fna', '../../data_temp/267894/3.fna', '../../data_temp/267894/0.fna', '../../data_temp/267894/6.fna', '../../data_temp/3120115/7.fna', '../../data_temp/3120115/2.fna', '../../data_temp/3120115/9.fna', '../../data_temp/3120115/5.fna', '../../data_temp/3120115/4.fna', '../../data_temp/3120115/1.fna', '../../data_temp/3120115/8.fna', '../../data_temp/3120115/3.fna', '../../data_temp/3120115/0.fna', '../../data_temp/3120115/6.fna', '../../data_temp/2786988/0.fna', '../../data_temp/2138247/7.fna', '../../data_temp/2138247/2.fna', '../../data_temp/2138247/9.fna', '../../data_temp/2138247/5.fna', '../../data_temp/2138247/4.fna', '../../data_temp/2138247/1.fna', '../../data_temp/2138247/8.fna', '../../data_temp/2138247/3.fna', '../../data_temp/2138247/0.fna', '../../data_temp/2138247/6.fna', '../../data_temp/2944141/0.fna', '../../data_temp/2762276/2.fna', '../../data_temp/2762276/1.fna', '../../data_temp/2762276/3.fna', '../../data_temp/2762276/0.fna', '../../data_temp/188786/7.fna', '../../data_temp/188786/2.fna', '../../data_temp/188786/9.fna', '../../data_temp/188786/5.fna', '../../data_temp/188786/4.fna', '../../data_temp/188786/1.fna', '../../data_temp/188786/8.fna', '../../data_temp/188786/3.fna', '../../data_temp/188786/0.fna', '../../data_temp/188786/6.fna', '../../data_temp/2900660/0.fna', '../../data_temp/2071613/1.fna', '../../data_temp/2071613/0.fna', '../../data_temp/2038695/0.fna', '../../data_temp/2049/64.fna', '../../data_temp/2049/32.fna', '../../data_temp/2049/99.fna', '../../data_temp/2049/43.fna', '../../data_temp/2049/7.fna', '../../data_temp/2049/98.fna', '../../data_temp/2049/58.fna', '../../data_temp/2049/90.fna', '../../data_temp/2049/14.fna', '../../data_temp/2049/41.fna', '../../data_temp/2049/67.fna', '../../data_temp/2049/73.fna', '../../data_temp/2049/17.fna', '../../data_temp/2049/56.fna', '../../data_temp/2049/50.fna', '../../data_temp/2049/2.fna', '../../data_temp/2049/46.fna', '../../data_temp/2049/52.fna', '../../data_temp/2049/65.fna', '../../data_temp/2049/95.fna', '../../data_temp/2049/45.fna', '../../data_temp/2049/74.fna', '../../data_temp/2049/44.fna', '../../data_temp/2049/12.fna', '../../data_temp/2049/94.fna', '../../data_temp/2049/25.fna', '../../data_temp/2049/60.fna', '../../data_temp/2049/39.fna', '../../data_temp/2049/22.fna', '../../data_temp/2049/48.fna', '../../data_temp/2049/88.fna', '../../data_temp/2049/24.fna', '../../data_temp/2049/9.fna', '../../data_temp/2049/86.fna', '../../data_temp/2049/5.fna', '../../data_temp/2049/89.fna', '../../data_temp/2049/71.fna', '../../data_temp/2049/54.fna', '../../data_temp/2049/23.fna', '../../data_temp/2049/49.fna', '../../data_temp/2049/40.fna', '../../data_temp/2049/18.fna', '../../data_temp/2049/4.fna', '../../data_temp/2049/20.fna', '../../data_temp/2049/63.fna', '../../data_temp/2049/33.fna', '../../data_temp/2049/87.fna', '../../data_temp/2049/80.fna', '../../data_temp/2049/26.fna', '../../data_temp/2049/1.fna', '../../data_temp/2049/47.fna', '../../data_temp/2049/92.fna', '../../data_temp/2049/51.fna', '../../data_temp/2049/37.fna', '../../data_temp/2049/84.fna', '../../data_temp/2049/53.fna', '../../data_temp/2049/70.fna', '../../data_temp/2049/42.fna', '../../data_temp/2049/57.fna', '../../data_temp/2049/72.fna', '../../data_temp/2049/91.fna', '../../data_temp/2049/27.fna', '../../data_temp/2049/19.fna', '../../data_temp/2049/66.fna', '../../data_temp/2049/13.fna', '../../data_temp/2049/16.fna', '../../data_temp/2049/29.fna', '../../data_temp/2049/8.fna', '../../data_temp/2049/21.fna', '../../data_temp/2049/3.fna', '../../data_temp/2049/10.fna', '../../data_temp/2049/30.fna', '../../data_temp/2049/38.fna', '../../data_temp/2049/93.fna', '../../data_temp/2049/78.fna', '../../data_temp/2049/77.fna', '../../data_temp/2049/75.fna', '../../data_temp/2049/35.fna', '../../data_temp/2049/31.fna', '../../data_temp/2049/28.fna', '../../data_temp/2049/82.fna', '../../data_temp/2049/59.fna', '../../data_temp/2049/69.fna', '../../data_temp/2049/62.fna', '../../data_temp/2049/85.fna', '../../data_temp/2049/0.fna', '../../data_temp/2049/79.fna', '../../data_temp/2049/11.fna', '../../data_temp/2049/34.fna', '../../data_temp/2049/6.fna', '../../data_temp/2049/68.fna', '../../data_temp/2049/97.fna', '../../data_temp/2049/15.fna', '../../data_temp/2049/55.fna', '../../data_temp/2049/61.fna', '../../data_temp/2049/36.fna', '../../data_temp/2049/81.fna', '../../data_temp/2049/96.fna', '../../data_temp/2049/76.fna', '../../data_temp/2049/83.fna', '../../data_temp/580372/7.fna', '../../data_temp/580372/2.fna', '../../data_temp/580372/9.fna', '../../data_temp/580372/5.fna', '../../data_temp/580372/4.fna', '../../data_temp/580372/1.fna', '../../data_temp/580372/8.fna', '../../data_temp/580372/3.fna', '../../data_temp/580372/0.fna', '../../data_temp/580372/6.fna', '../../data_temp/2790998/7.fna', '../../data_temp/2790998/2.fna', '../../data_temp/2790998/9.fna', '../../data_temp/2790998/5.fna', '../../data_temp/2790998/4.fna', '../../data_temp/2790998/1.fna', '../../data_temp/2790998/8.fna', '../../data_temp/2790998/3.fna', '../../data_temp/2790998/0.fna', '../../data_temp/2790998/6.fna', '../../data_temp/3050005/1.fna', '../../data_temp/3050005/0.fna', '../../data_temp/2843199/1.fna', '../../data_temp/2843199/0.fna', '../../data_temp/3118653/2.fna', '../../data_temp/3118653/1.fna', '../../data_temp/3118653/0.fna', '../../data_temp/3141701/7.fna', '../../data_temp/3141701/2.fna', '../../data_temp/3141701/9.fna', '../../data_temp/3141701/5.fna', '../../data_temp/3141701/4.fna', '../../data_temp/3141701/1.fna', '../../data_temp/3141701/8.fna', '../../data_temp/3141701/3.fna', '../../data_temp/3141701/0.fna', '../../data_temp/3141701/6.fna', '../../data_temp/2815910/7.fna', '../../data_temp/2815910/2.fna', '../../data_temp/2815910/9.fna', '../../data_temp/2815910/5.fna', '../../data_temp/2815910/4.fna', '../../data_temp/2815910/1.fna', '../../data_temp/2815910/8.fna', '../../data_temp/2815910/3.fna', '../../data_temp/2815910/0.fna', '../../data_temp/2815910/6.fna', '../../data_temp/1198451/0.fna', '../../data_temp/2600303/0.fna', '../../data_temp/188709/7.fna', '../../data_temp/188709/2.fna', '../../data_temp/188709/9.fna', '../../data_temp/188709/5.fna', '../../data_temp/188709/4.fna', '../../data_temp/188709/1.fna', '../../data_temp/188709/8.fna', '../../data_temp/188709/3.fna', '../../data_temp/188709/0.fna', '../../data_temp/188709/6.fna', '../../data_temp/31957/7.fna', '../../data_temp/31957/2.fna', '../../data_temp/31957/9.fna', '../../data_temp/31957/5.fna', '../../data_temp/31957/4.fna', '../../data_temp/31957/1.fna', '../../data_temp/31957/8.fna', '../../data_temp/31957/3.fna', '../../data_temp/31957/0.fna', '../../data_temp/31957/6.fna', '../../data_temp/412032/2.fna', '../../data_temp/412032/1.fna', '../../data_temp/412032/0.fna', '../../data_temp/2917697/0.fna', '../../data_temp/3048035/2.fna', '../../data_temp/3048035/5.fna', '../../data_temp/3048035/4.fna', '../../data_temp/3048035/1.fna', '../../data_temp/3048035/3.fna', '../../data_temp/3048035/0.fna', '../../data_temp/3048035/6.fna', '../../data_temp/2301193/1.fna', '../../data_temp/2301193/0.fna', '../../data_temp/3024225/0.fna', '../../data_temp/1553902/2.fna', '../../data_temp/1553902/1.fna', '../../data_temp/1553902/0.fna', '../../data_temp/2844887/2.fna', '../../data_temp/2844887/1.fna', '../../data_temp/2844887/0.fna', '../../data_temp/622454/2.fna', '../../data_temp/622454/1.fna', '../../data_temp/622454/0.fna', '../../data_temp/33925/7.fna', '../../data_temp/33925/2.fna', '../../data_temp/33925/9.fna', '../../data_temp/33925/5.fna', '../../data_temp/33925/4.fna', '../../data_temp/33925/1.fna', '../../data_temp/33925/8.fna', '../../data_temp/33925/3.fna', '../../data_temp/33925/0.fna', '../../data_temp/33925/6.fna', '../../data_temp/75787/7.fna', '../../data_temp/75787/2.fna', '../../data_temp/75787/9.fna', '../../data_temp/75787/5.fna', '../../data_temp/75787/4.fna', '../../data_temp/75787/1.fna', '../../data_temp/75787/8.fna', '../../data_temp/75787/3.fna', '../../data_temp/75787/0.fna', '../../data_temp/75787/6.fna', '../../data_temp/2951105/0.fna', '../../data_temp/909930/7.fna', '../../data_temp/909930/2.fna', '../../data_temp/909930/9.fna', '../../data_temp/909930/5.fna', '../../data_temp/909930/4.fna', '../../data_temp/909930/1.fna', '../../data_temp/909930/8.fna', '../../data_temp/909930/3.fna', '../../data_temp/909930/0.fna', '../../data_temp/909930/6.fna', '../../data_temp/485255/1.fna', '../../data_temp/485255/0.fna', '../../data_temp/1747769/2.fna', '../../data_temp/1747769/1.fna', '../../data_temp/1747769/0.fna', '../../data_temp/2767072/0.fna', '../../data_temp/453228/7.fna', '../../data_temp/453228/2.fna', '../../data_temp/453228/9.fna', '../../data_temp/453228/5.fna', '../../data_temp/453228/4.fna', '../../data_temp/453228/1.fna', '../../data_temp/453228/8.fna', '../../data_temp/453228/3.fna', '../../data_temp/453228/0.fna', '../../data_temp/453228/6.fna', '../../data_temp/1046/7.fna', '../../data_temp/1046/2.fna', '../../data_temp/1046/9.fna', '../../data_temp/1046/5.fna', '../../data_temp/1046/4.fna', '../../data_temp/1046/1.fna', '../../data_temp/1046/8.fna', '../../data_temp/1046/3.fna', '../../data_temp/1046/0.fna', '../../data_temp/1046/6.fna', '../../data_temp/191033/7.fna', '../../data_temp/191033/2.fna', '../../data_temp/191033/5.fna', '../../data_temp/191033/4.fna', '../../data_temp/191033/1.fna', '../../data_temp/191033/8.fna', '../../data_temp/191033/3.fna', '../../data_temp/191033/0.fna', '../../data_temp/191033/6.fna', '../../data_temp/255526/7.fna', '../../data_temp/255526/2.fna', '../../data_temp/255526/9.fna', '../../data_temp/255526/5.fna', '../../data_temp/255526/4.fna', '../../data_temp/255526/1.fna', '../../data_temp/255526/8.fna', '../../data_temp/255526/3.fna', '../../data_temp/255526/0.fna', '../../data_temp/255526/6.fna', '../../data_temp/2792775/2.fna', '../../data_temp/2792775/4.fna', '../../data_temp/2792775/1.fna', '../../data_temp/2792775/3.fna', '../../data_temp/2792775/0.fna', '../../data_temp/742031/7.fna', '../../data_temp/742031/2.fna', '../../data_temp/742031/9.fna', '../../data_temp/742031/5.fna', '../../data_temp/742031/4.fna', '../../data_temp/742031/1.fna', '../../data_temp/742031/8.fna', '../../data_temp/742031/3.fna', '../../data_temp/742031/0.fna', '../../data_temp/742031/6.fna', '../../data_temp/2066434/7.fna', '../../data_temp/2066434/14.fna', '../../data_temp/2066434/17.fna', '../../data_temp/2066434/2.fna', '../../data_temp/2066434/12.fna', '../../data_temp/2066434/21.zip', '../../data_temp/2066434/9.fna', '../../data_temp/2066434/5.fna', '../../data_temp/2066434/18.fna', '../../data_temp/2066434/4.fna', '../../data_temp/2066434/20.fna', '../../data_temp/2066434/1.fna', '../../data_temp/2066434/19.fna', '../../data_temp/2066434/13.fna', '../../data_temp/2066434/16.fna', '../../data_temp/2066434/8.fna', '../../data_temp/2066434/3.fna', '../../data_temp/2066434/10.fna', '../../data_temp/2066434/0.fna', '../../data_temp/2066434/11.fna', '../../data_temp/2066434/6.fna', '../../data_temp/2066434/15.fna', '../../data_temp/3118655/7.fna', '../../data_temp/3118655/2.fna', '../../data_temp/3118655/9.fna', '../../data_temp/3118655/5.fna', '../../data_temp/3118655/4.fna', '../../data_temp/3118655/1.fna', '../../data_temp/3118655/8.fna', '../../data_temp/3118655/3.fna', '../../data_temp/3118655/0.fna', '../../data_temp/3118655/6.fna', '../../data_temp/85021/7.fna', '../../data_temp/85021/2.fna', '../../data_temp/85021/9.fna', '../../data_temp/85021/5.fna', '../../data_temp/85021/4.fna', '../../data_temp/85021/1.fna', '../../data_temp/85021/8.fna', '../../data_temp/85021/3.fna', '../../data_temp/85021/0.fna', '../../data_temp/85021/6.fna', '../../data_temp/92713/7.fna', '../../data_temp/92713/2.fna', '../../data_temp/92713/9.fna', '../../data_temp/92713/5.fna', '../../data_temp/92713/4.fna', '../../data_temp/92713/1.fna', '../../data_temp/92713/8.fna', '../../data_temp/92713/3.fna', '../../data_temp/92713/0.fna', '../../data_temp/92713/6.fna', '../../data_temp/3039865/7.fna', '../../data_temp/3039865/2.fna', '../../data_temp/3039865/9.fna', '../../data_temp/3039865/5.fna', '../../data_temp/3039865/4.fna', '../../data_temp/3039865/1.fna', '../../data_temp/3039865/8.fna', '../../data_temp/3039865/3.fna', '../../data_temp/3039865/0.fna', '../../data_temp/3039865/6.fna', '../../data_temp/3118728/7.fna', '../../data_temp/3118728/2.fna', '../../data_temp/3118728/5.fna', '../../data_temp/3118728/4.fna', '../../data_temp/3118728/1.fna', '../../data_temp/3118728/8.fna', '../../data_temp/3118728/3.fna', '../../data_temp/3118728/0.fna', '../../data_temp/3118728/6.fna', '../../data_temp/74712/7.fna', '../../data_temp/74712/2.fna', '../../data_temp/74712/9.fna', '../../data_temp/74712/5.fna', '../../data_temp/74712/4.fna', '../../data_temp/74712/1.fna', '../../data_temp/74712/8.fna', '../../data_temp/74712/3.fna', '../../data_temp/74712/0.fna', '../../data_temp/74712/6.fna', '../../data_temp/2805586/7.fna', '../../data_temp/2805586/2.fna', '../../data_temp/2805586/9.fna', '../../data_temp/2805586/5.fna', '../../data_temp/2805586/4.fna', '../../data_temp/2805586/1.fna', '../../data_temp/2805586/8.fna', '../../data_temp/2805586/3.fna', '../../data_temp/2805586/0.fna', '../../data_temp/2805586/6.fna', '../../data_temp/414713/7.fna', '../../data_temp/414713/2.fna', '../../data_temp/414713/5.fna', '../../data_temp/414713/4.fna', '../../data_temp/414713/1.fna', '../../data_temp/414713/8.fna', '../../data_temp/414713/3.fna', '../../data_temp/414713/0.fna', '../../data_temp/414713/6.fna', '../../data_temp/1890499/0.fna', '../../data_temp/1129771/7.fna', '../../data_temp/1129771/2.fna', '../../data_temp/1129771/9.fna', '../../data_temp/1129771/5.fna', '../../data_temp/1129771/4.fna', '../../data_temp/1129771/1.fna', '../../data_temp/1129771/8.fna', '../../data_temp/1129771/3.fna', '../../data_temp/1129771/0.fna', '../../data_temp/1129771/6.fna', '../../data_temp/41297/64.fna', '../../data_temp/41297/32.fna', '../../data_temp/41297/99.fna', '../../data_temp/41297/43.fna', '../../data_temp/41297/7.fna', '../../data_temp/41297/98.fna', '../../data_temp/41297/58.fna', '../../data_temp/41297/90.fna', '../../data_temp/41297/14.fna', '../../data_temp/41297/41.fna', '../../data_temp/41297/67.fna', '../../data_temp/41297/73.fna', '../../data_temp/41297/17.fna', '../../data_temp/41297/56.fna', '../../data_temp/41297/50.fna', '../../data_temp/41297/2.fna', '../../data_temp/41297/46.fna', '../../data_temp/41297/52.fna', '../../data_temp/41297/65.fna', '../../data_temp/41297/95.fna', '../../data_temp/41297/45.fna', '../../data_temp/41297/74.fna', '../../data_temp/41297/44.fna', '../../data_temp/41297/12.fna', '../../data_temp/41297/94.fna', '../../data_temp/41297/25.fna', '../../data_temp/41297/60.fna', '../../data_temp/41297/39.fna', '../../data_temp/41297/22.fna', '../../data_temp/41297/48.fna', '../../data_temp/41297/88.fna', '../../data_temp/41297/24.fna', '../../data_temp/41297/9.fna', '../../data_temp/41297/86.fna', '../../data_temp/41297/5.fna', '../../data_temp/41297/89.fna', '../../data_temp/41297/71.fna', '../../data_temp/41297/54.fna', '../../data_temp/41297/23.fna', '../../data_temp/41297/49.fna', '../../data_temp/41297/40.fna', '../../data_temp/41297/18.fna', '../../data_temp/41297/4.fna', '../../data_temp/41297/20.fna', '../../data_temp/41297/63.fna', '../../data_temp/41297/33.fna', '../../data_temp/41297/87.fna', '../../data_temp/41297/80.fna', '../../data_temp/41297/26.fna', '../../data_temp/41297/1.fna', '../../data_temp/41297/47.fna', '../../data_temp/41297/92.fna', '../../data_temp/41297/51.fna', '../../data_temp/41297/37.fna', '../../data_temp/41297/84.fna', '../../data_temp/41297/53.fna', '../../data_temp/41297/70.fna', '../../data_temp/41297/42.fna', '../../data_temp/41297/57.fna', '../../data_temp/41297/72.fna', '../../data_temp/41297/91.fna', '../../data_temp/41297/27.fna', '../../data_temp/41297/19.fna', '../../data_temp/41297/66.fna', '../../data_temp/41297/13.fna', '../../data_temp/41297/16.fna', '../../data_temp/41297/29.fna', '../../data_temp/41297/8.fna', '../../data_temp/41297/21.fna', '../../data_temp/41297/3.fna', '../../data_temp/41297/10.fna', '../../data_temp/41297/30.fna', '../../data_temp/41297/38.fna', '../../data_temp/41297/93.fna', '../../data_temp/41297/78.fna', '../../data_temp/41297/77.fna', '../../data_temp/41297/75.fna', '../../data_temp/41297/35.fna', '../../data_temp/41297/31.fna', '../../data_temp/41297/28.fna', '../../data_temp/41297/82.fna', '../../data_temp/41297/59.fna', '../../data_temp/41297/69.fna', '../../data_temp/41297/62.fna', '../../data_temp/41297/85.fna', '../../data_temp/41297/0.fna', '../../data_temp/41297/79.fna', '../../data_temp/41297/11.fna', '../../data_temp/41297/34.fna', '../../data_temp/41297/6.fna', '../../data_temp/41297/68.fna', '../../data_temp/41297/97.fna', '../../data_temp/41297/15.fna', '../../data_temp/41297/55.fna', '../../data_temp/41297/61.fna', '../../data_temp/41297/36.fna', '../../data_temp/41297/81.fna', '../../data_temp/41297/96.fna', '../../data_temp/41297/76.fna', '../../data_temp/41297/83.fna', '../../data_temp/2024981/7.fna', '../../data_temp/2024981/2.fna', '../../data_temp/2024981/5.fna', '../../data_temp/2024981/4.fna', '../../data_temp/2024981/1.fna', '../../data_temp/2024981/3.fna', '../../data_temp/2024981/0.fna', '../../data_temp/2024981/6.fna', '../../data_temp/3029089/2.fna', '../../data_temp/3029089/1.fna', '../../data_temp/3029089/3.fna', '../../data_temp/3029089/0.fna', '../../data_temp/3085335/2.fna', '../../data_temp/3085335/1.fna', '../../data_temp/3085335/3.fna', '../../data_temp/3085335/0.fna', '../../data_temp/1191479/1.fna', '../../data_temp/1191479/0.fna', '../../data_temp/85030/7.fna', '../../data_temp/85030/2.fna', '../../data_temp/85030/9.fna', '../../data_temp/85030/5.fna', '../../data_temp/85030/4.fna', '../../data_temp/85030/1.fna', '../../data_temp/85030/8.fna', '../../data_temp/85030/3.fna', '../../data_temp/85030/0.fna', '../../data_temp/85030/6.fna', '../../data_temp/2805400/2.fna', '../../data_temp/2805400/5.fna', '../../data_temp/2805400/4.fna', '../../data_temp/2805400/1.fna', '../../data_temp/2805400/3.fna', '../../data_temp/2805400/0.fna', '../../data_temp/1648490/2.fna', '../../data_temp/1648490/1.fna', '../../data_temp/1648490/0.fna', '../../data_temp/2917811/1.fna', '../../data_temp/2917811/0.fna', '../../data_temp/189773/7.fna', '../../data_temp/189773/2.fna', '../../data_temp/189773/5.fna', '../../data_temp/189773/4.fna', '../../data_temp/189773/1.fna', '../../data_temp/189773/8.fna', '../../data_temp/189773/3.fna', '../../data_temp/189773/0.fna', '../../data_temp/189773/6.fna', '../../data_temp/2048906/7.fna', '../../data_temp/2048906/2.fna', '../../data_temp/2048906/9.fna', '../../data_temp/2048906/5.fna', '../../data_temp/2048906/4.fna', '../../data_temp/2048906/1.fna', '../../data_temp/2048906/8.fna', '../../data_temp/2048906/3.fna', '../../data_temp/2048906/0.fna', '../../data_temp/2048906/6.fna', '../../data_temp/186803/7.fna', '../../data_temp/186803/2.fna', '../../data_temp/186803/9.fna', '../../data_temp/186803/5.fna', '../../data_temp/186803/4.fna', '../../data_temp/186803/1.fna', '../../data_temp/186803/8.fna', '../../data_temp/186803/3.fna', '../../data_temp/186803/0.fna', '../../data_temp/186803/6.zip', '../../data_temp/186803/6.fna', '../../data_temp/2900661/0.fna', '../../data_temp/1182/7.fna', '../../data_temp/1182/2.fna', '../../data_temp/1182/9.fna', '../../data_temp/1182/5.fna', '../../data_temp/1182/4.fna', '../../data_temp/1182/1.fna', '../../data_temp/1182/8.fna', '../../data_temp/1182/3.fna', '../../data_temp/1182/0.fna', '../../data_temp/1182/6.fna', '../../data_temp/2976758/0.fna', '../../data_temp/361606/2.fna', '../../data_temp/361606/5.fna', '../../data_temp/361606/4.fna', '../../data_temp/361606/1.fna', '../../data_temp/361606/3.fna', '../../data_temp/361606/0.fna', '../../data_temp/206349/7.fna', '../../data_temp/206349/2.fna', '../../data_temp/206349/9.fna', '../../data_temp/206349/5.fna', '../../data_temp/206349/4.fna', '../../data_temp/206349/1.fna', '../../data_temp/206349/8.fna', '../../data_temp/206349/3.fna', '../../data_temp/206349/0.fna', '../../data_temp/206349/6.fna', '../../data_temp/1676650/2.fna', '../../data_temp/1676650/1.fna', '../../data_temp/1676650/0.fna', '../../data_temp/2772226/7.fna', '../../data_temp/2772226/2.fna', '../../data_temp/2772226/9.fna', '../../data_temp/2772226/5.fna', '../../data_temp/2772226/4.fna', '../../data_temp/2772226/1.fna', '../../data_temp/2772226/8.fna', '../../data_temp/2772226/3.fna', '../../data_temp/2772226/0.fna', '../../data_temp/2772226/6.fna', '../../data_temp/2941492/2.fna', '../../data_temp/2941492/1.fna', '../../data_temp/2941492/0.fna', '../../data_temp/2945022/2.fna', '../../data_temp/2945022/1.fna', '../../data_temp/2945022/0.fna', '../../data_temp/2008793/7.fna', '../../data_temp/2008793/2.fna', '../../data_temp/2008793/9.fna', '../../data_temp/2008793/5.fna', '../../data_temp/2008793/4.fna', '../../data_temp/2008793/1.fna', '../../data_temp/2008793/8.fna', '../../data_temp/2008793/3.fna', '../../data_temp/2008793/0.fna', '../../data_temp/2008793/6.fna', '../../data_temp/2162846/7.fna', '../../data_temp/2162846/2.fna', '../../data_temp/2162846/9.fna', '../../data_temp/2162846/5.fna', '../../data_temp/2162846/4.fna', '../../data_temp/2162846/1.fna', '../../data_temp/2162846/8.fna', '../../data_temp/2162846/3.fna', '../../data_temp/2162846/0.fna', '../../data_temp/2162846/6.fna', '../../data_temp/2717330/2.fna', '../../data_temp/2717330/1.fna', '../../data_temp/2717330/0.fna', '../../data_temp/539003/7.fna', '../../data_temp/539003/2.fna', '../../data_temp/539003/9.fna', '../../data_temp/539003/5.fna', '../../data_temp/539003/4.fna', '../../data_temp/539003/1.fna', '../../data_temp/539003/8.fna', '../../data_temp/539003/3.fna', '../../data_temp/539003/10.fna', '../../data_temp/539003/0.fna', '../../data_temp/539003/11.fna', '../../data_temp/539003/6.fna', '../../data_temp/1890430/0.fna', '../../data_temp/3029088/7.fna', '../../data_temp/3029088/2.fna', '../../data_temp/3029088/9.fna', '../../data_temp/3029088/5.fna', '../../data_temp/3029088/4.fna', '../../data_temp/3029088/1.fna', '../../data_temp/3029088/8.fna', '../../data_temp/3029088/3.fna', '../../data_temp/3029088/0.fna', '../../data_temp/3029088/6.fna', '../../data_temp/1890449/64.fna', '../../data_temp/1890449/32.fna', '../../data_temp/1890449/43.fna', '../../data_temp/1890449/7.fna', '../../data_temp/1890449/58.fna', '../../data_temp/1890449/14.fna', '../../data_temp/1890449/41.fna', '../../data_temp/1890449/67.fna', '../../data_temp/1890449/73.fna', '../../data_temp/1890449/17.fna', '../../data_temp/1890449/56.fna', '../../data_temp/1890449/50.fna', '../../data_temp/1890449/2.fna', '../../data_temp/1890449/46.fna', '../../data_temp/1890449/52.fna', '../../data_temp/1890449/65.fna', '../../data_temp/1890449/45.fna', '../../data_temp/1890449/74.fna', '../../data_temp/1890449/44.fna', '../../data_temp/1890449/12.fna', '../../data_temp/1890449/25.fna', '../../data_temp/1890449/60.fna', '../../data_temp/1890449/39.fna', '../../data_temp/1890449/22.fna', '../../data_temp/1890449/48.fna', '../../data_temp/1890449/24.fna', '../../data_temp/1890449/9.fna', '../../data_temp/1890449/5.fna', '../../data_temp/1890449/71.fna', '../../data_temp/1890449/54.fna', '../../data_temp/1890449/23.fna', '../../data_temp/1890449/49.fna', '../../data_temp/1890449/40.fna', '../../data_temp/1890449/18.fna', '../../data_temp/1890449/4.fna', '../../data_temp/1890449/20.fna', '../../data_temp/1890449/63.fna', '../../data_temp/1890449/33.fna', '../../data_temp/1890449/80.fna', '../../data_temp/1890449/26.fna', '../../data_temp/1890449/1.fna', '../../data_temp/1890449/47.fna', '../../data_temp/1890449/51.fna', '../../data_temp/1890449/37.fna', '../../data_temp/1890449/53.fna', '../../data_temp/1890449/70.fna', '../../data_temp/1890449/42.fna', '../../data_temp/1890449/57.fna', '../../data_temp/1890449/72.fna', '../../data_temp/1890449/27.fna', '../../data_temp/1890449/19.fna', '../../data_temp/1890449/66.fna', '../../data_temp/1890449/13.fna', '../../data_temp/1890449/16.fna', '../../data_temp/1890449/29.fna', '../../data_temp/1890449/8.fna', '../../data_temp/1890449/21.fna', '../../data_temp/1890449/3.fna', '../../data_temp/1890449/10.fna', '../../data_temp/1890449/30.fna', '../../data_temp/1890449/38.fna', '../../data_temp/1890449/78.fna', '../../data_temp/1890449/77.fna', '../../data_temp/1890449/75.fna', '../../data_temp/1890449/35.fna', '../../data_temp/1890449/31.fna', '../../data_temp/1890449/28.fna', '../../data_temp/1890449/59.fna', '../../data_temp/1890449/69.fna', '../../data_temp/1890449/62.fna', '../../data_temp/1890449/0.fna', '../../data_temp/1890449/79.fna', '../../data_temp/1890449/11.fna', '../../data_temp/1890449/34.fna', '../../data_temp/1890449/6.fna', '../../data_temp/1890449/68.fna', '../../data_temp/1890449/15.fna', '../../data_temp/1890449/55.fna', '../../data_temp/1890449/61.fna', '../../data_temp/1890449/36.fna', '../../data_temp/1890449/76.fna', '../../data_temp/2493628/2.fna', '../../data_temp/2493628/1.fna', '../../data_temp/2493628/0.fna', '../../data_temp/3120748/2.fna', '../../data_temp/3120748/1.fna', '../../data_temp/3120748/0.fna', '../../data_temp/449732/7.fna', '../../data_temp/449732/2.fna', '../../data_temp/449732/9.fna', '../../data_temp/449732/5.fna', '../../data_temp/449732/4.fna', '../../data_temp/449732/1.fna', '../../data_temp/449732/8.fna', '../../data_temp/449732/3.fna', '../../data_temp/449732/0.fna', '../../data_temp/449732/6.fna', '../../data_temp/31/7.fna', '../../data_temp/31/2.fna', '../../data_temp/31/9.fna', '../../data_temp/31/5.fna', '../../data_temp/31/4.fna', '../../data_temp/31/1.fna', '../../data_temp/31/8.fna', '../../data_temp/31/3.fna', '../../data_temp/31/0.fna', '../../data_temp/31/6.fna', '../../data_temp/2692416/0.fna', '../../data_temp/1763524/7.fna', '../../data_temp/1763524/2.fna', '../../data_temp/1763524/9.fna', '../../data_temp/1763524/5.fna', '../../data_temp/1763524/4.fna', '../../data_temp/1763524/1.fna', '../../data_temp/1763524/8.fna', '../../data_temp/1763524/3.fna', '../../data_temp/1763524/0.fna', '../../data_temp/1763524/6.fna', '../../data_temp/2802652/2.fna', '../../data_temp/2802652/4.fna', '../../data_temp/2802652/1.fna', '../../data_temp/2802652/3.fna', '../../data_temp/2802652/0.fna', '../../data_temp/2810280/7.fna', '../../data_temp/2810280/2.fna', '../../data_temp/2810280/9.fna', '../../data_temp/2810280/5.fna', '../../data_temp/2810280/4.fna', '../../data_temp/2810280/1.fna', '../../data_temp/2810280/8.fna', '../../data_temp/2810280/3.fna', '../../data_temp/2810280/0.fna', '../../data_temp/2810280/6.fna', '../../data_temp/2760872/1.fna', '../../data_temp/2760872/0.fna', '../../data_temp/543347/2.fna', '../../data_temp/543347/4.fna', '../../data_temp/543347/1.fna', '../../data_temp/543347/3.fna', '../../data_temp/543347/0.fna', '../../data_temp/186333/7.fna', '../../data_temp/186333/2.fna', '../../data_temp/186333/9.fna', '../../data_temp/186333/5.fna', '../../data_temp/186333/4.fna', '../../data_temp/186333/1.fna', '../../data_temp/186333/8.fna', '../../data_temp/186333/3.fna', '../../data_temp/186333/0.fna', '../../data_temp/186333/6.fna', '../../data_temp/2922432/0.fna', '../../data_temp/768668/2.fna', '../../data_temp/768668/5.fna', '../../data_temp/768668/4.fna', '../../data_temp/768668/1.fna', '../../data_temp/768668/3.fna', '../../data_temp/768668/0.fna', '../../data_temp/768668/6.fna', '../../data_temp/1217100/7.fna', '../../data_temp/1217100/2.fna', '../../data_temp/1217100/9.fna', '../../data_temp/1217100/5.fna', '../../data_temp/1217100/4.fna', '../../data_temp/1217100/1.fna', '../../data_temp/1217100/8.fna', '../../data_temp/1217100/3.fna', '../../data_temp/1217100/0.fna', '../../data_temp/1217100/6.fna', '../../data_temp/1313209/1.fna', '../../data_temp/1313209/0.fna', '../../data_temp/2303527/7.fna', '../../data_temp/2303527/2.fna', '../../data_temp/2303527/5.fna', '../../data_temp/2303527/4.fna', '../../data_temp/2303527/1.fna', '../../data_temp/2303527/8.fna', '../../data_temp/2303527/3.fna', '../../data_temp/2303527/0.fna', '../../data_temp/2303527/6.fna', '../../data_temp/213117/7.fna', '../../data_temp/213117/2.fna', '../../data_temp/213117/5.fna', '../../data_temp/213117/4.fna', '../../data_temp/213117/1.fna', '../../data_temp/213117/8.fna', '../../data_temp/213117/3.fna', '../../data_temp/213117/0.fna', '../../data_temp/213117/6.fna', '../../data_temp/2562242/7.fna', '../../data_temp/2562242/2.fna', '../../data_temp/2562242/5.fna', '../../data_temp/2562242/4.fna', '../../data_temp/2562242/1.fna', '../../data_temp/2562242/3.fna', '../../data_temp/2562242/0.fna', '../../data_temp/2562242/6.fna', '../../data_temp/2997384/1.fna', '../../data_temp/2997384/0.fna', '../../data_temp/3085110/2.fna', '../../data_temp/3085110/1.fna', '../../data_temp/3085110/0.fna', '../../data_temp/1853233/1.fna', '../../data_temp/1853233/0.fna', '../../data_temp/775/7.fna', '../../data_temp/775/2.fna', '../../data_temp/775/9.fna', '../../data_temp/775/5.fna', '../../data_temp/775/4.fna', '../../data_temp/775/1.fna', '../../data_temp/775/8.fna', '../../data_temp/775/3.fna', '../../data_temp/775/0.fna', '../../data_temp/775/6.fna', '../../data_temp/1890428/7.fna', '../../data_temp/1890428/2.fna', '../../data_temp/1890428/9.fna', '../../data_temp/1890428/5.fna', '../../data_temp/1890428/4.fna', '../../data_temp/1890428/1.fna', '../../data_temp/1890428/8.fna', '../../data_temp/1890428/3.fna', '../../data_temp/1890428/0.fna', '../../data_temp/1890428/6.fna', '../../data_temp/2930104/0.fna', '../../data_temp/72276/7.fna', '../../data_temp/72276/2.fna', '../../data_temp/72276/9.fna', '../../data_temp/72276/5.fna', '../../data_temp/72276/4.fna', '../../data_temp/72276/1.fna', '../../data_temp/72276/8.fna', '../../data_temp/72276/3.fna', '../../data_temp/72276/0.fna', '../../data_temp/72276/6.fna', '../../data_temp/1937968/2.fna', '../../data_temp/1937968/5.fna', '../../data_temp/1937968/4.fna', '../../data_temp/1937968/1.fna', '../../data_temp/1937968/3.fna', '../../data_temp/1937968/0.fna', '../../data_temp/1937968/6.fna', '../../data_temp/85015/7.fna', '../../data_temp/85015/2.fna', '../../data_temp/85015/9.fna', '../../data_temp/85015/5.fna', '../../data_temp/85015/4.fna', '../../data_temp/85015/1.fna', '../../data_temp/85015/8.fna', '../../data_temp/85015/3.fna', '../../data_temp/85015/0.fna', '../../data_temp/85015/6.fna', '../../data_temp/1486721/2.fna', '../../data_temp/1486721/1.fna', '../../data_temp/1486721/3.fna', '../../data_temp/1486721/0.fna', '../../data_temp/171552/7.fna', '../../data_temp/171552/2.fna', '../../data_temp/171552/9.fna', '../../data_temp/171552/5.fna', '../../data_temp/171552/4.fna', '../../data_temp/171552/1.fna', '../../data_temp/171552/8.fna', '../../data_temp/171552/3.fna', '../../data_temp/171552/0.fna', '../../data_temp/171552/6.fna', '../../data_temp/335928/7.fna', '../../data_temp/335928/2.fna', '../../data_temp/335928/9.fna', '../../data_temp/335928/5.fna', '../../data_temp/335928/4.fna', '../../data_temp/335928/1.fna', '../../data_temp/335928/8.fna', '../../data_temp/335928/3.fna', '../../data_temp/335928/0.fna', '../../data_temp/335928/6.fna', '../../data_temp/2035710/7.fna', '../../data_temp/2035710/2.fna', '../../data_temp/2035710/5.fna', '../../data_temp/2035710/4.fna', '../../data_temp/2035710/1.fna', '../../data_temp/2035710/3.fna', '../../data_temp/2035710/0.fna', '../../data_temp/2035710/6.fna', '../../data_temp/320583/7.fna', '../../data_temp/320583/2.fna', '../../data_temp/320583/9.fna', '../../data_temp/320583/5.fna', '../../data_temp/320583/4.fna', '../../data_temp/320583/1.fna', '../../data_temp/320583/8.fna', '../../data_temp/320583/3.fna', '../../data_temp/320583/0.fna', '../../data_temp/320583/6.fna', '../../data_temp/2024974/7.fna', '../../data_temp/2024974/2.fna', '../../data_temp/2024974/9.fna', '../../data_temp/2024974/5.fna', '../../data_temp/2024974/4.fna', '../../data_temp/2024974/1.fna', '../../data_temp/2024974/8.fna', '../../data_temp/2024974/3.fna', '../../data_temp/2024974/0.fna', '../../data_temp/2024974/6.fna', '../../data_temp/2808923/2.fna', '../../data_temp/2808923/1.fna', '../../data_temp/2808923/0.fna', '../../data_temp/267892/7.fna', '../../data_temp/267892/2.fna', '../../data_temp/267892/9.fna', '../../data_temp/267892/5.fna', '../../data_temp/267892/4.fna', '../../data_temp/267892/1.fna', '../../data_temp/267892/8.fna', '../../data_temp/267892/3.fna', '../../data_temp/267892/0.fna', '../../data_temp/267892/6.fna', '../../data_temp/2800062/7.fna', '../../data_temp/2800062/2.fna', '../../data_temp/2800062/9.fna', '../../data_temp/2800062/5.fna', '../../data_temp/2800062/4.fna', '../../data_temp/2800062/1.fna', '../../data_temp/2800062/8.fna', '../../data_temp/2800062/3.fna', '../../data_temp/2800062/0.fna', '../../data_temp/2800062/6.fna', '../../data_temp/506/7.fna', '../../data_temp/506/2.fna', '../../data_temp/506/9.fna', '../../data_temp/506/5.fna', '../../data_temp/506/4.fna', '../../data_temp/506/1.fna', '../../data_temp/506/8.fna', '../../data_temp/506/3.fna', '../../data_temp/506/0.fna', '../../data_temp/506/6.fna', '../../data_temp/3029087/32.fna', '../../data_temp/3029087/7.fna', '../../data_temp/3029087/14.fna', '../../data_temp/3029087/17.fna', '../../data_temp/3029087/2.fna', '../../data_temp/3029087/12.fna', '../../data_temp/3029087/25.fna', '../../data_temp/3029087/22.fna', '../../data_temp/3029087/24.fna', '../../data_temp/3029087/9.fna', '../../data_temp/3029087/5.fna', '../../data_temp/3029087/23.fna', '../../data_temp/3029087/18.fna', '../../data_temp/3029087/4.fna', '../../data_temp/3029087/20.fna', '../../data_temp/3029087/33.fna', '../../data_temp/3029087/26.fna', '../../data_temp/3029087/1.fna', '../../data_temp/3029087/27.fna', '../../data_temp/3029087/19.fna', '../../data_temp/3029087/13.fna', '../../data_temp/3029087/16.fna', '../../data_temp/3029087/29.fna', '../../data_temp/3029087/8.fna', '../../data_temp/3029087/21.fna', '../../data_temp/3029087/3.fna', '../../data_temp/3029087/10.fna', '../../data_temp/3029087/30.fna', '../../data_temp/3029087/35.zip', '../../data_temp/3029087/31.fna', '../../data_temp/3029087/28.fna', '../../data_temp/3029087/0.fna', '../../data_temp/3029087/11.fna', '../../data_temp/3029087/34.fna', '../../data_temp/3029087/6.fna', '../../data_temp/3029087/15.fna', '../../data_temp/85025/7.fna', '../../data_temp/85025/2.fna', '../../data_temp/85025/9.fna', '../../data_temp/85025/5.fna', '../../data_temp/85025/4.fna', '../../data_temp/85025/1.fna', '../../data_temp/85025/8.fna', '../../data_temp/85025/3.fna', '../../data_temp/85025/0.fna', '../../data_temp/85025/6.fna', '../../data_temp/3137695/2.fna', '../../data_temp/3137695/1.fna', '../../data_temp/3137695/0.fna', '../../data_temp/31989/7.fna', '../../data_temp/31989/2.fna', '../../data_temp/31989/9.fna', '../../data_temp/31989/5.fna', '../../data_temp/31989/4.fna', '../../data_temp/31989/1.fna', '../../data_temp/31989/8.fna', '../../data_temp/31989/3.fna', '../../data_temp/31989/0.fna', '../../data_temp/31989/6.fna', '../../data_temp/2897348/0.fna', '../../data_temp/3142802/7.fna', '../../data_temp/3142802/2.fna', '../../data_temp/3142802/9.fna', '../../data_temp/3142802/5.fna', '../../data_temp/3142802/4.fna', '../../data_temp/3142802/1.fna', '../../data_temp/3142802/8.fna', '../../data_temp/3142802/3.fna', '../../data_temp/3142802/0.fna', '../../data_temp/3142802/6.fna', '../../data_temp/1853234/0.fna', '../../data_temp/2689614/7.fna', '../../data_temp/2689614/2.fna', '../../data_temp/2689614/9.fna', '../../data_temp/2689614/5.fna', '../../data_temp/2689614/4.fna', '../../data_temp/2689614/1.fna', '../../data_temp/2689614/8.fna', '../../data_temp/2689614/3.fna', '../../data_temp/2689614/0.fna', '../../data_temp/2689614/6.fna', '../../data_temp/2900657/0.fna', '../../data_temp/1892258/0.fna', '../../data_temp/134623/7.fna', '../../data_temp/134623/2.fna', '../../data_temp/134623/9.fna', '../../data_temp/134623/5.fna', '../../data_temp/134623/4.fna', '../../data_temp/134623/1.fna', '../../data_temp/134623/8.fna', '../../data_temp/134623/3.fna', '../../data_temp/134623/0.fna', '../../data_temp/134623/6.fna', '../../data_temp/2785443/2.fna', '../../data_temp/2785443/1.fna', '../../data_temp/2785443/0.fna', '../../data_temp/126/7.fna', '../../data_temp/126/2.fna', '../../data_temp/126/9.fna', '../../data_temp/126/5.fna', '../../data_temp/126/4.fna', '../../data_temp/126/1.fna', '../../data_temp/126/8.fna', '../../data_temp/126/3.fna', '../../data_temp/126/0.fna', '../../data_temp/126/6.fna', '../../data_temp/2887331/2.fna', '../../data_temp/2887331/1.fna', '../../data_temp/2887331/0.fna', '../../data_temp/1655514/7.fna', '../../data_temp/1655514/2.fna', '../../data_temp/1655514/9.fna', '../../data_temp/1655514/5.fna', '../../data_temp/1655514/4.fna', '../../data_temp/1655514/1.fna', '../../data_temp/1655514/8.fna', '../../data_temp/1655514/3.fna', '../../data_temp/1655514/0.fna', '../../data_temp/1655514/6.fna', '../../data_temp/2211325/7.fna', '../../data_temp/2211325/2.fna', '../../data_temp/2211325/9.fna', '../../data_temp/2211325/5.fna', '../../data_temp/2211325/4.fna', '../../data_temp/2211325/1.fna', '../../data_temp/2211325/8.fna', '../../data_temp/2211325/3.fna', '../../data_temp/2211325/0.fna', '../../data_temp/2211325/6.fna', '../../data_temp/84994/7.fna', '../../data_temp/84994/2.fna', '../../data_temp/84994/9.fna', '../../data_temp/84994/5.fna', '../../data_temp/84994/4.fna', '../../data_temp/84994/1.fna', '../../data_temp/84994/8.fna', '../../data_temp/84994/3.fna', '../../data_temp/84994/0.fna', '../../data_temp/84994/6.fna', '../../data_temp/2419832/1.fna', '../../data_temp/2419832/0.fna', '../../data_temp/171551/7.fna', '../../data_temp/171551/2.fna', '../../data_temp/171551/9.fna', '../../data_temp/171551/5.fna', '../../data_temp/171551/4.fna', '../../data_temp/171551/1.fna', '../../data_temp/171551/8.fna', '../../data_temp/171551/3.fna', '../../data_temp/171551/0.fna', '../../data_temp/171551/6.fna', '../../data_temp/203557/7.fna', '../../data_temp/203557/2.fna', '../../data_temp/203557/9.fna', '../../data_temp/203557/5.fna', '../../data_temp/203557/4.fna', '../../data_temp/203557/1.fna', '../../data_temp/203557/8.fna', '../../data_temp/203557/3.fna', '../../data_temp/203557/0.fna', '../../data_temp/203557/6.fna', '../../data_temp/224027/7.fna', '../../data_temp/224027/2.fna', '../../data_temp/224027/9.fna', '../../data_temp/224027/5.fna', '../../data_temp/224027/4.fna', '../../data_temp/224027/1.fna', '../../data_temp/224027/8.fna', '../../data_temp/224027/3.fna', '../../data_temp/224027/0.fna', '../../data_temp/224027/6.fna', '../../data_temp/90627/7.fna', '../../data_temp/90627/2.fna', '../../data_temp/90627/9.fna', '../../data_temp/90627/5.fna', '../../data_temp/90627/4.fna', '../../data_temp/90627/1.fna', '../../data_temp/90627/8.fna', '../../data_temp/90627/3.fna', '../../data_temp/90627/0.fna', '../../data_temp/90627/6.fna', '../../data_temp/2821832/7.fna', '../../data_temp/2821832/2.fna', '../../data_temp/2821832/9.fna', '../../data_temp/2821832/5.fna', '../../data_temp/2821832/4.fna', '../../data_temp/2821832/1.fna', '../../data_temp/2821832/8.fna', '../../data_temp/2821832/3.fna', '../../data_temp/2821832/0.fna', '../../data_temp/2821832/6.fna', '../../data_temp/1505234/0.fna', '../../data_temp/1890500/7.fna', '../../data_temp/1890500/2.fna', '../../data_temp/1890500/9.fna', '../../data_temp/1890500/5.fna', '../../data_temp/1890500/4.fna', '../../data_temp/1890500/1.fna', '../../data_temp/1890500/8.fna', '../../data_temp/1890500/3.fna', '../../data_temp/1890500/0.fna', '../../data_temp/1890500/6.fna', '../../data_temp/224464/7.fna', '../../data_temp/224464/2.fna', '../../data_temp/224464/9.fna', '../../data_temp/224464/5.fna', '../../data_temp/224464/4.fna', '../../data_temp/224464/1.fna', '../../data_temp/224464/8.fna', '../../data_temp/224464/3.fna', '../../data_temp/224464/0.fna', '../../data_temp/224464/6.fna', '../../data_temp/189770/2.fna', '../../data_temp/189770/1.fna', '../../data_temp/189770/3.fna', '../../data_temp/189770/0.fna', '../../data_temp/1331809/7.fna', '../../data_temp/1331809/2.fna', '../../data_temp/1331809/9.fna', '../../data_temp/1331809/5.fna', '../../data_temp/1331809/4.fna', '../../data_temp/1331809/1.fna', '../../data_temp/1331809/8.fna', '../../data_temp/1331809/3.fna', '../../data_temp/1331809/0.fna', '../../data_temp/1331809/6.fna', '../../data_temp/1106/7.fna', '../../data_temp/1106/2.fna', '../../data_temp/1106/9.fna', '../../data_temp/1106/5.fna', '../../data_temp/1106/4.fna', '../../data_temp/1106/1.fna', '../../data_temp/1106/8.fna', '../../data_temp/1106/3.fna', '../../data_temp/1106/0.fna', '../../data_temp/1106/6.fna', '../../data_temp/1840213/7.fna', '../../data_temp/1840213/2.fna', '../../data_temp/1840213/9.fna', '../../data_temp/1840213/5.fna', '../../data_temp/1840213/4.fna', '../../data_temp/1840213/1.fna', '../../data_temp/1840213/8.fna', '../../data_temp/1840213/3.fna', '../../data_temp/1840213/0.fna', '../../data_temp/1840213/6.fna', '../../data_temp/1293609/0.fna', '../../data_temp/2900659/7.fna', '../../data_temp/2900659/2.fna', '../../data_temp/2900659/5.fna', '../../data_temp/2900659/4.fna', '../../data_temp/2900659/1.fna', '../../data_temp/2900659/3.fna', '../../data_temp/2900659/0.fna', '../../data_temp/2900659/6.fna', '../../data_temp/3031144/2.fna', '../../data_temp/3031144/5.fna', '../../data_temp/3031144/4.fna', '../../data_temp/3031144/1.fna', '../../data_temp/3031144/3.fna', '../../data_temp/3031144/0.fna', '../../data_temp/3079752/2.fna', '../../data_temp/3079752/4.fna', '../../data_temp/3079752/1.fna', '../../data_temp/3079752/3.fna', '../../data_temp/3079752/0.fna', '../../data_temp/566277/7.fna', '../../data_temp/566277/2.fna', '../../data_temp/566277/9.fna', '../../data_temp/566277/5.fna', '../../data_temp/566277/4.fna', '../../data_temp/566277/1.fna', '../../data_temp/566277/8.fna', '../../data_temp/566277/3.fna', '../../data_temp/566277/0.fna', '../../data_temp/566277/6.fna', '../../data_temp/2762317/1.fna', '../../data_temp/2762317/0.fna', '../../data_temp/2791015/7.fna', '../../data_temp/2791015/2.fna', '../../data_temp/2791015/9.fna', '../../data_temp/2791015/5.fna', '../../data_temp/2791015/4.fna', '../../data_temp/2791015/1.fna', '../../data_temp/2791015/8.fna', '../../data_temp/2791015/3.fna', '../../data_temp/2791015/0.fna', '../../data_temp/2791015/6.fna', '../../data_temp/2823145/2.fna', '../../data_temp/2823145/5.fna', '../../data_temp/2823145/4.fna', '../../data_temp/2823145/1.fna', '../../data_temp/2823145/3.fna', '../../data_temp/2823145/0.fna', '../../data_temp/809/7.fna', '../../data_temp/809/2.fna', '../../data_temp/809/9.fna', '../../data_temp/809/5.fna', '../../data_temp/809/4.fna', '../../data_temp/809/1.fna', '../../data_temp/809/8.fna', '../../data_temp/809/3.fna', '../../data_temp/809/0.fna', '../../data_temp/809/6.fna', '../../data_temp/186814/7.fna', '../../data_temp/186814/2.fna', '../../data_temp/186814/9.fna', '../../data_temp/186814/5.fna', '../../data_temp/186814/4.fna', '../../data_temp/186814/1.fna', '../../data_temp/186814/8.fna', '../../data_temp/186814/3.fna', '../../data_temp/186814/0.fna', '../../data_temp/186814/6.fna', '../../data_temp/3031625/2.fna', '../../data_temp/3031625/1.fna', '../../data_temp/3031625/0.fna', '../../data_temp/2951104/2.fna', '../../data_temp/2951104/1.fna', '../../data_temp/2951104/0.fna', '../../data_temp/2762284/1.fna', '../../data_temp/2762284/0.fna', '../../data_temp/75682/7.fna', '../../data_temp/75682/2.fna', '../../data_temp/75682/9.fna', '../../data_temp/75682/5.fna', '../../data_temp/75682/4.fna', '../../data_temp/75682/1.fna', '../../data_temp/75682/8.fna', '../../data_temp/75682/3.fna', '../../data_temp/75682/0.fna', '../../data_temp/75682/6.fna', '../../data_temp/267890/7.fna', '../../data_temp/267890/2.fna', '../../data_temp/267890/9.fna', '../../data_temp/267890/5.fna', '../../data_temp/267890/4.fna', '../../data_temp/267890/1.fna', '../../data_temp/267890/8.fna', '../../data_temp/267890/3.fna', '../../data_temp/267890/0.fna', '../../data_temp/267890/6.fna', '../../data_temp/1213/2.fna', '../../data_temp/1213/4.fna', '../../data_temp/1213/1.fna', '../../data_temp/1213/3.fna', '../../data_temp/1213/0.fna', '../../data_temp/3123439/0.fna', '../../data_temp/2008790/7.fna', '../../data_temp/2008790/2.fna', '../../data_temp/2008790/9.fna', '../../data_temp/2008790/5.fna', '../../data_temp/2008790/4.fna', '../../data_temp/2008790/1.fna', '../../data_temp/2008790/8.fna', '../../data_temp/2008790/3.fna', '../../data_temp/2008790/0.fna', '../../data_temp/2008790/6.fna', '../../data_temp/3118658/2.fna', '../../data_temp/3118658/5.fna', '../../data_temp/3118658/4.fna', '../../data_temp/3118658/1.fna', '../../data_temp/3118658/3.fna', '../../data_temp/3118658/0.fna', '../../data_temp/1127830/7.fna', '../../data_temp/1127830/2.fna', '../../data_temp/1127830/9.fna', '../../data_temp/1127830/5.fna', '../../data_temp/1127830/4.fna', '../../data_temp/1127830/1.fna', '../../data_temp/1127830/8.fna', '../../data_temp/1127830/3.fna', '../../data_temp/1127830/0.fna', '../../data_temp/1127830/6.fna', '../../data_temp/1508635/7.fna', '../../data_temp/1508635/2.fna', '../../data_temp/1508635/9.fna', '../../data_temp/1508635/5.fna', '../../data_temp/1508635/4.fna', '../../data_temp/1508635/1.fna', '../../data_temp/1508635/8.fna', '../../data_temp/1508635/3.fna', '../../data_temp/1508635/0.fna', '../../data_temp/1508635/6.fna', '../../data_temp/2886822/7.fna', '../../data_temp/2886822/2.fna', '../../data_temp/2886822/9.fna', '../../data_temp/2886822/5.fna', '../../data_temp/2886822/4.fna', '../../data_temp/2886822/1.fna', '../../data_temp/2886822/8.fna', '../../data_temp/2886822/3.fna', '../../data_temp/2886822/0.fna', '../../data_temp/2886822/6.fna', '../../data_temp/1268/7.fna', '../../data_temp/1268/2.fna', '../../data_temp/1268/9.fna', '../../data_temp/1268/5.fna', '../../data_temp/1268/4.fna', '../../data_temp/1268/1.fna', '../../data_temp/1268/8.fna', '../../data_temp/1268/3.fna', '../../data_temp/1268/0.fna', '../../data_temp/1268/6.fna', '../../data_temp/541019/2.fna', '../../data_temp/541019/1.fna', '../../data_temp/541019/0.fna', '../../data_temp/186827/7.fna', '../../data_temp/186827/2.fna', '../../data_temp/186827/9.fna', '../../data_temp/186827/5.fna', '../../data_temp/186827/4.fna', '../../data_temp/186827/1.fna', '../../data_temp/186827/8.fna', '../../data_temp/186827/3.fna', '../../data_temp/186827/0.fna', '../../data_temp/186827/6.fna', '../../data_temp/31977/7.fna', '../../data_temp/31977/2.fna', '../../data_temp/31977/9.fna', '../../data_temp/31977/5.fna', '../../data_temp/31977/4.fna', '../../data_temp/31977/1.fna', '../../data_temp/31977/8.fna', '../../data_temp/31977/3.fna', '../../data_temp/31977/0.fna', '../../data_temp/31977/6.fna', '../../data_temp/2900655/0.fna', '../../data_temp/1892259/7.fna', '../../data_temp/1892259/2.fna', '../../data_temp/1892259/9.fna', '../../data_temp/1892259/5.fna', '../../data_temp/1892259/4.fna', '../../data_temp/1892259/1.fna', '../../data_temp/1892259/8.fna', '../../data_temp/1892259/3.fna', '../../data_temp/1892259/0.fna', '../../data_temp/1892259/6.fna', '../../data_temp/563835/7.fna', '../../data_temp/563835/2.fna', '../../data_temp/563835/9.fna', '../../data_temp/563835/5.fna', '../../data_temp/563835/4.fna', '../../data_temp/563835/1.fna', '../../data_temp/563835/8.fna', '../../data_temp/563835/3.fna', '../../data_temp/563835/0.fna', '../../data_temp/563835/6.fna', '../../data_temp/2705999/1.fna', '../../data_temp/2705999/0.fna', '../../data_temp/68298/7.fna', '../../data_temp/68298/2.fna', '../../data_temp/68298/9.fna', '../../data_temp/68298/5.fna', '../../data_temp/68298/4.fna', '../../data_temp/68298/1.fna', '../../data_temp/68298/8.fna', '../../data_temp/68298/3.fna', '../../data_temp/68298/0.fna', '../../data_temp/68298/6.fna', '../../data_temp/1813606/7.fna', '../../data_temp/1813606/2.fna', '../../data_temp/1813606/9.fna', '../../data_temp/1813606/5.fna', '../../data_temp/1813606/4.fna', '../../data_temp/1813606/1.fna', '../../data_temp/1813606/8.fna', '../../data_temp/1813606/3.fna', '../../data_temp/1813606/0.fna', '../../data_temp/1813606/6.fna', '../../data_temp/1706375/7.fna', '../../data_temp/1706375/2.fna', '../../data_temp/1706375/9.fna', '../../data_temp/1706375/5.fna', '../../data_temp/1706375/4.fna', '../../data_temp/1706375/1.fna', '../../data_temp/1706375/8.fna', '../../data_temp/1706375/3.fna', '../../data_temp/1706375/0.fna', '../../data_temp/1706375/6.fna', '../../data_temp/2762316/2.fna', '../../data_temp/2762316/4.fna', '../../data_temp/2762316/1.fna', '../../data_temp/2762316/3.fna', '../../data_temp/2762316/0.fna', '../../data_temp/1903411/7.fna', '../../data_temp/1903411/2.fna', '../../data_temp/1903411/9.fna', '../../data_temp/1903411/5.fna', '../../data_temp/1903411/4.fna', '../../data_temp/1903411/1.fna', '../../data_temp/1903411/8.fna', '../../data_temp/1903411/3.fna', '../../data_temp/1903411/0.fna', '../../data_temp/1903411/6.fna', '../../data_temp/596498/7.fna', '../../data_temp/596498/2.fna', '../../data_temp/596498/9.fna', '../../data_temp/596498/5.fna', '../../data_temp/596498/4.fna', '../../data_temp/596498/1.fna', '../../data_temp/596498/8.fna', '../../data_temp/596498/3.fna', '../../data_temp/596498/0.fna', '../../data_temp/596498/6.fna', '../../data_temp/2212440/1.fna', '../../data_temp/2212440/0.fna', '../../data_temp/2036754/7.fna', '../../data_temp/2036754/2.fna', '../../data_temp/2036754/9.fna', '../../data_temp/2036754/5.fna', '../../data_temp/2036754/4.fna', '../../data_temp/2036754/1.fna', '../../data_temp/2036754/8.fna', '../../data_temp/2036754/3.fna', '../../data_temp/2036754/0.fna', '../../data_temp/2036754/6.fna', '../../data_temp/2900654/7.fna', '../../data_temp/2900654/2.fna', '../../data_temp/2900654/9.fna', '../../data_temp/2900654/5.fna', '../../data_temp/2900654/4.fna', '../../data_temp/2900654/1.fna', '../../data_temp/2900654/8.fna', '../../data_temp/2900654/3.fna', '../../data_temp/2900654/0.fna', '../../data_temp/2900654/6.fna', '../../data_temp/689704/7.fna', '../../data_temp/689704/2.fna', '../../data_temp/689704/5.fna', '../../data_temp/689704/4.fna', '../../data_temp/689704/1.fna', '../../data_temp/689704/3.fna', '../../data_temp/689704/0.fna', '../../data_temp/689704/6.fna', '../../data_temp/3031143/7.fna', '../../data_temp/3031143/2.fna', '../../data_temp/3031143/12.fna', '../../data_temp/3031143/9.fna', '../../data_temp/3031143/5.fna', '../../data_temp/3031143/4.fna', '../../data_temp/3031143/1.fna', '../../data_temp/3031143/13.fna', '../../data_temp/3031143/8.fna', '../../data_temp/3031143/3.fna', '../../data_temp/3031143/10.fna', '../../data_temp/3031143/0.fna', '../../data_temp/3031143/11.fna', '../../data_temp/3031143/6.fna', '../../data_temp/186821/7.fna', '../../data_temp/186821/2.fna', '../../data_temp/186821/9.fna', '../../data_temp/186821/5.fna', '../../data_temp/186821/4.fna', '../../data_temp/186821/1.fna', '../../data_temp/186821/8.fna', '../../data_temp/186821/3.fna', '../../data_temp/186821/0.fna', '../../data_temp/186821/6.fna', '../../data_temp/1836792/7.fna', '../../data_temp/1836792/2.fna', '../../data_temp/1836792/9.fna', '../../data_temp/1836792/5.fna', '../../data_temp/1836792/4.fna', '../../data_temp/1836792/1.fna', '../../data_temp/1836792/8.fna', '../../data_temp/1836792/3.fna', '../../data_temp/1836792/0.fna', '../../data_temp/1836792/6.fna', '../../data_temp/2805591/7.fna', '../../data_temp/2805591/2.fna', '../../data_temp/2805591/9.fna', '../../data_temp/2805591/5.fna', '../../data_temp/2805591/4.fna', '../../data_temp/2805591/1.fna', '../../data_temp/2805591/8.fna', '../../data_temp/2805591/3.fna', '../../data_temp/2805591/0.fna', '../../data_temp/2805591/6.fna', '../../data_temp/513556/7.fna', '../../data_temp/513556/2.fna', '../../data_temp/513556/9.fna', '../../data_temp/513556/5.fna', '../../data_temp/513556/4.fna', '../../data_temp/513556/1.fna', '../../data_temp/513556/8.fna', '../../data_temp/513556/3.fna', '../../data_temp/513556/0.fna', '../../data_temp/513556/6.fna', '../../data_temp/2303507/7.fna', '../../data_temp/2303507/2.fna', '../../data_temp/2303507/9.fna', '../../data_temp/2303507/5.fna', '../../data_temp/2303507/4.fna', '../../data_temp/2303507/1.fna', '../../data_temp/2303507/8.fna', '../../data_temp/2303507/3.fna', '../../data_temp/2303507/0.fna', '../../data_temp/2303507/6.fna', '../../data_temp/1853225/7.fna', '../../data_temp/1853225/2.fna', '../../data_temp/1853225/9.fna', '../../data_temp/1853225/5.fna', '../../data_temp/1853225/4.fna', '../../data_temp/1853225/1.fna', '../../data_temp/1853225/8.fna', '../../data_temp/1853225/3.fna', '../../data_temp/1853225/0.fna', '../../data_temp/1853225/6.fna', '../../data_temp/2814874/2.fna', '../../data_temp/2814874/1.fna', '../../data_temp/2814874/0.fna', '../../data_temp/3118657/7.fna', '../../data_temp/3118657/2.fna', '../../data_temp/3118657/5.fna', '../../data_temp/3118657/4.fna', '../../data_temp/3118657/1.fna', '../../data_temp/3118657/3.fna', '../../data_temp/3118657/0.fna', '../../data_temp/3118657/6.fna', '../../data_temp/533207/1.fna', '../../data_temp/533207/0.fna', '../../data_temp/3078170/0.fna', '../../data_temp/2770092/7.fna', '../../data_temp/2770092/2.fna', '../../data_temp/2770092/9.fna', '../../data_temp/2770092/5.fna', '../../data_temp/2770092/4.fna', '../../data_temp/2770092/1.fna', '../../data_temp/2770092/8.fna', '../../data_temp/2770092/3.fna', '../../data_temp/2770092/0.fna', '../../data_temp/2770092/6.fna', '../../data_temp/3133414/1.fna', '../../data_temp/3133414/0.fna', '../../data_temp/412447/1.fna', '../../data_temp/412447/0.fna', '../../data_temp/213421/7.fna', '../../data_temp/213421/2.fna', '../../data_temp/213421/9.fna', '../../data_temp/213421/5.fna', '../../data_temp/213421/4.fna', '../../data_temp/213421/1.fna', '../../data_temp/213421/8.fna', '../../data_temp/213421/3.fna', '../../data_temp/213421/0.fna', '../../data_temp/213421/6.fna', '../../data_temp/2937910/7.fna', '../../data_temp/2937910/2.fna', '../../data_temp/2937910/9.fna', '../../data_temp/2937910/5.fna', '../../data_temp/2937910/4.fna', '../../data_temp/2937910/1.fna', '../../data_temp/2937910/8.fna', '../../data_temp/2937910/3.fna', '../../data_temp/2937910/0.fna', '../../data_temp/2937910/6.fna', '../../data_temp/135620/7.fna', '../../data_temp/135620/2.fna', '../../data_temp/135620/9.fna', '../../data_temp/135620/5.fna', '../../data_temp/135620/4.fna', '../../data_temp/135620/1.fna', '../../data_temp/135620/8.fna', '../../data_temp/135620/3.fna', '../../data_temp/135620/0.fna', '../../data_temp/135620/6.fna', '../../data_temp/225058/7.fna', '../../data_temp/225058/2.fna', '../../data_temp/225058/9.fna', '../../data_temp/225058/5.fna', '../../data_temp/225058/4.fna', '../../data_temp/225058/1.fna', '../../data_temp/225058/8.fna', '../../data_temp/225058/3.fna', '../../data_temp/225058/0.fna', '../../data_temp/225058/6.fna', '../../data_temp/2893042/2.fna', '../../data_temp/2893042/1.fna', '../../data_temp/2893042/3.fna', '../../data_temp/2893042/0.fna', '../../data_temp/1777752/7.fna', '../../data_temp/1777752/14.fna', '../../data_temp/1777752/17.fna', '../../data_temp/1777752/2.fna', '../../data_temp/1777752/12.fna', '../../data_temp/1777752/9.fna', '../../data_temp/1777752/5.fna', '../../data_temp/1777752/4.fna', '../../data_temp/1777752/1.fna', '../../data_temp/1777752/13.fna', '../../data_temp/1777752/16.fna', '../../data_temp/1777752/8.fna', '../../data_temp/1777752/3.fna', '../../data_temp/1777752/10.fna', '../../data_temp/1777752/0.fna', '../../data_temp/1777752/11.fna', '../../data_temp/1777752/6.fna', '../../data_temp/1777752/15.fna', '../../data_temp/2092/64.fna', '../../data_temp/2092/32.fna', '../../data_temp/2092/43.fna', '../../data_temp/2092/7.fna', '../../data_temp/2092/98.fna', '../../data_temp/2092/58.fna', '../../data_temp/2092/90.fna', '../../data_temp/2092/14.fna', '../../data_temp/2092/41.fna', '../../data_temp/2092/67.fna', '../../data_temp/2092/73.fna', '../../data_temp/2092/17.fna', '../../data_temp/2092/56.fna', '../../data_temp/2092/50.fna', '../../data_temp/2092/2.fna', '../../data_temp/2092/46.fna', '../../data_temp/2092/52.fna', '../../data_temp/2092/65.fna', '../../data_temp/2092/95.fna', '../../data_temp/2092/45.fna', '../../data_temp/2092/74.fna', '../../data_temp/2092/44.fna', '../../data_temp/2092/12.fna', '../../data_temp/2092/94.fna', '../../data_temp/2092/25.fna', '../../data_temp/2092/60.fna', '../../data_temp/2092/39.fna', '../../data_temp/2092/22.fna', '../../data_temp/2092/48.fna', '../../data_temp/2092/88.fna', '../../data_temp/2092/24.fna', '../../data_temp/2092/9.fna', '../../data_temp/2092/86.fna', '../../data_temp/2092/5.fna', '../../data_temp/2092/89.fna', '../../data_temp/2092/71.fna', '../../data_temp/2092/54.fna', '../../data_temp/2092/23.fna', '../../data_temp/2092/49.fna', '../../data_temp/2092/40.fna', '../../data_temp/2092/18.fna', '../../data_temp/2092/4.fna', '../../data_temp/2092/20.fna', '../../data_temp/2092/63.fna', '../../data_temp/2092/33.fna', '../../data_temp/2092/87.fna', '../../data_temp/2092/80.fna', '../../data_temp/2092/26.fna', '../../data_temp/2092/1.fna', '../../data_temp/2092/47.fna', '../../data_temp/2092/92.fna', '../../data_temp/2092/51.fna', '../../data_temp/2092/37.fna', '../../data_temp/2092/84.fna', '../../data_temp/2092/53.fna', '../../data_temp/2092/70.fna', '../../data_temp/2092/42.fna', '../../data_temp/2092/57.fna', '../../data_temp/2092/72.fna', '../../data_temp/2092/91.fna', '../../data_temp/2092/27.fna', '../../data_temp/2092/19.fna', '../../data_temp/2092/66.fna', '../../data_temp/2092/13.fna', '../../data_temp/2092/16.fna', '../../data_temp/2092/29.fna', '../../data_temp/2092/8.fna', '../../data_temp/2092/21.fna', '../../data_temp/2092/3.fna', '../../data_temp/2092/10.fna', '../../data_temp/2092/99.zip', '../../data_temp/2092/30.fna', '../../data_temp/2092/38.fna', '../../data_temp/2092/93.fna', '../../data_temp/2092/78.fna', '../../data_temp/2092/77.fna', '../../data_temp/2092/75.fna', '../../data_temp/2092/35.fna', '../../data_temp/2092/31.fna', '../../data_temp/2092/28.fna', '../../data_temp/2092/82.fna', '../../data_temp/2092/59.fna', '../../data_temp/2092/69.fna', '../../data_temp/2092/62.fna', '../../data_temp/2092/85.fna', '../../data_temp/2092/0.fna', '../../data_temp/2092/79.fna', '../../data_temp/2092/11.fna', '../../data_temp/2092/34.fna', '../../data_temp/2092/6.fna', '../../data_temp/2092/68.fna', '../../data_temp/2092/97.fna', '../../data_temp/2092/15.fna', '../../data_temp/2092/55.fna', '../../data_temp/2092/61.fna', '../../data_temp/2092/36.fna', '../../data_temp/2092/81.fna', '../../data_temp/2092/96.fna', '../../data_temp/2092/76.fna', '../../data_temp/2092/83.fna', '../../data_temp/3085095/7.fna', '../../data_temp/3085095/2.fna', '../../data_temp/3085095/9.fna', '../../data_temp/3085095/5.fna', '../../data_temp/3085095/4.fna', '../../data_temp/3085095/1.fna', '../../data_temp/3085095/8.fna', '../../data_temp/3085095/3.fna', '../../data_temp/3085095/0.fna', '../../data_temp/3085095/6.fna', '../../data_temp/1185/2.fna', '../../data_temp/1185/4.fna', '../../data_temp/1185/1.fna', '../../data_temp/1185/3.fna', '../../data_temp/1185/0.fna', '../../data_temp/1921568/1.fna', '../../data_temp/1921568/0.fna', '../../data_temp/2907186/0.fna', '../../data_temp/2897176/7.fna', '../../data_temp/2897176/2.fna', '../../data_temp/2897176/9.fna', '../../data_temp/2897176/5.fna', '../../data_temp/2897176/4.fna', '../../data_temp/2897176/1.fna', '../../data_temp/2897176/8.fna', '../../data_temp/2897176/3.fna', '../../data_temp/2897176/0.fna', '../../data_temp/2897176/6.fna', '../../data_temp/2897178/2.fna', '../../data_temp/2897178/1.fna', '../../data_temp/2897178/0.fna', '../../data_temp/3031659/7.fna', '../../data_temp/3031659/2.fna', '../../data_temp/3031659/9.fna', '../../data_temp/3031659/5.fna', '../../data_temp/3031659/4.fna', '../../data_temp/3031659/1.fna', '../../data_temp/3031659/8.fna', '../../data_temp/3031659/3.fna', '../../data_temp/3031659/0.fna', '../../data_temp/3031659/6.fna', '../../data_temp/2005520/7.fna', '../../data_temp/2005520/2.fna', '../../data_temp/2005520/9.fna', '../../data_temp/2005520/5.fna', '../../data_temp/2005520/4.fna', '../../data_temp/2005520/1.fna', '../../data_temp/2005520/8.fna', '../../data_temp/2005520/3.fna', '../../data_temp/2005520/0.fna', '../../data_temp/2005520/6.fna', '../../data_temp/1903416/7.fna', '../../data_temp/1903416/2.fna', '../../data_temp/1903416/9.fna', '../../data_temp/1903416/5.fna', '../../data_temp/1903416/4.fna', '../../data_temp/1903416/1.fna', '../../data_temp/1903416/8.fna', '../../data_temp/1903416/3.fna', '../../data_temp/1903416/0.fna', '../../data_temp/1903416/6.fna', '../../data_temp/2690158/1.fna', '../../data_temp/2690158/0.fna', '../../data_temp/1643685/7.fna', '../../data_temp/1643685/2.fna', '../../data_temp/1643685/9.fna', '../../data_temp/1643685/5.fna', '../../data_temp/1643685/4.fna', '../../data_temp/1643685/1.fna', '../../data_temp/1643685/8.fna', '../../data_temp/1643685/3.fna', '../../data_temp/1643685/0.fna', '../../data_temp/1643685/6.fna', '../../data_temp/1643949/7.fna', '../../data_temp/1643949/2.fna', '../../data_temp/1643949/9.fna', '../../data_temp/1643949/5.fna', '../../data_temp/1643949/4.fna', '../../data_temp/1643949/1.fna', '../../data_temp/1643949/8.fna', '../../data_temp/1643949/3.fna', '../../data_temp/1643949/0.fna', '../../data_temp/1643949/6.fna', '../../data_temp/72275/7.fna', '../../data_temp/72275/2.fna', '../../data_temp/72275/9.fna', '../../data_temp/72275/5.fna', '../../data_temp/72275/4.fna', '../../data_temp/72275/1.fna', '../../data_temp/72275/8.fna', '../../data_temp/72275/3.fna', '../../data_temp/72275/0.fna', '../../data_temp/72275/6.fna', '../../data_temp/3127005/0.fna', '../../data_temp/1643948/7.fna', '../../data_temp/1643948/2.fna', '../../data_temp/1643948/9.fna', '../../data_temp/1643948/5.fna', '../../data_temp/1643948/4.fna', '../../data_temp/1643948/1.fna', '../../data_temp/1643948/8.fna', '../../data_temp/1643948/3.fna', '../../data_temp/1643948/0.fna', '../../data_temp/1643948/6.fna', '../../data_temp/3085112/1.fna', '../../data_temp/3085112/0.fna', '../../data_temp/3056373/0.fna', '../../data_temp/213121/7.fna', '../../data_temp/213121/2.fna', '../../data_temp/213121/9.fna', '../../data_temp/213121/5.fna', '../../data_temp/213121/4.fna', '../../data_temp/213121/1.fna', '../../data_temp/213121/8.fna', '../../data_temp/213121/3.fna', '../../data_temp/213121/0.fna', '../../data_temp/213121/6.fna', '../../data_temp/1779135/2.fna', '../../data_temp/1779135/4.fna', '../../data_temp/1779135/1.fna', '../../data_temp/1779135/3.fna', '../../data_temp/1779135/0.fna', '../../data_temp/2795691/7.fna', '../../data_temp/2795691/2.fna', '../../data_temp/2795691/9.fna', '../../data_temp/2795691/5.fna', '../../data_temp/2795691/4.fna', '../../data_temp/2795691/1.fna', '../../data_temp/2795691/8.fna', '../../data_temp/2795691/3.fna', '../../data_temp/2795691/0.fna', '../../data_temp/2795691/6.fna', '../../data_temp/203488/7.fna', '../../data_temp/203488/2.fna', '../../data_temp/203488/5.fna', '../../data_temp/203488/4.fna', '../../data_temp/203488/1.fna', '../../data_temp/203488/3.fna', '../../data_temp/203488/0.fna', '../../data_temp/203488/6.fna', '../../data_temp/2070/7.fna', '../../data_temp/2070/2.fna', '../../data_temp/2070/9.fna', '../../data_temp/2070/5.fna', '../../data_temp/2070/4.fna', '../../data_temp/2070/1.fna', '../../data_temp/2070/8.fna', '../../data_temp/2070/3.fna', '../../data_temp/2070/0.fna', '../../data_temp/2070/6.fna', '../../data_temp/45401/7.fna', '../../data_temp/45401/2.fna', '../../data_temp/45401/9.fna', '../../data_temp/45401/5.fna', '../../data_temp/45401/4.fna', '../../data_temp/45401/1.fna', '../../data_temp/45401/5.zip', '../../data_temp/45401/8.fna', '../../data_temp/45401/3.fna', '../../data_temp/45401/0.fna', '../../data_temp/45401/6.fna', '../../data_temp/2900656/0.fna', '../../data_temp/2805416/2.fna', '../../data_temp/2805416/1.fna', '../../data_temp/2805416/3.fna', '../../data_temp/2805416/0.fna', '../../data_temp/3024551/7.fna', '../../data_temp/3024551/2.fna', '../../data_temp/3024551/5.fna', '../../data_temp/3024551/4.fna', '../../data_temp/3024551/1.fna', '../../data_temp/3024551/3.fna', '../../data_temp/3024551/0.fna', '../../data_temp/3024551/6.fna', '../../data_temp/2682227/1.fna', '../../data_temp/2682227/0.fna', '../../data_temp/2066485/2.fna', '../../data_temp/2066485/1.fna', '../../data_temp/2066485/3.fna', '../../data_temp/2066485/0.fna', '../../data_temp/2831100/7.fna', '../../data_temp/2831100/2.fna', '../../data_temp/2831100/9.fna', '../../data_temp/2831100/5.fna', '../../data_temp/2831100/4.fna', '../../data_temp/2831100/1.fna', '../../data_temp/2831100/8.fna', '../../data_temp/2831100/3.fna', '../../data_temp/2831100/0.fna', '../../data_temp/2831100/6.fna', '../../data_temp/2904715/2.fna', '../../data_temp/2904715/1.fna', '../../data_temp/2904715/0.fna', '../../data_temp/2844601/2.fna', '../../data_temp/2844601/1.fna', '../../data_temp/2844601/3.fna', '../../data_temp/2844601/0.fna', '../../data_temp/2793121/0.fna', '../../data_temp/1508536/0.fna', '../../data_temp/28256/7.fna', '../../data_temp/28256/2.fna', '../../data_temp/28256/9.fna', '../../data_temp/28256/5.fna', '../../data_temp/28256/4.fna', '../../data_temp/28256/1.fna', '../../data_temp/28256/8.fna', '../../data_temp/28256/3.fna', '../../data_temp/28256/0.fna', '../../data_temp/28256/6.fna', '../../data_temp/2691358/32.fna', '../../data_temp/2691358/7.fna', '../../data_temp/2691358/14.fna', '../../data_temp/2691358/41.fna', '../../data_temp/2691358/17.fna', '../../data_temp/2691358/2.fna', '../../data_temp/2691358/12.fna', '../../data_temp/2691358/25.fna', '../../data_temp/2691358/39.fna', '../../data_temp/2691358/22.fna', '../../data_temp/2691358/24.fna', '../../data_temp/2691358/9.fna', '../../data_temp/2691358/5.fna', '../../data_temp/2691358/23.fna', '../../data_temp/2691358/40.fna', '../../data_temp/2691358/18.fna', '../../data_temp/2691358/4.fna', '../../data_temp/2691358/20.fna', '../../data_temp/2691358/33.fna', '../../data_temp/2691358/26.fna', '../../data_temp/2691358/1.fna', '../../data_temp/2691358/37.fna', '../../data_temp/2691358/42.fna', '../../data_temp/2691358/27.fna', '../../data_temp/2691358/19.fna', '../../data_temp/2691358/13.fna', '../../data_temp/2691358/16.fna', '../../data_temp/2691358/29.fna', '../../data_temp/2691358/8.fna', '../../data_temp/2691358/21.fna', '../../data_temp/2691358/3.fna', '../../data_temp/2691358/10.fna', '../../data_temp/2691358/30.fna', '../../data_temp/2691358/38.fna', '../../data_temp/2691358/35.fna', '../../data_temp/2691358/31.fna', '../../data_temp/2691358/28.fna', '../../data_temp/2691358/0.fna', '../../data_temp/2691358/11.fna', '../../data_temp/2691358/34.fna', '../../data_temp/2691358/6.fna', '../../data_temp/2691358/15.fna', '../../data_temp/2691358/36.fna', '../../data_temp/2603322/2.fna', '../../data_temp/2603322/5.fna', '../../data_temp/2603322/4.fna', '../../data_temp/2603322/1.fna', '../../data_temp/2603322/3.fna', '../../data_temp/2603322/0.fna', '../../data_temp/2603322/6.fna', '../../data_temp/1970190/7.fna', '../../data_temp/1970190/2.fna', '../../data_temp/1970190/9.fna', '../../data_temp/1970190/5.fna', '../../data_temp/1970190/4.fna', '../../data_temp/1970190/1.fna', '../../data_temp/1970190/8.fna', '../../data_temp/1970190/3.fna', '../../data_temp/1970190/0.fna', '../../data_temp/1970190/6.fna']

In [10]:
print(checked_genome_files)
print(len(checked_genome_files))

['../../data_temp/388449/7.fna', '../../data_temp/388449/2.fna', '../../data_temp/388449/9.fna', '../../data_temp/388449/5.fna', '../../data_temp/388449/4.fna', '../../data_temp/388449/1.fna', '../../data_temp/388449/8.fna', '../../data_temp/388449/3.fna', '../../data_temp/388449/0.fna', '../../data_temp/388449/6.fna', '../../data_temp/1902584/1.fna', '../../data_temp/1902584/0.fna', '../../data_temp/1331736/7.fna', '../../data_temp/1331736/2.fna', '../../data_temp/1331736/9.fna', '../../data_temp/1331736/5.fna', '../../data_temp/1331736/4.fna', '../../data_temp/1331736/1.fna', '../../data_temp/1331736/8.fna', '../../data_temp/1331736/3.fna', '../../data_temp/1331736/0.fna', '../../data_temp/1331736/6.fna', '../../data_temp/1892252/7.fna', '../../data_temp/1892252/2.fna', '../../data_temp/1892252/9.fna', '../../data_temp/1892252/5.fna', '../../data_temp/1892252/4.fna', '../../data_temp/1892252/1.fna', '../../data_temp/1892252/8.fna', '../../data_temp/1892252/3.fna', '../../data_temp/18

In [11]:

def get_genome_length(genome_file):
    genome_kmer = KMerSet(1)
    genome_kmer.insert_file(genome_file)
    return genome_kmer.length

def compute_length_parallel(genome_files):
    with Pool(max_pool) as p:
        return p.starmap(get_genome_length,([(g_file,) for g_file in genome_files]))

    
checked_genome_lengths = compute_length_parallel(checked_genome_files[:lim_file])


In [12]:
# ortho_vals = [] 

# def compute_ortho_ani_parallel(genome_files):
#     args = [(g1,g2) for g1,g2 in zip(genome_files[:lim_file],genome_files[1:lim_file])]
#     with Pool(max_pool) as p:
#         return p.starmap(compute_ortho_ani,args)
    
# ortho_vals = compute_ortho_ani_parallel(checked_genome_files)
# print(ortho_vals)

In [13]:
# kmer_length = 21
# kmer_class = FracMinHash
    
# kmer_vals = [[]] * 40
# for kmer_length in range(10,39+1):
#     kmer_vals[kmer_length] = compute_kmer_ani_parallel(checked_genome_files,kmer_length,kmer_class,poly_cond)

# print(kmer_vals)

In [16]:
def get_cmash(genome_file,cond,k):
    cmash_kmer = CMashKmerSet(cond,k)
    cmash_kmer.insert_file(genome_file)
    return cmash_kmer

def compute_cmash_parallel(genome_files, initial_kmer_length, condition):
    args = [(g,condition,initial_kmer_length) for g in genome_files]
    with Pool(max_pool) as p:
        return p.starmap(get_cmash,args)
    
def compute_cmash_ani(cmash1, cmash2, main_kmer_length, low_kmer_length):
    kspec_containment = [cmash1.k_specific_containment(cmash2,k_len) for k_len in range(5,41)]
    for k_len in range(6,41):
        print(k_len,f"Containment: {kspec_containment[k_len-5]}",f"Ratio = {kspec_containment[k_len-5]/kspec_containment[k_len-6] if kspec_containment[k_len-6] > 0 else 0}")

    main_containment = cmash1.k_specific_containment(cmash2,main_kmer_length)
    low_containment = cmash1.k_specific_containment(cmash2,low_kmer_length)
    if main_containment <= 0:
        return 0
    ani_estimate = main_containment / ((main_containment / low_containment) ** ((main_kmer_length - 1)/(main_kmer_length - low_kmer_length)))
    return ani_estimate

def compute_cmash_ani_series(cmash_kmer_sets, main_kmer_length, low_kmer_length):
    return [
        compute_cmash_ani(c1,c2,main_kmer_length,low_kmer_length)
        for c1,c2 in zip(cmash_kmer_sets,cmash_kmer_sets[1:])
    ]

def compute_cmash_ani_parallel(cmash_kmer_sets, main_kmer_length, low_kmer_length):
    args = [(c1,c2,main_kmer_length,low_kmer_length) for c1,c2 in zip(cmash_kmer_sets,cmash_kmer_sets[1:])]
    with Pool(max_pool) as p:
        return p.starmap(compute_cmash_ani,args)


In [17]:
main_kmer_length = 41
low_kmer_length = 20

cmash_kmer_sets = compute_cmash_parallel(checked_genome_files[:lim_file],main_kmer_length,poly_cond)
# cmash_ani = compute_cmash_ani_parallel(cmash_kmer_sets,main_kmer_length,low_kmer_length)
cmash_ani = compute_cmash_ani_series(cmash_kmer_sets,main_kmer_length,low_kmer_length)

6 Containment: 0.9994877049180327 Ratio = 0.9994877049180327
7 Containment: 0.9973203939745076 Ratio = 0.9978315781846432
8 Containment: 0.9772965778494459 Ratio = 0.9799223837735204
9 Containment: 0.8559267227156941 Ratio = 0.8758106209674572
10 Containment: 0.5310807966785926 Ratio = 0.6204746067438733
11 Containment: 0.21847890170976164 Ratio = 0.4113854296298044
12 Containment: 0.071449266031419 Ratio = 0.32703050716693827
13 Containment: 0.020908549890894662 Ratio = 0.29263491498569577
14 Containment: 0.005560206612222977 Ratio = 0.26592980580850123
15 Containment: 0.001710978648248021 Ratio = 0.30771853774044733
16 Containment: 0.00048066694508577145 Ratio = 0.28093100143473837
17 Containment: 0.00010241704219582139 Ratio = 0.21307277990073942
18 Containment: 5.5142859393586095e-05 Ratio = 0.5384148791189746
19 Containment: 2.363116478011201e-05 Ratio = 0.4285444215259656
20 Containment: 2.3630420227639716e-05 Ratio = 0.9999684927730299
21 Containment: 1.575311714805567e-05 Ratio

In [ ]:
# print(cmash_ani)

In [ ]:
ortho_vals = [0.6365614184397164, 0.6844516210045662, 0.877343023308958, 0.6291772486772487, 0.629662520661157, 0.6879192810150376, 0.689969785670545, 0.6883388260869565, 0.7031486872727273, 0.5998172549019608, 0.7113680276816609, 0.6274515384615384, 0.7458501844941957, 0.8596176090116279, 0.746394787008708, 0.7449114282330649, 0.7452413389021479, 0.8185315478026214, 0.7445563756368689, 0.8153205903145988, 0.7458778778937812, 0.6237357333333333, 0.6924438438914027, 0.9074765761743169, 0.6796802946768061, 0.6796657196087564, 0.9767430027998133, 0.6767753015616585, 0.6727879976442874, 0.6999582350292661, 0.6815503967065868, 0.6243225471698113, 0.7410200523731587, 0.7275068041634541, 0.6121577419354839, 0.7543466666666667, 0.749216392733564, 0.7252024251805985, 0.6260803465346535, 0.8673194349206349, 0.9840955604938272, 0.877905855774985, 0.7003834817813765, 0.6992046968403074, 0.7218278381458967, 0.727200487197724, 0.7024489756507137, 0.6972845484477893, 0.5920594736842105, 0.7647226716016151, 0.7515077179135209, 0.7214189697908598, 0.736157540091638, 0.7790928778467909, 0.7404318734177215, 0.7354144406651549, 0.9858289143341815, 0.7163618260869565, 0.6097007317073171, 0.681517982017982, 0.6785804978118162, 0.7048418660488627, 0.622686052631579, 0.634826, 0.6132984210526315, 0.6249170731707318, 0.636704945054945, 0.6353019636363636, 0.9703001290217214, 0.6307242156862745, 0.6216462385321101, 0.6350808979591837, 0.6318313142857143, 0.6329657389162562, 0.6283045562130177, 0.7795999227129338, 0.7470898722415795, 0.7466438034682081, 0.8923565830396019, 0.6946489893359186, 0.69821302649304, 0.8003807548966267, 0.8075094737525987, 0.6998963124696749, 0.6237074050632911, 0.7818334710121545, 0.8590762863122645, 0.7453148082901554, 0.738414434565157, 0.8572863769740193, 0.767964388942774, 0.7701931763870815, 0.6248613934426229, 0.7107778084415585, 0.6767132367549669, 0.6759318514644351, 0.70356546875, 0.684938619246862, 0.6885777684563759, 0.6682699784172662, 0.6757629450261781, 0.9835857282981854, 0.617999, 0.7805837572254335, 0.6908734015594542, 0.6890211837748345, 0.8881078381437465, 0.8814704594921403, 0.7356880669398908, 0.6261563380281691, 0.7067916117045983, 0.9237763225172074, 0.923071933066933, 0.7079531717647058, 0.7205453870387039, 0.8816956688005391, 0.5992182352941177, 0.9676724927829099, 0.9899802985285795, 0.7157309301745636, 0.7142784224759615, 0.7103646534368708, 0.9900862930186825, 0.9916532774734128, 0.9904346118721461, 0.7234950475737393, 0.613686062992126, 0.6383655135135136, 0.6267450306748467, 0.6534174744897959, 0.6097630120481927, 0.6090752525252525, 0.629092375, 0.6439745794392523, 0.6391186666666666, 0.60117, 0.6178786046511628, 0.7206897801009373, 0.836663720508167, 0.7143208764271323, 0.7096804977029096, 0.7332883159268929, 0.7237311472715319, 0.6163925, 0.6794533218390805, 0.6791106809338522, 0.6782036274509804, 0.636155960591133, 0.6253726984126984, 0.6372540322580645, 0.6531448805460751, 0.6412430078125, 0.6463333333333333, 0.6079796774193549, 0.792832143928036, 0.6228279716024341, 0.6462922509578544, 0.9957559115462525, 0.657002083958021, 0.6603977835820896, 0.7095957612267251, 0.6423100173010381, 0.6676611943319838, 0.6080696428571428, 0.6649464542936289, 0.6305124242424243, 0.703977414403778, 0.6814396531791908, 0.6808196166134185, 0.7152756032770605, 0.6996768002322881, 0.683837072622108, 0.6919769139722863, 0.7058859294117648, 0.6975438417926566, 0.623853125, 0.6237575757575757, 0.6426259495798319, 0.9731284682596291, 0.7202124055232558, 0.6397272903225807, 0.6401538261851015, 0.6207852434456929, 0.6395409859154929, 0.6442872466216216, 0.6447023977695168, 0.6192969798657718, 0.7094020102163462, 0.7089650775656324, 0.7040302692743764, 0.6874714262295082, 0.6897051589704769, 0.7471348330596607, 0.6797997849462366, 0.6878957205240175, 0.810713957027753, 0.6751919309927361, 0.6733814077669903, 0.7292615568110483]

In [ ]:
kmer_vals = [[], [], [], [], [], [], [], [], [], [], [0.9997690263448022, 0.9957648508388901, 0.9986973683848038, 0.9815901524059681, 0.9942757816096581, 0.9941694414214921, 0.9957458544432762, 0.99991769885553, 0.9980418412678892, 0.9730671241286697, 0.9820729452867621, 0.9532732076357401, 0.9746821699716747, 0.9863949663217925, 0.9815080025471447, 0.9853462906276877, 0.9728287728491452, 0.9927668637629175, 0.9853252019132832, 0.9763670181410675, 0.9870500630112566, 0.9935383923669654, 0.9976159018875139, 0.998655318843455, 0.9932477339650116, 0.9987121523608109, 0.998947060384032, 0.9972447230722508, 0.993975834612432, 0.9910246257774825, 0.9988491884991061, 0.9991107819579933, 0.9962150682679626, 0.9981249200856798, 0.9661489153285523, 0.9843266480394107, 0.9818915172920706, 0.9872859905749682, 0.9667044086583043, 0.9827718236591162, 0.9950392881923221, 0.9817073523242666, 0.9901436325175954, 0.9752985444319672, 0.9768658576526005, 0.9853434112559355, 0.9845668543422685, 0.9767622786166879, 0.9711689536278102, 0.9796545556719493, 0.977527586022518, 0.9899571540512329, 0.9875746931463308, 0.9697423828496369, 0.9867093978607908, 0.9828192145453313, 0.9796680168837288, 0.9900119042849412, 0.9576412168782851, 0.9937239919137009, 0.9656835496790324, 0.9796318129319357, 0.9794457203475346, 0.9894220983603121, 0.9626922815685659, 0.9739146063358891, 0.9694948597905578, 0.9901102692044548, 0.9928090987524478, 0.9863763650758498, 0.9717846620328414, 0.9952769145908327, 0.9380996294547708, 0.9879347814066982, 0.9980844952535809, 0.9975970019327287, 0.9882267108935153, 0.9986669433410572, 0.9981983302507081, 0.9980351826566278, 0.998402522252063, 0.9984847593474725, 0.997647842446051, 0.998797339819745, 0.9685944593388071, 0.98411416808669, 0.9835512484056589, 0.9961436755202716, 0.9712578601688175, 0.9859510652387475, 0.9866379105077673, 0.9820661068551996, 0.9878508338495273, 0.9870827184298198, 0.9939529758715883, 0.9876903772138117, 0.9934188456297418, 0.9859404039623998, 0.9956328135391702, 0.9903966269131179, 0.9881456844801908, 0.9945030796320774, 0.9883081383610506, 0.9907493152247223, 0.9799363180107112, 0.9933375980792233, 0.9922504935138001, 0.99068759175349, 0.9899253538905177, 0.9894066606726398, 0.9915041662047401, 0.9935199421012162, 0.9937058849360191, 0.9938504411999409, 0.9654792474324541, 0.9860825615681853, 0.9911584858456334, 0.9962808459856846, 0.9977637622508266, 0.9948353496510718, 0.99377278594225, 0.9948555190591449, 0.9965117413738583, 0.9987203921725765, 0.9980840292012007, 0.9965837866962636, 0.9577826021654597, 0.9523682052506702, 0.9869476681271446, 0.9865498682193762, 0.9562128130352101, 0.9737768838191974, 0.9692941365197276, 0.9708526631853432, 0.9626462222263908, 0.9740350462792606, 0.9807975422164386, 0.9893636898329944, 0.9863776655891834, 0.9873946056360435, 0.978049126789434, 0.9738119183010097, 0.991951855299258, 0.9868431703860725, 0.9867479719728581, 0.9896444108513529, 0.9832711131080103, 0.9822266128289477, 0.9663400508952571, 0.9838827318317072, 0.9699335771590053, 0.9904010630411099, 0.9617089721656866, 0.9906218052698648, 0.9868359015121726, 0.9811592736250286, 0.9938860224085629, 0.999370869288191, 0.9948991575565487, 0.9925881124466274, 0.9776175264195598, 0.9825419728553888, 0.9797723675246712, 0.9652954354965531, 0.9737772384426941, 0.9854844863580854, 0.9848259201314472, 0.9908766766063904, 0.9867231099443005, 0.9943193546441456, 0.9915760947117944, 0.9957816741494971, 0.9933188784689067, 0.9722705808611133, 0.9949770156108702, 0.9535022278954088, 0.9746577121882417, 0.9986606714558782, 0.9981107628971018, 0.9982588190054674, 0.9872766589607332, 0.9982498603794153, 0.9916741355988836, 0.9792352232325203, 0.9980326743922933, 0.9708833659130032, 0.9981834200739366, 0.9968739230109062, 0.997128009427798, 0.9979595299247604, 0.9960903929817445, 0.9964404666056835, 0.9963065609420451, 0.9964605474981091, 0.9958551926204096, 0.998468735229279, 0.9942233856707338, 0.9976575816501223, 0.9965880886628109], [0.9951559313913947, 0.9695635567907746, 0.9857843044935285, 0.9426086935002471, 0.9618821420231398, 0.9771998069345743, 0.9668179303327985, 0.9968698024003749, 0.9864981843702291, 0.9331263847509977, 0.9486388124803632, 0.9052711209681389, 0.967561414576168, 0.9800512757573809, 0.9730028225199576, 0.9781800136824906, 0.9642768676653579, 0.9861855614524788, 0.9762759961334071, 0.9670340987986497, 0.9787315559417202, 0.9710371504209272, 0.986008901663531, 0.9913684243067077, 0.9754854065447467, 0.9885657386390866, 0.9957101247298566, 0.985721829230181, 0.9782591019860157, 0.9734997265686886, 0.9886851173810043, 0.9874163427407434, 0.980201890288719, 0.9833819552487032, 0.9198633206542732, 0.9653138622710113, 0.9625324728021782, 0.9677115397996423, 0.9456857304840918, 0.969243732430507, 0.9901885441563631, 0.9666125252253276, 0.9804834254315757, 0.9530852011860257, 0.9646051280181721, 0.9708600549799431, 0.9706385772145697, 0.9559088948262818, 0.9194271324717581, 0.9604378280081188, 0.9602757551680874, 0.975129041060053, 0.9709390529529067, 0.9507136469420271, 0.9756140425016395, 0.9648883960342496, 0.9696269933407476, 0.9732874065643451, 0.9085060265218071, 0.970343811914363, 0.9380207031405708, 0.9579137480381023, 0.9267571871359407, 0.9684449010791067, 0.9217887938984918, 0.9381787456881429, 0.9419074744680045, 0.9767840296306035, 0.9871057699122113, 0.9586872058963342, 0.9324514095171647, 0.9769806463058053, 0.9066067777572302, 0.969809971920578, 0.9854175271602181, 0.9820330276855064, 0.9611136275979928, 0.9871714247244425, 0.9855609490756612, 0.9847463520529525, 0.9858089750356495, 0.9836998794938191, 0.9828049108654076, 0.9843974818887616, 0.9373655944780065, 0.9723116029678286, 0.9752501091516038, 0.9916362464660861, 0.9559184439473201, 0.9775858956652792, 0.9780167664878617, 0.9683081985667592, 0.961919691532134, 0.9600036175807335, 0.967950437827775, 0.9531147919703679, 0.9692099075553963, 0.9541637810170113, 0.9746910016882986, 0.958164643578527, 0.9594466553411594, 0.9871378519697908, 0.9525800993596683, 0.9615606977979956, 0.9519214604692565, 0.9673300642236305, 0.9717177719478333, 0.9675557641198672, 0.9616423753854549, 0.961128065838675, 0.9777553438826554, 0.9844536399501221, 0.9847802569364322, 0.9772832901488687, 0.9433600907343956, 0.976472583484534, 0.9484065842541581, 0.9868610833929549, 0.994579938306505, 0.9753973819259997, 0.9753467584888387, 0.9706723959656058, 0.9911703618081346, 0.9950070720216395, 0.9942321330790497, 0.9813646912956354, 0.9287866740127613, 0.9273576912517008, 0.9640656662706026, 0.9689773704697516, 0.9031796161035321, 0.9383546539745669, 0.9388693790776931, 0.9514207242316681, 0.9363095136366074, 0.9258184156518895, 0.9479266581495098, 0.9710832911267476, 0.9711352637949455, 0.9670616071051782, 0.9582235735535658, 0.9579910920065208, 0.9790295569330609, 0.9413801280126642, 0.9545876076204106, 0.960219343893416, 0.9528586184147705, 0.9397804489111512, 0.9152217307400132, 0.9520714660330392, 0.9304549689983367, 0.9621305793375725, 0.9149983601668725, 0.9625810175413068, 0.9579499227995295, 0.9455008704183056, 0.9679374915746023, 0.9974648755351223, 0.9769578449035932, 0.9680523823573917, 0.9341809863458905, 0.9525228197489057, 0.9414097814857567, 0.9135730950171406, 0.9301772232327995, 0.9452518039121345, 0.9668606175906405, 0.9672121740639104, 0.9688921281761043, 0.9793039506958612, 0.9735828963624181, 0.9796792592412884, 0.9751117890581338, 0.9416177888454901, 0.9781803582668981, 0.8883501397659915, 0.9422969095637421, 0.979665973092654, 0.9887067649474335, 0.9798039687498143, 0.9575394926508127, 0.9780190895830241, 0.9660955699247036, 0.9474525781065493, 0.9755047400279672, 0.938045530814784, 0.9821790192797024, 0.9829723056524384, 0.9802591007986221, 0.9844851897073352, 0.9807892496174269, 0.9810300130222944, 0.9756080565195528, 0.9813260832969783, 0.9802269730313027, 0.9861122225369343, 0.9652818972243343, 0.9841795149056752, 0.9793718338426789], [0.973820705500011, 0.9148232856891948, 0.9529285680304868, 0.8884474416951427, 0.8992925742483988, 0.938111100124971, 0.9132827862402624, 0.9774849763875256, 0.9529569901122141, 0.8720672438017626, 0.9071348911318292, 0.847511348432834, 0.9549972561509709, 0.9707792570687194, 0.958389371355752, 0.9664735596411242, 0.949232058188967, 0.9737842916903514, 0.9601225008047862, 0.9527836601243553, 0.9666228886762771, 0.9161552844884828, 0.9541048627427965, 0.9739461636272957, 0.9391444386192768, 0.9586741386218204, 0.9906793870437122, 0.9556562948325736, 0.9410131112436909, 0.9401187263499273, 0.9594467091602948, 0.94854123878328, 0.9485973595148164, 0.944440003407483, 0.860037227747323, 0.9363200117885029, 0.9334747573866041, 0.9359454635622965, 0.9151144046013064, 0.949008029036206, 0.9866004448816076, 0.9507351888004962, 0.9613752244864735, 0.9264927111916318, 0.9453532356898939, 0.9471276914569222, 0.9474064406343795, 0.9300591179812702, 0.8427678210484039, 0.9406542827588361, 0.9425926177645855, 0.9527648154339048, 0.9516167823112905, 0.9281852894532467, 0.9581460769513038, 0.9443474318168542, 0.9571673313505985, 0.944548923724493, 0.8400008268162725, 0.9329293852637313, 0.9052127471746563, 0.9268103169733722, 0.8556561933767407, 0.9318444515635641, 0.8708306138401105, 0.895590284858526, 0.9090131234831744, 0.945197907814688, 0.9788808660773911, 0.9149515830569598, 0.8874897147125637, 0.9400389817035965, 0.8731452140782368, 0.9424689537823596, 0.95188771218356, 0.9487810811402504, 0.9184868725954599, 0.9560299975855444, 0.9617390178454664, 0.9507320277904412, 0.9538760527110205, 0.951519850116532, 0.9516957043456336, 0.9466124695595598, 0.9016559914668477, 0.965784746872272, 0.9657368093074571, 0.9833738280071354, 0.9390857884581756, 0.9705522003973373, 0.9658785982640006, 0.9585327781455447, 0.9217128730668739, 0.9222383245427783, 0.9207562303435756, 0.9010382114642297, 0.9248158176673918, 0.9088975522698276, 0.9291586177626686, 0.9091969480948526, 0.9125398974103628, 0.9814698617107541, 0.8958288086402375, 0.9198597160642799, 0.9113668501682255, 0.9267073608385408, 0.9464302948022928, 0.9399204810449691, 0.9159205149843473, 0.9049724025241804, 0.9471740500225396, 0.9697639358159631, 0.9689457681893698, 0.938730057897616, 0.9150881805366426, 0.9635436424409409, 0.8749603933291589, 0.9775650828729943, 0.9910702665500314, 0.938830539330141, 0.9369717893735706, 0.9316119301515822, 0.9855464518474639, 0.9914300394489896, 0.9907928251809911, 0.9497353112194954, 0.8961944561309851, 0.8980648921371356, 0.9301568824829977, 0.9369826953780418, 0.8514624975624623, 0.8914558746524824, 0.901411434141344, 0.9272376301913696, 0.9050893833220166, 0.8619703865687232, 0.9017174099752816, 0.95051000978745, 0.9525425663404536, 0.9387717457517504, 0.9384341334144572, 0.9380328997011703, 0.9603709962935798, 0.8678917778639214, 0.9079603082350957, 0.9157202235623922, 0.9098215226634621, 0.8828015645698016, 0.862555932357125, 0.9055999840606893, 0.8837528289547092, 0.9134647743083036, 0.8579376246530379, 0.917163734004886, 0.917683186294935, 0.8910847699255132, 0.9154776723723445, 0.996298874282335, 0.9386077482937983, 0.922925579926473, 0.8810430202987165, 0.9041782384107008, 0.8929494466924879, 0.8409183997612795, 0.8735109009488322, 0.8852120768068072, 0.9427601861014931, 0.924596897500158, 0.9372372841672129, 0.9544543425384759, 0.944787097621467, 0.9432957745020055, 0.9424812540958141, 0.910497100490191, 0.9481126439737073, 0.8194096258902712, 0.8980519512602272, 0.921928452442976, 0.9781980471221066, 0.9270424489259222, 0.9088780623101255, 0.9199380455452977, 0.9160083903892142, 0.9081281086264684, 0.9162300779760881, 0.8913748346789905, 0.9387760799979906, 0.9481731654815381, 0.9459021624721532, 0.9489700904356447, 0.946586340705979, 0.9450556906195893, 0.9349367480174438, 0.9417799865174523, 0.942122725797234, 0.955365785677929, 0.9138370697267737, 0.9459369059765772, 0.943725567870823], [0.9262042559566107, 0.8505519148818558, 0.9168028478770204, 0.8296300741763302, 0.8345971303961772, 0.8826159875955896, 0.8503265221320004, 0.9315671379464818, 0.9037336239288492, 0.8073029624980583, 0.8563668128560341, 0.7841771677949717, 0.9370516975315974, 0.9572633670395385, 0.937027396714407, 0.9475257342794823, 0.9274291680085887, 0.9522559649935203, 0.9380668584535539, 0.9346600313973247, 0.9486940535202113, 0.845672480915285, 0.9048083652725581, 0.9560930332780252, 0.8957174033321647, 0.9121196828776987, 0.9882492311202977, 0.9065920336413883, 0.8896744515809898, 0.8961394980628801, 0.912419010094543, 0.8906385756895362, 0.9045205220825071, 0.8922053103939606, 0.7999479498002072, 0.8978421439769853, 0.8951738587212068, 0.8967550861518144, 0.8769242775892858, 0.9260437466278917, 0.984292791374635, 0.9298017651931728, 0.9325980163437398, 0.8961950710529917, 0.9163548995529777, 0.9177307173239672, 0.9162910843015588, 0.8975975039271669, 0.7646601817820542, 0.9164695944689097, 0.9222553985240198, 0.9239497212522675, 0.9269054753088207, 0.9062042306062644, 0.9351638818190012, 0.9147203827788171, 0.9469017374221365, 0.9086821858754336, 0.7762382283136052, 0.8849463779003888, 0.8613857114212629, 0.8882895314215866, 0.7865688849241962, 0.8816696376480184, 0.8185990985798931, 0.8542419255862584, 0.8709461489988956, 0.8987082245010216, 0.9717771455347434, 0.8596998996463038, 0.8425035934674998, 0.8965395888136862, 0.837150079070866, 0.9011679074491747, 0.9022740297864091, 0.9077457927030641, 0.8753494182861162, 0.913349386812116, 0.9363735718309906, 0.9023534591794771, 0.9093018714694637, 0.9094398561904106, 0.9155452712627011, 0.8959734287480026, 0.8618531520898208, 0.9528202628110402, 0.9547984038363388, 0.9667204174674482, 0.924191547809993, 0.9587563105412322, 0.9529162108069484, 0.9479097990196469, 0.8739642975513424, 0.8766311263289428, 0.8641676092094337, 0.8419565576149588, 0.8701092156114472, 0.853149743321156, 0.8654552046243132, 0.851593063476253, 0.8567735755282004, 0.9795566004567505, 0.8314407847273074, 0.8750355397396312, 0.8684236335975944, 0.8787607459101588, 0.9282436067269596, 0.9180755303048905, 0.8663403752751254, 0.8398952725566082, 0.9024612836577428, 0.9522134276229267, 0.9515843125128951, 0.8868296126699444, 0.8810508031662914, 0.9483785795267979, 0.7983382809275675, 0.971344112655112, 0.989497869940375, 0.8880089879059008, 0.8827006141520286, 0.8813915565902687, 0.9824043729012822, 0.9899752182815494, 0.9887196356110304, 0.9063187772795787, 0.8585803354310294, 0.8635368946922101, 0.8879531238817043, 0.8933026068953391, 0.7912650123086193, 0.8406820015095835, 0.855604336477726, 0.8937340128199781, 0.8706498550627179, 0.8008453454531302, 0.8501656277236026, 0.9261453428325565, 0.928830350704831, 0.9064007721414724, 0.9110645823276404, 0.9188191046350133, 0.9341522086289561, 0.7928264038234105, 0.8570279216768956, 0.865504849454877, 0.8586133447975014, 0.8188016158788035, 0.7981452722992605, 0.8489928955509644, 0.8342110967693227, 0.8560576726678242, 0.8053396063359768, 0.8687943421018149, 0.8789447380111691, 0.8332150955833609, 0.8540466548792024, 0.9956361114191291, 0.8859496875751631, 0.8686951919931604, 0.8275426349829286, 0.84531720668999, 0.843719140727557, 0.7779399272854053, 0.8232496388277527, 0.8180216820485025, 0.9095567006937791, 0.870256448983706, 0.8949116169251445, 0.9151850394982886, 0.9047451998471273, 0.8916217799279836, 0.893046922637956, 0.8791040339314237, 0.907409062910663, 0.7571556777433233, 0.8424362652061058, 0.8507422591344621, 0.9742828251628629, 0.8616823175467593, 0.8488995487169066, 0.8523897195418323, 0.8519667951504929, 0.858729462540058, 0.8451094691287798, 0.8319351248986864, 0.8763165642593724, 0.8966242786850788, 0.8933076280759485, 0.8968109843764058, 0.8949712153256786, 0.8919792241905657, 0.8827485209056009, 0.8887515281120203, 0.8891424943460063, 0.909986540722581, 0.8528261558965504, 0.8856999056610496, 0.8925409074842672], [0.8696999487133266, 0.7943097057580002, 0.8983358560455955, 0.7740020525121434, 0.7670291146546663, 0.818648919681744, 0.7951370170343802, 0.8756502283712719, 0.851198928578718, 0.7510413125874973, 0.80947611023102, 0.7329438968564587, 0.9148150208438868, 0.9398966278012834, 0.9098175605139159, 0.9219730266422634, 0.9022173620600118, 0.924033128790239, 0.9088833230273737, 0.9135720393982424, 0.924072637921285, 0.7794328158907984, 0.8512553882816801, 0.9447298184521727, 0.8442587952996818, 0.8593233064421455, 0.9867597588360906, 0.8504731422010723, 0.8328073677654422, 0.8460457321013216, 0.8575260988484029, 0.8282476750180334, 0.8565598093553388, 0.8371472431018561, 0.7457124978259294, 0.8636287843905425, 0.8581623992121983, 0.8540860777718912, 0.8293263527139393, 0.9055327134388328, 0.9826718384708386, 0.9113175932198485, 0.8956665497226405, 0.8601102169374475, 0.882623984533283, 0.8818208691648792, 0.8790670633767106, 0.8641658628744185, 0.7185171859615076, 0.8894326773052238, 0.8981021295450023, 0.8877993503868431, 0.8984265196910748, 0.8807547261516486, 0.9096388445696569, 0.883229905733188, 0.9404785384663216, 0.8638171542275236, 0.7203276932539354, 0.840850301000685, 0.8202824455725009, 0.8519072166264218, 0.7228734867548121, 0.8331515533668798, 0.7644979894591165, 0.8091065691019368, 0.8241103877061059, 0.8397503416930041, 0.9687629885407067, 0.8009680537113647, 0.7985032856544663, 0.8470433544225352, 0.7944381619480652, 0.851773499132195, 0.8506757835296934, 0.8656487151848463, 0.8324945166624503, 0.8693101764653116, 0.9189740574290196, 0.851293740716596, 0.8596112915416092, 0.8731093855447024, 0.8832510356665798, 0.8451267866719958, 0.8187097908209431, 0.937416386210879, 0.9423451452885859, 0.9447476786505176, 0.9048824141234971, 0.9447850077600718, 0.9363000179800691, 0.9331364966824777, 0.8237020903768045, 0.832564504671055, 0.8081802057888106, 0.7894345203371983, 0.8248047548693348, 0.8040656872122186, 0.8100715058957988, 0.7938976000249935, 0.8069924190977342, 0.9791226257942016, 0.7677842816249175, 0.8396472308246036, 0.8288486051559362, 0.837762670763845, 0.9179378979253321, 0.9065465439262841, 0.8192912819619078, 0.7756931969476903, 0.8485502604282128, 0.9384634794801593, 0.9378622900780105, 0.8316437923120705, 0.8391500863915305, 0.9320919443733894, 0.7288452520236655, 0.9680444757599873, 0.988708437198862, 0.8389032835578153, 0.8287718152773855, 0.8318506387048187, 0.9814908050545302, 0.9892393507738708, 0.9879154497506237, 0.8585797667886594, 0.8140168361248976, 0.8258003024828645, 0.8422884161472494, 0.8517223784020888, 0.7389911375637459, 0.7854645774245778, 0.8043459581304929, 0.8525924967228086, 0.8358397767052727, 0.746585421337921, 0.7995385708837189, 0.894190234106778, 0.9048984201303139, 0.8699199435849719, 0.8778615496784813, 0.8937146822607921, 0.9028404774676431, 0.7302905207372694, 0.8065535316553311, 0.814921805512184, 0.8073384031739218, 0.7659204187841956, 0.7445032968425265, 0.7947969271673192, 0.7827725955839028, 0.7978992543974545, 0.7576229443346243, 0.8127328644449173, 0.8462869500081557, 0.7745666810608517, 0.795324235896414, 0.996066490063985, 0.8344036983640445, 0.8148143135854413, 0.774844282978023, 0.7872371476340277, 0.794764829886155, 0.7265323911317767, 0.7765679274769827, 0.7595205733584125, 0.8675693918776448, 0.8128644385440841, 0.8434737148163713, 0.8687968876368584, 0.8574437213595958, 0.8384353362483679, 0.8385659661434426, 0.8423796415839281, 0.8598629417536711, 0.709764300295522, 0.7908375270704917, 0.7900875363734212, 0.9744440914115231, 0.8099209133153256, 0.7924869021913754, 0.7875047226194621, 0.7866120871184907, 0.8073996554316439, 0.7819003706279655, 0.776472526994552, 0.8121731885244214, 0.8433016844310864, 0.841354342416412, 0.8398941335860761, 0.839634875069705, 0.8360299032442385, 0.8333414295912557, 0.8322117341211099, 0.8333953733640082, 0.8662921524921786, 0.7926279019163335, 0.8286749197616612, 0.8403498911477636], [0.8160505410517201, 0.7487734907109823, 0.8926660575591453, 0.722277192519351, 0.7131648337267947, 0.7751539626763482, 0.7508815103743177, 0.8205138312061006, 0.8048077628883553, 0.7005981032521241, 0.7707073805383222, 0.697211990443132, 0.8832507705042703, 0.9225724152079835, 0.8780654693357366, 0.8914332196728536, 0.8730626858004623, 0.8959060290577405, 0.8732466981284698, 0.8897307724599888, 0.8916800030144686, 0.7207024835047704, 0.8054662449169, 0.9396164823849715, 0.7953293305980693, 0.8079970542128334, 0.9870520135883935, 0.7974506537016881, 0.7817036245868689, 0.8024023957679124, 0.81002466785352, 0.7732555151802518, 0.8153885936150321, 0.7964719113313092, 0.6883798467199334, 0.8345867647487659, 0.831040654452458, 0.8153440087302297, 0.7793428767651149, 0.8892383935438531, 0.9823695401452257, 0.8951899603595218, 0.8517954208069484, 0.8249629597838619, 0.85183916349002, 0.84886551551782, 0.8441945368323049, 0.8263511915857027, 0.6609437428233883, 0.8609739213976674, 0.8697956507344732, 0.8565274842973306, 0.8702244913375669, 0.8591255920776065, 0.8784135056486837, 0.8499240613825276, 0.9401388064549063, 0.8265156386417859, 0.6563934310248765, 0.8012464629188828, 0.7790250477934065, 0.8195957785782638, 0.6784757484244973, 0.7806021873638944, 0.7202125376361043, 0.7687034199421084, 0.7920545316456127, 0.7901555136975217, 0.9675918941255742, 0.7497696751683669, 0.7502768418714373, 0.8015050933115866, 0.7556102472799986, 0.8084770613149169, 0.8014642282742788, 0.8353312068942459, 0.8000777447810891, 0.827956701258271, 0.9106400075394738, 0.8090734341843517, 0.8177765381172856, 0.8463390840744872, 0.8594449545756597, 0.7974736811912893, 0.7750911388010702, 0.9202781124024545, 0.9285251246652616, 0.9166644691261302, 0.8830651703989643, 0.9305735160346387, 0.9184162205385065, 0.9159015456632016, 0.7842497194075859, 0.7970632935009094, 0.7659735935894908, 0.74626695711861, 0.7849208602769718, 0.7639752437480489, 0.7657272319000192, 0.7480312536278354, 0.7611011780537252, 0.9787521515082668, 0.7200835617916227, 0.8258180363253594, 0.7887254331205794, 0.8006530639739279, 0.916251113232861, 0.9029849370919333, 0.7908374620889721, 0.7258321843579847, 0.8019119309574638, 0.9343371042679721, 0.93219681396349, 0.7865755794448783, 0.8039346496090041, 0.9201039227871655, 0.683931636641679, 0.9680412698636823, 0.9889938173905893, 0.8015167999119015, 0.7904704514692977, 0.7894285562352287, 0.9820360745103709, 0.989075380331466, 0.9881739196830943, 0.8201713047165923, 0.7700215820077395, 0.786279894681296, 0.794450255586745, 0.8035349060028337, 0.6967092794919882, 0.7274475382049473, 0.7591539684081546, 0.8129163011890762, 0.7971228336052285, 0.7115929154771609, 0.7445141398977889, 0.859730945820175, 0.8839333869513805, 0.8316890771211624, 0.8420399102156041, 0.8657941854235414, 0.8684187035267061, 0.6672848797907566, 0.7612413261738801, 0.7677904075121548, 0.7673985497422106, 0.7261920131134861, 0.700567188220016, 0.7479981375421414, 0.7462279476828202, 0.7489507741732928, 0.7104231110997251, 0.7677847272074945, 0.8279082727068702, 0.7293257753237856, 0.754835402157943, 0.9961475608772081, 0.7782328233069605, 0.7691585164684253, 0.7393625468945393, 0.7330000951694405, 0.7561546345369902, 0.6819576681613579, 0.7596565873229044, 0.6997902753580577, 0.8280817071047046, 0.7700817649809395, 0.7967996037281101, 0.8297505336192789, 0.812671800977521, 0.791828772428629, 0.7924151336251778, 0.8097561563851133, 0.8200874822681888, 0.6659240125843534, 0.7341940349214446, 0.7424913581861314, 0.9741056799958467, 0.7718362496085429, 0.7388731895300787, 0.733675687212419, 0.7406739904786941, 0.7659043611361115, 0.7354750214141322, 0.7338686887764785, 0.759453669892584, 0.7947802116775753, 0.793766634001037, 0.7928598913189073, 0.7910562911727244, 0.786260805603405, 0.8005670197333561, 0.7829284704364534, 0.7834794269706951, 0.8371751072169964, 0.7475832352248378, 0.7774403747299657, 0.7942401080988102], [0.7721374182711546, 0.7105969435781225, 0.8922923833350546, 0.6875689423598244, 0.685615090305452, 0.7443712081191501, 0.720620958587985, 0.7815701842515651, 0.7713506902991719, 0.6616190011712284, 0.7438364949831736, 0.6452689590715345, 0.8518367733449662, 0.9065458287272311, 0.847005662455718, 0.8585460017508006, 0.8434666290649335, 0.8775471287807632, 0.8419119610570843, 0.868434312347448, 0.8620493768288419, 0.6809504841554913, 0.7666365682478947, 0.9394190642459451, 0.7599818402983869, 0.7714697586285325, 0.9874170411275249, 0.751937909656294, 0.7406779254874055, 0.7705486616084308, 0.7690099285496884, 0.7229079522615879, 0.7950187045131922, 0.776135751093633, 0.6631480155178097, 0.8223783014504206, 0.8098509571754811, 0.7927553153139244, 0.7391711159764757, 0.8797657405166114, 0.9818662198088618, 0.8864285560859414, 0.8148372695918016, 0.7816910701163597, 0.8128629557003682, 0.8140312765559, 0.8051342786196688, 0.7891667622438028, 0.6228181203828069, 0.8371130393023333, 0.8469190734892168, 0.8234935671537774, 0.8411844904100911, 0.8359853090005767, 0.8481777279289524, 0.8208975286899256, 0.9417962857664267, 0.7935361231556441, 0.6351984625049296, 0.770950507171191, 0.7398066951724985, 0.7944733112380332, 0.6584146987753908, 0.7351281634245743, 0.6784043918677737, 0.7283988646587519, 0.7490347077218821, 0.7439974120039453, 0.9681057762654504, 0.7123596111247246, 0.7067908206053435, 0.7608847659596344, 0.7208812088537653, 0.7628466582544317, 0.7566032243146312, 0.8178531763792608, 0.7721603708547922, 0.8025548375778953, 0.9090333076634289, 0.7785585812307083, 0.791757205747589, 0.8330259483468331, 0.848524564600598, 0.7678122974802387, 0.7374164507783884, 0.9011519024378548, 0.9146064884802956, 0.8856077292572009, 0.8575424903675541, 0.916013324621028, 0.8993504103059226, 0.8973844804242774, 0.7450136970466719, 0.7765937124067508, 0.7431123251929779, 0.7310655550311013, 0.7645216415980727, 0.7418043129612208, 0.738733053498529, 0.7134537293303526, 0.7321368254805286, 0.9791374933882467, 0.6779735912264321, 0.8197140103630866, 0.7717649947726892, 0.7784003663017711, 0.9174901588559767, 0.9038069010245326, 0.7747845171436056, 0.6850516655226178, 0.7764499717978381, 0.9319566418064563, 0.9310538319810109, 0.7615485105241285, 0.7879640550330901, 0.9132943308958287, 0.6500074611284348, 0.9676220615264269, 0.9892460578877608, 0.7766523048859509, 0.7632245690738404, 0.7672590670461296, 0.9823740188563006, 0.9897283848444386, 0.9882340497706421, 0.797909707137393, 0.7281709224547745, 0.7558893730333148, 0.7559095514457802, 0.7675659066937143, 0.6551817139638809, 0.6848945286303646, 0.7137108715353851, 0.7759300044867025, 0.758176968009934, 0.6856565684818657, 0.7048895702255975, 0.8299232064095496, 0.8690896455610884, 0.8051592345078962, 0.8115579160582768, 0.8428778335417159, 0.8351555434543877, 0.6244941576483459, 0.7274367666807521, 0.7272737880317963, 0.7397856456555184, 0.6733680529955243, 0.6609312586273387, 0.6964599706109357, 0.6987420003831422, 0.7233187034691765, 0.6867277725996823, 0.7273214280387889, 0.8205989614162628, 0.6955844504622306, 0.7113520577693455, 0.9962361802716169, 0.739932643407058, 0.7282739071766406, 0.7128793325728402, 0.7127449778046862, 0.7273560248768199, 0.6724640565132781, 0.757762731172506, 0.6562128042520387, 0.7986719178686065, 0.7329732442581836, 0.7655082602864072, 0.8001719105217115, 0.7778907710104084, 0.7564754682387094, 0.7578932067010994, 0.7796559788520987, 0.789103412920737, 0.6403709623490753, 0.6992540973709306, 0.7033227808902639, 0.9751737213587149, 0.7545375769800279, 0.6891814919173508, 0.6913290028342279, 0.688364334070247, 0.7247906873945463, 0.6907872440869627, 0.6946590266652461, 0.7118792247997314, 0.7672702087645479, 0.7643546460819185, 0.7644474149984307, 0.7569461332884816, 0.7513196852672305, 0.7852855928494079, 0.735593937302902, 0.7448673863762447, 0.8299365501128205, 0.7154647724211005, 0.7359168968083674, 0.7658527856564263], [0.732340821999459, 0.6886245606280924, 0.8914149685786259, 0.6657558505802149, 0.6616906202358277, 0.7204420153999698, 0.6993011350866273, 0.7547634835502648, 0.7530220224947185, 0.6324518270858995, 0.7392778721300537, 0.5905730796696613, 0.8291561868176066, 0.8981024426023962, 0.8243548664369655, 0.8341244661162441, 0.8148793779815224, 0.8640653395023701, 0.8188382505766442, 0.8555148655927699, 0.8384731256502648, 0.6573475959042784, 0.7439304486254119, 0.9394528792646301, 0.7339316684769147, 0.7447397695648674, 0.987970203375619, 0.7241562665146365, 0.7046648638985645, 0.746396732445164, 0.7483971473885371, 0.6808514179604092, 0.7863457753754765, 0.7680961561520608, 0.6081010729801798, 0.8211896902547425, 0.7997853225131345, 0.7850738596477573, 0.7118126020060611, 0.8742513397814126, 0.9825393355111961, 0.8817386642440742, 0.785340859628429, 0.7567324747071044, 0.7857326745019713, 0.7909985009316304, 0.777690017209586, 0.7615551586341661, 0.5995569325267399, 0.8202945250825296, 0.8269474795678989, 0.8025609851782546, 0.8171630046154946, 0.8223928848938041, 0.8269764175940347, 0.7991585806620176, 0.9456686950685388, 0.7707853430379027, 0.6179145774005198, 0.7746784583835269, 0.730057420174261, 0.7801748465315558, 0.6555991990207548, 0.7080715458476173, 0.6315141568234927, 0.6788230355312269, 0.7099548789497441, 0.711442315509457, 0.9684399756528205, 0.6822748337961885, 0.6581487942465372, 0.7264273365285729, 0.6990051267290079, 0.6968548562422558, 0.7150185031908798, 0.8119357882959837, 0.7635784649694516, 0.7904345245704696, 0.9087854340102798, 0.7583309403587558, 0.771635987479452, 0.8284003550102831, 0.8450990540927216, 0.7435448291797385, 0.7066942065757755, 0.8823903831962507, 0.9037807139125568, 0.8583700018183851, 0.8308699928279414, 0.904344017055117, 0.8794496937631774, 0.8792861112440572, 0.7128395869475743, 0.7706340280230684, 0.7320717537645665, 0.7081283023998854, 0.7572501648619978, 0.7250095315070695, 0.7135700885670536, 0.6998670626277881, 0.7247374801435386, 0.9800507951466471, 0.674219718420126, 0.8226471343887729, 0.7493910837703822, 0.7719722490753118, 0.9180959676893362, 0.9062084671265601, 0.7664125231064881, 0.6643214400916041, 0.7638555673979928, 0.9319925120542362, 0.9312392690143126, 0.7495621484592057, 0.7775439785839737, 0.9101486873576686, 0.6011786038837169, 0.9678564633713052, 0.9895149661862116, 0.7616028314744611, 0.7530361422072235, 0.7538269752175331, 0.9834573608547698, 0.9894719158659254, 0.9885184382955531, 0.7867997797749949, 0.6967224133978878, 0.727654154960267, 0.733295439680638, 0.7416578521899159, 0.6155693940158852, 0.6778708737021755, 0.6900298106868412, 0.7418174623596626, 0.7288197342066741, 0.6268742339800204, 0.6655677038720408, 0.8050589545975759, 0.8605137564325623, 0.7832705818363859, 0.7851837319930657, 0.8174416997085018, 0.8138380100870011, 0.5945103828367411, 0.7135711042283195, 0.7228481888975947, 0.7111371884340698, 0.641365011308618, 0.6537792684843976, 0.683739553677333, 0.6955426970476691, 0.6835864842237167, 0.6541856762338268, 0.6971266899000739, 0.8216108160750345, 0.6719859218100896, 0.6901153571435473, 0.996395497083296, 0.7096572966224776, 0.7039542507506513, 0.7087787821553053, 0.7096013748927195, 0.7095470086742619, 0.6117293906642014, 0.7508691514699138, 0.6662102225941388, 0.7736460119721749, 0.7102344375224636, 0.7350146397913351, 0.7745170880553297, 0.7547645777065658, 0.7369530372347572, 0.7374547357640721, 0.7608570452735306, 0.7642173214929323, 0.5931093617520699, 0.6580856927183057, 0.685404551610968, 0.9763278863667177, 0.7481190076987897, 0.6631324726863163, 0.6730928435170336, 0.6605454959618351, 0.7008415605210626, 0.6906172207023831, 0.6839230041117437, 0.6802671109140617, 0.7514544922439105, 0.7449411962059446, 0.7484310800613543, 0.7414578757490398, 0.735126804456459, 0.7831348570007917, 0.7218297073566599, 0.7242690634147806, 0.830466187806885, 0.6928076993460003, 0.7049807519242315, 0.7505917135675472], [0.6988739953942336, 0.6811055337829858, 0.89289943424799, 0.6286537141978009, 0.59294396491741, 0.7132497097618933, 0.6814589950275509, 0.7497824741042938, 0.7479796463279109, 0.6099474471963047, 0.7301851526863989, 0.6083755400462797, 0.8149135087899638, 0.8923026627340733, 0.8078654777675026, 0.8194743836876947, 0.8021735458885638, 0.8577842225364701, 0.8108569115468128, 0.8508775822504664, 0.8236141510396375, 0.6243246494728674, 0.7281061821078664, 0.9404062546783656, 0.7197080275966304, 0.7289111951804653, 0.9882303459760451, 0.714294977322909, 0.6973324838016556, 0.7382690126505556, 0.7281482989448106, 0.6437248222163645, 0.7772978776277127, 0.7664551816833414, 0.0, 0.8158150672131355, 0.8025988666416325, 0.7839991569415422, 0.6913160541667164, 0.8752585109459053, 0.9826052807499086, 0.8837337614404779, 0.7586459907361887, 0.7350951701956869, 0.7689020988091522, 0.7756162520568874, 0.7495490532296024, 0.7351926263118783, 0.0, 0.8154944967021196, 0.8151229197203865, 0.7921670100947839, 0.8108749851192577, 0.8138968383403768, 0.8118427849224933, 0.7903568079234291, 0.9474592672002731, 0.7533198392657469, 0.0, 0.7726797326434662, 0.7271838552316117, 0.7719956969837198, 0.6611855440259894, 0.6917336969501983, 0.6311327766231747, 0.6611056582781986, 0.7087727918795047, 0.67547402246062, 0.9694438477286811, 0.6538112302553649, 0.6522188476969433, 0.7115169217325571, 0.6633736781837363, 0.7023371714205354, 0.676126192920521, 0.8111116224548329, 0.7607910186589235, 0.7850358142891176, 0.9090529063027748, 0.739217529170889, 0.7587266897611854, 0.8266103315197356, 0.8436632459481525, 0.7287136550999259, 0.688655894673697, 0.8655779957467473, 0.8950971419406234, 0.8367708009050046, 0.8112164386740739, 0.8951807345672448, 0.8625605756401562, 0.8614418938769108, 0.679822236850691, 0.7614369922014592, 0.720981143022926, 0.6953097544032786, 0.7583432602246704, 0.723290604937802, 0.6983949856560672, 0.680860520219838, 0.7201897554910847, 0.9809021330666241, 0.6652278350770107, 0.8238734148021747, 0.7501314832271242, 0.7601788458195323, 0.9204052673742872, 0.906316506069074, 0.7616131065130696, 0.6427764280872296, 0.7536246056583515, 0.9326816031407893, 0.9317131672643316, 0.742363305860211, 0.7693776410638394, 0.9089714518904236, 0.0, 0.9684036595594482, 0.9899683039042209, 0.7625674816349649, 0.7490939174260904, 0.7538964589772161, 0.9835492747782725, 0.989863202743722, 0.9886756489611622, 0.788356632725625, 0.6558174690442484, 0.6934770444140652, 0.6670038919166849, 0.7233676073491196, 0.0, 0.6271288020176194, 0.6757056251508476, 0.7244215140421082, 0.6883384302460271, 0.0, 0.6355940486683072, 0.7826089361550305, 0.8574075419013031, 0.7638928354580684, 0.7673384781481714, 0.8048971324239845, 0.8014595193447802, 0.6293643932532509, 0.7162050149860891, 0.7045748138209212, 0.6901145044268427, 0.6014385154624334, 0.6113509031637367, 0.6420881994215626, 0.6751698248253484, 0.0, 0.6017768944056252, 0.6652478848029312, 0.8252018046327809, 0.598134735169224, 0.6822121575014292, 0.9964968101825357, 0.701722608149035, 0.693436055182727, 0.7209874084500251, 0.718268497217112, 0.7212715614361953, 0.0, 0.7563458336544686, 0.6814666010636058, 0.7564109795690789, 0.6991164033235102, 0.7129894404935234, 0.7680995428198079, 0.7469805540300666, 0.721174553949581, 0.7278335525373362, 0.7500563901750955, 0.7425028050690401, 0.0, 0.6345824148701514, 0.6685955207600612, 0.9766448363582237, 0.7592807979680499, 0.6359634050897472, 0.6602652322558522, 0.6572239051797294, 0.6685712605247375, 0.6712949369864623, 0.6755014205020157, 0.6568802944175385, 0.7459890716527725, 0.7380949089636429, 0.7387988055805309, 0.7238860543071679, 0.7243778300531158, 0.7792926051991562, 0.7115071687918177, 0.7192318195396178, 0.8316128339576857, 0.6919315819871906, 0.6949239418349262, 0.7555539296715582], [0.673687669216141, 0.6891302552352913, 0.8945703908060573, 0.6136364566789902, 0.6315715645082112, 0.7258653644050843, 0.680956867560784, 0.7445572257837809, 0.7492807182864204, 0.0, 0.7216968724758162, 0.0, 0.806515798236972, 0.8921343018852494, 0.8022070923603484, 0.8107548385014418, 0.7962332284801457, 0.8589147309125827, 0.8081856255523305, 0.8480770238664679, 0.8142931570340384, 0.6038789796612261, 0.7216041966908417, 0.9410496623518186, 0.7206817420387411, 0.7284807774571546, 0.9884869819963733, 0.7066116816591177, 0.6975486793899606, 0.7452822822891472, 0.7254476772649884, 0.6095866873325755, 0.7854828704779664, 0.7767401526694594, 0.6108393362606019, 0.8184079773462302, 0.8077490062794456, 0.7866787356576693, 0.6699518182002343, 0.8761793406927091, 0.9829357069882502, 0.8828367759924102, 0.7474305568290376, 0.7157675387273946, 0.75723873095957, 0.7685445129028996, 0.744972036049071, 0.7352546445457496, 0.6333883986047244, 0.8108765573552323, 0.8054438538062635, 0.7825610888531326, 0.8072015162182142, 0.8133424715431703, 0.8016194344575314, 0.7894719134898971, 0.9502087851103794, 0.7636394498102779, 0.0, 0.7752531029489306, 0.7362751556262387, 0.7644180946074087, 0.6614796660775897, 0.6760592785646751, 0.6159196024987658, 0.6752846099839268, 0.6764589361567528, 0.6460744692620423, 0.9699238471495728, 0.663566149829194, 0.6490532982408576, 0.6669222792965983, 0.6470329186306001, 0.6641500708457101, 0.6761641326413933, 0.8101345021178292, 0.764342263655078, 0.7805655316697285, 0.9114427868571604, 0.7393342069231944, 0.7553878760815784, 0.8305480242522537, 0.8462740704232762, 0.7305495935985987, 0.6594667661944019, 0.8527332741261401, 0.8918785347462893, 0.8234638899794648, 0.7996330036710314, 0.8907916321296345, 0.8518800835508861, 0.8462181919705857, 0.6633197654536419, 0.7541282149563374, 0.7173135450705075, 0.6946890955375141, 0.7617029283874128, 0.7245582187526761, 0.7118047710469403, 0.6820196162814548, 0.7201171004786427, 0.9807235336168265, 0.6320201613535994, 0.8270455509319224, 0.7562091142079099, 0.7703770478339459, 0.9242277961113022, 0.9102382579610562, 0.7781144361231286, 0.6651632467495417, 0.7652163726311138, 0.9340096738024676, 0.9334975287008468, 0.7534669701482521, 0.7753158102106376, 0.9104569615272682, 0.0, 0.9687269376004333, 0.9905143575726861, 0.7631326654389875, 0.7522578976625766, 0.7625978637377813, 0.9840517148211599, 0.9902582780292221, 0.9892991693936963, 0.7934990215962242, 0.6517900579882925, 0.6766100053520472, 0.6805442330270903, 0.7254232827456212, 0.6340227820295748, 0.0, 0.6725839970240198, 0.7132376485638011, 0.64700391462504, 0.0, 0.6506532483960904, 0.7818826056154662, 0.8593005861021779, 0.7579954403207987, 0.7699398096986299, 0.8000214421197447, 0.7957929598091903, 0.6279124070226988, 0.7050437152027389, 0.6929918648377704, 0.7144926662293376, 0.663781585678602, 0.6275797303598308, 0.6901736789598343, 0.6598399732539287, 0.6571289523843882, 0.6546576501791762, 0.0, 0.8212353093816215, 0.6755888385377987, 0.6693382660354689, 0.996827701698612, 0.705400220558142, 0.6950710460136548, 0.724116283217638, 0.707979789364162, 0.7175008055350389, 0.6211308498225052, 0.7793590525354704, 0.6948360602243041, 0.75717305454969, 0.7047577972370763, 0.7119923232715384, 0.7690369981301302, 0.7551606246438343, 0.7327073468631946, 0.7343136747768291, 0.7598770594972436, 0.752236454717814, 0.0, 0.6734487355250192, 0.6762814296652598, 0.9776180702200082, 0.7676780872894349, 0.6208552289876472, 0.6898167975705889, 0.6515502078663703, 0.6825245449492815, 0.6679219995357288, 0.6764274053557878, 0.6380976540932055, 0.7497241787047679, 0.7367275026408734, 0.7438105689462589, 0.7233805174238195, 0.7135241709804401, 0.7841069553727253, 0.7105948677193644, 0.71853370151602, 0.8358109962928463, 0.679266719683656, 0.6947358365175071, 0.754228877549237], [0.6453364737021984, 0.6912975684982525, 0.8963348888236039, 0.6069124388565897, 0.0, 0.7275494224804759, 0.6687456553483044, 0.7479561136239409, 0.7535558631604059, 0.0, 0.7173192885277893, 0.0, 0.800924801140114, 0.8922433844299519, 0.8013634900672709, 0.8065734166853711, 0.7927693424470061, 0.8590327490353102, 0.8014098723021393, 0.8492847858390624, 0.8094886503034265, 0.6323586755500133, 0.7285154979046089, 0.9421448569478974, 0.7168199488769366, 0.7242718849250752, 0.9890867591396623, 0.693389744403138, 0.6868989883469758, 0.7323163840719438, 0.7338398115930926, 0.0, 0.7919564033678363, 0.7809866709137117, 0.0, 0.8203177227403905, 0.819067300338463, 0.796464061539167, 0.0, 0.878907937206215, 0.9835736081429343, 0.8843674972083813, 0.7438406865146765, 0.7152082738016154, 0.7631914550833789, 0.7760882277070803, 0.7388505766852751, 0.7308045363213244, 0.6255090582825225, 0.8110302844703009, 0.8059990969253035, 0.7777591016780443, 0.7983661715790764, 0.8128173915800141, 0.8048306180280339, 0.7850438793551827, 0.9520075426797688, 0.7719464701592439, 0.6192050299287447, 0.781646042098417, 0.7382055985207802, 0.7666339066597075, 0.6750322690937274, 0.6656919607307461, 0.6311948899344952, 0.6350789930982979, 0.6725305361134758, 0.6504014533615102, 0.9701469991323456, 0.6490052400734483, 0.0, 0.6882897488331562, 0.6540975319276108, 0.0, 0.6655102146160184, 0.8107136361002337, 0.7730412617928881, 0.7883351714879289, 0.9130327427388951, 0.7447589815793039, 0.7610471017122834, 0.8319591296322266, 0.8471993140227433, 0.7432781162634884, 0.6614401621127723, 0.8454615308806516, 0.8894122127152968, 0.8179869336629997, 0.7945561498946212, 0.8883588935182826, 0.8422618858188315, 0.8387937485305791, 0.6477165033374591, 0.7638164228992139, 0.7241647641483344, 0.7055125349252539, 0.7547891954367961, 0.7244734674812006, 0.7263332244402521, 0.703271525764626, 0.7270269107892492, 0.9816254128870527, 0.6467244404715082, 0.8329183524942758, 0.7533070300105137, 0.7685014277791772, 0.9256542617913612, 0.9135509562672611, 0.7781804610305203, 0.6619093618886663, 0.7686255168390215, 0.9352500516013004, 0.9346456039700557, 0.757319053664715, 0.7786142077044429, 0.9093462814626576, 0.0, 0.9689793531840053, 0.9903419046458387, 0.7691349335940936, 0.7634516816793694, 0.7651286801733251, 0.9845004623173726, 0.9901205415204737, 0.9893159499025567, 0.7940294761929685, 0.6040663563647513, 0.6898326889365299, 0.6838305520458344, 0.7324310392595186, 0.6269584077480013, 0.6571879918351633, 0.6329831849027645, 0.6604666692009873, 0.6612240265680607, 0.0, 0.6650615495314798, 0.7792020197793021, 0.8585985476741776, 0.7551224206328102, 0.7576852388298357, 0.8050289406272654, 0.7941457710296113, 0.0, 0.7255192105181621, 0.7147847496136187, 0.7250861910505976, 0.677610913400481, 0.6648242045522132, 0.6847611915483437, 0.6597620100560356, 0.6484064524757245, 0.6918905333123102, 0.0, 0.8294821382433128, 0.6828106954536143, 0.66680184820218, 0.9970200669051584, 0.6841138608392864, 0.6785720872579127, 0.7202774156661683, 0.712561220069376, 0.7044203374926474, 0.6353734011171621, 0.7958758942965518, 0.6998816391957831, 0.7607257694597759, 0.7146013883757171, 0.7191601192084404, 0.7720564396279842, 0.7584299330855936, 0.739635545230473, 0.7427996723414715, 0.7544581245578529, 0.7503546661689405, 0.0, 0.0, 0.6826635714161923, 0.9778066008471052, 0.7674994534998202, 0.6487779168202064, 0.6922640204419425, 0.6490318703790808, 0.672163173453719, 0.6773419128219319, 0.6900354477888653, 0.6529305464491213, 0.7569961338458917, 0.752816984730033, 0.7543403146573309, 0.7377618830439354, 0.7299118400444845, 0.7868507693004313, 0.7095682061408162, 0.7303092486094547, 0.8366996089404449, 0.6757540322071757, 0.6837841822897308, 0.7571439874279412], [0.6377602569542418, 0.7055974447975024, 0.8973099830901962, 0.0, 0.0, 0.7092274744125766, 0.6595063883880534, 0.7401963547010473, 0.755444840683491, 0.0, 0.7092054064797833, 0.0, 0.8038737736710594, 0.8925966128973875, 0.80279934441345, 0.8059120564623852, 0.7921617303737902, 0.8621533384513684, 0.8058640505695455, 0.850877622531064, 0.8136904682410606, 0.6129772128088234, 0.7324790698051926, 0.9425515636912736, 0.719113811113249, 0.7221436447215162, 0.9894907414479891, 0.7071802161979879, 0.6956687680859268, 0.7417834999423639, 0.7427568754576679, 0.0, 0.7973795803059867, 0.781441239058789, 0.0, 0.8296549199517245, 0.819253275703004, 0.8035856059559123, 0.6605117529630704, 0.8781848446100491, 0.983825724193461, 0.8873682447271163, 0.731066414886407, 0.7136196773808929, 0.7680644648145477, 0.7708809014721346, 0.7304921249579055, 0.719510505286533, 0.0, 0.812239536344107, 0.8106917856052673, 0.7846063257251019, 0.8025731732409557, 0.8165202336492478, 0.8021804015426611, 0.7941667718153537, 0.9541409232838853, 0.7606839218187816, 0.0, 0.7767813246392229, 0.7352151244864333, 0.7728720860117606, 0.6950481449246565, 0.6690047136450391, 0.0, 0.6489092260354447, 0.6505159942391188, 0.6516641870100608, 0.970326177142523, 0.6295406716500598, 0.0, 0.6488425315278096, 0.6248156732554087, 0.6561733798880276, 0.6859986647001952, 0.8168398972692131, 0.7679876660197875, 0.7943954094758343, 0.9144003719640058, 0.7466069242874166, 0.7628468245410226, 0.8347218802417996, 0.848636261946659, 0.739600811682692, 0.6572033142416887, 0.8391906254025159, 0.8898504484863604, 0.8153449995636779, 0.7896309867559969, 0.8883396160293258, 0.8384716704675331, 0.8352806931744567, 0.6346982506512546, 0.7713833669436847, 0.7173281520230003, 0.7090580894838566, 0.7580259370143212, 0.7367403468673652, 0.7353055469517618, 0.7154049885093092, 0.7157625120366419, 0.9817419704610629, 0.6604340063278866, 0.8347973111665543, 0.7688460242794252, 0.785271362141661, 0.9275553269034891, 0.9152526629958407, 0.7759080721779927, 0.6405045802901287, 0.7675434369335068, 0.9358540900843861, 0.9353712640885488, 0.757308065892435, 0.7868007893170653, 0.9107171789716567, 0.0, 0.9696812385052205, 0.9906454312056608, 0.7675679352532849, 0.7714976673629405, 0.7700409687187916, 0.9849977874226761, 0.9906699202810404, 0.9895117857801364, 0.8027484555507017, 0.6189791674887015, 0.661662294090146, 0.6961753995339415, 0.7409852410489763, 0.640700447446498, 0.0, 0.0, 0.6740938838380036, 0.6401845139586239, 0.0, 0.0, 0.7806340525899644, 0.8618541553298181, 0.7709986261397618, 0.765642111556767, 0.8033887796369238, 0.7949664983176591, 0.6232152534356383, 0.7267332540464565, 0.705782812798679, 0.7061031538159024, 0.0, 0.0, 0.6624822236159419, 0.6510968190241166, 0.6613885321738159, 0.0, 0.0, 0.8289519645076561, 0.0, 0.6506312503675935, 0.9970611253885251, 0.6925101875047228, 0.6724673028944204, 0.7259703911205623, 0.7149216994550475, 0.7082726965596219, 0.0, 0.7950579028043206, 0.6661797922712543, 0.7627036156928929, 0.7001359486424492, 0.710747584601575, 0.7692163779630917, 0.751553850517157, 0.7289436853622838, 0.7382485773848003, 0.7532341391971434, 0.7410572723017985, 0.0, 0.0, 0.6947205780157751, 0.9789945298671704, 0.7683341302354892, 0.0, 0.7039309556921166, 0.0, 0.6626377818343803, 0.689785233043889, 0.6882881436990378, 0.6531249392890107, 0.7616524104525956, 0.7530084323851289, 0.7570838465814563, 0.7427712434910152, 0.7314303392167181, 0.7933840452615041, 0.7262649739727489, 0.7231762544482665, 0.8379182761535156, 0.6739671050903413, 0.6780495960908949, 0.7592824820093323], [0.6511259557117466, 0.6961986763727375, 0.8974108203843606, 0.0, 0.0, 0.7075892113429697, 0.6310212027514415, 0.7420945306386907, 0.7631688729020284, 0.0, 0.7165387567712588, 0.0, 0.8069631929961781, 0.8924220310547463, 0.8062806771330193, 0.8090312670210602, 0.8029613600084391, 0.8661353045488095, 0.8092912099052636, 0.8529861508877582, 0.8186841465061546, 0.6267334700200677, 0.7353911479648766, 0.9443650821194106, 0.7105943218190958, 0.7296049860483625, 0.9898222209464536, 0.7085807617045156, 0.6889361823087295, 0.7523384319761323, 0.7413524354767073, 0.0, 0.799449284614039, 0.7878963589454427, 0.63268906452256, 0.8283006193773582, 0.819353379367761, 0.8025548441484013, 0.0, 0.8814112374485167, 0.9839962951319687, 0.8876238810605431, 0.7300901977675517, 0.7156344733285666, 0.7699664264505349, 0.7861628445826303, 0.7454665257435749, 0.7206781724131913, 0.0, 0.8197636822141346, 0.8113984899198845, 0.7794420054899444, 0.8028044638833601, 0.8211777119891572, 0.8019005058005443, 0.7915931984678337, 0.9565905032194313, 0.761528147944435, 0.0, 0.7920565021697322, 0.7453235409318333, 0.7782393162037934, 0.6775853219532673, 0.6816585327963701, 0.6374966688167403, 0.6619869469633886, 0.0, 0.64396056175625, 0.971185846419205, 0.6636549106518573, 0.6547687162180272, 0.0, 0.6381601884433626, 0.6901397519370632, 0.6910141523881405, 0.8195111424970153, 0.7746981246872275, 0.7965280423698675, 0.9151370818488117, 0.7538114800528292, 0.7700453447872343, 0.8386085919676778, 0.8492476376795578, 0.7381826902920237, 0.628641866491107, 0.8402869559562067, 0.8911286999831388, 0.8157506895107767, 0.792896785882268, 0.8893576066474598, 0.8409032528757703, 0.8369699055025548, 0.6361065202422964, 0.7783437109946435, 0.7189653965743596, 0.7136414706476127, 0.7598060217622227, 0.7389377748019972, 0.724484323412573, 0.6965556607605564, 0.7163768738550397, 0.9815108927108013, 0.0, 0.8370092764718846, 0.7591826234885024, 0.7806058159438182, 0.9295007655043981, 0.9162463502551801, 0.7854131965671866, 0.0, 0.7654127083694321, 0.9366304710688839, 0.935707004587215, 0.7558846203158135, 0.7865811886960341, 0.9128848298205166, 0.0, 0.9697134982782217, 0.9906588046803194, 0.7669609550411692, 0.7692489157782066, 0.7712660096865863, 0.9855007156321653, 0.9906632011235582, 0.9897042404037434, 0.8064152007584182, 0.0, 0.6739865696365047, 0.7174704964588225, 0.7364194864471258, 0.0, 0.0, 0.0, 0.6861139507772736, 0.0, 0.0, 0.0, 0.7849107542447112, 0.8644889017900835, 0.7765640843649122, 0.7621631839987154, 0.8058808906621437, 0.7994399014326832, 0.6362583468596326, 0.7496522569177386, 0.736877500534752, 0.7336289103990745, 0.6595068836843074, 0.0, 0.0, 0.6984754616347963, 0.0, 0.6591302257175087, 0.0, 0.8301820032296968, 0.6574998957067052, 0.6918852581449418, 0.9971089122410395, 0.704152880612, 0.7028767040776506, 0.7279870119235065, 0.7310725895204573, 0.7195435616058286, 0.0, 0.7975464750390913, 0.0, 0.7558441941828735, 0.6920613730573439, 0.7047197453875101, 0.7714935308561642, 0.7544857744131779, 0.7316751967973543, 0.7494964960752292, 0.7580613856531756, 0.7543611606108379, 0.0, 0.0, 0.6845509875388983, 0.9797088447404532, 0.7733223487072937, 0.6902377606884657, 0.6956673771830323, 0.0, 0.6421597833773198, 0.6839867680545246, 0.6783011301846832, 0.6454429242410791, 0.7626405351698167, 0.7642636604646504, 0.7641945295830671, 0.7414548787922715, 0.7235785342532897, 0.7998498661642204, 0.7303441767555104, 0.7388716672950821, 0.8394579497112878, 0.703091635808345, 0.7196079713631951, 0.7647697138903984], [0.6955527176076405, 0.6973308414458967, 0.898446262923187, 0.0, 0.0, 0.7307285785592514, 0.7116488369188932, 0.738280845658226, 0.7744393001889888, 0.0, 0.7267082723162666, 0.0, 0.8072048130363279, 0.894039604344496, 0.8085931892234894, 0.8098017380185036, 0.8038447992001924, 0.865853535828532, 0.8122841601243347, 0.8574860129372113, 0.8184709143542215, 0.6591239070129482, 0.7340848493035373, 0.9452710410114248, 0.722485336747072, 0.7304855991518121, 0.990339164897913, 0.7152525038751683, 0.6934559823424243, 0.7490767971794337, 0.7411364962328065, 0.0, 0.804171364756018, 0.7912331655971643, 0.645560137072564, 0.8350991255846181, 0.8243186774743526, 0.8035083472271445, 0.6842273075292992, 0.882153122695718, 0.9839329675132321, 0.8900699377568276, 0.7420265295627461, 0.711198314912805, 0.7705196938356791, 0.7820996552104591, 0.7451243532716366, 0.7338391530505286, 0.0, 0.824403846133738, 0.8128568879234803, 0.788093093078407, 0.810451802112213, 0.8208412191027034, 0.8066409272254635, 0.7990397922031997, 0.9577917854944259, 0.7517602476573131, 0.0, 0.8042299564310167, 0.7540351100653744, 0.7742093830630207, 0.6689499745952094, 0.6814028095106932, 0.6502211782085758, 0.6734418865132332, 0.6751276677295758, 0.6558607807937737, 0.9714992993633411, 0.6551410098739995, 0.6669933381541168, 0.6942539152219938, 0.0, 0.0, 0.6809438688909759, 0.8201386186719009, 0.7833810251048797, 0.8000544903820358, 0.9153762523256609, 0.7498406033528451, 0.7590365124410182, 0.8392361240013649, 0.8500412615553129, 0.7397435089013078, 0.6609565800011251, 0.8417583131266949, 0.8934968817365483, 0.8216527300055472, 0.7965706085146737, 0.8916887974683272, 0.8415182334448392, 0.8363027914010208, 0.0, 0.7777853346579894, 0.7254606844720348, 0.7201585347321869, 0.7680557002554739, 0.7412346044502663, 0.7494846968841054, 0.7263822027499889, 0.711883708518245, 0.9821745559969344, 0.0, 0.8416790853502829, 0.7797628517889518, 0.7905455599015342, 0.9295117453191768, 0.9155251972517369, 0.7922397758519735, 0.6984730919668033, 0.7823662796738529, 0.9366556062509716, 0.9366926108717522, 0.7725807149859428, 0.7950082965475492, 0.9142691263750492, 0.0, 0.9700426813055756, 0.9909426113440598, 0.7752985450667107, 0.7802380669544849, 0.7787170350825929, 0.9862972440937061, 0.9911384810825398, 0.9899161248185783, 0.8077894130601475, 0.0, 0.6851159072021358, 0.0, 0.7551942512571667, 0.666506332246803, 0.0, 0.0, 0.6969899787820655, 0.6973641829957796, 0.0, 0.0, 0.7847688486730189, 0.8665539211298733, 0.7726639027588103, 0.7691518922533469, 0.8075169285359054, 0.7957243230066613, 0.6490580662975675, 0.749601978207106, 0.7216738155732159, 0.7316480531714594, 0.671190768607634, 0.0, 0.0, 0.6967617863311683, 0.0, 0.0, 0.0, 0.8336746406767461, 0.6693083280451396, 0.6873725459645952, 0.9973077873334999, 0.689060794111224, 0.7140517373780911, 0.7379674879478908, 0.7409899158751885, 0.7323816637001663, 0.0, 0.8088230899319677, 0.0, 0.7692361648921529, 0.7032950564543304, 0.7248268249024403, 0.7807015939566986, 0.7631920173206466, 0.7403846077998676, 0.746798397243146, 0.7650479864963609, 0.7552055156079243, 0.0, 0.0, 0.7025995949454672, 0.9803392937125619, 0.7776286844478041, 0.6542828329182436, 0.7115498017717886, 0.0, 0.6552124800254087, 0.7025977740788442, 0.6987570520131013, 0.6575948755279868, 0.7665551066739827, 0.7660020055220448, 0.7691965358194307, 0.7484241073222286, 0.7402989429254403, 0.8065348613695851, 0.7404838521221925, 0.7426873484858485, 0.8460887709296036, 0.7165126320829107, 0.7136746222254465, 0.7645082790436893], [0.6740603722859694, 0.7141144619530644, 0.9017225017715622, 0.0, 0.0, 0.7323924968338401, 0.6944856759544092, 0.7592061438982475, 0.7689941762689199, 0.0, 0.7272595871313983, 0.0, 0.8071382265987217, 0.8958059628511437, 0.807641862819797, 0.8068198087556346, 0.8048175546520706, 0.8649428332711878, 0.812018390141627, 0.8574704443945254, 0.8134782461847082, 0.6517688515458746, 0.748020931188578, 0.9455163270434411, 0.7379257995514572, 0.7414347373658489, 0.9905991452782356, 0.7025522198095986, 0.6837461051109016, 0.755873656118647, 0.7491662424373091, 0.0, 0.8073865193930826, 0.7972578697076188, 0.0, 0.8381169260886738, 0.8302324466153574, 0.8088507409635103, 0.6955501110885215, 0.8836313731098311, 0.9840283487372641, 0.891579134431677, 0.7363843842016915, 0.7212482820967544, 0.7651759069522153, 0.7896475725695697, 0.7564706981294375, 0.745817150983886, 0.6766336489925885, 0.8228797044740197, 0.8180975706849641, 0.793117835416649, 0.8063322081984378, 0.8233896449581694, 0.8100711676658995, 0.7986138232993465, 0.9592486200964283, 0.7583766132247575, 0.670850731231206, 0.8086344207189121, 0.7650216230770902, 0.7980864416304675, 0.680403992294029, 0.0, 0.0, 0.0, 0.0, 0.6674114417504228, 0.971632183383381, 0.0, 0.0, 0.0, 0.0, 0.6923156467571169, 0.0, 0.8205260489672243, 0.783954450493877, 0.7981724094616415, 0.9168621953847363, 0.7560865009016108, 0.7682538340018616, 0.8417056855420104, 0.8539862612847903, 0.7466324914510233, 0.6534719658632038, 0.8434974757455694, 0.8937368830483919, 0.822068396540877, 0.7985450648631748, 0.8907363111881609, 0.8457853330594605, 0.8390758022601998, 0.6418112378928199, 0.7751205157746146, 0.7206644893828165, 0.7096428912167366, 0.7736088979986863, 0.7272858768332561, 0.7584871920729698, 0.7125610467870315, 0.7363125747655773, 0.9823114304312782, 0.0, 0.8420876827662341, 0.7884726154513516, 0.7944352556464, 0.9313535311075458, 0.9171338680209357, 0.7899970624751257, 0.6973660865817616, 0.7826407138034973, 0.9377481094184847, 0.9372582247127302, 0.7730860543503075, 0.7953200148033974, 0.9126300122175339, 0.0, 0.9708449623028118, 0.991520444340084, 0.7875789941115416, 0.7818013655985994, 0.78762652260718, 0.986853881250505, 0.9912419008694751, 0.9902770661958497, 0.8201765029613988, 0.0, 0.6765119681964673, 0.0, 0.7513301630653821, 0.6777416465881634, 0.7045765081392094, 0.0, 0.0, 0.7169546468454452, 0.0, 0.0, 0.7769733178312013, 0.8691119314709348, 0.7755585249778207, 0.7770615706043781, 0.8071488676306501, 0.8006283272219721, 0.0, 0.7474309493243313, 0.7265291853014612, 0.7473939092700588, 0.7145449151584718, 0.6920834091257752, 0.696578923790177, 0.7281579402746367, 0.7167318979053676, 0.6829872948048628, 0.0, 0.840814102931788, 0.7003923041647705, 0.6668323744170042, 0.9974800415332078, 0.7078150991404233, 0.7117902804266161, 0.743005908395076, 0.7582652904243818, 0.7331962693835289, 0.0, 0.8161631716189424, 0.0, 0.7730522027865643, 0.7089608623866761, 0.6930395058519592, 0.783887782421823, 0.768050019896356, 0.7416420032503299, 0.7465829424518573, 0.7691114741563724, 0.7546948093517365, 0.0, 0.0, 0.7230074819174832, 0.980577624813087, 0.7767744094328051, 0.6970040067092189, 0.7257378027967732, 0.0, 0.0, 0.7230828305557161, 0.715928342847603, 0.6886255934909922, 0.7810665399449284, 0.7704159266731447, 0.7759729706444167, 0.75124697199513, 0.7485798863015906, 0.8115082621326343, 0.7401589966239837, 0.7397561618410192, 0.8470914343744539, 0.7078132553557717, 0.7236051670810126, 0.7758280722663367], [0.0, 0.7073375582155241, 0.90144349665893, 0.0, 0.0, 0.7373925274940979, 0.7113493552418741, 0.7616100842734731, 0.7713437216832009, 0.0, 0.7411960239544194, 0.0, 0.8168614631835387, 0.8960757779312423, 0.8138688259880457, 0.8155738920209402, 0.8122124149343255, 0.8681978154003396, 0.8184970512822913, 0.8588913578036523, 0.8134677074216803, 0.6630221809937891, 0.7473809308634074, 0.9462754145418453, 0.7370164784531738, 0.7407769479011049, 0.9906550198721461, 0.7197658860184077, 0.7055473399476908, 0.7659696071554306, 0.7476325882087301, 0.0, 0.810091665532743, 0.8012005366967309, 0.0, 0.8415206360632322, 0.8299358569092775, 0.8157933770958452, 0.686394995584177, 0.8826811942640729, 0.9844048619591187, 0.8900503002838606, 0.7526220903201544, 0.7266542807527673, 0.7746423991042016, 0.7886235046632498, 0.7588913565764864, 0.7337884223716759, 0.0, 0.8278769878758865, 0.8153382951285384, 0.7976258351027403, 0.8160936378393148, 0.8249445005939033, 0.807097063618861, 0.8006280228988283, 0.9605492626074473, 0.7713980008242621, 0.0, 0.8064381102548763, 0.7676669257588492, 0.7961401030166817, 0.710360605466991, 0.7137380506602834, 0.0, 0.0, 0.0, 0.0, 0.9723218087633448, 0.6778132829070219, 0.0, 0.6952703420484834, 0.0, 0.0, 0.0, 0.8244156931583086, 0.7867003884509968, 0.8067797850762746, 0.9184578553476357, 0.7677786193549908, 0.782742490191625, 0.8423166346003628, 0.8555880972602496, 0.7520949310935664, 0.0, 0.844730636593764, 0.8952601106290824, 0.8245894162860488, 0.7979013641107441, 0.8926537851109266, 0.8509077768391422, 0.8421505633451996, 0.0, 0.7743111996368065, 0.7403820988523291, 0.7108687214748765, 0.7673392599141373, 0.7500988811609167, 0.7529227669267761, 0.7360004401160821, 0.7371178169321592, 0.982563093563121, 0.0, 0.8441251589809755, 0.7969851899423889, 0.8008261989380693, 0.9323197736417609, 0.9183107920812853, 0.7940680289809918, 0.6879717033931005, 0.7968899418536123, 0.9387584877392289, 0.9385391965187548, 0.7878238504674784, 0.8071901532083997, 0.9152876263893648, 0.0, 0.9710994177004801, 0.9913629117580068, 0.7875736263090148, 0.7890130874562402, 0.7902845519665787, 0.986627324373699, 0.9911668702241276, 0.9903668241456107, 0.8253892599491255, 0.0, 0.6871820421670688, 0.0, 0.7466738501958167, 0.0, 0.0, 0.0, 0.7174151840345525, 0.6874551627335167, 0.0, 0.0, 0.788595748933286, 0.8716282476526324, 0.7753903326843224, 0.78039064037343, 0.8140085408923072, 0.8051169518621519, 0.6717921495896824, 0.7444959941600372, 0.7213754188983383, 0.7470050939103243, 0.712747845536334, 0.0, 0.0, 0.6975940851843654, 0.0, 0.0, 0.0, 0.8393575646297735, 0.691520086790972, 0.7081213223897516, 0.9974686791632597, 0.7241467794110632, 0.7332136489327302, 0.7491966916890608, 0.754087393370395, 0.7426172281945326, 0.6961276388375678, 0.8162540200114237, 0.7318749828646182, 0.773839690912886, 0.7270572260029476, 0.7395437382051883, 0.7797833720253394, 0.7643390050318986, 0.7543003027038261, 0.7656138928494441, 0.7709193410254365, 0.7711630670463574, 0.0, 0.0, 0.7161007496547828, 0.9813086644001039, 0.7831680518452097, 0.6956619929389728, 0.7386711830441376, 0.0, 0.0, 0.7079428298560827, 0.6996316881786345, 0.6992309559029661, 0.7834621424280043, 0.7804709455813501, 0.7843227167360972, 0.7692357067232468, 0.7571729919752318, 0.8183386547170041, 0.7511082907654877, 0.7523053232326091, 0.8511744522763965, 0.7131733660944972, 0.7278002938116663, 0.7868155800851515], [0.0, 0.7136633101424326, 0.9019549360010243, 0.0, 0.0, 0.7152689430791317, 0.7205794741946719, 0.7769583890077325, 0.7859774116921863, 0.0, 0.7401447832519789, 0.0, 0.8160889578952182, 0.8972799961052615, 0.8193102220882863, 0.818776644530168, 0.810121399133808, 0.8696549518560653, 0.8148849415939236, 0.8590993138802328, 0.8234170913714767, 0.0, 0.7480852206381265, 0.9468754025696556, 0.7363022155478803, 0.738694682357111, 0.9910367566415907, 0.7064082209789384, 0.7040750407476336, 0.7662594363906583, 0.7403574862815558, 0.0, 0.81557394768859, 0.8046959259718022, 0.6972226198662879, 0.842524545824679, 0.8404577711829924, 0.8210877654542156, 0.6962436859329746, 0.8847342208288036, 0.9842453422484626, 0.8920814331929646, 0.7344180578651537, 0.6827701038517174, 0.7875965236102258, 0.7847378199225855, 0.7597397247341338, 0.7382470562599153, 0.0, 0.8253257894345514, 0.824680088065565, 0.7979446023914039, 0.820105135793746, 0.8318539877341135, 0.8140068121207296, 0.8041829001201316, 0.9616200075066451, 0.7870054021982543, 0.0, 0.8120605296762459, 0.7739986110389623, 0.8012251149157325, 0.7003472687877854, 0.6937777775383362, 0.6834298557160636, 0.7050577903578308, 0.0, 0.6891695188821325, 0.9722215553434661, 0.6881213344631548, 0.0, 0.7239969620659071, 0.0, 0.0, 0.6935110581544367, 0.8266404577848674, 0.7898705204334499, 0.8032391058481239, 0.9189729080603466, 0.7660507206348594, 0.7862105853872234, 0.8458609511003745, 0.8567023906201437, 0.7507730065739717, 0.693263620982624, 0.8454751632194756, 0.8966189997901821, 0.8272518928650914, 0.7991275758185219, 0.8947078593821053, 0.8532326452208088, 0.8438085340123062, 0.0, 0.7817616404784039, 0.7132355277963156, 0.7092972799715993, 0.7805062248879826, 0.7554647872811959, 0.7667141705289512, 0.7167974015283759, 0.7506646724248431, 0.9828034221523757, 0.0, 0.8435856749269023, 0.8014691059359614, 0.8063103225351381, 0.9342874297023317, 0.9211700252665227, 0.8021681858110347, 0.6977605346572147, 0.7950560730129508, 0.9390981889859075, 0.9385920781317217, 0.7866541522623063, 0.7957533750034788, 0.9163897172019918, 0.0, 0.9711249197395349, 0.9916714540893529, 0.7890229655966031, 0.7846917440654085, 0.7922993436551486, 0.9869432144390963, 0.9913605078372544, 0.9907495094015101, 0.8295002512089272, 0.0, 0.6963115302995416, 0.0, 0.755405430372956, 0.6981835213380966, 0.0, 0.0, 0.7266306452350537, 0.6974586566875736, 0.0, 0.0, 0.7910463640528009, 0.871027117642634, 0.773526001010327, 0.7799305882782377, 0.8161725719515943, 0.8045541271283941, 0.6823607793841417, 0.7667003595797178, 0.7444446238876596, 0.7466832880185749, 0.0, 0.0, 0.0, 0.7074913764613457, 0.0, 0.0, 0.0, 0.848884762944244, 0.7009799183822317, 0.7181698778713428, 0.9976112349323716, 0.7430373037133439, 0.7477554167851941, 0.7664960630417103, 0.7745746493784729, 0.7511597777595846, 0.7054848799706841, 0.8231115697773832, 0.7409657393363082, 0.7749368338090159, 0.7165662839591538, 0.7247515878819267, 0.7880000440463302, 0.7731311135936657, 0.7591754267941051, 0.7637392061962104, 0.7794949256371005, 0.7661621182536119, 0.0, 0.0, 0.7064883466701517, 0.9814716548537005, 0.7737912005131701, 0.0, 0.7245553217341275, 0.0, 0.6877122703840564, 0.7063090114987765, 0.0, 0.6903893072392319, 0.7802404003320845, 0.7765219085850344, 0.7877707508429798, 0.7783485366603868, 0.7750318824439553, 0.8201421840489818, 0.7646912250355784, 0.7607677776406272, 0.8531436829487827, 0.7176090574260011, 0.7113572605397287, 0.7848796896799365], [0.0, 0.7100499039302232, 0.9049284529196773, 0.0, 0.0, 0.7242544590195373, 0.7343855138731374, 0.784211340869061, 0.7789302973169878, 0.0, 0.7342845869859433, 0.0, 0.8165623891350329, 0.8993428881723732, 0.8189625096559033, 0.8164376545082634, 0.815006425742583, 0.871659066235987, 0.8189322832265709, 0.8619250298119273, 0.8285084830270947, 0.0, 0.7414330154031217, 0.9476290409136001, 0.744693894212349, 0.7444648909704565, 0.9913287780576433, 0.7303481676838997, 0.7056848053492807, 0.7638050243603736, 0.7346895456569029, 0.0, 0.8169421118736437, 0.807779391757201, 0.6888973632546415, 0.8401293145570023, 0.8348198750797925, 0.8189187080392347, 0.7053649192049976, 0.8841145848054038, 0.9844932215735792, 0.8925535643061102, 0.7241485553412957, 0.7100275673815429, 0.7809968613779625, 0.7894720932188528, 0.7392595146951434, 0.7546658825130532, 0.0, 0.8342707808815844, 0.826565380344777, 0.803404080833557, 0.8239358136344062, 0.8363310290553763, 0.8183573917711242, 0.8080948888458208, 0.9633389005783153, 0.7880815655265896, 0.0, 0.8161948326119395, 0.7641074962117714, 0.781313539681285, 0.0, 0.7029507538431206, 0.0, 0.7143060110204571, 0.0, 0.0, 0.9728057689588592, 0.6975219650342851, 0.0, 0.7326634016422504, 0.0, 0.0, 0.0, 0.8263434668196511, 0.7856695712597773, 0.8010329902314381, 0.9201119445056182, 0.7713053951529493, 0.7867685806026279, 0.846173367243609, 0.8562441155729457, 0.7520311664013207, 0.0, 0.8468355726152302, 0.8989145872796246, 0.8276873259692821, 0.8066137178710598, 0.8952130838539024, 0.8523671888958329, 0.8432720286448939, 0.0, 0.7882529190671218, 0.7474210952464918, 0.7001518069371249, 0.7695995506012746, 0.7602200850857115, 0.7765624276828448, 0.7145427554662653, 0.7539952767306338, 0.9834379138343315, 0.0, 0.8535600443529935, 0.8104123639893263, 0.8125428569465695, 0.9358521545058386, 0.9229333728697368, 0.8105665930913978, 0.7258196977017771, 0.8025820216645273, 0.9400314942134048, 0.9396408640286903, 0.794178509050347, 0.8132344425816815, 0.9157310116171223, 0.0, 0.971790087556431, 0.9917007676770125, 0.7943186819877892, 0.8000774183841255, 0.8000291460549375, 0.9872759210282593, 0.9915068030130209, 0.9907170746104872, 0.8357388397983365, 0.0, 0.7058893288746128, 0.0, 0.763156063725371, 0.0, 0.0, 0.0, 0.7350272089933334, 0.7068419653312156, 0.0, 0.0, 0.7980674598994053, 0.8728299359008556, 0.7830546000026382, 0.7765198506463129, 0.820342932618871, 0.8139649107013643, 0.0, 0.7640810909953475, 0.7577047623853496, 0.7592643174956255, 0.7121844184547813, 0.0, 0.0, 0.7165250360647604, 0.0, 0.0, 0.0, 0.8516416140601107, 0.0, 0.0, 0.9978251170376261, 0.7417822398599998, 0.7391882934749001, 0.7484766819475049, 0.7872176982358243, 0.7478364577246852, 0.7148245701453375, 0.8293306918259624, 0.7300053973310057, 0.792985114435964, 0.7407353952738319, 0.732606447382138, 0.7951687427193297, 0.7881061909946705, 0.7668407747956183, 0.7730010218602771, 0.7841965031999711, 0.7776506927890069, 0.0, 0.0, 0.7263927828256447, 0.9823348122923568, 0.7810840942186176, 0.0, 0.7388114872760958, 0.0, 0.0, 0.7257805836962642, 0.7184011350883596, 0.0, 0.7805373917646865, 0.7850864775821088, 0.7903368176788277, 0.7683268282451339, 0.7641673498950895, 0.8220024049040299, 0.7584643606027671, 0.7592820898717241, 0.8538556470079259, 0.712859870624658, 0.7203491862796485, 0.7900818245547153], [0.0, 0.7272360999313014, 0.9034245249353469, 0.7003099033426455, 0.0, 0.7570019247869126, 0.7425561116030732, 0.788016700586719, 0.7864280741837623, 0.0, 0.7319079263144715, 0.0, 0.8166206772065607, 0.8989504980915087, 0.8215580344669466, 0.8164248133158858, 0.8176410075393028, 0.8747843541550328, 0.8288764170257724, 0.8644401577627742, 0.827294241475601, 0.7102474685686068, 0.7584908369031909, 0.9481461840893113, 0.7544874290827861, 0.7464301898282261, 0.9913910114005196, 0.7453364319724577, 0.7218853274618162, 0.7698425207780594, 0.756341201807146, 0.0, 0.8183344259476547, 0.8115827738415039, 0.0, 0.8494188433626003, 0.8383273293667681, 0.8247231563485525, 0.0, 0.8843977806090106, 0.9851836974410921, 0.8945177487169406, 0.7507708721866929, 0.7552547110962883, 0.7862562163906526, 0.7972678826039089, 0.758708394682744, 0.7250999095681444, 0.0, 0.833117647191891, 0.8280160602887302, 0.8047840399403419, 0.8191370575117428, 0.8392751147209517, 0.8261808827318022, 0.8127831493367752, 0.965356303704452, 0.7859432207152619, 0.0, 0.827552398752172, 0.7751197386196049, 0.8052356254089783, 0.7365375375060689, 0.7402595707897243, 0.0, 0.0, 0.0, 0.0, 0.9729346597783048, 0.7064693544895201, 0.0, 0.0, 0.0, 0.0, 0.7118424417660947, 0.8305518093270751, 0.7959884771708632, 0.8107847086930247, 0.9203720401434015, 0.7690204217267413, 0.791051846190869, 0.8486349131433938, 0.8596207160177595, 0.7661612309918424, 0.6942895702265249, 0.8501354169124548, 0.8992488583421576, 0.8247736330972256, 0.8082745465134982, 0.8967023163354556, 0.8537450797154753, 0.8437608724808336, 0.0, 0.7947567868080568, 0.7128076744572671, 0.0, 0.7770134981385303, 0.7383715575049851, 0.7666472356033558, 0.7234662967626656, 0.7141798936101534, 0.9829592179099069, 0.0, 0.8527280877314878, 0.8159845095421805, 0.816341390078912, 0.9364208036486619, 0.9247923500869504, 0.8081836342044028, 0.7523326827932882, 0.8042730556010936, 0.9408469332959288, 0.9402339953648529, 0.7966032869401518, 0.8140255908830342, 0.9172678721310746, 0.0, 0.9717996849662549, 0.9916155581701225, 0.798701062748609, 0.8024640477151456, 0.8035845152627029, 0.9875732704000642, 0.9917294205244416, 0.9906901005093757, 0.8382637664627889, 0.0, 0.0, 0.0, 0.7570375781931392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7879653483600144, 0.8768581052571104, 0.7833193319080193, 0.7766281524757221, 0.82056357276514, 0.8171069426322145, 0.0, 0.7768446958192461, 0.765683390535085, 0.7625296511150527, 0.7209727706398155, 0.0, 0.7335229581299096, 0.7425380757908041, 0.0, 0.0, 0.0, 0.8524300509407872, 0.0, 0.0, 0.9977334452605441, 0.7359419199014345, 0.729166118044005, 0.7561551493879903, 0.7817602377642199, 0.7509934814266978, 0.0, 0.8329189895455748, 0.7674981378608875, 0.8009929125129521, 0.7393268809257219, 0.7123222245472355, 0.7957290029804636, 0.784610866512226, 0.7773269165632738, 0.7840546127555501, 0.7849232968791038, 0.7794165480223715, 0.7002808003252396, 0.0, 0.7423415683312787, 0.9828006301757414, 0.7909075375777975, 0.7056833368350844, 0.741274835147556, 0.0, 0.0, 0.7420386841001757, 0.7454530039357817, 0.7089936115035546, 0.7919352092942921, 0.7907098938861954, 0.7923688150411158, 0.7755765038566705, 0.7650772090484388, 0.8295516626163515, 0.7658437856359203, 0.7632521675692967, 0.8572197228082238, 0.7347104425956726, 0.7579605059798644, 0.7951982515057292], [0.7218913691998363, 0.7386095295575955, 0.9042676939345002, 0.0, 0.0, 0.7507612002459977, 0.7326604392550649, 0.7961296456329754, 0.7927360555834938, 0.0, 0.7639032642173287, 0.0, 0.8168863088446178, 0.9007851143236181, 0.8222136102126589, 0.8191519958394364, 0.8203849130155512, 0.8761501507132267, 0.8249718406181595, 0.8667059615840114, 0.8328413174862498, 0.7186030688825128, 0.7675208578549801, 0.9492891840846094, 0.761842658278451, 0.7706993317151956, 0.9916146140354881, 0.7423178283880605, 0.7225590000404357, 0.7700671623085964, 0.7609029798594701, 0.0, 0.8206774817928607, 0.8087406370851667, 0.0, 0.8504054443815707, 0.8437473188388942, 0.8272591175458572, 0.0, 0.8847199681810453, 0.9850765926360111, 0.8944891554045553, 0.7644606373458355, 0.7504031909849534, 0.7862998536627506, 0.8069259167861729, 0.7476550339205832, 0.7435904262758268, 0.0, 0.8397739167683763, 0.8264002010153376, 0.8077360491993292, 0.8285522927571339, 0.8378017258067149, 0.8300617507792437, 0.8198702522257892, 0.9652911128022079, 0.7874522044379695, 0.0, 0.8325437652317543, 0.7864239939996178, 0.8109501479364009, 0.7445101038032307, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9727132020238672, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8308368328034622, 0.793966664111695, 0.8074912965938742, 0.9218722475656744, 0.7677996118746876, 0.7903683685829054, 0.8495847035537333, 0.8629349733058311, 0.7716895048702637, 0.0, 0.8516512824579395, 0.899054102940207, 0.829432797367989, 0.8089248488076768, 0.8984897850295803, 0.8574030593122128, 0.8476137205887094, 0.0, 0.7870435148771945, 0.7385779363377402, 0.7350711063987383, 0.780710668502455, 0.7359525629464312, 0.7410702033784392, 0.7494036818146644, 0.7230073043206375, 0.9839591500366726, 0.0, 0.8530726706470937, 0.8202998494098619, 0.8239780824655082, 0.937218635083141, 0.9249113941752154, 0.8110880683512098, 0.0, 0.8042642221617823, 0.9419806983893488, 0.9416929599502136, 0.7964444738658706, 0.8150215596852657, 0.9172291519716216, 0.0, 0.9719027661006218, 0.9916864514270466, 0.8058077869328822, 0.8036446812038678, 0.8067246744022625, 0.9875932971491759, 0.9920543881593059, 0.9907307097970702, 0.8422421901849126, 0.0, 0.0, 0.0, 0.7819412082081197, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8010940063344427, 0.8777673694246038, 0.7923253657524273, 0.7901823917694021, 0.8257117063075564, 0.8179913719941189, 0.0, 0.7905704890389981, 0.7763404025324802, 0.7655380387304772, 0.7466820534665977, 0.0, 0.0, 0.7507072978939746, 0.7411995157191096, 0.0, 0.0, 0.8560429401398196, 0.7269816124827585, 0.7327981985794726, 0.9979538761679931, 0.7333367296132423, 0.7443304486416626, 0.7713332982021625, 0.7790326777719668, 0.7630133180901912, 0.0, 0.828129767181671, 0.7457716547990624, 0.7925455452331037, 0.7238545653420062, 0.7212443036617258, 0.7957500739407399, 0.780136617197152, 0.7609844748133161, 0.7743909814877479, 0.7859627334320772, 0.7658998229797939, 0.0, 0.0, 0.714891402782933, 0.9827909564725269, 0.802879198226961, 0.7312316858281172, 0.7415287083100986, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7930911954124824, 0.7963912883524443, 0.7963399818646268, 0.7811736327756071, 0.7822412620885352, 0.8309506276319742, 0.7862295818958659, 0.7864179954985789, 0.8619628024673145, 0.7578710900901788, 0.7605788647437141, 0.7948540992598293], [0.0, 0.7492469163143737, 0.907336734554475, 0.0, 0.0, 0.7753545292794578, 0.713336905369993, 0.7851653955838301, 0.7966100261672471, 0.0, 0.7577045808156183, 0.0, 0.8236566993844557, 0.9025597183810803, 0.8290826908778116, 0.8162287860235162, 0.8216344571657084, 0.8761590717508323, 0.824676952115608, 0.864527085600384, 0.8337313613716717, 0.0, 0.7464863316463886, 0.950242334283312, 0.7601444933034256, 0.7672012151032166, 0.9918275285855266, 0.7326319594258476, 0.7208278211683214, 0.7624694184314393, 0.7704553911084123, 0.0, 0.8255965120693964, 0.812407974951118, 0.0, 0.85229434391665, 0.8412109146062352, 0.833087194878045, 0.0, 0.8845154891280146, 0.985332207261066, 0.8930336056470402, 0.7308322457120313, 0.7179897524056819, 0.7806391939132304, 0.809540138283034, 0.7447905680843335, 0.740704467340971, 0.0, 0.8388806116142026, 0.8296504666302049, 0.8104039276212336, 0.828019130113828, 0.8407418242883117, 0.8208269320859212, 0.8099085276040988, 0.9665081680200394, 0.7780851658115606, 0.0, 0.8336444076504991, 0.7929473832587957, 0.8143774223894831, 0.7345613468865883, 0.0, 0.0, 0.0, 0.0, 0.7236508189627959, 0.9738115607672043, 0.7229899325358705, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8377756253156353, 0.7991190258125426, 0.8130482407414534, 0.9221904680075785, 0.7764493132179707, 0.7939123990905984, 0.852041419259145, 0.8615998919458877, 0.773561796928054, 0.0, 0.8541548783770858, 0.9023333081994381, 0.8331352169352586, 0.8115321714926536, 0.8998721545003078, 0.8599819211265783, 0.8481923910149977, 0.0, 0.7863055547321698, 0.7296175353918546, 0.7594130634648414, 0.7902518939096306, 0.7666651374261922, 0.7803517641667183, 0.7565494652602494, 0.747623584568668, 0.9836520489013462, 0.0, 0.8627464146072179, 0.81624944179058, 0.8156871060810224, 0.9385727270220295, 0.9243786445708086, 0.8098755942262139, 0.0, 0.8082931770216492, 0.9419747786981454, 0.94153678070706, 0.8005188502663082, 0.8147179399251242, 0.9192100775355269, 0.0, 0.9723047474765768, 0.9920767179738242, 0.8052430849212998, 0.8045340924301903, 0.8067460353610296, 0.9883364242631667, 0.9920535905934471, 0.9911964094246297, 0.8397039622788453, 0.0, 0.0, 0.7659365445776061, 0.7835292408127519, 0.0, 0.0, 0.0, 0.0, 0.7318127849721736, 0.0, 0.0, 0.8008550715094082, 0.8791567957127214, 0.7919212955354719, 0.7966595839100759, 0.8297268190560582, 0.8203025084020322, 0.0, 0.7925244158924987, 0.7737824033680732, 0.7765768842963632, 0.7541200696838515, 0.0, 0.0, 0.7575244464360638, 0.0, 0.0, 0.0, 0.860972024933704, 0.7349654032389791, 0.723476156087608, 0.9980820229469479, 0.7583787669629392, 0.7516913757108843, 0.770173233091751, 0.7857284348541839, 0.7597555479640193, 0.0, 0.839960000806874, 0.0, 0.7975647306527864, 0.7417601764263132, 0.7288657378705556, 0.8020837368436775, 0.7870486880179776, 0.773581105587709, 0.7785378015421909, 0.7906531712151977, 0.7825291833998383, 0.0, 0.0, 0.7398231604889287, 0.9829651869194367, 0.8106812373647521, 0.7389915849892512, 0.7561407856208688, 0.0, 0.0, 0.7225515551574326, 0.7256941023054753, 0.0, 0.7945228563331512, 0.7979021198736832, 0.7990746497631178, 0.7899903545320012, 0.7876814101175238, 0.8345595703304187, 0.7821659523789045, 0.7806215465661043, 0.8623363935156584, 0.0, 0.7272014866576636, 0.7940551130846979], [0.0, 0.7462358789283926, 0.9073686356823274, 0.0, 0.0, 0.764767424635764, 0.0, 0.8046506948121906, 0.8021898323480554, 0.0, 0.7819488497290628, 0.0, 0.8227963939009856, 0.9034522412864721, 0.8297867448134888, 0.8292050401013974, 0.8221947857024127, 0.8747579256174911, 0.8218861144103319, 0.8652900947636375, 0.8292197100305376, 0.0, 0.761981310847521, 0.9504838844829886, 0.7614999103934533, 0.7608465207750588, 0.9921087317991099, 0.746857158145391, 0.7124103225657451, 0.7812833373432672, 0.7545695671422489, 0.0, 0.825340864098714, 0.8140147922098445, 0.0, 0.8608256076220013, 0.85233950154554, 0.8407081111959953, 0.0, 0.8894183513273496, 0.985972901882517, 0.893804056264884, 0.0, 0.0, 0.8043499655760503, 0.8088978466604034, 0.751601457979404, 0.0, 0.0, 0.8419317229769322, 0.8247656612965907, 0.802212345911677, 0.8303394128829115, 0.8446511512207233, 0.8282490041897043, 0.816925380279492, 0.9675668860578391, 0.784348847499094, 0.0, 0.8346235488744858, 0.7848585500255544, 0.8064024211740028, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9735825071812519, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8387374991791616, 0.7957255872403359, 0.8020625064215757, 0.9232431605315232, 0.7700106295734744, 0.7997122318785342, 0.8531838228631647, 0.8649372851455638, 0.7661360599644511, 0.0, 0.8544540433678575, 0.902080667856532, 0.8310510805745901, 0.8139977621643467, 0.8999754476394498, 0.8608643901599966, 0.8525662223344607, 0.0, 0.7926734684371343, 0.7536481786494849, 0.7767746027038417, 0.7955917381041306, 0.7602453887644877, 0.7978191549879865, 0.77720917848693, 0.7647009941496767, 0.9838781195670191, 0.0, 0.8626863708179402, 0.8221540522817082, 0.8281594552022289, 0.9398421519529826, 0.9266450918850334, 0.8139733398952818, 0.7391576913288134, 0.8141048560569522, 0.9428203190272162, 0.9418934447148622, 0.8066843518929936, 0.8242553694548821, 0.919352138552101, 0.0, 0.9725474019571896, 0.9920642741066072, 0.808298232827639, 0.8033878476248769, 0.8152273888873354, 0.9884487248212591, 0.9921781174459839, 0.9914696173417441, 0.8470236814431966, 0.0, 0.7385456825524271, 0.7720322116941621, 0.7898413032099556, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7963005455748701, 0.8788566042603515, 0.7730311891213405, 0.789468434368974, 0.8298773634923412, 0.8170771232273315, 0.0, 0.784734069101519, 0.7408707573640241, 0.7556661401042776, 0.0, 0.0, 0.0, 0.7648937482284707, 0.0, 0.0, 0.0, 0.8629136116584041, 0.7692842133121799, 0.7311186069676902, 0.9979506253076037, 0.7653897850377555, 0.7642217645709876, 0.7769377621908198, 0.8046866144331434, 0.776566424916041, 0.0, 0.8292866017227226, 0.0, 0.8017130876326907, 0.7390835907563559, 0.7530890525771916, 0.8096771378486383, 0.7845681678963984, 0.7828615021292906, 0.7850806945557866, 0.7953139436609977, 0.7793825984448576, 0.0, 0.0, 0.0, 0.9834951681653714, 0.8119888181804932, 0.0, 0.729599692243733, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7997450978774482, 0.7956959996941011, 0.7904123602383405, 0.7804071680288104, 0.7692763045735854, 0.8395917285665571, 0.7663068444642154, 0.7485596600470202, 0.8606221815151623, 0.7185467145886228, 0.0, 0.8053972587940668], [0.0, 0.7278217635630211, 0.908095949501386, 0.0, 0.0, 0.7451953612641307, 0.7287034233861118, 0.7996144185251536, 0.8028747909872277, 0.0, 0.7615519703345827, 0.0, 0.8280475412622075, 0.9035611311244258, 0.8291063118286379, 0.826343893602909, 0.8252327763064686, 0.8797656304194137, 0.8263351374745802, 0.8681552175748162, 0.8323096806652774, 0.0, 0.7601894408350575, 0.9510615581334747, 0.7567894906598324, 0.7576312649245351, 0.9921762308151967, 0.7471848368664966, 0.7198917421651997, 0.7780294132932866, 0.770795672399117, 0.0, 0.8307947464276402, 0.8226005399115347, 0.0, 0.8653085810728779, 0.8545416878667063, 0.8403251717104534, 0.0, 0.8876196180394347, 0.9860667156710005, 0.8994118173195947, 0.0, 0.7330902337545854, 0.7956173386412118, 0.817826842387904, 0.7806208681712117, 0.0, 0.0, 0.8371936655696807, 0.8348048923854908, 0.7961970496316536, 0.8282345312662117, 0.8475232373258035, 0.8285017858579803, 0.817076290816712, 0.9687011203979707, 0.7515791030386244, 0.0, 0.8411568062635663, 0.8020681053814738, 0.8120719734274929, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.973613442530285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.837091554261513, 0.8056050084195636, 0.8189100402850973, 0.9243627098251063, 0.783773361310867, 0.798456278115853, 0.8539674604256636, 0.8663904281201115, 0.7755066300769665, 0.0, 0.8536925339234439, 0.9053614053099264, 0.8383157928562754, 0.8128771068314466, 0.9011401296434641, 0.8621807659314786, 0.8511826844234267, 0.0, 0.781139729639782, 0.7770656346679127, 0.7731177318145658, 0.792828242877796, 0.7868926933742296, 0.7882969842044997, 0.7532000116772493, 0.7614857851134281, 0.9843032188406716, 0.0, 0.8645543329787135, 0.8281413786513152, 0.8393323944194091, 0.9402230310082882, 0.9266104623842405, 0.8072591376175926, 0.0, 0.8136834362804266, 0.9432268593900801, 0.9431519164480944, 0.8058547822929554, 0.8274458234207446, 0.9208435332154369, 0.0, 0.9728543566761405, 0.9921900445282447, 0.815975393365948, 0.8210799670629276, 0.8233620549018763, 0.9885817221026949, 0.9920755112561962, 0.9914897016998488, 0.8533301285553533, 0.0, 0.0, 0.7621523335226282, 0.7955834901848268, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7832191774593487, 0.8815429420531795, 0.7941118821362787, 0.7808103322561827, 0.8321187813973709, 0.8193063293560999, 0.0, 0.7698271569036021, 0.7640444089189425, 0.7717616775925036, 0.0, 0.0, 0.0, 0.7545793354751689, 0.0, 0.0, 0.0, 0.8590328743928515, 0.7493775464661565, 0.0, 0.9980595473538558, 0.7390183502001585, 0.0, 0.7661018042660978, 0.775846255041684, 0.756343744017181, 0.0, 0.8385843158073429, 0.0, 0.8050435571665185, 0.7302563644956661, 0.0, 0.8052554073452695, 0.798392082695722, 0.7889358095122601, 0.7909664563909334, 0.7993142369822044, 0.7899177553799624, 0.0, 0.0, 0.0, 0.9837114898598034, 0.817127678893079, 0.7371401740445644, 0.7625668570818341, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7995173037489883, 0.7980889036290117, 0.7991210353451075, 0.7903555541921174, 0.778394828641608, 0.8417352017132026, 0.7807519907729772, 0.7759612517857372, 0.8661929572359077, 0.7258793146422894, 0.0, 0.802612637303254], [0.7508487544872258, 0.7347600380080215, 0.9088248501246518, 0.0, 0.0, 0.7517401771867347, 0.7511569489334727, 0.7996904615245214, 0.8080336154721466, 0.0, 0.7677539002542109, 0.0, 0.8252822836105149, 0.9034412718295158, 0.8408447483816652, 0.8335396499256555, 0.8291150361745138, 0.8829725638024509, 0.8347249887138225, 0.8688427900006829, 0.8328754430243144, 0.0, 0.7614527470525093, 0.9518978465395926, 0.7710345010733948, 0.7574172138163983, 0.9922379430978379, 0.7448284267370133, 0.7517568571958484, 0.7839497773674088, 0.7807940404567191, 0.0, 0.8344169190138238, 0.8246555908465903, 0.0, 0.8633122178284801, 0.8586309357234527, 0.8327654386664294, 0.0, 0.8905123749056609, 0.986045557719321, 0.901263799440405, 0.7515924609286253, 0.7558450530438763, 0.8067188798443078, 0.8148447056418998, 0.7742380294509407, 0.7614090760702426, 0.0, 0.8406317163006451, 0.8374842154462819, 0.8166159513681828, 0.8323826662078208, 0.8510629330547217, 0.8311601388774069, 0.8288032638680991, 0.9699604751456098, 0.7903914414261022, 0.0, 0.8539219666029256, 0.8118896123864059, 0.8271644031568631, 0.0, 0.74956307646859, 0.0, 0.0, 0.0, 0.0, 0.9741008505158958, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7494141791645739, 0.8398558235193173, 0.8071297825000104, 0.8030681355545353, 0.9249331344884891, 0.7825497825913985, 0.805234692608516, 0.8567964498038786, 0.8653746102019051, 0.778489996667412, 0.0, 0.8575433731809493, 0.9064107616309588, 0.8392698451049946, 0.814046772223917, 0.9032595868586833, 0.8655993112501451, 0.8536767858935947, 0.0, 0.7750013176239648, 0.7506838886869486, 0.0, 0.8020931661413192, 0.7731936228093478, 0.7944079394630196, 0.759663718604575, 0.7676422696777518, 0.9844658686850684, 0.0, 0.8651714510934322, 0.8270098428531752, 0.8342777184958899, 0.9410509021313787, 0.9294334781717466, 0.7914285164461565, 0.0, 0.822508133755568, 0.9423263085652174, 0.9416155750008995, 0.8155351303259867, 0.8320943434822824, 0.9213944480957299, 0.0, 0.9734941293971151, 0.9922363759732951, 0.820971384474229, 0.8141326061049635, 0.8202347157265928, 0.9887916114730185, 0.9922979892694679, 0.9914974810380969, 0.8550614875300212, 0.0, 0.0, 0.0, 0.7789655411940335, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7953880349244821, 0.8830538378055369, 0.7996426832438726, 0.800602628304778, 0.8360028465741914, 0.8109398752873809, 0.0, 0.7882752916454441, 0.7799057458488005, 0.7780047661231683, 0.0, 0.0, 0.0, 0.7608887669283926, 0.0, 0.0, 0.0, 0.8637820675365612, 0.7560499906183786, 0.744890767763431, 0.9981158862357805, 0.7777368761615291, 0.7648199129683669, 0.7888563907266549, 0.7980505861237112, 0.7466623395613815, 0.0, 0.8470417739604799, 0.7724410228702417, 0.8050062482546838, 0.0, 0.0, 0.8071280512346035, 0.8066454838821578, 0.7944468548344231, 0.7967368351814249, 0.7866692271925063, 0.7910094941605061, 0.0, 0.0, 0.0, 0.9841568882949343, 0.8224994521978422, 0.7691423791206985, 0.7687795725659627, 0.0, 0.7446478292882459, 0.0, 0.0, 0.0, 0.8016871500441611, 0.8103841813154865, 0.8161228249796824, 0.7961572637288916, 0.7867945290294046, 0.8389472928041807, 0.7898389517481947, 0.7927138426807296, 0.8658595988757433, 0.7739827997219176, 0.7645369498667947, 0.8100198337339052], [0.0, 0.7415416799760339, 0.9106095824632426, 0.0, 0.0, 0.0, 0.0, 0.797903774968142, 0.8034334151934138, 0.0, 0.7830387571765756, 0.0, 0.8394996780821485, 0.9069343632966621, 0.8395849479401777, 0.8358926017279338, 0.8335772508917567, 0.8852735630007831, 0.8437966235271347, 0.8707311100982548, 0.8431685368917509, 0.0, 0.7729111956114081, 0.9519686231495117, 0.7693483009965035, 0.7637745367761645, 0.992387400696126, 0.7718365272619451, 0.7581754948814396, 0.7710326850839211, 0.767080007773079, 0.0, 0.8396341610081801, 0.8366706573124357, 0.0, 0.8667239599895465, 0.8587077724381003, 0.8388109899273534, 0.0, 0.8933219967392302, 0.9862415165608195, 0.9019578556982767, 0.7831668620067119, 0.7778397096854938, 0.8121524259888109, 0.8239717977026679, 0.7963134796945327, 0.7925752958257105, 0.0, 0.8476674357026383, 0.8402602548296123, 0.8218194372430807, 0.8344938927300272, 0.8519683153670161, 0.8338970389443074, 0.8281947891085679, 0.9703475440675455, 0.8096865836409647, 0.0, 0.8559457621433425, 0.821517835096421, 0.832897706607365, 0.7613925990274754, 0.7555108431482813, 0.0, 0.0, 0.0, 0.0, 0.9742118847197689, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7556903466670511, 0.8416258577572379, 0.8183814827203668, 0.8267501301821223, 0.9248837826839971, 0.7968367053975938, 0.8083631631941596, 0.8571137938993509, 0.8619428885708093, 0.797425525447708, 0.7404442832676396, 0.8588475863941787, 0.9063905458300318, 0.8414215625643678, 0.816576994896514, 0.9039453519480114, 0.869353179204891, 0.8548373820750719, 0.0, 0.8086512084862263, 0.7725919272414281, 0.7690095701105361, 0.8003161832041421, 0.7905826466515841, 0.7954032215695465, 0.7869059400304221, 0.7832653217903146, 0.9844753688399996, 0.0, 0.871701743809959, 0.8364455556862121, 0.8435699051193717, 0.9431502463862692, 0.9307886727454725, 0.8226273349477393, 0.0, 0.83047637734603, 0.9436144014196431, 0.9432475990469327, 0.8234849794219384, 0.8353363921759325, 0.9225083845639982, 0.0, 0.973488724525818, 0.9924427714428277, 0.8158040543618086, 0.8275316161910083, 0.8309609501125433, 0.989043673707866, 0.9923177736922405, 0.9913810922343891, 0.8581946893435045, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8213366942208976, 0.8866510607314481, 0.8029930197157571, 0.7983479694258506, 0.8396064818582116, 0.8241215258841478, 0.0, 0.8018322205919193, 0.7764680235723314, 0.7839532378338085, 0.7637910398358968, 0.0, 0.7904553063147074, 0.0, 0.0, 0.0, 0.0, 0.868802914711247, 0.0, 0.0, 0.9982030724770098, 0.7772097032924875, 0.7617633377459854, 0.7943116831205742, 0.8030918080533842, 0.7844558717535, 0.0, 0.8521493232429572, 0.0, 0.8087476767651076, 0.7440529538478063, 0.7567691217851599, 0.8141266666577242, 0.8031704609802784, 0.7857502972514756, 0.8041216496340705, 0.8101407977987043, 0.798933064514158, 0.0, 0.0, 0.7512930000304221, 0.9844833187589002, 0.8270575723664754, 0.7658211968520591, 0.7749288881371079, 0.0, 0.0, 0.7508022008646945, 0.7534117033305225, 0.7531430991539698, 0.8099406658267954, 0.815262649119553, 0.8180049563237056, 0.8015550846558505, 0.7869438046857568, 0.8432983740825152, 0.7922030721741024, 0.7876319068330201, 0.8698116272405462, 0.0, 0.0, 0.8212798286091318], [0.0, 0.7331086748446345, 0.9132606620262202, 0.0, 0.0, 0.7642862019349587, 0.7488533833109663, 0.7936343402359785, 0.8115113207040782, 0.0, 0.7995774439888917, 0.0, 0.8396355790359762, 0.9078004624753783, 0.8409587037080519, 0.8370120590736898, 0.8388036465928012, 0.8830705420962005, 0.8435142638337276, 0.8736960615038636, 0.8439872955724224, 0.0, 0.758199360634591, 0.9525528242122929, 0.7791695108671804, 0.7696994626365321, 0.9927831484365697, 0.0, 0.0, 0.7997594716327436, 0.7792251745827176, 0.0, 0.8424071750975474, 0.8386607105482475, 0.0, 0.8701559098381634, 0.8608668785297725, 0.8491401727746534, 0.0, 0.8926857151444106, 0.9865317410167825, 0.9024325865394207, 0.7640068270891986, 0.7529277060000719, 0.7955626328987808, 0.8159455518962466, 0.7857919719048835, 0.7938460761735234, 0.0, 0.8465943176194849, 0.8372341370485479, 0.8216155316296342, 0.8355701602913587, 0.8558917012310714, 0.8279715832249974, 0.8335301796676355, 0.9708382003816228, 0.8018801222574483, 0.0, 0.8563225535635445, 0.8070019757018426, 0.8298378029114142, 0.0, 0.7613751966039856, 0.0, 0.0, 0.0, 0.0, 0.974925278623644, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.843767871539425, 0.8161801929969924, 0.8236263769195834, 0.9251037769300551, 0.793522194303051, 0.8011185343872624, 0.8572625298120352, 0.865668994339311, 0.7898718810632785, 0.0, 0.8601951751605483, 0.9084089126836171, 0.8446305540445233, 0.8241220737256204, 0.9060957206713715, 0.8708462117798295, 0.8593207498634645, 0.0, 0.7924694989699688, 0.7627690721678115, 0.7746358460036812, 0.7647607648935763, 0.760051393921479, 0.7886438703546892, 0.7805759150204504, 0.7791820675862469, 0.9849779462875659, 0.0, 0.8703358721791917, 0.8432222136970895, 0.844775460098426, 0.9431965779783922, 0.9307495673237768, 0.8260135538993899, 0.0, 0.8287167675883581, 0.9442004764725349, 0.9437196354344873, 0.8225084878986276, 0.8387171069493242, 0.9226297053384264, 0.0, 0.9734480945759846, 0.9926556941747655, 0.8228458985149335, 0.832172534964277, 0.8261016610568767, 0.9892586654025816, 0.9924345662327212, 0.9918002439956467, 0.861228404369812, 0.0, 0.0, 0.0, 0.8058055058397409, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8132924409715615, 0.8882301837364873, 0.8020836814120842, 0.8036608329143099, 0.8364794037232123, 0.8252906183321331, 0.0, 0.8208950245632646, 0.8031201990821293, 0.7954264435845714, 0.7698982200463439, 0.0, 0.796170479758623, 0.7885360722544289, 0.0, 0.0, 0.0, 0.8694283773075191, 0.0, 0.0, 0.9981410485886076, 0.7826380187041254, 0.776490095649063, 0.8033228706081441, 0.8015935803084735, 0.7951998172185519, 0.0, 0.846991465523441, 0.7841782801927962, 0.8136397722128075, 0.7742036812009985, 0.7627374121059928, 0.811888303165659, 0.8047053108761986, 0.7820314226388103, 0.799707002369977, 0.802631717330154, 0.7927382854571098, 0.0, 0.0, 0.0, 0.9850897603397116, 0.8177918325199968, 0.7566762446892964, 0.7805453192933479, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8200005986990683, 0.8233360660986035, 0.826197976512747, 0.8066218373483077, 0.809794911635599, 0.8518070338104371, 0.7941242418187539, 0.8031533876213651, 0.8734104112999435, 0.761175515401835, 0.0, 0.8267958816670135], [0.0, 0.0, 0.9135607651592557, 0.0, 0.0, 0.0, 0.0, 0.7988895174777154, 0.8140409333266171, 0.0, 0.7934738633183129, 0.0, 0.8325115929575141, 0.9075726793403959, 0.844221503266033, 0.8319607686059327, 0.8343775212598523, 0.8843080656211144, 0.8449223209045895, 0.873167423043559, 0.8421291907053591, 0.0, 0.764216882622231, 0.9536509253904941, 0.7694656299116169, 0.7520287280017056, 0.9928965027499218, 0.7629530684490249, 0.7696907000568807, 0.7870720471437379, 0.7549391888246104, 0.0, 0.8377732833901518, 0.8373454556527724, 0.0, 0.8704837846341685, 0.8582925560398537, 0.846563063319769, 0.0, 0.8935835832604234, 0.9864788423948113, 0.9010274539902866, 0.0, 0.7589223162973289, 0.8095684226707356, 0.828797655249502, 0.0, 0.7939986319696499, 0.0, 0.8468117523206117, 0.8392557752920519, 0.814673614751208, 0.8403325488441143, 0.8561884787887678, 0.831297125414125, 0.8344962356993888, 0.9717140029428519, 0.7756943875764452, 0.0, 0.8582249042184121, 0.8083049817122864, 0.8290208991605019, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9747317318689237, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7677273357074389, 0.8473613214071278, 0.8148826766571184, 0.8273056794253277, 0.9272823669354711, 0.8050159733285157, 0.8168811848967592, 0.8605895214095391, 0.8692853677927855, 0.804106037989949, 0.0, 0.8629975513460294, 0.9089907889276762, 0.8479244523064159, 0.8274243908247433, 0.9054058081817743, 0.8752281162056557, 0.8626943082517646, 0.0, 0.8068850200791104, 0.7838064979030871, 0.0, 0.800928198655368, 0.7655496489012038, 0.7702030921696794, 0.7974487745807883, 0.7696010675205438, 0.9850746959718013, 0.0, 0.8741111564297243, 0.841019729007691, 0.8458653402540265, 0.9439173022436781, 0.9323591286665434, 0.8175981238102202, 0.0, 0.8304812989417937, 0.9451473632108256, 0.9445248009478683, 0.8238728506313809, 0.8419507754268344, 0.9236583462535464, 0.0, 0.9736594155709419, 0.992595502728921, 0.8231376644657582, 0.8303527258105039, 0.8354593846159615, 0.9893276810471239, 0.9923973581314851, 0.9918518233465539, 0.8641605961050217, 0.0, 0.0, 0.0, 0.8234636752943558, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.811110608245484, 0.8857283536287863, 0.8040599675327803, 0.793332303782345, 0.8397019616015603, 0.8238101126817746, 0.0, 0.7925224038596114, 0.7725467319215559, 0.7702846941697021, 0.0, 0.0, 0.0, 0.7788410922319086, 0.0, 0.7753087871686096, 0.0, 0.8740314482466551, 0.7891520568494805, 0.7635345147633209, 0.9981988717351562, 0.7880696385243389, 0.7819512233014851, 0.816082933489277, 0.8220871085781863, 0.8001018347274476, 0.0, 0.8548640719477576, 0.0, 0.799686436150802, 0.7711400847307758, 0.7834275211202716, 0.8121895957726202, 0.7974973445685807, 0.772327976242491, 0.7831260901324539, 0.7963079828996884, 0.7886266083320611, 0.0, 0.0, 0.0, 0.985460030698762, 0.8205704149050497, 0.0, 0.762512987054726, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8215037327213431, 0.8198286328141479, 0.8213406701723055, 0.8016229901238991, 0.8001942923097587, 0.8506141554453608, 0.7991283081691681, 0.7947483959190044, 0.8725806327884926, 0.0, 0.0, 0.8172892321701655], [0.0, 0.7786877368221086, 0.9117474524250079, 0.0, 0.0, 0.7758456993421827, 0.7750456658364937, 0.8190387525131706, 0.8212947232341337, 0.0, 0.7987586038149989, 0.0, 0.8348775399755497, 0.908899669499417, 0.846938145227193, 0.8352036129860068, 0.8388785736617919, 0.8872485545265613, 0.8478046866791332, 0.8750038830112374, 0.8419707943482792, 0.0, 0.7697262875630186, 0.9539708216082317, 0.7896511711234911, 0.7720752060655541, 0.992991982164896, 0.7770167185331716, 0.7861217075169611, 0.7924871227234016, 0.7751723272632167, 0.0, 0.8404957893520334, 0.8407155291315597, 0.0, 0.8786192426980121, 0.8721170399498426, 0.8530935219256944, 0.0, 0.8933380680092957, 0.9866502151759619, 0.9015417547918908, 0.0, 0.7650551325581315, 0.8142569491952963, 0.8199406051711526, 0.7725782642519611, 0.7841832354090306, 0.0, 0.845797717515782, 0.8398280130911554, 0.8190476414360287, 0.8433599475305321, 0.8577394141873663, 0.8323148356988583, 0.8274162880388585, 0.9724619071335758, 0.7813880630134726, 0.0, 0.8616861322668086, 0.8164275189378101, 0.8352283419329325, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9751056803781925, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8495858423879545, 0.8185901755627388, 0.8313987348520301, 0.9284933832371031, 0.8057803097364755, 0.8215144336408613, 0.8624312924796623, 0.8728594149527957, 0.8047328940533729, 0.0, 0.862881752134451, 0.9102827059456856, 0.8451694906116407, 0.8249541308377533, 0.9062419561264654, 0.873230762659933, 0.8618937079041676, 0.0, 0.7875713823182666, 0.7742693488383401, 0.7707836557905393, 0.7996485267488816, 0.77191794864514, 0.8102672329262851, 0.8023657566844771, 0.7984747810547964, 0.9851159864912656, 0.0, 0.8727039366376608, 0.8378930853668513, 0.847670449540475, 0.9439321624327301, 0.9324889324243023, 0.8247666121163366, 0.0, 0.8315048164409026, 0.9453099779840618, 0.94490155634426, 0.8250295035161426, 0.8447040209227938, 0.924231584928971, 0.0, 0.973727070599119, 0.9928024413352718, 0.829512743557573, 0.8326295597166995, 0.8369218327360998, 0.9898356927688583, 0.9926258681737526, 0.991861363674627, 0.8684492422847365, 0.0, 0.0, 0.0, 0.8245531001728738, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7854570034759221, 0.8894666987305709, 0.8057231424967163, 0.8103180672348885, 0.8457504785810094, 0.8325132904042707, 0.0, 0.8084412687941445, 0.7925645286709242, 0.7759475050454128, 0.7806289235016316, 0.0, 0.0, 0.7840235322652155, 0.0, 0.0, 0.0, 0.8780062172866137, 0.7789705394414603, 0.7692549172772651, 0.998247240424693, 0.7844131685436814, 0.7786527597412779, 0.8159211726544365, 0.8223858514510751, 0.8087193860881962, 0.7830754918579648, 0.8622788959642286, 0.0, 0.8287983703597765, 0.7764192190306143, 0.7739047372126419, 0.8215039190639853, 0.8143615457490785, 0.8100285914994676, 0.8182193039146776, 0.8161323959871407, 0.8112929462411821, 0.0, 0.0, 0.7833210458686862, 0.9854755594825808, 0.8299368357165615, 0.0, 0.782623428664754, 0.0, 0.0, 0.7684360817047734, 0.7709454062457111, 0.7707032616857148, 0.8187982767507764, 0.8176218376316512, 0.8246314403437789, 0.8158621187268272, 0.8113322931136677, 0.8562675164845762, 0.8130862504256754, 0.8104181340487835, 0.8769723156442194, 0.7583490861392316, 0.7723601928944215, 0.8296212323679432], [0.0, 0.7908735830356587, 0.9135530400439708, 0.0, 0.0, 0.8097765318420203, 0.7886583005092447, 0.8232672558961167, 0.8177803377361216, 0.0, 0.794724901056666, 0.0, 0.8431981049837324, 0.9092320362140047, 0.8478215705518946, 0.8375672145461788, 0.840155461322076, 0.8886188677350685, 0.8500057910884233, 0.8772806927782393, 0.8482374441053028, 0.0, 0.0, 0.9543672781383848, 0.790633358236161, 0.7855396753407703, 0.993153188428244, 0.7738915962915319, 0.7721758324400322, 0.778260089229124, 0.803173444671961, 0.0, 0.8375979598651766, 0.8341952931264165, 0.0, 0.8736305027779963, 0.865162799877379, 0.8493571376320878, 0.0, 0.8958313548060303, 0.9868201116276138, 0.9005306664572237, 0.7807552509859416, 0.0, 0.8100415393445634, 0.8116701639700428, 0.8069316109241681, 0.7979602273135225, 0.0, 0.8516784102745568, 0.8462227403097227, 0.8166970858472145, 0.8385898514440943, 0.8607712784546732, 0.830952197284147, 0.8303081270702639, 0.9729149730893946, 0.8009340157549704, 0.0, 0.8631934067474113, 0.8207508463509698, 0.8407068426688606, 0.0, 0.7785125157185601, 0.0, 0.0, 0.0, 0.0, 0.9750252657118321, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8483965987223102, 0.8037697491267435, 0.8312875945979031, 0.9284999595137061, 0.7958249082143168, 0.8135203238793852, 0.8633126473419256, 0.8734905530863575, 0.7986983683549124, 0.0, 0.8665908249789047, 0.9103425863672564, 0.8465763468664953, 0.8300476693984997, 0.9061505223131172, 0.8767148805488846, 0.8663440637457657, 0.0, 0.8013235992883787, 0.7938731007197919, 0.0, 0.8042147115644391, 0.7769806468627276, 0.8036552670275539, 0.8024235973463826, 0.7804397298056652, 0.9852385638106238, 0.0, 0.8744281746387881, 0.8475935246631848, 0.8459687629801125, 0.9446274696918597, 0.9328174353369182, 0.8199405430448728, 0.0, 0.8286353522780129, 0.9457756482526135, 0.9452164453249027, 0.8226019158784382, 0.8385677237043553, 0.923189312435372, 0.0, 0.9739238857297917, 0.9927350780454408, 0.8260145526165559, 0.8305649951345598, 0.8438063643677254, 0.9895608931520231, 0.9928575558647226, 0.991951843883904, 0.8687501308050823, 0.0, 0.0, 0.0, 0.8201519508529366, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8171317909506927, 0.8887656526463963, 0.8181406539971384, 0.797027905546359, 0.8479174915692511, 0.831032047333401, 0.0, 0.8022293592870696, 0.797374326656242, 0.7955891805875207, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8744244150143832, 0.0, 0.0, 0.9982767834819061, 0.7751568258101308, 0.7698368001533598, 0.8051090771660124, 0.8065886125752375, 0.7987250579393788, 0.0, 0.8590605490288359, 0.0, 0.8366870986364564, 0.7815418700276203, 0.0, 0.8330966890789797, 0.8233541521691604, 0.8224041685646996, 0.8259754226146362, 0.8252341835540805, 0.8158816557705091, 0.0, 0.0, 0.0, 0.9858367149832353, 0.8190219595615581, 0.0, 0.7736112719428999, 0.0, 0.0, 0.0, 0.0, 0.7758590086652473, 0.8192763212134853, 0.8193589272857561, 0.822687008165928, 0.815160488172796, 0.807017488110147, 0.8541522546554745, 0.800140727662041, 0.8043250279179341, 0.8756830000988254, 0.7775704036840873, 0.0, 0.8221036693635683], [0.7850795626884814, 0.7704863939196087, 0.9136627426416075, 0.0, 0.0, 0.7858421242730393, 0.0, 0.8246823930537213, 0.818084605333637, 0.0, 0.7852747380686159, 0.0, 0.8387624883077816, 0.9090394298773781, 0.8486801591599407, 0.8388056968372232, 0.8411227314987186, 0.8886607615896205, 0.8401418440483911, 0.8748382197117127, 0.8500829805364054, 0.7683576552009996, 0.7664909342330941, 0.9540451130245355, 0.7909233699141532, 0.7821398995035609, 0.9931283014100516, 0.0, 0.7636441470564174, 0.8019596961223566, 0.8040611016242671, 0.0, 0.8481533154578204, 0.8439042275388198, 0.0, 0.8832251819702227, 0.8723301597520565, 0.8557723507580484, 0.7854489627825456, 0.8987906993713661, 0.9867952913336528, 0.9022711614075105, 0.0, 0.0, 0.8140836330420221, 0.8366193374124129, 0.8115509259960119, 0.7943533335202728, 0.0, 0.852207572037406, 0.8497224449204329, 0.8178742874639723, 0.8472950594504619, 0.8623333642448866, 0.8384177773357127, 0.8355858747411019, 0.9738087273169266, 0.8051413152850155, 0.0, 0.8673284995298259, 0.8180373217478255, 0.8492032567319215, 0.0, 0.7972561331281632, 0.0, 0.0, 0.0, 0.0, 0.975316008561996, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8574472669891362, 0.8255298831915456, 0.8307482850205252, 0.9287909344302151, 0.8102971178143623, 0.8237739943041594, 0.8653477405805158, 0.8729599270158782, 0.8115166864775446, 0.7694921805667726, 0.8653681978666008, 0.9114780439658378, 0.8481433805384053, 0.8275664127005598, 0.9060285725568612, 0.8811178079717681, 0.8638750551758761, 0.0, 0.7835205172455405, 0.798388295791005, 0.7812461736548006, 0.8005705305098265, 0.0, 0.8000854953087364, 0.8007956467373278, 0.0, 0.9856233187261341, 0.0, 0.8784301286428233, 0.8541629445844163, 0.8610197981764889, 0.9455212390238714, 0.9336374003373288, 0.8351268741440104, 0.7869542429037345, 0.8354173969637623, 0.9457891946984455, 0.9453672581322902, 0.8295303498373818, 0.8432359766165294, 0.9244873172614072, 0.0, 0.9743380855822947, 0.9928245406645669, 0.8326711761288864, 0.8356811595390982, 0.8453994119816425, 0.9899237984786974, 0.99304914097729, 0.9920205762117681, 0.8715452065357328, 0.0, 0.0, 0.0, 0.8093252840592269, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8244069908483048, 0.8906045331987444, 0.8118177490283008, 0.8077230103339034, 0.8504046219050451, 0.828871168048521, 0.0, 0.8356931259538727, 0.8105384914964573, 0.8087589509490906, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7907663664927465, 0.0, 0.873534833252816, 0.7891268347346296, 0.7936980418505977, 0.998340552570422, 0.0, 0.0, 0.8144503133324797, 0.0, 0.794856432748124, 0.0, 0.8718826526043045, 0.0, 0.8359489636116764, 0.7944223437176409, 0.7842853190572014, 0.8333946512042623, 0.8315906848148252, 0.8189831037538163, 0.8299634364120202, 0.8324087806383038, 0.8223499094161214, 0.0, 0.0, 0.7930661067707381, 0.9857628914738824, 0.8258811047574279, 0.0, 0.8064254505020093, 0.0, 0.0, 0.7929700962166342, 0.7954959448572543, 0.7811408877182011, 0.8260175613140038, 0.829523812955141, 0.8291341920915308, 0.8212606453190917, 0.8202156313766801, 0.8630009228318344, 0.8190360265967063, 0.8171951422978075, 0.878834972526616, 0.7909096991144062, 0.7966260284646697, 0.8287299722200311]]

In [ ]:

for kmer_length in range(10,39+1):
    fig = plt.figure(figsize=(10,10),dpi=300)
    ax = fig.add_subplot()
    filter_o, filter_k = zip(*[(o,k) for o,k,g in zip(ortho_vals,kmer_vals[kmer_length],checked_genome_lengths) if k > 0 and o > 0 and g < 5e6])
    lin_fit = np.poly1d(np.polyfit(filter_o, filter_k, 1))
    ax.plot(ortho_vals,ortho_vals,"r-")
    ax.scatter(filter_o,filter_k,marker="^",s=[2 for _ in filter_o])
    ax.plot(np.unique(ortho_vals), lin_fit(np.unique(ortho_vals)))
    ax.text(0.885,0.585,f"Zeros dropped : {len(ortho_vals) - len(filter_o)}")
    pearson_coeff = scipy.stats.pearsonr(filter_o,filter_k)

    ax.text(0.885,0.625,f"Pearson Corr. Coeff.: {pearson_coeff.statistic:.3f}")
    ax.text(0.885,0.605,f"Pearson Corr. p.: {pearson_coeff.pvalue:.3e}")

    ax.legend(["OrthoANI",f"{kmer_length}-mer dots",f"{kmer_length}-mer line : {lin_fit}"])
    plt.plot()
    plt.savefig(f"../../plots/{len(ortho_vals)}genomes-{kmer_length}mer-{c=}-estimated-ANI.png")
    plt.close()

    



In [ ]:
fig = plt.figure(figsize=(5,5),dpi=300)
ax = fig.add_subplot()
for kmer_length in range(10,19+1):
    filter_o, filter_k, gen_length = zip(*[(o,k,l) for o,k,l in zip(ortho_vals,kmer_vals[kmer_length],checked_genome_lengths) if (k > 0 and o > 0 and o < 0.8 and l > 0)])
    diff_vals = [(k-o)/g for o,k,g in zip(filter_o,filter_k,gen_length)]
    print(diff_vals)
    ax.scatter([g for g in gen_length],diff_vals,marker='^',s=[2 for _ in diff_vals])

    # plt.savefig(f"../../plots/{len(ortho_vals)}genomes-{kmer_length}mer-{c=}-length-vs-diff.png")
    # plt.close()

ax.legend([kmer_length for kmer_length in range(10,19+1)])
plt.plot()